<a href="https://colab.research.google.com/github/Tar-ive/proposal_critique/blob/main/proposal_critique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import PyPDF2
import re
from typing import Dict, List, Any
import io

class NIHGrantParser:
    def __init__(self):
        self.data = {}

    def extract_basic_info(self, text: str) -> Dict[str, Any]:
        """Extract basic application information"""
        basic_info = {}

        # Extract PI name
        pi_match = re.search(r'Contact PD/PI:\s*([^,\n]+)', text)
        basic_info['PI_Name'] = pi_match.group(1).strip() if pi_match else ''

        # Extract application ID
        app_id_match = re.search(r'Application Identifier\s*([A-Z0-9]+)', text)
        basic_info['Application_ID'] = app_id_match.group(1) if app_id_match else ''

        # Extract funding opportunity
        funding_match = re.search(r'Funding Opportunity Number:\s*([A-Z0-9-]+)', text)
        basic_info['Funding_Opportunity'] = funding_match.group(1) if funding_match else ''

        # Extract title
        title_match = re.search(r'DESCRIPTIVE TITLE OF APPLICANT\'S PROJECT\*\s*([^\n]+)', text)
        if not title_match:
            title_match = re.search(r'Title:\s*([^\n]+)', text)
        basic_info['Project_Title'] = title_match.group(1).strip() if title_match else ''

        # Extract organization
        org_match = re.search(r'Legal Name\*:\s*([^\n]+)', text)
        basic_info['Organization'] = org_match.group(1).strip() if org_match else ''

        # Extract project dates
        start_date_match = re.search(r'Start Date\*\s*([0-9/]+)', text)
        end_date_match = re.search(r'Ending Date\*\s*([0-9/]+)', text)
        basic_info['Start_Date'] = start_date_match.group(1) if start_date_match else ''
        basic_info['End_Date'] = end_date_match.group(1) if end_date_match else ''

        return basic_info

    def extract_personnel(self, text: str) -> List[Dict[str, Any]]:
        """Extract key personnel information"""
        personnel = []

        # Find personnel section
        personnel_patterns = [
            r'Senior/Key Personnel:\s*Organization:\s*Role Category:(.*?)(?=\n\n|\nAlways follow)',
            r'PROFILE - Project Director/Principal Investigator(.*?)(?=PROFILE - Senior/Key Person|$)',
            r'PROFILE - Senior/Key Person(.*?)(?=PROFILE - Senior/Key Person|$)'
        ]

        for pattern in personnel_patterns:
            matches = re.finditer(pattern, text, re.DOTALL)
            for match in matches:
                person_text = match.group(1)
                person_info = self._parse_person_info(person_text)
                if person_info:
                    personnel.append(person_info)

        return personnel

    def _parse_person_info(self, text: str) -> Dict[str, Any]:
        """Parse individual person information"""
        person = {}

        # Extract name
        name_match = re.search(r'First Name\*:\s*([^\n]+).*?Last Name\*:\s*([^\n]+)', text, re.DOTALL)
        if name_match:
            person['First_Name'] = name_match.group(1).strip()
            person['Last_Name'] = name_match.group(2).strip()
            person['Full_Name'] = f"{person['First_Name']} {person['Last_Name']}"

        # Extract role
        role_match = re.search(r'Project Role\*:\s*([^\n]+)', text)
        person['Role'] = role_match.group(1).strip() if role_match else ''

        # Extract organization
        org_match = re.search(r'Organization Name\*:\s*([^\n]+)', text)
        person['Organization'] = org_match.group(1).strip() if org_match else ''

        # Extract position
        position_match = re.search(r'Position/Title\*:\s*([^\n]+)', text)
        person['Position'] = position_match.group(1).strip() if position_match else ''

        # Extract degree
        degree_match = re.search(r'Degree Type:\s*([^\n]+)', text)
        person['Degree'] = degree_match.group(1).strip() if degree_match else ''

        return person if person else None

    def extract_budget_info(self, text: str) -> Dict[str, Any]:
        """Extract budget information"""
        budget_info = {}

        # Extract total federal funds
        federal_funds_match = re.search(r'Total Federal Funds Requested\*\s*\$([0-9,]+)', text)
        budget_info['Total_Federal_Funds'] = federal_funds_match.group(1) if federal_funds_match else ''

        # Extract total project funding
        total_funds_match = re.search(r'Total Federal & Non-Federal Funds\*\s*\$([0-9,]+)', text)
        budget_info['Total_Project_Funds'] = total_funds_match.group(1) if total_funds_match else ''

        # Extract budget periods (simplified)
        budget_periods = re.findall(r'Budget Period ([0-9]+).*?\$([0-9,]+)', text)
        for period, amount in budget_periods:
            budget_info[f'Budget_Period_{period}'] = amount

        return budget_info

    def extract_research_info(self, text: str) -> Dict[str, Any]:
        """Extract research-specific information"""
        research_info = {}

        # Extract abstract/summary
        abstract_match = re.search(r'Summary:(.*?)(?=Project Narrative|Narrative:|$)', text, re.DOTALL)
        if abstract_match:
            research_info['Abstract'] = abstract_match.group(1).strip()[:1000]  # Limit length

        # Extract specific aims
        aims_match = re.search(r'Specific Aims?\.(.*?)(?=Research Strategy|B\. Innovation|$)', text, re.DOTALL)
        if aims_match:
            research_info['Specific_Aims'] = aims_match.group(1).strip()[:1000]

        # Extract human subjects info
        human_subjects_match = re.search(r'Are Human Subjects Involved\?\*\s*(Yes|No)', text)
        research_info['Human_Subjects'] = human_subjects_match.group(1) if human_subjects_match else ''

        # Extract vertebrate animals info
        animals_match = re.search(r'Are Vertebrate Animals Used\?\*\s*(Yes|No)', text)
        research_info['Vertebrate_Animals'] = animals_match.group(1) if animals_match else ''

        # Extract clinical trial info
        clinical_trial_match = re.search(r'Clinical Trial:\s*(Y|N)', text)
        research_info['Clinical_Trial'] = clinical_trial_match.group(1) if clinical_trial_match else ''

        return research_info

    def extract_sbir_info(self, text: str) -> Dict[str, Any]:
        """Extract SBIR/STTR specific information"""
        sbir_info = {}

        # Extract program type
        program_match = re.search(r'Program Type.*?(SBIR|STTR)', text)
        sbir_info['Program_Type'] = program_match.group(1) if program_match else ''

        # Extract application type
        app_type_match = re.search(r'Application Type.*?(Phase I|Phase II|Fast-Track)', text)
        sbir_info['Application_Type'] = app_type_match.group(1) if app_type_match else ''

        # Extract small business info
        sb_match = re.search(r'small business eligibility.*?(Yes|No)', text)
        sbir_info['Small_Business_Eligible'] = sb_match.group(1) if sb_match else ''

        return sbir_info

    def parse_pdf_text(self, pdf_path: str) -> str:
        """Extract text from PDF"""
        text = ""
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
        except Exception as e:
            print(f"Error reading PDF: {e}")
        return text

    def parse_grant_application(self, pdf_path: str) -> Dict[str, Any]:
        """Main parsing function"""
        print("Extracting text from PDF...")
        text = self.parse_pdf_text(pdf_path)

        print("Parsing basic information...")
        basic_info = self.extract_basic_info(text)

        print("Parsing personnel information...")
        personnel = self.extract_personnel(text)

        print("Parsing budget information...")
        budget_info = self.extract_budget_info(text)

        print("Parsing research information...")
        research_info = self.extract_research_info(text)

        print("Parsing SBIR information...")
        sbir_info = self.extract_sbir_info(text)

        # Combine all information
        grant_data = {
            **basic_info,
            **budget_info,
            **research_info,
            **sbir_info
        }

        return grant_data, personnel

    def save_to_csv(self, pdf_path: str, output_prefix: str = "nih_grant"):
        """Parse PDF and save to CSV files"""
        grant_data, personnel = self.parse_grant_application(pdf_path)

        # Save main grant information
        grant_df = pd.DataFrame([grant_data])
        grant_df.to_csv(f"{output_prefix}_main.csv", index=False)
        print(f"Main grant data saved to {output_prefix}_main.csv")

        # Save personnel information
        if personnel:
            personnel_df = pd.DataFrame(personnel)
            personnel_df.to_csv(f"{output_prefix}_personnel.csv", index=False)
            print(f"Personnel data saved to {output_prefix}_personnel.csv")

        # Save summary information
        summary_data = {
            'Field': list(grant_data.keys()),
            'Value': list(grant_data.values())
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv(f"{output_prefix}_summary.csv", index=False)
        print(f"Summary data saved to {output_prefix}_summary.csv")

        return grant_df, personnel_df if personnel else None


# Usage example
if __name__ == "__main__":
    parser = NIHGrantParser()

    # Replace with your PDF path
    pdf_path = "harty-application.pdf"

    try:
        grant_df, personnel_df = parser.save_to_csv(pdf_path, "nih_grant_harty")

        print("\nExtracted Grant Information:")
        print(grant_df.head())

        if personnel_df is not None:
            print("\nExtracted Personnel Information:")
            print(personnel_df.head())

    except Exception as e:
        print(f"Error processing PDF: {e}")

Extracting text from PDF...
Parsing basic information...
Parsing personnel information...
Parsing budget information...
Parsing research information...
Parsing SBIR information...
Main grant data saved to nih_grant_harty_main.csv
Personnel data saved to nih_grant_harty_personnel.csv
Summary data saved to nih_grant_harty_summary.csv

Extracted Grant Information:
  PI_Name Application_ID Funding_Opportunity  \
0   Harty              1           PA-20-265   

                                       Project_Title  \
0  Development of Small Molecule Therapeutics Tar...   

                              Organization Start_Date    End_Date  \
0  Fox Chase Chemical Diversity Center Inc             04/01/2021   

  Total_Federal_Funds Total_Project_Funds  \
0                                           

                                            Abstract  \
0  The ultimate goal of this Phase II application...   

                                       Specific_Aims Human_Subjects  \
0  The ultim

In [ ]:
import pandas as pd
import PyPDF2
import re
from typing import Dict, List, Any, Optional, Callable
import json
import yaml
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
import logging
from pathlib import Path

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class ExtractionRule:
    """Defines a rule for extracting data from text"""
    name: str
    pattern: str
    extraction_type: str = "single"  # single, multiple, table
    post_process: Optional[str] = None
    required: bool = False
    default_value: Any = ""
    max_length: Optional[int] = None

@dataclass
class TableExtractionRule:
    """Defines rules for extracting tabular data"""
    name: str
    start_pattern: str
    end_pattern: str
    column_patterns: List[str]
    row_separator: str = "\n"

@dataclass
class SectionConfig:
    """Configuration for a document section"""
    name: str
    start_pattern: Optional[str] = None
    end_pattern: Optional[str] = None
    extraction_rules: List[ExtractionRule] = field(default_factory=list)
    table_rules: List[TableExtractionRule] = field(default_factory=list)
    subsections: List['SectionConfig'] = field(default_factory=list)

class TextExtractor(ABC):
    """Abstract base class for text extraction"""

    @abstractmethod
    def extract_text(self, file_path: str) -> str:
        pass

class PDFExtractor(TextExtractor):
    """PDF text extraction using PyPDF2"""

    def extract_text(self, file_path: str) -> str:
        text = ""
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page_num, page in enumerate(pdf_reader.pages):
                    try:
                        page_text = page.extract_text()
                        text += f"\n--- PAGE {page_num + 1} ---\n{page_text}\n"
                    except Exception as e:
                        logger.warning(f"Could not extract text from page {page_num + 1}: {e}")
        except Exception as e:
            logger.error(f"Error reading PDF: {e}")
            raise
        return text

class DataProcessor:
    """Handles post-processing of extracted data"""

    @staticmethod
    def clean_text(text: str, max_length: Optional[int] = None) -> str:
        """Clean and normalize text"""
        if not text:
            return ""

        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text.strip())

        # Limit length if specified
        if max_length and len(text) > max_length:
            text = text[:max_length] + "..."

        return text

    @staticmethod
    def extract_numbers(text: str) -> str:
        """Extract numeric values from text"""
        numbers = re.findall(r'[\d,]+\.?\d*', text)
        return numbers[0] if numbers else ""

    @staticmethod
    def extract_dates(text: str) -> str:
        """Extract date patterns from text"""
        date_patterns = [
            r'\d{1,2}/\d{1,2}/\d{4}',
            r'\d{4}-\d{2}-\d{2}',
            r'\w+ \d{1,2}, \d{4}'
        ]
        for pattern in date_patterns:
            dates = re.findall(pattern, text)
            if dates:
                return dates[0]
        return ""

    @staticmethod
    def extract_emails(text: str) -> List[str]:
        """Extract email addresses from text"""
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        return re.findall(email_pattern, text)

class DataExtractor:
    """Main data extraction engine"""

    def __init__(self, processor: DataProcessor = None):
        self.processor = processor or DataProcessor()

    def extract_by_rule(self, text: str, rule: ExtractionRule) -> Any:
        """Extract data based on a single rule"""
        try:
            if rule.extraction_type == "single":
                match = re.search(rule.pattern, text, re.DOTALL | re.IGNORECASE)
                if match:
                    result = match.group(1) if match.groups() else match.group(0)
                else:
                    result = rule.default_value

            elif rule.extraction_type == "multiple":
                matches = re.findall(rule.pattern, text, re.DOTALL | re.IGNORECASE)
                result = matches if matches else [rule.default_value]

            elif rule.extraction_type == "all_text":
                result = text

            else:
                result = rule.default_value

            # Post-processing
            if rule.post_process and isinstance(result, str):
                if rule.post_process == "clean_text":
                    result = self.processor.clean_text(result, rule.max_length)
                elif rule.post_process == "extract_numbers":
                    result = self.processor.extract_numbers(result)
                elif rule.post_process == "extract_dates":
                    result = self.processor.extract_dates(result)
                elif rule.post_process == "extract_emails":
                    result = self.processor.extract_emails(result)

            return result

        except Exception as e:
            logger.warning(f"Error extracting data for rule {rule.name}: {e}")
            return rule.default_value

    def extract_table_data(self, text: str, table_rule: TableExtractionRule) -> List[Dict[str, str]]:
        """Extract tabular data"""
        try:
            # Find table boundaries
            start_match = re.search(table_rule.start_pattern, text, re.IGNORECASE)
            end_match = re.search(table_rule.end_pattern, text, re.IGNORECASE)

            if not start_match:
                return []

            table_start = start_match.end()
            table_end = end_match.start() if end_match else len(text)
            table_text = text[table_start:table_end]

            # Split into rows
            rows = table_text.split(table_rule.row_separator)

            # Extract data using column patterns
            table_data = []
            for row in rows:
                row_data = {}
                for i, pattern in enumerate(table_rule.column_patterns):
                    match = re.search(pattern, row, re.IGNORECASE)
                    column_name = f"column_{i}" if i < len(table_rule.column_patterns) else f"column_{i}"
                    row_data[column_name] = match.group(1) if match and match.groups() else ""

                # Only add non-empty rows
                if any(row_data.values()):
                    table_data.append(row_data)

            return table_data

        except Exception as e:
            logger.warning(f"Error extracting table data for {table_rule.name}: {e}")
            return []

class DocumentPipeline:
    """Main pipeline for processing documents"""

    def __init__(self, config_path: Optional[str] = None, text_extractor: TextExtractor = None):
        self.text_extractor = text_extractor or PDFExtractor()
        self.data_extractor = DataExtractor()
        self.config = self.load_config(config_path) if config_path else None

    def load_config(self, config_path: str) -> Dict[str, Any]:
        """Load configuration from file"""
        config_path = Path(config_path)

        if config_path.suffix.lower() == '.json':
            with open(config_path, 'r') as f:
                return json.load(f)
        elif config_path.suffix.lower() in ['.yml', '.yaml']:
            with open(config_path, 'r') as f:
                return yaml.safe_load(f)
        else:
            raise ValueError(f"Unsupported config file format: {config_path.suffix}")

    def create_extraction_rules_from_config(self, config: Dict[str, Any]) -> List[SectionConfig]:
        """Convert configuration to extraction rules"""
        sections = []

        for section_data in config.get('sections', []):
            # Create extraction rules
            extraction_rules = []
            for rule_data in section_data.get('extraction_rules', []):
                rule = ExtractionRule(**rule_data)
                extraction_rules.append(rule)

            # Create table rules
            table_rules = []
            for table_data in section_data.get('table_rules', []):
                table_rule = TableExtractionRule(**table_data)
                table_rules.append(table_rule)

            # Create section config
            section = SectionConfig(
                name=section_data['name'],
                start_pattern=section_data.get('start_pattern'),
                end_pattern=section_data.get('end_pattern'),
                extraction_rules=extraction_rules,
                table_rules=table_rules
            )
            sections.append(section)

        return sections

    def extract_section_text(self, full_text: str, section: SectionConfig) -> str:
        """Extract text for a specific section"""
        if not section.start_pattern:
            return full_text

        start_match = re.search(section.start_pattern, full_text, re.DOTALL | re.IGNORECASE)
        if not start_match:
            logger.warning(f"Could not find start pattern for section: {section.name}")
            return ""

        section_start = start_match.start()

        if section.end_pattern:
            end_match = re.search(section.end_pattern, full_text[section_start:], re.DOTALL | re.IGNORECASE)
            section_end = section_start + end_match.start() if end_match else len(full_text)
        else:
            section_end = len(full_text)

        return full_text[section_start:section_end]

    def process_document(self, file_path: str, sections: List[SectionConfig] = None) -> Dict[str, Any]:
        """Process a document and extract structured data"""
        logger.info(f"Processing document: {file_path}")

        # Extract text
        full_text = self.text_extractor.extract_text(file_path)

        if not sections and self.config:
            sections = self.create_extraction_rules_from_config(self.config)
        elif not sections:
            raise ValueError("No extraction configuration provided")

        extracted_data = {}
        all_tables = {}

        for section in sections:
            logger.info(f"Processing section: {section.name}")

            # Get section text
            section_text = self.extract_section_text(full_text, section)

            # Extract data using rules
            section_data = {}
            for rule in section.extraction_rules:
                section_data[rule.name] = self.data_extractor.extract_by_rule(section_text, rule)

            # Extract table data
            for table_rule in section.table_rules:
                table_data = self.data_extractor.extract_table_data(section_text, table_rule)
                all_tables[f"{section.name}_{table_rule.name}"] = table_data

            extracted_data[section.name] = section_data

        return {
            'sections': extracted_data,
            'tables': all_tables,
            'metadata': {
                'file_path': file_path,
                'extraction_timestamp': pd.Timestamp.now().isoformat()
            }
        }

    def save_to_csv(self, extracted_data: Dict[str, Any], output_prefix: str):
        """Save extracted data to CSV files"""
        output_prefix = Path(output_prefix)

        # Save main sections data
        sections_data = []
        for section_name, section_data in extracted_data['sections'].items():
            flat_data = {'section': section_name}
            flat_data.update(section_data)
            sections_data.append(flat_data)

        if sections_data:
            sections_df = pd.DataFrame(sections_data)
            sections_file = f"{output_prefix}_sections.csv"
            sections_df.to_csv(sections_file, index=False)
            logger.info(f"Sections data saved to {sections_file}")

        # Save tables data
        for table_name, table_data in extracted_data['tables'].items():
            if table_data:
                table_df = pd.DataFrame(table_data)
                table_file = f"{output_prefix}_table_{table_name}.csv"
                table_df.to_csv(table_file, index=False)
                logger.info(f"Table data saved to {table_file}")

        # Save flattened data
        flattened_data = {}
        for section_name, section_data in extracted_data['sections'].items():
            for key, value in section_data.items():
                flattened_key = f"{section_name}_{key}"
                if isinstance(value, list):
                    flattened_data[flattened_key] = "; ".join(map(str, value))
                else:
                    flattened_data[flattened_key] = value

        # Add metadata
        flattened_data.update(extracted_data['metadata'])

        flattened_df = pd.DataFrame([flattened_data])
        flattened_file = f"{output_prefix}_flattened.csv"
        flattened_df.to_csv(flattened_file, index=False)
        logger.info(f"Flattened data saved to {flattened_file}")

        return flattened_df

# Configuration builder helper
class ConfigBuilder:
    """Helper class to build extraction configurations"""

    @staticmethod
    def create_nih_grant_config() -> Dict[str, Any]:
        """Create configuration for NIH grant applications"""
        return {
            "document_type": "nih_grant",
            "sections": [
                {
                    "name": "basic_info",
                    "start_pattern": r"APPLICATION FOR FEDERAL ASSISTANCE",
                    "end_pattern": r"Table of Contents",
                    "extraction_rules": [
                        {
                            "name": "pi_name",
                            "pattern": r"Contact PD/PI:\s*([^,\n]+)",
                            "post_process": "clean_text",
                            "required": True
                        },
                        {
                            "name": "application_id",
                            "pattern": r"Application Identifier\s*([A-Z0-9]+)",
                            "required": True
                        },
                        {
                            "name": "project_title",
                            "pattern": r"DESCRIPTIVE TITLE.*?\n([^\n]+)",
                            "post_process": "clean_text",
                            "max_length": 200
                        },
                        {
                            "name": "organization",
                            "pattern": r"Legal Name\*:\s*([^\n]+)",
                            "post_process": "clean_text"
                        },
                        {
                            "name": "start_date",
                            "pattern": r"Start Date\*\s*([0-9/]+)",
                            "post_process": "extract_dates"
                        },
                        {
                            "name": "end_date",
                            "pattern": r"Ending Date\*\s*([0-9/]+)",
                            "post_process": "extract_dates"
                        }
                    ]
                },
                {
                    "name": "personnel",
                    "start_pattern": r"Senior/Key Personnel:",
                    "end_pattern": r"Research & Related Budget",
                    "extraction_rules": [
                        {
                            "name": "personnel_names",
                            "pattern": r"([A-Z][a-z]+ [A-Z][a-z]+) Ph\.D",
                            "extraction_type": "multiple"
                        },
                        {
                            "name": "organizations",
                            "pattern": r"([A-Z][a-z]+ [A-Za-z ]+(?:University|Institute|Center))",
                            "extraction_type": "multiple"
                        }
                    ]
                },
                {
                    "name": "budget",
                    "start_pattern": r"Total Federal Funds Requested",
                    "end_pattern": r"SBIR/STTR Information",
                    "extraction_rules": [
                        {
                            "name": "total_federal_funds",
                            "pattern": r"Total Federal Funds Requested\*\s*\$([0-9,]+)",
                            "post_process": "extract_numbers"
                        },
                        {
                            "name": "total_project_funds",
                            "pattern": r"Total Federal & Non-Federal Funds\*\s*\$([0-9,]+)",
                            "post_process": "extract_numbers"
                        }
                    ]
                },
                {
                    "name": "research_info",
                    "start_pattern": r"Project Summary/Abstract",
                    "end_pattern": r"Bibliography",
                    "extraction_rules": [
                        {
                            "name": "abstract",
                            "pattern": r"Summary:(.*?)(?=Project Narrative|Narrative:)",
                            "post_process": "clean_text",
                            "max_length": 1000
                        },
                        {
                            "name": "human_subjects",
                            "pattern": r"Are Human Subjects Involved\?\*\s*(Yes|No)"
                        },
                        {
                            "name": "vertebrate_animals",
                            "pattern": r"Are Vertebrate Animals Used\?\*\s*(Yes|No)"
                        }
                    ]
                }
            ]
        }

    @staticmethod
    def save_config(config: Dict[str, Any], file_path: str):
        """Save configuration to file"""
        file_path = Path(file_path)

        if file_path.suffix.lower() == '.json':
            with open(file_path, 'w') as f:
                json.dump(config, f, indent=2)
        elif file_path.suffix.lower() in ['.yml', '.yaml']:
            with open(file_path, 'w') as f:
                yaml.dump(config, f, default_flow_style=False)
        else:
            raise ValueError(f"Unsupported config file format: {file_path.suffix}")

# Usage example
if __name__ == "__main__":
    # Create and save NIH grant configuration
    config = ConfigBuilder.create_nih_grant_config()
    ConfigBuilder.save_config(config, "nih_grant_config.json")

    # Initialize pipeline
    pipeline = DocumentPipeline(config_path="nih_grant_config.json")

    # Process document
    try:
        pdf_path = "harty-application.pdf"  # Replace with your PDF path
        extracted_data = pipeline.process_document(pdf_path)

        # Save to CSV
        output_files = pipeline.save_to_csv(extracted_data, "extracted_grant_data")

        print("Processing completed successfully!")
        print(f"Extracted data from {len(extracted_data['sections'])} sections")
        print(f"Found {len(extracted_data['tables'])} tables")

    except Exception as e:
        logger.error(f"Error processing document: {e}")

Processing completed successfully!
Extracted data from 4 sections
Found 0 tables


In [ ]:
import json
import re
import PyPDF2
from typing import Dict, Any, List, Optional
from dataclasses import dataclass, asdict
from pathlib import Path
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class ExtractionConfig:
    """Configuration for data extraction"""
    field_name: str
    patterns: List[str]
    extraction_type: str = "single"  # single, multiple, nested, table
    post_process: Optional[str] = None
    required: bool = False
    default_value: Any = ""

class JSONDocumentExtractor:
    """Generalizable document extractor that outputs structured JSON"""

    def __init__(self, document_type: str = "nsf_grant"):
        self.document_type = document_type
        self.extraction_configs = self._load_extraction_configs()

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file"""
        text = ""
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page_num, page in enumerate(pdf_reader.pages):
                    try:
                        page_text = page.extract_text()
                        text += f"\n{page_text}\n"
                    except Exception as e:
                        logger.warning(f"Could not extract text from page {page_num + 1}: {e}")
        except Exception as e:
            logger.error(f"Error reading PDF: {e}")
            raise
        return text

    def _load_extraction_configs(self) -> Dict[str, List[ExtractionConfig]]:
        """Load extraction configurations based on document type"""
        if self.document_type == "nsf_grant":
            return self._get_nsf_grant_configs()
        elif self.document_type == "nih_grant":
            return self._get_nih_grant_configs()
        else:
            raise ValueError(f"Unsupported document type: {self.document_type}")

    def _get_nsf_grant_configs(self) -> Dict[str, List[ExtractionConfig]]:
        """NSF Grant proposal extraction configurations"""
        return {
            "administrative_info": [
                ExtractionConfig("proposal_id", [r"Proposal No\.?\s*:?\s*([A-Z0-9-]+)", r"NSF Proposal.*?([0-9]{7})"]),
                ExtractionConfig("nsf_program", [r"Program:\s*([^\n]+)", r"NSF Program.*?:\s*([^\n]+)"]),
                ExtractionConfig("submission_date", [r"Submitted.*?:\s*([0-9/\-]+)", r"Date Submitted.*?:\s*([0-9/\-]+)"]),
                ExtractionConfig("proposal_title", [r"Project Title:\s*([^\n]+)", r"Title:\s*([^\n]+)"]),
                ExtractionConfig("pi_name", [r"Principal Investigator.*?:\s*([^\n,]+)", r"PI:\s*([^\n,]+)"]),
                ExtractionConfig("pi_email", [r"([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})", r"Email.*?:\s*([^\s\n]+@[^\s\n]+)"]),
                ExtractionConfig("institution", [r"Institution.*?:\s*([^\n]+)", r"Organization.*?:\s*([^\n]+)"]),
                ExtractionConfig("requested_amount", [r"Total Budget.*?:?\s*\$?([0-9,]+)", r"Requested Amount.*?:?\s*\$?([0-9,]+)"]),
            ],
            "budget": [
                ExtractionConfig("total_budget", [r"Total.*?Budget.*?:?\s*\$?([0-9,]+)"]),
                ExtractionConfig("personnel_costs", [r"Personnel.*?:?\s*\$?([0-9,]+)"]),
                ExtractionConfig("equipment_costs", [r"Equipment.*?:?\s*\$?([0-9,]+)"]),
                ExtractionConfig("travel_costs", [r"Travel.*?:?\s*\$?([0-9,]+)"]),
                ExtractionConfig("indirect_costs", [r"Indirect.*?Cost.*?:?\s*\$?([0-9,]+)", r"F&A.*?:?\s*\$?([0-9,]+)"]),
            ],
            "project_description": [
                ExtractionConfig("overview", [r"Project Summary.*?:\s*(.*?)(?=Intellectual Merit|Project Description)", r"Overview.*?:\s*(.*?)(?=\n\n|\n[A-Z])"]),
                ExtractionConfig("intellectual_merit", [r"Intellectual Merit.*?:\s*(.*?)(?=Broader Impact|References)", r"Merit.*?:\s*(.*?)(?=\n\n|\n[A-Z])"]),
                ExtractionConfig("broader_impacts", [r"Broader Impact.*?:\s*(.*?)(?=References|Bibliography)", r"Impact.*?:\s*(.*?)(?=\n\n|\n[A-Z])"]),
                ExtractionConfig("keywords", [r"Keywords?.*?:\s*([^\n]+)", r"Key words?.*?:\s*([^\n]+)"], extraction_type="multiple"),
            ],
            "research_plan": [
                ExtractionConfig("objectives", [r"Objective.*?:\s*(.*?)(?=Methodology|Approach)", r"Goals?.*?:\s*(.*?)(?=\n\n|\n[A-Z])"], extraction_type="multiple"),
                ExtractionConfig("methodology", [r"Methodology.*?:\s*(.*?)(?=Timeline|Expected)", r"Approach.*?:\s*(.*?)(?=\n\n|\n[A-Z])"]),
                ExtractionConfig("timeline", [r"Timeline.*?:\s*(.*?)(?=Expected|Results)", r"Schedule.*?:\s*(.*?)(?=\n\n|\n[A-Z])"]),
                ExtractionConfig("innovation", [r"Innovation.*?:\s*(.*?)(?=Significance|Impact)", r"Novel.*?:\s*(.*?)(?=\n\n|\n[A-Z])"]),
            ],
            "compliance": [
                ExtractionConfig("human_subjects", [r"Human Subjects.*?:\s*(Yes|No)", r"IRB.*?:\s*(Yes|No|Approved|Pending)"]),
                ExtractionConfig("vertebrate_animals", [r"Vertebrate Animals.*?:\s*(Yes|No)", r"IACUC.*?:\s*(Yes|No|Approved|Pending)"]),
                ExtractionConfig("biohazards", [r"Biohazard.*?:\s*(Yes|No)", r"Biological.*?Agent.*?:\s*(Yes|No)"]),
            ],
            "personnel": [
                ExtractionConfig("co_investigators", [r"Co-?PI.*?:\s*([^\n]+)", r"Co-?Investigator.*?:\s*([^\n]+)"], extraction_type="multiple"),
                ExtractionConfig("senior_personnel", [r"Senior Personnel.*?:\s*([^\n]+)"], extraction_type="multiple"),
                ExtractionConfig("collaborators", [r"Collaborator.*?:\s*([^\n]+)", r"Partner.*?:\s*([^\n]+)"], extraction_type="multiple"),
            ]
        }

    def _get_nih_grant_configs(self) -> Dict[str, List[ExtractionConfig]]:
        """NIH Grant application extraction configurations"""
        return {
            "administrative_info": [
                ExtractionConfig("application_id", [r"Application.*?ID.*?:\s*([A-Z0-9-]+)", r"Grant.*?Number.*?:\s*([A-Z0-9-]+)"]),
                ExtractionConfig("funding_opportunity", [r"Funding Opportunity.*?:\s*([A-Z0-9-]+)", r"FOA.*?:\s*([A-Z0-9-]+)"]),
                ExtractionConfig("pi_name", [r"Principal Investigator.*?:\s*([^\n,]+)", r"PI.*?:\s*([^\n,]+)"]),
                ExtractionConfig("institution", [r"Institution.*?:\s*([^\n]+)", r"Organization.*?:\s*([^\n]+)"]),
                ExtractionConfig("project_title", [r"Project Title.*?:\s*([^\n]+)", r"Title.*?:\s*([^\n]+)"]),
            ],
            "research_info": [
                ExtractionConfig("specific_aims", [r"Specific Aims.*?:\s*(.*?)(?=Research Strategy|Background)"]),
                ExtractionConfig("significance", [r"Significance.*?:\s*(.*?)(?=Innovation|Approach)"]),
                ExtractionConfig("innovation", [r"Innovation.*?:\s*(.*?)(?=Approach|Methods)"]),
                ExtractionConfig("approach", [r"Approach.*?:\s*(.*?)(?=References|Bibliography)"]),
            ]
        }

    def extract_single_value(self, text: str, patterns: List[str]) -> str:
        """Extract single value using multiple pattern attempts"""
        for pattern in patterns:
            match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
            if match:
                result = match.group(1) if match.groups() else match.group(0)
                return self._clean_text(result)
        return ""

    def extract_multiple_values(self, text: str, patterns: List[str]) -> List[str]:
        """Extract multiple values using patterns"""
        results = []
        for pattern in patterns:
            matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
            results.extend([self._clean_text(match) for match in matches])
        return list(set(results))  # Remove duplicates

    def _clean_text(self, text: str) -> str:
        """Clean and normalize extracted text"""
        if not text:
            return ""

        # Remove extra whitespace and newlines
        text = re.sub(r'\s+', ' ', text.strip())

        # Remove common artifacts
        text = re.sub(r'[^\w\s\-.,;:()$@]', '', text)

        return text.strip()

    def extract_section_data(self, text: str, section_configs: List[ExtractionConfig]) -> Dict[str, Any]:
        """Extract data for a specific section"""
        section_data = {}

        for config in section_configs:
            try:
                if config.extraction_type == "single":
                    value = self.extract_single_value(text, config.patterns)
                elif config.extraction_type == "multiple":
                    value = self.extract_multiple_values(text, config.patterns)
                else:
                    value = config.default_value

                # Set default if empty and required
                if not value and config.required:
                    value = config.default_value

                section_data[config.field_name] = value

            except Exception as e:
                logger.warning(f"Error extracting {config.field_name}: {e}")
                section_data[config.field_name] = config.default_value

        return section_data

    def extract_references(self, text: str) -> List[Dict[str, str]]:
        """Extract bibliography/references"""
        references = []

        # Find references section
        ref_patterns = [
            r"References.*?:\s*(.*?)(?=Appendix|$)",
            r"Bibliography.*?:\s*(.*?)(?=Appendix|$)",
            r"Works Cited.*?:\s*(.*?)(?=Appendix|$)"
        ]

        ref_text = ""
        for pattern in ref_patterns:
            match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
            if match:
                ref_text = match.group(1)
                break

        if ref_text:
            # Split references (simple approach)
            ref_lines = re.split(r'\n(?=[A-Z])', ref_text)

            for i, ref_line in enumerate(ref_lines):
                if len(ref_line.strip()) > 50:  # Filter out short lines
                    references.append({
                        "id": i + 1,
                        "citation": self._clean_text(ref_line),
                        "type": "unknown"
                    })

        return references

    def process_document(self, file_path: str) -> Dict[str, Any]:
        """Main processing function that extracts all data into JSON structure"""
        logger.info(f"Processing {self.document_type} document: {file_path}")

        # Extract text
        full_text = self.extract_text_from_pdf(file_path)

        # Initialize result structure
        result = {
            "document_info": {
                "file_path": str(file_path),
                "document_type": self.document_type,
                "extraction_timestamp": pd.Timestamp.now().isoformat()
            }
        }

        # Extract data for each section
        for section_name, section_configs in self.extraction_configs.items():
            logger.info(f"Extracting {section_name} data...")
            section_data = self.extract_section_data(full_text, section_configs)
            result[section_name] = section_data

        # Extract references
        logger.info("Extracting references...")
        result["references"] = self.extract_references(full_text)

        # Add full text if needed (optional)
        # result["full_text"] = full_text

        return result

    def save_json(self, data: Dict[str, Any], output_path: str):
        """Save extracted data to JSON file"""
        output_path = Path(output_path)

        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

        logger.info(f"Data saved to {output_path}")

    def validate_extracted_data(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Validate and add quality metrics to extracted data"""
        validation_results = {
            "completeness_score": 0,
            "missing_required_fields": [],
            "data_quality_issues": []
        }

        total_fields = 0
        filled_fields = 0

        for section_name, section_data in data.items():
            if section_name == "document_info":
                continue

            if isinstance(section_data, dict):
                for field_name, field_value in section_data.items():
                    total_fields += 1
                    if field_value and field_value != "":
                        filled_fields += 1
                    else:
                        validation_results["missing_required_fields"].append(f"{section_name}.{field_name}")

        validation_results["completeness_score"] = filled_fields / total_fields if total_fields > 0 else 0

        return validation_results

# Usage example
if __name__ == "__main__":
    import pandas as pd

    # Initialize extractor for NSF grant
    extractor = JSONDocumentExtractor(document_type="nsf_grant")

    try:
        # Process document
        pdf_path = "/content/1-k08-ai155816-01a1-aladra-application-508.pdf"  # Replace with your PDF path
        extracted_data = extractor.process_document(pdf_path)

        # Validate data
        validation_results = extractor.validate_extracted_data(extracted_data)
        extracted_data["validation"] = validation_results

        # Save to JSON
        extractor.save_json(extracted_data, "extracted_nsf_grant5.json")

        print("Extraction completed successfully!")
        print(f"Completeness score: {validation_results['completeness_score']:.2%}")
        print(f"Total sections extracted: {len(extracted_data) - 2}")  # Excluding document_info and validation

        # Print summary
        for section_name, section_data in extracted_data.items():
            if section_name not in ["document_info", "validation"]:
                if isinstance(section_data, dict):
                    filled_fields = sum(1 for v in section_data.values() if v and v != "")
                    print(f"{section_name}: {filled_fields}/{len(section_data)} fields extracted")

    except Exception as e:
        logger.error(f"Error processing document: {e}")

Extraction completed successfully!
Completeness score: 66.67%
Total sections extracted: 7
administrative_info: 5/8 fields extracted
budget: 5/5 fields extracted
project_description: 2/4 fields extracted
research_plan: 4/4 fields extracted
compliance: 0/3 fields extracted
personnel: 2/3 fields extracted


In [ ]:
# prompt: load "/content/extracted_nsf_grant1.json" , /content/extracted_nsf_grant2.json' .... /content/extracted_nsf_grant5.json and give me a list of all of the columns or keys of the json, whatever it is called.

file_paths = [f'/content/extracted_nsf_grant{i}.json' for i in range(1, 6)]

all_keys = set()

for file_path in file_paths:
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)

        # Recursively get all keys from nested dictionaries
        def get_keys(obj, current_path=""):
            if isinstance(obj, dict):
                for key, value in obj.items():
                    new_path = f"{current_path}.{key}" if current_path else key
                    all_keys.add(new_path)
                    get_keys(value, new_path)
            elif isinstance(obj, list):
                # Optionally add keys for list items if they are dicts
                for i, item in enumerate(obj):
                    # new_path = f"{current_path}[{i}]" # You can add array index if needed
                    get_keys(item, current_path)

        get_keys(data)

    except FileNotFoundError:
        print(f"Warning: File not found at {file_path}")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {file_path}")
    except Exception as e:
        print(f"An error occurred processing {file_path}: {e}")

print("List of all unique keys across all JSON files:")
for key in sorted(list(all_keys)):
    print(key)

List of all unique keys across all JSON files:
administrative_info
administrative_info.institution
administrative_info.nsf_program
administrative_info.pi_email
administrative_info.pi_name
administrative_info.proposal_id
administrative_info.proposal_title
administrative_info.requested_amount
administrative_info.submission_date
budget
budget.equipment_costs
budget.indirect_costs
budget.personnel_costs
budget.total_budget
budget.travel_costs
compliance
compliance.biohazards
compliance.human_subjects
compliance.vertebrate_animals
document_info
document_info.document_type
document_info.extraction_timestamp
document_info.file_path
personnel
personnel.co_investigators
personnel.collaborators
personnel.senior_personnel
project_description
project_description.broader_impacts
project_description.intellectual_merit
project_description.keywords
project_description.overview
references
references.citation
references.id
references.type
research_plan
research_plan.innovation
research_plan.methodology


In [ ]:
import json
import re
import PyPDF2
from typing import Dict, Any, List, Optional, Union
from dataclasses import dataclass
import pandas as pd
from pathlib import Path
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class GeneralizedNIHSummaryParser:
    """Generalized parser for various types of NIH Grant Summary Statements"""

    def __init__(self):
        self.data = {}

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file"""
        text = ""
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page_num, page in enumerate(pdf_reader.pages):
                    try:
                        page_text = page.extract_text()
                        text += f"\n{page_text}\n"
                    except Exception as e:
                        logger.warning(f"Could not extract text from page {page_num + 1}: {e}")
        except Exception as e:
            logger.error(f"Error reading PDF: {e}")
            raise
        return text

    def detect_grant_type(self, text: str) -> str:
        """Detect the type of NIH grant from the document"""
        # Look for common grant type indicators
        if re.search(r'\bR43\b|\bR44\b|SBIR|Small Business', text, re.IGNORECASE):
            return "SBIR"
        elif re.search(r'\bK\d+\b.*?(Career|Fellowship)', text, re.IGNORECASE):
            return "Career_Development"
        elif re.search(r'\bR01\b|\bR21\b|\bR03\b', text, re.IGNORECASE):
            return "Research_Grant"
        elif re.search(r'\bU\d+\b', text, re.IGNORECASE):
            return "Cooperative_Agreement"
        elif re.search(r'\bP\d+\b', text, re.IGNORECASE):
            return "Program_Project"
        else:
            return "Unknown"

    def extract_administrative_info(self, text: str) -> Dict[str, Any]:
        """Extract basic administrative information"""
        admin_info = {}

        # Application number - more flexible pattern
        app_patterns = [
            r'Application Number:\s*([A-Z0-9\s\-]+)',
            r'(\d+\s+[A-Z]\d+\s+[A-Z]{2}\d+\-\d+)',
            r'Grant Number:\s*([A-Z0-9\s\-]+)'
        ]

        for pattern in app_patterns:
            match = re.search(pattern, text)
            if match:
                admin_info['application_number'] = match.group(1).strip()
                break

        # Principal Investigator(s) - handle multiple formats
        pi_patterns = [
            r'Principal Investigator[^\n]*?:\s*([A-Z\-,\s]+?)(?:\(Contact\)|Applicant|Review)',
            r'Principal Investigators[^\n]*?:\s*(.*?)(?=Applicant Organization|Review Group)',
            r'PI.*?:\s*([A-Z\-,\s]+?)(?:\n|Applicant)'
        ]

        for pattern in pi_patterns:
            match = re.search(pattern, text, re.DOTALL)
            if match:
                pi_text = match.group(1).strip()
                # Clean up the PI text
                pi_text = re.sub(r'\(Contact\)', '', pi_text)
                pi_text = re.sub(r'\s+', ' ', pi_text)
                admin_info['principal_investigator'] = pi_text
                break

        # Extract individual PIs if listed alphabetically
        pi_list_match = re.search(r'Principal Investigators \(Listed Alphabetically\):\s*(.*?)(?=Applicant Organization)', text, re.DOTALL)
        if pi_list_match:
            pi_list_text = pi_list_match.group(1)
            # Split by lines and clean
            pis = [line.strip() for line in pi_list_text.split('\n') if line.strip() and not re.match(r'^\d+', line)]
            admin_info['principal_investigators_list'] = pis

        # Organization
        org_patterns = [
            r'Applicant Organization:\s*([^\n]+)',
            r'Institution:\s*([^\n]+)',
            r'Organization:\s*([^\n]+)'
        ]

        for pattern in org_patterns:
            match = re.search(pattern, text)
            if match:
                admin_info['applicant_organization'] = match.group(1).strip()
                break

        # Review Group
        review_patterns = [
            r'Review Group:\s*([^\n]+(?:\n[^\n]+)*?)(?=Meeting Date|Center for)',
            r'Study Section:\s*([^\n]+)'
        ]

        for pattern in review_patterns:
            match = re.search(pattern, text, re.DOTALL)
            if match:
                review_text = match.group(1).strip()
                review_text = re.sub(r'\s+', ' ', review_text)
                admin_info['review_group'] = review_text
                break

        # Meeting Date
        meeting_patterns = [
            r'Meeting Date:\s*([0-9/]+)',
            r'Review Date:\s*([0-9/]+)'
        ]

        for pattern in meeting_patterns:
            match = re.search(pattern, text)
            if match:
                admin_info['meeting_date'] = match.group(1)
                break

        # Council
        council_match = re.search(r'Council:\s*([A-Z0-9\s]+)', text)
        admin_info['council'] = council_match.group(1).strip() if council_match else ""

        # RFA/PA
        rfa_match = re.search(r'RFA/PA:\s*([A-Z0-9\-]+)', text)
        admin_info['rfa_pa'] = rfa_match.group(1) if rfa_match else ""

        # Requested Start Date
        start_patterns = [
            r'Requested Start:\s*([0-9/]+)',
            r'Start Date:\s*([0-9/]+)'
        ]

        for pattern in start_patterns:
            match = re.search(pattern, text)
            if match:
                admin_info['requested_start'] = match.group(1)
                break

        # Project Title - more flexible extraction
        title_patterns = [
            r'Project Title:\s*([^\n]+(?:\n[^\n]+)*?)(?=SRG Action:|Impact Score:|Next Steps:|Human Subjects)',
            r'Title:\s*([^\n]+(?:\n[^\n]+)*?)(?=SRG Action:|Impact Score:|PI:|Principal)'
        ]

        for pattern in title_patterns:
            match = re.search(pattern, text, re.DOTALL)
            if match:
                title = match.group(1).strip()
                title = re.sub(r'\s+', ' ', title)
                admin_info['project_title'] = title
                break

        # Impact Score - handle both numeric and text formats
        impact_patterns = [
            r'Impact Score:\s*(\d+)',
            r'Impact Score:\s*([^\n]+)',
            r'Priority Score:\s*(\d+)'
        ]

        for pattern in impact_patterns:
            match = re.search(pattern, text)
            if match:
                admin_info['impact_score'] = match.group(1).strip()
                break

        # Human and Animal Subjects
        human_subjects_match = re.search(r'Human Subjects:\s*([^\n]+)', text)
        admin_info['human_subjects'] = human_subjects_match.group(1).strip() if human_subjects_match else ""

        animal_subjects_match = re.search(r'Animal Subjects:\s*([^\n]+)', text)
        admin_info['animal_subjects'] = animal_subjects_match.group(1).strip() if animal_subjects_match else ""

        # Grant type detection
        admin_info['grant_type'] = self.detect_grant_type(text)

        return admin_info

    def extract_budget_info(self, text: str) -> Dict[str, Any]:
        """Extract budget information - handles various formats"""
        budget_info = {}

        # Look for budget table patterns
        budget_patterns = [
            r'Project\s+Direct Costs\s+Estimated\s+Year\s+Requested\s+Total Cost\s+((?:\d+.*?\n.*?\n.*?\n)+)',
            r'Budget Year\s+Direct Costs\s+Total Costs?\s+((?:\d+.*?\n.*?\n)+)',
            r'Year\s+\d+.*?(\$[\d,]+).*?(\$[\d,]+)'
        ]

        for pattern in budget_patterns:
            match = re.search(pattern, text, re.DOTALL)
            if match:
                budget_text = match.group(1) if len(match.groups()) > 0 else match.group(0)
                years = re.findall(r'Year\s+(\d+)|^(\d+)\s', budget_text, re.MULTILINE)
                if years:
                    budget_info['project_years'] = [y[0] or y[1] for y in years]
                break

        # Total budget - multiple patterns
        total_patterns = [
            r'TOTAL\s+([0-9,\$]+)\s+([0-9,\$]+)',
            r'Total.*?(\$[\d,]+).*?(\$[\d,]+)',
            r'Direct Costs.*?(\$[\d,]+)',
            r'Total Cost.*?(\$[\d,]+)'
        ]

        for pattern in total_patterns:
            match = re.search(pattern, text)
            if match:
                if len(match.groups()) >= 2:
                    budget_info['total_direct_costs'] = match.group(1)
                    budget_info['total_estimated_cost'] = match.group(2)
                else:
                    budget_info['total_amount'] = match.group(1)
                break

        return budget_info

    def extract_project_description(self, text: str) -> Dict[str, Any]:
        """Extract project description and related sections"""
        description_info = {}

        # Main description - multiple possible headers
        desc_patterns = [
            r'DESCRIPTION \(provided by applicant\):\s*(.*?)(?=PUBLIC HEALTH RELEVANCE:|CRITIQUE|$)',
            r'Project Description:\s*(.*?)(?=PUBLIC HEALTH RELEVANCE:|CRITIQUE|$)',
            r'Abstract:\s*(.*?)(?=PUBLIC HEALTH RELEVANCE:|CRITIQUE|$)'
        ]

        for pattern in desc_patterns:
            match = re.search(pattern, text, re.DOTALL)
            if match:
                description_info['description'] = self._clean_text(match.group(1))
                break

        # Public Health Relevance
        relevance_patterns = [
            r'PUBLIC HEALTH RELEVANCE:\s*(.*?)(?=CRITIQUE|RESUME|$)',
            r'Public Health Relevance Statement:\s*(.*?)(?=CRITIQUE|RESUME|$)',
            r'Relevance:\s*(.*?)(?=CRITIQUE|RESUME|$)'
        ]

        for pattern in relevance_patterns:
            match = re.search(pattern, text, re.DOTALL)
            if match:
                description_info['public_health_relevance'] = self._clean_text(match.group(1))
                break

        return description_info

    def extract_critique_sections(self, text: str) -> List[Dict[str, Any]]:
        """Extract all critique sections - adaptable to different formats"""
        critiques = []

        # First try the standard pattern with colon
        critique_pattern = r'CRITIQUE (\d+):(.*?)(?=CRITIQUE \d+:|THE FOLLOWING|COMMITTEE BUDGET|$)'
        critique_matches = list(re.finditer(critique_pattern, text, re.DOTALL))

        # If no matches, try without colon (like your document)
        if not critique_matches:
            critique_pattern = r'CRITIQUE (\d+)\s*\n(.*?)(?=CRITIQUE \d+|THE FOLLOWING|COMMITTEE BUDGET|$)'
            critique_matches = list(re.finditer(critique_pattern, text, re.DOTALL))

        # If still no matches, try finding them by looking for score patterns
        if not critique_matches:
            # Look for patterns like "Significance: 2" which indicate start of critiques
            score_sections = list(re.finditer(r'(Significance:\s*\d+.*?)(?=Significance:\s*\d+|THE FOLLOWING|COMMITTEE BUDGET|$)', text, re.DOTALL))
            for i, match in enumerate(score_sections):
                critique_matches.append(type('obj', (object,), {
                    'group': lambda self, x: str(i+1) if x == 1 else match.group(1)
                })())

        for match in critique_matches:
            critique_num = match.group(1)
            critique_text = match.group(2)

            # Extract scores if present
            scores = self._extract_critique_scores(critique_text)

            critique_data = {
                'critique_number': critique_num,
                'scores': scores,
                'overall_impact': self._extract_section_content(critique_text, 'Overall Impact'),
                'significance': self._extract_evaluation_section(critique_text, 'Significance'),
                'investigator': self._extract_evaluation_section(critique_text, 'Investigator'),
                'innovation': self._extract_evaluation_section(critique_text, 'Innovation'),
                'approach': self._extract_evaluation_section(critique_text, 'Approach'),
                'environment': self._extract_evaluation_section(critique_text, 'Environment'),
                # Additional sections that might be present
                'candidate': self._extract_evaluation_section(critique_text, 'Candidate'),
                'career_development': self._extract_evaluation_section(critique_text, 'Career Development'),
                'research_plan': self._extract_evaluation_section(critique_text, 'Research Plan'),
                'mentor': self._extract_evaluation_section(critique_text, 'Mentor'),
                'training_plan': self._extract_evaluation_section(critique_text, 'Training Plan')
            }

            # Extract compliance sections
            critique_data['compliance'] = self._extract_compliance_from_critique(critique_text)

            # Remove empty sections
            critique_data = {k: v for k, v in critique_data.items() if v}

            critiques.append(critique_data)

        return critiques

    def _extract_critique_scores(self, text: str) -> Dict[str, str]:
        """Extract numerical scores from critique"""
        scores = {}

        score_patterns = [
            (r'Significance:\s*(\d+)', 'significance'),
            (r'Investigator(?:\(s\))?:\s*(\d+)', 'investigator'),
            (r'Innovation:\s*(\d+)', 'innovation'),
            (r'Approach:\s*(\d+)', 'approach'),
            (r'Environment:\s*(\d+)', 'environment'),
            (r'Candidate:\s*(\d+)', 'candidate'),
            (r'Career Development.*?:\s*(\d+)', 'career_development'),
            (r'Research Plan:\s*(\d+)', 'research_plan'),
            (r'Mentor.*?:\s*(\d+)', 'mentor')
        ]

        for pattern, key in score_patterns:
            match = re.search(pattern, text)
            if match:
                scores[key] = match.group(1)

        return scores

    def _extract_evaluation_section(self, text: str, section_name: str) -> Dict[str, Any]:
        """Extract strengths and weaknesses for a specific evaluation section"""
        section_data = {}

        # Multiple patterns to try for section headers
        section_patterns = [
            rf'\d+\.\s*{section_name}.*?:(.*?)(?=\d+\.\s*\w+.*?:|Protections for Human|Vertebrate Animals|Budget and Period|$)',
            rf'{section_name}:\s*(.*?)(?=\d+\.\s*\w+.*?:|Protections for Human|Vertebrate Animals|Budget and Period|$)',
            rf'\d+\.\s*{section_name}\s*\n(.*?)(?=\d+\.\s*\w+|Protections for Human|Vertebrate Animals|Budget and Period|$)'
        ]

        section_match = None
        for pattern in section_patterns:
            section_match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
            if section_match:
                break

        if section_match:
            section_text = section_match.group(1)

            # Extract strengths - handle both "Strengths" and bullet points directly
            strengths_patterns = [
                r'Strengths?\s*(.*?)(?=Weaknesses?|\d+\.|$)',
                r'Strengths?\s*\n(.*?)(?=Weaknesses?|\d+\.|$)'
            ]

            for pattern in strengths_patterns:
                strengths_match = re.search(pattern, section_text, re.DOTALL)
                if strengths_match:
                    strengths_text = strengths_match.group(1)
                    section_data['strengths'] = self._extract_bullet_points(strengths_text)
                    break

            # Extract weaknesses
            weaknesses_patterns = [
                r'Weaknesses?\s*(.*?)(?=Strengths?|\d+\.|$)',
                r'Weaknesses?\s*\n(.*?)(?=Strengths?|\d+\.|$)'
            ]

            for pattern in weaknesses_patterns:
                weaknesses_match = re.search(pattern, section_text, re.DOTALL)
                if weaknesses_match:
                    weaknesses_text = weaknesses_match.group(1)
                    section_data['weaknesses'] = self._extract_bullet_points(weaknesses_text)
                    break

        return section_data

    def _extract_bullet_points(self, text: str) -> List[str]:
        """Extract bullet points or numbered items from text"""
        if not text:
            return []

        # Split by various bullet point indicators
        items = re.split(r'\n\s*[•·▪▫]\s*|\n\s*\d+\.\s*|\n\s*[a-z]\)\s*|\n\s*-\s+', text)

        # Also try splitting by line breaks if no bullet points found
        if len(items) <= 1:
            # Look for sentences that start new thoughts
            items = re.split(r'\n\s*(?=[A-Z])', text)

        # Clean and filter items
        cleaned_items = []
        for item in items:
            cleaned = self._clean_text(item)
            # More flexible filtering - accept shorter meaningful statements
            if cleaned and len(cleaned) > 5 and not re.match(r'^\s*(None|N/A|Not applicable)', cleaned, re.IGNORECASE):
                cleaned_items.append(cleaned)

        return cleaned_items

    def _extract_section_content(self, text: str, section_name: str) -> str:
        """Extract content from a named section"""
        pattern = rf'{section_name}:\s*(.*?)(?=\d+\.|Protections for Human|Vertebrate Animals|Budget and Period|$)'
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        return self._clean_text(match.group(1)) if match else ""

    def _extract_compliance_from_critique(self, text: str) -> Dict[str, str]:
        """Extract compliance information from critique section"""
        compliance = {}

        compliance_patterns = [
            (r'Protections for Human Subjects:\s*([^\n]+)', 'human_subjects'),
            (r'Vertebrate Animals:\s*([^\n]+)', 'vertebrate_animals'),
            (r'Biohazards:\s*([^\n]+)', 'biohazards'),
            (r'Authentication of Key Biological.*?:\s*([^\n]+)', 'resource_authentication'),
            (r'Budget and Period of Support:\s*([^\n]+)', 'budget_recommendation')
        ]

        for pattern, key in compliance_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                compliance[key] = match.group(1).strip()

        return compliance

    def extract_resume_summary(self, text: str) -> Dict[str, Any]:
        """Extract resume and summary of discussion"""
        resume_info = {}

        # Multiple possible patterns for summary
        summary_patterns = [
            r'RESUME AND SUMMARY OF DISCUSSION:\s*(.*?)(?=DESCRIPTION|CRITIQUE|$)',
            r'Summary of Discussion:\s*(.*?)(?=DESCRIPTION|CRITIQUE|$)',
            r'Review Summary:\s*(.*?)(?=DESCRIPTION|CRITIQUE|$)'
        ]

        for pattern in summary_patterns:
            match = re.search(pattern, text, re.DOTALL)
            if match:
                resume_info['summary_of_discussion'] = self._clean_text(match.group(1))
                break

        return resume_info

    def extract_committee_notes(self, text: str) -> Dict[str, Any]:
        """Extract scientific review officer's notes and committee recommendations"""
        notes_info = {}

        # Scientific Review Officer's Notes
        sro_match = re.search(r'SCIENTIFIC REVIEW OFFICER\'S NOTES:\s*(.*?)(?=COMMITTEE BUDGET|$)', text, re.DOTALL)
        if sro_match:
            notes_info['scientific_review_officer_notes'] = self._clean_text(sro_match.group(1))

        # Committee Budget Recommendations
        budget_rec_patterns = [
            r'COMMITTEE BUDGET RECOMMENDATIONS:\s*(.*?)(?=Footnotes|$)',
            r'Budget Recommendations:\s*(.*?)(?=Footnotes|$)'
        ]

        for pattern in budget_rec_patterns:
            match = re.search(pattern, text, re.DOTALL)
            if match:
                notes_info['committee_budget_recommendations'] = self._clean_text(match.group(1))
                break

        return notes_info

    def _clean_text(self, text: str) -> str:
        """Clean and normalize text"""
        if not text:
            return ""

        # Remove extra whitespace and normalize
        text = re.sub(r'\s+', ' ', text.strip())

        # Remove common page headers/footers patterns
        text = re.sub(r'\d+\s+[A-Z]\d+\s+[A-Z]{2}\d+\-\d+.*?ZRG\d+\s+[A-Z\-]+\s*\(\d+\)', '', text)
        text = re.sub(r'[A-Z\-]+,\s*[A-Z]\s*$', '', text)

        return text.strip()

    def process_summary_statement(self, pdf_path: str, debug: bool = False) -> Dict[str, Any]:
        """Main processing function - works with various NIH grant types"""
        logger.info(f"Processing NIH Summary Statement: {pdf_path}")

        # Extract text
        full_text = self.extract_text_from_pdf(pdf_path)

        # Debug: Print text sections if requested
        if debug:
            print("=== DEBUG: Looking for CRITIQUE sections ===")
            critique_positions = []
            for match in re.finditer(r'CRITIQUE \d+', full_text):
                start_pos = match.start()
                end_pos = min(start_pos + 200, len(full_text))
                print(f"Found CRITIQUE at position {start_pos}: {full_text[start_pos:end_pos]}...")
                critique_positions.append(start_pos)

            if critique_positions:
                print(f"\nFound {len(critique_positions)} CRITIQUE sections")
                # Print a sample of the first critique
                if len(critique_positions) > 0:
                    start = critique_positions[0]
                    end = critique_positions[1] if len(critique_positions) > 1 else min(start + 1000, len(full_text))
                    print(f"\n=== SAMPLE OF FIRST CRITIQUE ===")
                    print(full_text[start:end])
                    print("=== END SAMPLE ===\n")
            else:
                print("No CRITIQUE sections found!")
                # Look for other patterns
                print("\n=== Looking for alternative patterns ===")
                for pattern in ['Significance:', 'Investigator', 'Innovation:', 'Approach:', 'Environment:']:
                    matches = list(re.finditer(pattern, full_text))
                    print(f"Found {len(matches)} instances of '{pattern}'")

        # Detect grant type first
        grant_type = self.detect_grant_type(full_text)
        logger.info(f"Detected grant type: {grant_type}")

        # Extract all sections
        result = {
            "document_info": {
                "file_path": str(pdf_path),
                "document_type": "nih_summary_statement",
                "grant_type": grant_type,
                "extraction_timestamp": pd.Timestamp.now().isoformat()
            },
            "administrative_info": self.extract_administrative_info(full_text),
            "budget_info": self.extract_budget_info(full_text),
            "project_description": self.extract_project_description(full_text),
            "resume_and_summary": self.extract_resume_summary(full_text),
            "critiques": self.extract_critique_sections(full_text),
            "committee_notes": self.extract_committee_notes(full_text)
        }

        if debug:
            print(f"\n=== EXTRACTION RESULTS ===")
            print(f"Found {len(result['critiques'])} critiques")
            for i, critique in enumerate(result['critiques']):
                print(f"Critique {i+1}: {list(critique.keys())}")

        return result

    def save_json(self, data: Dict[str, Any], output_path: str):
        """Save extracted data to JSON file"""
        output_path = Path(output_path)

        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

        logger.info(f"Data saved to {output_path}")

    def create_summary_report(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Create a summary report of the extraction"""
        admin_info = data.get("administrative_info", {})

        summary = {
            "extraction_summary": {
                "grant_type": data.get("document_info", {}).get("grant_type", ""),
                "application_number": admin_info.get("application_number", ""),
                "principal_investigator": admin_info.get("principal_investigator", ""),
                "project_title": admin_info.get("project_title", ""),
                "impact_score": admin_info.get("impact_score", ""),
                "review_group": admin_info.get("review_group", ""),
                "organization": admin_info.get("applicant_organization", ""),
                "number_of_critiques": len(data.get("critiques", [])),
                "has_description": bool(data.get("project_description", {}).get("description", "")),
                "has_public_health_relevance": bool(data.get("project_description", {}).get("public_health_relevance", "")),
                "has_committee_notes": bool(data.get("committee_notes", {})),
                "sections_extracted": list(data.keys())
            }
        }

        return summary

    def save_summary_csv(self, data: Dict[str, Any], output_path: str):
        """Save a CSV summary of key information"""
        summary = self.create_summary_report(data)

        # Flatten the data for CSV
        flat_data = {}
        flat_data.update(data.get("administrative_info", {}))
        flat_data.update(summary.get("extraction_summary", {}))

        # Add critique summary
        critiques = data.get("critiques", [])
        for i, critique in enumerate(critiques):
            if 'overall_impact' in critique:
                flat_data[f"critique_{i+1}_overall_impact"] = critique["overall_impact"][:200]
            if 'scores' in critique:
                for score_type, score_value in critique['scores'].items():
                    flat_data[f"critique_{i+1}_{score_type}_score"] = score_value

        # Create DataFrame and save
        df = pd.DataFrame([flat_data])
        df.to_csv(output_path, index=False)
        logger.info(f"Summary CSV saved to {output_path}")

# Usage example
if __name__ == "__main__":
    parser = GeneralizedNIHSummaryParser()

    try:
        # Process the summary statement
        pdf_path = "/content/R43-Summary-Statement_ MacLeod-1R43AI145704-01.pdf"  # Replace with your PDF path
        extracted_data = parser.process_summary_statement(pdf_path)

        # Save to JSON
        parser.save_json(extracted_data, "nih_summary_extracted4.json")

        # Save summary CSV
        parser.save_summary_csv(extracted_data, "nih_summary_key_info.csv")

        # Print summary
        summary = parser.create_summary_report(extracted_data)
        print("Extraction Summary:")
        print(f"Grant Type: {summary['extraction_summary']['grant_type']}")
        print(f"Application: {summary['extraction_summary']['application_number']}")
        print(f"PI: {summary['extraction_summary']['principal_investigator']}")
        print(f"Impact Score: {summary['extraction_summary']['impact_score']}")
        print(f"Number of Critiques: {summary['extraction_summary']['number_of_critiques']}")

        # Print critique overview
        critiques = extracted_data.get("critiques", [])
        for i, critique in enumerate(critiques):
            print(f"\nCritique {i+1} Overview:")
            if 'scores' in critique:
                for score_type, score_value in critique['scores'].items():
                    print(f"  {score_type.title()} Score: {score_value}")

            for section in ['significance', 'investigator', 'innovation', 'approach', 'environment']:
                if section in critique and critique[section]:
                    strengths = len(critique[section].get('strengths', []))
                    weaknesses = len(critique[section].get('weaknesses', []))
                    if strengths or weaknesses:
                        print(f"  {section.title()} - Strengths: {strengths}, Weaknesses: {weaknesses}")

    except Exception as e:
        logger.error(f"Error processing summary statement: {e}")
        raise

Extraction Summary:
Grant Type: SBIR
Application: 1 R43 AI145704- 01 
P
PI: 
Impact Score: 
Number of Critiques: 1

Critique 1 Overview:


In [ ]:
# Enhanced AI Grant Reviewer using Claude API for Google Colab
# Make sure to install the required package first

# Install required packages
!pip install anthropic

import json
import os
from typing import Dict, Any, List
from IPython.display import display, Markdown
import anthropic
import os
import anthropic
from google.colab import userdata

class ClaudeGrantReviewer:
    """
    Enhanced AI Grant Reviewer that uses Claude API to analyze grant proposals
    and provide detailed critiques similar to NIH summary statements.
    """

    def __init__(self, api_key: str = None):
        """
        Initialize the Claude Grant Reviewer.

        Args:
            api_key: Anthropic API key. If None, will try to get from Colab secrets.
        """
        if api_key is None:
            api_key = userdata.get('ANTHROPIC_API_KEY')
            if api_key is None:
                raise ValueError("API key must be provided either as parameter or stored as 'ANTHROPIC_API_KEY' in Colab secrets.")

        self.client = anthropic.Anthropic(api_key=api_key)
        self.system_prompt = self._get_system_prompt()

    def _get_system_prompt(self) -> str:
        """
        Placeholder for system prompt logic.
        Replace with actual prompt generation based on document type or config.
        """
        # Example placeholder prompt
        return """You are an AI assistant specialized in reviewing grant proposals.
Provide a detailed critique of the grant based on the provided content,
following the structure of an NIH summary statement critique."""

    def _get_system_prompt(self) -> str:
        """Get the enhanced system prompt for Claude."""
        return """<role>
You are an expert AI Grant Reviewer, acting as a coach for PhD students. Your persona is that of a seasoned, constructive, and meticulous reviewer from a major funding agency like the NSF or NIH. Your goal is not to be overly harsh, but to provide clear, actionable feedback that will tangibly improve the proposal's chances of getting funded. You will analyze the structured JSON data of a grant proposal provided in the input.
</role>

<instructions>
1. **Analyze Holistically:** Carefully review all sections of the provided JSON input, paying special attention to the `project_description`, `research_plan`, and `budget` objects.

2. **Synthesize Key Points:** Identify the core research question, the proposed methods, and the expected outcomes.

3. **Structure Your Output:** Your final response must be in Markdown and contain exactly these sections:

   **SUMMARY**
   - A brief, 2-3 sentence paragraph summarizing the project's goals and approach.

   **OVERALL IMPACT AND SIGNIFICANCE**
   - Assess the potential impact of the proposed work
   - Evaluate the significance to the field and broader scientific community
   - Consider the urgency and unmet need addressed

   **STRENGTHS**
   - A bulleted list of 3-5 key strengths of the proposal
   - Focus on elements like intellectual merit, innovation, approach, team expertise, and resources

   **WEAKNESSES AND AREAS FOR IMPROVEMENT**
   - A numbered list of the most critical weaknesses that need to be addressed
   - For each point, first state the issue clearly, then provide a concrete suggestion for how to fix it
   - This is the most important section for improvement

   **DETAILED CRITIQUE BY CATEGORY**

   *Significance and Innovation:*
   - Evaluate the importance and novelty of the research
   - Assess potential to advance the field

   *Approach and Methodology:*
   - Review the soundness and feasibility of the research plan
   - Evaluate appropriateness of methods and experimental design

   *Team and Resources:*
   - Assess investigator qualifications and team composition
   - Evaluate institutional resources and environment

   *Budget and Timeline:*
   - Review budget justification and appropriateness
   - Assess timeline feasibility

   **RECOMMENDATIONS**
   - Specific recommendations for strengthening the proposal
   - Priority order for addressing weaknesses
   - Suggestions for enhancing impact and innovation

**Critical Analysis Criteria:**
- **Intellectual Merit & Innovation:** Is the idea novel and significant? Does the research plan present a compelling case for advancing knowledge?
- **Methodology:** Is the approach sound, well-described, and feasible within the proposed timeline?
- **Broader Impacts:** Does the proposal articulate a convincing benefit to society beyond the immediate research?
- **Budget Justification:** Do the costs align logically with the proposed research plan?
- **Team Expertise:** Does the team have the necessary qualifications and experience?
- **Feasibility:** Can the proposed work realistically be completed with the requested resources?

You must respond only with the markdown-formatted review following the exact structure specified above."""

    def validate_json_structure(self, proposal_data: Dict[str, Any]) -> List[str]:
        """
        Validates the structure of the grant proposal JSON data.

        Args:
            proposal_data: Dictionary containing the proposal data

        Returns:
            List of validation errors (empty if valid)
        """
        errors = []

        # Check required top-level keys
        required_keys = ['administrative_info', 'project_description']
        for key in required_keys:
            if key not in proposal_data:
                errors.append(f"Missing required section: {key}")

        # Check administrative_info structure
        if 'administrative_info' in proposal_data:
            admin_keys = ['proposal_title', 'pi_name']
            for key in admin_keys:
                if key not in proposal_data['administrative_info']:
                    errors.append(f"Missing administrative_info field: {key}")

        return errors

    def analyze_proposal(self, proposal_json: str, model: str = "claude-3-5-sonnet-20241022") -> str:
        """
        Analyzes a grant proposal using Claude API and returns structured feedback.

        Args:
            proposal_json: JSON string containing the proposal data
            model: Claude model to use (default: claude-3-5-sonnet-20241022)

        Returns:
            Markdown-formatted review
        """
        try:
            # Parse and validate JSON
            proposal_data = json.loads(proposal_json)
            validation_errors = self.validate_json_structure(proposal_data)

            if validation_errors:
                return "**Validation Errors:**\n" + "\n".join(f"- {error}" for error in validation_errors)

            # Prepare the user message
            user_message = f"""Please review the following grant proposal JSON data and provide a comprehensive critique following the format of NIH summary statements:

```json
{proposal_json}
```

Provide your review following the exact format specified in your instructions, with particular attention to:
1. Overall impact and significance
2. Detailed strengths and weaknesses
3. Specific recommendations for improvement
4. Assessment of feasibility and innovation

Make your critique constructive and actionable, similar to how expert reviewers provide feedback in NIH summary statements."""

            # Call Claude API
            response = self.client.messages.create(
                model=model,
                max_tokens=3000,
                temperature=0.1,
                system=self.system_prompt,
                messages=[
                    {
                        "role": "user",
                        "content": user_message
                    }
                ]
            )

            return response.content[0].text

        except json.JSONDecodeError as e:
            return f"**JSON Parsing Error:** {str(e)}\n\nPlease ensure your input is valid JSON."
        except Exception as e:
            return f"**API Error:** {str(e)}\n\nPlease check your API key and try again."

    def compare_with_summary_statement(self, proposal_json: str, summary_statement_json: str, model: str = "claude-3-5-sonnet-20241022") -> str:
        """
        Compares the proposal with an existing summary statement to provide enhanced feedback.

        Args:
            proposal_json: JSON string containing the proposal data
            summary_statement_json: JSON string containing the summary statement data
            model: Claude model to use

        Returns:
            Comparative analysis in markdown format
        """
        try:
            # Parse JSONs
            proposal_data = json.loads(proposal_json)
            summary_data = json.loads(summary_statement_json)

            # Prepare comparative analysis message
            user_message = f"""Please analyze this grant proposal and compare it with the provided summary statement to give enhanced feedback:

**ORIGINAL PROPOSAL:**
```json
{proposal_json}
```

**SUMMARY STATEMENT (for reference):**
```json
{summary_statement_json}
```

Provide a comprehensive analysis that:
1. Reviews the proposal using the same standards as the summary statement
2. Identifies how the proposal could be improved based on the feedback patterns in the summary statement
3. Provides specific recommendations for strengthening weak areas
4. Suggests how to maintain and enhance the strong aspects

Format your response according to the structured format specified in your instructions."""

            # Call Claude API
            response = self.client.messages.create(
                model=model,
                max_tokens=3500,
                temperature=0.1,
                system=self.system_prompt,
                messages=[
                    {
                        "role": "user",
                        "content": user_message
                    }
                ]
            )

            return response.content[0].text

        except json.JSONDecodeError as e:
            return f"**JSON Parsing Error:** {str(e)}\n\nPlease ensure both inputs are valid JSON."
        except Exception as e:
            return f"**API Error:** {str(e)}\n\nPlease check your API key and try again."

# Convenience functions for Google Colab usage
def setup_claude_reviewer(api_key: str = None):
    """
    Set up the Claude Grant Reviewer. Call this first with your API key.

    Args:
        api_key: Your Anthropic API key

    Returns:
        ClaudeGrantReviewer instance
    """
    if api_key:
        os.environ['ANTHROPIC_API_KEY'] = api_key

    try:
        reviewer = ClaudeGrantReviewer(api_key)
        print("✅ Enhanced Claude Grant Reviewer successfully initialized!")
        return reviewer
    except Exception as e:
        print(f"❌ Error initializing reviewer: {e}")
        return None

def review_grant_proposal(reviewer: ClaudeGrantReviewer, proposal_json: str, display_review: bool = True):
    """
    Review a grant proposal using Claude API.

    Args:
        reviewer: ClaudeGrantReviewer instance
        proposal_json: JSON string containing the proposal data
        display_review: Whether to display the review in Colab (default: True)

    Returns:
        Review text
    """
    if reviewer is None:
        print("❌ Reviewer not initialized. Please run setup_claude_reviewer() first.")
        return None

    print("🔍 Analyzing proposal with Claude...")
    review = reviewer.analyze_proposal(proposal_json)

    if display_review:
        display(Markdown(review))

    return review

def compare_proposal_with_summary(reviewer: ClaudeGrantReviewer, proposal_json: str, summary_statement_json: str, display_review: bool = True):
    """
    Compare a proposal with a summary statement for enhanced feedback.

    Args:
        reviewer: ClaudeGrantReviewer instance
        proposal_json: JSON string containing the proposal data
        summary_statement_json: JSON string containing the summary statement data
        display_review: Whether to display the review in Colab (default: True)

    Returns:
        Comparative analysis text
    """
    if reviewer is None:
        print("❌ Reviewer not initialized. Please run setup_claude_reviewer() first.")
        return None

    print("🔍 Performing comparative analysis with Claude...")
    analysis = reviewer.compare_with_summary_statement(proposal_json, summary_statement_json)

    if display_review:
        display(Markdown(analysis))

    return analysis

# Example usage and test function
def demo_grant_reviewer():
    """
    Demo function showing how to use the Enhanced Claude Grant Reviewer.
    """

    # Example proposal JSON (simplified version)
    example_proposal = """{
  "document_info": {
    "file_path": "harty-application.pdf",
    "document_type": "nsf_grant",
    "extraction_timestamp": "2025-07-10T23:27:28.701138"
  },
  "administrative_info": {
    "proposal_id": "",
    "nsf_program": "",
    "submission_date": "18902",
    "proposal_title": "Development of Small Molecule Therapeutics Targeting Hemorrhagic Fever",
    "pi_name": "4040- 0010",
    "pi_email": "deaweb@niaid.nih.gov",
    "institution": "Host-Pathogen Interactions is the study of the basic biology of infection in humans and animals  the immediate i ntersection of host and",
    "requested_amount": ""
  },
  "budget": {
    "total_budget": "1",
    "personnel_costs": ",",
    "equipment_costs": "19",
    "travel_costs": ",",
    "indirect_costs": "40"
  },
  "project_description": {
    "overview": "Development of Host -Oriented Small Molecule Therapeutics for Filoviruses and Arena viruses 154 Inclusion Enrollment Reports _________________________________________________________ 160 Vertebrate Animals 167 Select Agent Research 173 Bibliography  References Cited 175 ConsortiumContractual Arrangements ______________________________________________________ 178 Letters of Support 180 Resource Sharing Plan(s) 189 Authentication of Key Biological andor Chemical Resources ____________________________________ 190 Table of Contents Page 3 Contact PDPI: Harty, Ronald N. OMB Number: 4040- 0010 Expiration Date: 12312022 ProjectPerformance Site Location(s) ProjectPerformance Site Primary Location  I am submitting an application as an individual, and not on behalf of a company, state, local or tribal government, academia, or other type of organization. Organization Name: Fox Chase Chemical Diversity Center, Inc. Duns Number: Street1: Pennsylvania Biotechnology Center Street2: City: Doylestown County: Bucks State: PA: Pennsylvania Province: Country: USA: UNITED STATES Zip  Postal Code: ProjectPerformance Site Congressional District: PA-001 ProjectPerformance Site Location 1  I am submitting an application as an individual, and not on behalf of a company, state, local or tribal government, academia, or other type of organization. Organization Name: University of Pennsylvania DUNS Number: Street1: Street2: City: Philadelphia County: State: PA: Pennsylvania Province: Country: USA: UNITED STATES Zip  Postal Code: 19104- 6205 ProjectPerformance Site Congressional District: PA-002 Page 4 Tracking Number: GRANT13199760 Funding Opportunity Number: PA-20-265. Received Date: 2020-09-08T12:43:45.000- 04:00 Contact PDPI: Harty, Ronald N. ProjectPerformance Site Location 2  I am submitting an application as an individual, and not on behalf of a company, state, local or tribal government, academia, or other type of organization. Organization Name: Texas Biomedical Research Institute DUNS Number: Street1: 8715 W. Military Dr. Street2: San City Antonio County: State: TX: Texas Province: Country: USA: UNITED STATES Zip  Postal Code: 78227- 5302 ProjectPerformance Site Congressional District: TX-020 Additional Location(s) File Name: Page 5 Tracking Number: GRANT13199760 Funding Opportunity Number: PA-20-265. Received Date: 2020-09-08T12:43:45.000- 04:00 Contact PDPI: Harty, Ronald N. OMB Number: 4040- 0010 Expiration Date: 12312022 RESEARCH  RELATED Other Project Information 1. Are Human Subjects Involved  Yes  No 1.a. If YES to Human Subjects Is the Project Exempt from Federal regulations  Yes  No If YES, check appropriate exemption number: 1 2 3 4 5 6 7 ___ If NO, is the IRB review Pending  Yes  No IRB Approval Date: Human Subject Assurance Number none 8 2. Are Vertebrate Animals Used  Yes  No 2.a. If YES to Vertebrate Animals Is the IACUC review Pending  Yes  No IACUC Approval Date: Animal Welfare Assurance Number none 3. Is proprietaryprivileged information included in the application  Yes  No 4.a. Does this project have an actual or potential impact -positive or negative -on the environment  Yes 4.b. If yes, please explain: 4.c. If this project has an actual or potential impact on the environment, has an exemption been authorized or an  Yes environmental assessment (EA) or environmental impact statement (EIS) been performed 4.d. If yes, please explain:  No  No 5. Is the research performance site designated, or eligible to be designated, as a historic place  Yes 5.a. If yes, please explain:  No 6. Does this project involve activities outside the United States or partnership with international  Yes collaborators 6.a. If yes, identify countries: 6.b. Optional Explanation:  No Filename 7. Project SummaryAbstract Summary_Harty_Final.pdf 8. Project Narrative Narrative_Harty_Final.pdf 9. Bibliography  References Cited References -Harty_Final_2.pdf 10.Facilities  Other Resources Facilities_FCCDC.9_combined.pdf 11.Equipment Equipment_FCCDC.2_combined.pdf Page 6 Tracking Number: GRANT13199760 Funding Opportunity Number: PA-20-265. Received Date: 2020-09-08T12:43:45.000- 04:00 Contact PDPI: Harty, Ronald N. Summary: The ultimate goal of this Phase II application is to develop novel small molecule, broad-spectrum therapeutics against viral infections caused by filoviruses, arenaviruses, and other viruses that depend on the PPxY L -domain motif for egress and spread of infection. Some of these viruses, including Ebola (EBOV), Marburg (MARV), and Lassa fever (LAFV) viruses, are highly pathogenic and classified as Category A bioterror pathogens. We and others hav e determined that efficient budding of these emerging human pathogens depends on the subversion of host proteins, such as neural precursor cell expressed developmentally down-regulated protein 4 (Nedd4), by PPxY L-domains in the matrix proteins of these RNA viruses. The identification and development of small molecule inhibitors that interfere with these virus-host interactions should effectively block virus egress, disease progression, and transmission. In these efforts we have discovered several chemical series of small molecule inhibitors of the host Nedd4virus PPxY complex important for viral egress which led to one analog possessing proof of concept in vivo activity in a Marburg virus challenged mouse model. As FDA approved therapeutic agents for the treatment of these most of these viral infections are not available, our identification of virus -host inhibitors that may prevent virus spread will fill a significant unmet need. Moreover, these inhibitors will be broad-spectrum, and therefore will likely be effective against newly emerging viruses as well as viral variants. As described below, we will use a rigorous multifaceted approach to identify, develop, and validate PPxY budding inhibitors identified in Phase I as potent, broad-spectrum antivirals. The goal of this Phase II STTR grant application is to optimize our lead inhibitors of VP40 PPxY -Nedd4 interactions to generate full - fledged predevelopment drug candidates ready for IND directed studies. This will be accomplishe d by combining the pharmaceutical and medicinal chemistry expertise of the scientists at the Fox Chase Chemical Diversity Center, Inc. (FCCDC) with the expertise and experience in the experimental aspects of antiviral therapy of the Harty Lab at the University of Pennsylvania. We will realize this goal by optimizing our existing series of inhibitors, exemplified by in vivo active FC -10696, for improved potency and oral drug properties (Aim 1), evaluating new compounds based on two potent series for their ability to specifically inhibit PPxY -Nedd4 interactions and subsequent VLP and surrogate virus egress (Aim 2), identifying compounds having suitable drug properties and selectivity using in vitro and in vivo ADMET evaluation (Aim 3), and evaluating compounds for their antiviral efficacy against authentic BSL-4 viruses in vitro and in vivo (A im 4). Project SummaryAbstract Page 7 Contact PDPI: Harty, Ronald N. Narrative : There is an urgent need to develop antiviral therapy against emerging human RNA viruses that represent potential agents of bioterrorism (Ebola, Marburg, and Lassa fever). We have discovered small molecule compounds that disrupt budding and spread of live viruses; a process that is critical for virus dissemination and disease progression. Here, the Harty group at the University of Pennsylvania, experts in the antiviral technology of this proposal, have partnered with the small business Fox Chase Chemical Diversity Center, Inc. to further develop these broad spectrum antiviral budding inhibitors by using medicinal chemistry, live virus budding assays, and live cell imaging techniques. Project Narrative Page 8 Contact PDPI: Harty, Ronald N. Resources  Laboratories of Fox Chase Chemical Diversity Center, Inc. Fox Chase Chemical Diversity Center, Inc. (FCCDC) is located in a 112,000 ft2 facility known as the Pennsylvania Biotechnology Center (PA Biotech Center) at Doylestown, PA. The Center is a recently renovated, $14 million, non-profit business incubator, funded in part by a grant from the Commonwealth of Pennsylvania, whose mission is to nurture entrepreneurship and biotechnology in the region. It is home to 30 biotechnology companies, and the non-profit Blumberg Institute. The FCCDC has 1,600 ft2 of laboratory space within the facility including six chemistry fume hoods and one BSL2 incubator, and 2,400 ft2 of office space. FCCDC also conducts laboratory operations at a second nearby facility at 3700 Horizon Dr., King of Prussia, PA, the former Bachem site, where it leases 6,600 ft2 of combined lab and office space including 11 chemistry fume hoods. Offices. The main FCCDC offices are located within the PA Biotech Center. They are on the same floor as the laboratories and with dedicated administrative assistance available within a reception area. The offices are equipped with a computer, conferencing-capable phone system to facilitate collaborative discussions, and locking file cabinets for storage of confidential study information. The office suite includes two conferences rooms, one of which is set-up for computer based presentations. FCCDC staff are fully supported by an associated administrative infrastructure (copy machines, fax, reception, video conferencing, etc.) A computer network connected by T1 lines for scientific writingediting, data analysis, and graphicsillustration is available at the PA Biotech Center. FCCDC laboratories have the following equipment that will be specifically used for supporting the work described in this grant application: two 300-MHz 1H NMRs with multinuclear probe capability (13C, 31P, 18F), four LCMSs including three Micromass ZQ mass spectrometers with Waters 2695 HPLC with diode array detectors, multiple ISCO and Biotage personal chromatography purification equipment, multiple Gilson 215 semi - preparative HPLC systems, Biotage Initiator microwave, two Parr hydrogenators, 18 rotary evaporators, Genevac EZ2 Evaporation System, Mettler microbalances, JChem temperature controllers, Innova platform shaker, miscellaneous smaller glassware, PCs, printers, and related equipment, BSL2 incubators for cell culture, laminar flow biosafety cabinets, multiple pipettors, common refridgerators and freez ers. Chemical compound information is stored within an MS Excel workbook and is accessible via Instant JChem which is also used to calculate biophysical properties such as LogPD and tPSA ( www.chemaxon.com ). FCCDC has a companylicense for the structure and literature searching tool SciFinder-n, and free on-site access via the Pennsylvania Drug Discovery Institute to 30,000 reagents and starting materials (including 800 boronic acids) that are binned in functionality appropriate trays and can be searched electronically. Records Retention. All FCCDC final compound samples are stored in a locked repository. All laboratory notebooks are signed daily and witnessed on a monthly basis. We observe standard industry practices in the area of confidentiality. Laboratory notebooks are kept under lock and key in the evenings and weekends. NMR data is stored on a primary server which is backed up to a second server. We have a license to ACD software, allowing us to print out coupling constants, multiplicities and chemical shifts of NMR spectra quickly, w hich is very useful when writing patent applications or manuscripts for publication. Collaborative Scientific Environment. Fox Chase Chemical Diversity Center, Inc. holds regular and periodicgeneral research review and specific project review meetings. Project meetings would be held with the relevant research staff at our academic partner either in person, by go-to-meeting web review, or by phone. The PA Biotech Center and the non-profit Pennsylvania Drug Discovery Institute hosts scientific seminars on approximately a monthly basis, including eminent researchers in the area of drug discovery. Speakers have included Bill Greenlee, Magid Abou-Gharbia, Donna Huryn, and Dan Skronovsky. Facilities  Other Resources Page 9 Contact PDPI: Harty, Ronald N. Facilities and Resources Ha rty, Ronald N. Laboratory : 1,008 sq. ft. of BSL-2 laboratory space including a tissue culture area. The laboratory can accommodate up to 7 people and is equipped with one 6-ft and one 4-ft biohazard hoods for cell culture and virus experiments, three tissue culture incubators, two ultra low freezers, two 37 C floor shak erincubators, two Beckman L90 ultracentrifuges, two Beckman Avanti J -E centrifuge, and several refrigeratorsfreezers. An additional shared equipment suite and darkroom are available on the floor. Animal : NA Clinical: NA Computer: I have a Mac Desktop, a MacBook Pro laptop, and an HP Laserjet M1212 laser printerscannerfax in my office. The laboratory is equipped with two Dell Optiplex GX260 systems and a Mac Desktop with all necessary software along with two laser printerscannerfax units. All computers have internet and email access. Office: I have a 92 sq. ft. office within my research lab. Other: Secretarial services are provided by the Department of Pathobiology. Building facilities, information technology (IT), and glassware services are located in the building, along with a meetingconference room. Scientific Environment: The laboratory is within the School of Veterinary Medicine at the University of Pennsylvania. There are a plethora of active virology, immunology, molecular biology, biochemistry and parasitology laboratories within the School of Veterinary Medicine and University wide with an abundance of both pre- and postdoctoral trainees. Biohazards : The work in my laboratory involves studies approved for BSL-2 conditions. Penn Vet Imaging Facility : The Veterinary School Core Imaging Facility houses a two-photonconfocal (4 laser) spectral imaging Leica SP5 system on a DM6000 microscope with tandem scanner, five internal spectral detectors, and four external non-descanned detectors and motorized XY stage. This system is optimized for multicolor intravital imaging. The core also has a Yokagawa CSU X1 spinning disk confocal head on a Leica DMI4000 microscope and an LMM5 dual laser excitation source, a Hamamatsu 512 EM camera, and an ASI motorized and computer controlled XYZ stage. This instrument is housed within an environmental (temperature and CO2 controlled) chamber and is ideal for long term imaging of live cells. We just acquired a new Leica SP5 spectral imaging microscope on an inverted stand (DMI6000) that is equipped with 3 continuous wave lasers, 3 pulsed lasers and a LeicaPicoquant Fluorescence Lifetime Imaging (FLIM) system. This instrument is optimized for live cell imaging of cells and specifically for real time measurements of protein-protein interactions. This instrument is the only one of its kind at the University of Pennsylvania and is ideally configured for the FLIM measurement of virus -host interactions. The core also has several software packages for image analysis including Metamorph from Molecular Devices, Volocity from Improvision, and the LAS -AF suite from Leica. Facilities  Other Resources Page 10 Contact PDPI: Harty, Ronald N. FACILITIES AND OTHER RESOURCES Texas Biomedical Research Institute EN VIRONMENT: Texas Biomedical Research Institute (Texas Biomed) was founded as a private, not-for - profit research institution in 1941 by Thomas B. Slick Jr., with a goal to unravel the mysteries of disease through innovative thinking, creative problem solving, and cutting edge technologies. Located on a 200-acre campus on the northwest side of San Antonio, Texas, our scientists work each and every day to improve human health by being a world leader in infectious disease research. Texas Biomed partners with hundreds of researchers and institutions, targeting advances in the fight against infectious diseases impacting our communities around the world, including AIDS, TB, hepatitis, malaria, hemorrhagic fever viruses, parasitic infections and many other infectious diseases. We also study co-morbidities of infection and vulnerable populations, such as the very young and the elderly and those suffering from cardiovascular diseases and metabolic disorders. Texas Biomed is staffed by more than 350 employees including a multidisciplinary team of approximately 65 doctoral -level scientists  physiologists, geneticists, virologists, immunologists, molecular biologists, cell biologists, physicians and veterinarians  who lead more than 200 major research projects. Texas Biomed also supports 150 technical staff and an administrative support staff of more than 100 professionals. Texas Biomed recently underwent a strategic planning process and defined its focus as an institute that studies infectious diseases and conditions that make an individual susceptible to infectious diseases (aging, metabolic syndrome, etc.). The Institute developed three new programmatic areas: Host-Pathogen Interactions is the study of the basic biology of infection in humans and animals  the immediate i ntersection of host and pathogen. Disease Intervention  Prevention is the development of diagnostics, treatments and vaccines to reduce the severity ofor cure infection. Population Health is the identification of correlates of disease susceptibility or resistance to infectious diseases on a population level. Each program has two faculty leads who focus on the academic development of the programs and their themes. Administrative responsibilities are supported through the Office of the Vice President for Research, unencumbering scientists to pursue the academic mission of Texas Biomed. Texas Biomed supports a culture of learning and development. The Institute holds a weekly seminar that hosts an external guest approximately monthly, and internal presentations from faculty and trainees at all other times. Attendance at seminar ranges between 50 and 80 faculty and research staff. Administrative staff are also frequently in attendance to learn about the latest research at Texas Biomed. Texas Biomed supports a monthly journal club that rotates across the three programmatic areas that typically has an attendance of 20-40 researchers. Individual laboratories also hold their own journal clubs, frequently shared across multiple scientific groups. The Vice President for Research runs a monthly training course for Staff Scientists on how to manage a laboratory, and a monthly training course for a subset of Staff Scientists on how to write a grant. Staff Scientists at Texas Biomed are the equivalent of Research Faculty. Human Resources provide training through a Lunch  Learn lecture series and individualized training programs on management and leadership. Communication at Texas Biomed is facilitated by a monthly institute-wide newsletter and video blogs. Texas Biomed is part of a city wide, 4-institute collaborative (with University of Texas Health Science Center, University of Texas San Antonio, and Southwest Research Institute) that works synergistically to increase the biomedical reputation of the city of San Antonio. Initiatives include the San Antonio Vaccine Development Center and the Precision Therapeutics Program, which are supported by and foster collaboration across the four institutions. Administrative collaboration includes negotiated rates for use of each institutions cores, participation in graduate and training programs, and input on faculty search committees including consideration for dual hire appointments. RESEARCH FACILITI ES: Our faculty and staff scientists conduct their research amid 550,000 square feet (sq. ft.) of space that includes laboratories and core facilities, offices, auditorium, animal hospital, library, and animal facilities, including a six-acre corral. Research laboratories are spaced across three main buildings for a total of 55,361 sq. ft. of space. Facilities  Other Resources Page 11 Contact PDPI: Harty, Ronald N. Moorman (building 35; 31,954 sq. ft.; where Shtanko lab is located): The Moorman building has 12 laboratories totaling 9,459 sq. ft. for in vitro BSL2 studies. Each lab has a dedicated technician office. Faculty offices (approx. 300 sq. ft.) are peripheral to the laboratories and directly adjacent to offices and cubicles for staff and students. The Moorman can support between 6-12 faculty, depending on space requirements. The Moorman also supports approximately 1,200 square feet of select agent BSL3 laboratory, and a 2,100 sq. ft. of ABSL4 suite laboratory (1,200 operational, remaining space is mechanical and annex space). Dr. Shtanko has 800 sq. ft. of lab space which is outfitted with common equipment necessary for studies in cell biology, molecular biology, and immunology. She also has private office space located in the same building within 80 feet of the lab. Earl Slick (building 71; 59,556 square feet): The Earl Slick building has 14 laboratories totaling 12,298 sq. ft. for in vitro BSL2 studies, 1,099 square feet for BSL2 studies (for HIV), 1,840 sq. ft. of BSL3 prep space, and 10,500 sq. ft. of office space. Faculty offices (approx. 250 sq. ft.) and offices for support staff and trainees are in close proximity to associated laboratory space. Earl Slick can support between 7-14 faculty, depending on space requirements. Laboratories for individual faculty are outfitted with common equipment necessary for studies in cell biology, molecular biology, and immunology. There is a dedicated blood draw room immediately next to the BSL2 lab space, to ensure privacy during phlebotomy procedures. This room is equipped with a refrigerator for juice boxes for donors and a phlebotomy chair with upholstered arms and foot rest, and can recline to the Trendelenburg position. Only trained individuals will serve as phlebotomists. Urschel-Slick (building 12; 57,875 sq. ft.): The Urschel -Slick building has 16 laboratories totaling 11,995 sq. ft. for BSL2 studies, 7,500 sq. ft. for BSL3 studies, including 1,840 sq. ft. of BSL3 prep space, and 10,500 sq. ft. of office space. Faculty offices (approx. 250 sq. ft.) and offices for support staff and trainees are in close proximity to associated laboratory space. Urschel -Slick can support between 8-16 faculty, depending on space requirements. Urschel -Slick is the location of a recently renovated 7,500 sq. ft. ABSL3 and rodent vivarium. Ledford (building 38; located 4,876 sq. ft.): The Ledford building is a free-standing building with 2,579 sq. ft. of laboratory space and 1,305 sq. ft. of adjacent office space. The Ledford building houses the institute Central Core Facility and is no more than 3 minutes walking distance from any research space at Texas Biomed. The building is keycard access restricted and monitored. Core staff are located within the building to provide support for any instrument. The Ledford building has a darkroom for individual light sensitive equipment. UT Health San Antonio (10 min drive) has the following cores available to the PI: Mass Spectrometry, Bioanalytics  Single cell, Biobanking  Genome Analysis, Biomolecular NMR, Flow Cytometry, Macromolecular interactions, Microt CT, Electron Microscopy, Optical Imaging, and X -ray Crystallography (see http:research.uthscsa.eduRCLcores.shtml ). For detailed UT Health San Antonio Core Resources, see: http:research.uthscsa.eduRCLResourcepages.shtml CLINICAL: Human subject studies are reviewed by UT Health San Antonio Institutional Review Board through a joinder agreement. UT Health is located approximately a 15-minute drive from Texas Biomed and is affiliated with University Hospital, Audie L. Murphy VA Hospital, Robert B. Green Campus, and the County Public Health Hospital. ANIMAL: Texas Biomed has an independent Institutional Animal Care and Use Program which is responsible for monitoring vertebrate animal research to be compliant with USDA and AAALAC standards, and adheres to the Animal Welfare Act, Public Health Service Policy on Humane Care and Use of Laboratory Animals, and the Guide for the Care and Use of Laboratory Animals . Animal Welfare Assurance A308201. Texas Biomed has been accredited by AAALAC continuously since 1973 (AAALAC File 000246). A dedicated rodent vivarium is located in the Slick -Urschel building with current capacity to house rodents on ventilated racks or in static caging. Space is available for expansion and to accommodate additional species. The vivarium is supported by a dedicated supervisor and staff, and veterinary care. The vivarium staff also support rodent studies at ABSL3. Facilities  Other Resources Page 12 Contact PDPI: Harty, Ronald N. COMPUTER: In addition to personal computers, there are 57 desktop and 31 laptop computers in addition to printers and fax machines available within the Moorman and adjacent buildings, and the Library to be used for instrument operation, data management, word processing, graphics and manuscript preparation. These computers are connected to the central network and are protected by individual UPS devices. There is open access through the Internet to library resources at Texas Biomed. Scientists can use the institutional accounts of the Slack discussion platform and the Dropbox file share service to increase communication and data sharing internally and externally with collaborators. On servers, scientists have access to a wide variety of software and programming and scripting languages such as FORTRAN, C, C, Java, Tcl, Perl, Python, Ruby, bash, R, etc. Statistical and mathematical packages available include GraphPad, Maple, S-PLUS, and R. Software available for genetic analysis includes PAP, S.A.G.E., MENDEL, FISHER, LINKAGE, CRI-MAP, PREST, Merlin, GeneHunter, SimWalk2, Loki, PLINK, GATK, vcftools, Ingenuity Pathway Analysis, kinship2, and SOLAR. Specific software or packages can be installed on demand. A dedicated staff of 9 persons led by the Vice President for Information Technology maintains and updates current systems and assists scientists in specific requests (installation of software, libraries and packages, help in debugging failed compilation). A ticket system is in place to efficiently address issues. The technical staff has expertise in database management in both SQL-based software and the PEDSYS database system, developed in-house for management and analysis of genetic, demographic, epidemiological and laboratory data. Data and System Security We have implemented extensive security measures to protect our computers and data files against unauthorized access. Our security measures are compliant with IRB requirements for human data. Our internet connection is a 100 Mbs Ethernet connection, using a leased router managed by our internet provider, and protected by our Palo Alto Networks next generation security appliances, in a redundant configuration. Users connect to the system over the network using either SSHSFTP or VPN connections. Access to network is removed on or before the day that an individuals employment is terminated, unless prior arrangements have been made. Access logs are checked periodically for repeated login failures which might suggest an attempt to break into our system. Countermeasures are taken if evidence of such an attempt is found. Access to central files containing research, clinical, support, or administrative data is restricted to a list of approved users. Separate lists are maintained for each research or administrative unit (laboratory, project, etc.). Disk-based daily incremental and monthly full backups of software and data are performed using the Veritas NetBackup system. Long-term data archiving is provided by a single six drives, 113- slot 800 GB tape library, and four 25-slot 400 GB tape libraries. REDCap ( Research Electronic Data Capture ) is available for sharing and managing data across computer platforms and between research institutes. REDCap is a secure, web -based application for building and managing online surveys and databases. REDCap provides automated export procedures for seamless data downloads to Excel and common statistical packages (SPSS, SAS, Stata, R), as well as a built-in project calendar, a scheduling module, ad hoc reporting tools, and advanced features, such as branching logic, file uploading, and calculated fields. B IOSAFETY: ABSL4 laboratory Tex as Biomed is home to the only operational ABSL4 lab owned by a private institution. The ABSL4 lab supports basic research as well as vaccine and therapeutics testing of agents with no cure. This unique resource has supported the research of faculty in Texas Biomed in emerging diseases and bioterror agents including the hemorrhagic fever viruses Ebola virus and Marburg virus. This full -suit lab is CDC and USDA certified for work on human and animal pathogens. Access to the ABSL4 area is secured via three successive magnetic cardkeypad controlled airlock passages: The first is located at the entry foyer to the BSL3 suite, the second at the entry foyer to the outer changer room, and the third at the entry foyer to the ABSL4 area. The ABSL4 area is equipped with an entry and exit decontamination airlock and an emergency exitdecontamination chamber. The latter also serves as an equipment decontamination Facilities  Other Resources Page 13 Contact PDPI: Harty, Ronald N. chamber to allow for the repair, removal andor replacement of faulty equipment without requiring a complete laboratory decontamination process. All supply and exhaust air for the ABSL4 passes through dual (and tandem) HEPA filters. Critical parameters (air supply and exhaust flow rates, pressure differentials, normal and emergency breathing air, decontamination systems, etc.) are continuously monitored. Personnel within the lab communicate, and are actively monitored by outside personnel, through a scrambled UHF communication system worn inside the laboratory suit. The 500 sq. ft. ABSL4 laboratory staging area includes an air locked suit change room, two inner change rooms with pass -through showers, and an outer clean change room equipped with sinks, lockers, toilets, etc. Critical mechanical support devices are located in an attached 3,000 sq. ft. mechanical support complex. All critical equipment and devices (valves, etc.) are redundant, including duplicate air-supply and exhaust systems, decontamination shower systems, as well as the compressors, pumps, valves, heater cores, etc. that comprise the liquid waste decontamination system. All critical mechanical devices operate on a system with automatic emergency electricity backup generators. The ABSL4 area (including mechanical support rooms) is equipped with an elaborate intrusion detection system that includes electronic door and roof hatch sensors, motion detectors, etc. The laboratory contains 3 x class IIB biological safety hoods, 4 x water -jacketed CO2 incubators, low -, high- and ultra-speed centrifuges equipped with both analytical and preparative scale rotors, a microfuge, 2 x 4C refrigerators, 2 x liquid nitrogen storage systems, and 3 x -80C freezers. The ABSL4 also has an optical ELISA plate reader, MAGPIX multiplexing 96-well format unit, luminometer, Innova 4000 environmental shaker, microplate shaker, BioFlo III fermenters with chillers, Evos fluorescent microscope, pass -through autoclave, dunk tank, facsimile machine and networked computers. Texas Biomed maintains an experienced and trained staff of scientists, veterinarians, research technicians and veterinary technicians available to perform studies at high biocontainment. These individuals have demonstrated proficiency at conducting in vitro and animal studies with the agent identified in the proposal. The ABSL4 Operations and Safety Manuals specify policies, procedures, and standard operating procedures (SOPs) for the safe handling of biological materials in biosafety laboratories. The policies, procedures, and SOPs comply with applicable Federal, State, and municipal regulations and with the guidelines Biosafety in Microbiological and Biomedical Laboratories issued by the CDC and the National Institute of Health (NIH). Employees are trained from these manuals on each facilities mechanical systems, biosafety, biocontainment and security. Employees are also trained according to project specific and SOPs. These procedures apply to all that use, generate, store, or dispose of potentially infectious materials in Texas Biomed biosafety laboratories and to persons who must enter these laboratories to perform services. Select agent use, transfer or possession is forbidden without the permission of the Responsible Official (RO) and until the required forms filed and written approval received from the CDC Select Agent Program. Upon approval, ABSL4 investigators desiring to work on a BSL4 project must also submit an application to the Biohazard and Safety Committee for approval. The Committee is responsible for evaluating the facility, equipment, and staff capabilities to perform work in a safe manner. Infectious cultures and inventory stocks are stored inside the ABSL4 laboratory in locked refrigerators and freezers. An electronic Inventory (eInventory) system is used to document usage of infectious stocks. All infectious material stored in refrigerators or freezers is properly labeled and stored in containers capable of withstanding thermal shock of freezing and thawing. Each container is labeled with the identity of the infectious agent, the date of the preparation, and a barcode that links the material to the more inclusive information contained in the inventory database. When work is completed, all infectious cultures are removed from workbenches and cabinets and stored in a designated refrigerator or freezer. Materials to be discarded are placed in a sealable container filled with a suitable disinfectant. The container is placed in a discard pan containing the disinfectant. Discard pans are placed on a cart and transported to the autoclave. Labware containing infectious liquids are stored and transported in leak -proof containers large enough to contain the fluid in case of leakage. Facilities  Other Resources Page 14 Contact PDPI: Harty, Ronald N. Biocontainment Program Tex as Biomed has established an institution-wide comprehensive BSL3BSL4 and CDC certified Select Agent BSL3 facilities program. The development of a centralized program for the campus ensures that 1) the growth and development of BSL4BSL3BSL2 facilities will remain well -coordinated both scientifically and administratively at all levels of the program; 2) spaces will be developed in the most effective manner based on an accurate assessment of need and cost; 3) technical support will be coordinated among the facilities; 4) training will be optimized, including sending key personnel to important national meetings for training and education on an ongoing basis, a critical need considering the rapidly changing landscape regarding security requirements and control s; and 5) assurance that broad expertise on campus is represented in BSL432 discussions regarding a) work with bacteria, viruses, toxins, etc; b) entry criteria for users, including the CDCFBI program; c) exposure protocols for users; d) proper use and maintenance of animals; e) user issues and concerns; and f) infrastructure issues and concerns. Institutional BSL34 compliance oversight is provided by Environmental Health and Safety (EHS). EHS provides the following services for the entire institute operation: radiation safety, environmental compliance, occupational health and safety, research safety and biosafety. Texas Biomed has a program director for the BSL3BSL4Biosecurity program, a Responsible Official and alternate for the Select Agent Program and a Biohazardous and Safety Committee (BSC). Complementing and interacting with the regulatory and compliance arm of the BSL3BSL4Biosecurity activities on campus is the centralized research oversight structure. The Biocontainment Program is managed by a BSL3 Director, BSL4 Director, and two biocontainment program coordinators. These staff work closely with the Director of Maximum Containment Contract research. Program operations include an institute-wide Biosafety -34 Advisory Committee (BAC) that consists of the EHS Director, BSL3 Director, BSL4 DirectorIBC Chair, and coordinators. The VP for ResearchRO attends the meeting monthly. The charge of the BAC is to develop and review institution BSL34 rules and regulations, develop and refine BSL34 user entry requirements for certification, evaluate the use and efficiency of campus BSL34 fac ilities, develop and refine the BSL34 medical surveillance program, handle user issues as they arise, and provide oversight for the operation of the facilities. A complete Texas Biomed BSL3Select agentBSL4 education and entry program for users is in place. Texas Biomed users meet as a group (BUG) with leadership on a monthly basis. Access to BSL34 facilities is granted only when personnel receive thorough biosafety and biosecurity training and proper on-site training. Training materials are reviewed by the Responsible Officer (RO) andor Alternative Responsible Officer (ARO). Biosafety and biosecurity training emphasizes the facility design and system in place as well as the current rules and regulations users must follow. Refresher training is provided on an annual basis and time sensitive issues are discussed in the monthly BSL34 user group (BUG) meeting. On-site training is focused on demonstrating proper application of personal protective equipment (PPE), facility features, and proper usage of equipment. Junior scientists must be accompanied and supervised by senior scientists who are experienced in BSL34-related techniques to ensure proper handling of infectious materials. The BSL34 program at Texas Biomed ensures that all BSL34 users are fully trained and supported by operational staff, relieving some of the burden of training and oversight from the PI. Select Agent Research Texas Biomed is a select agent registered entity with Health and Human Services, Centers for Disease Control and Prevention and U.S. Department of Agriculture, Animal Plant Health Inspection Service, National Select Agent Program. The Institute has been inspected by the CDC National Select Agent Program for use of HHS Select Agents and Toxins, Overlap Select Agents and Toxins and USDA Select Agents and Toxins. The registration number is C20171120 1976. Facilities  Other Resources Page 15 Contact PDPI: Harty, Ronald N. OTHER INSTITUTIONAL RESOURCES: Biology Core instrumentation and services The Biology Core, housed in the Central Core facility, is operated by a Director and two PhD level Managers. Microscopy: The Biology Core operates a Zeiss 3-Channel Confocal System Microscope LLC LSM800 and a Nikon C2 Confocal Module microscope. Microscopes are each located within 2 separate buildings (Earl Slick and Moorman) to be in close proximity to the most frequent users. Microscopy in Moorman building is performed using a Nikon Ti Eclipse microscope housed in a separate, dedicated microscopy room. The Nikon Ti Eclipse microscope is necessary for high resolution and high-speed image acquisition of cells and fluorescently labeled virus- like particles. It is fitted for both high speed widefield image capture as well as confocal capabilities using the Nikon C2 scan head with high sensitivity photomultipliers. The microscope has automated image acquisition ability with electronic XYZ piezo stage and Nikon elements software suitable for high content screening of drug libraries, siRNA and expression plasmids. Objective lenses are of the highest quality including oil immersion 40x NA 1.30 lens, 100x oil immersion lambda NA 1.45 lens as well as conventional air 4x, 10x, 40x, and 60x lenses. The scope is fitted with a heated, CO 2 fed, incubator stage for live cell work. The camera for widefield imaging is an ANDOR iXon3 EMCCD camera, which provides rapid, highly sensitive acquisition of 14-bit and 16- bit images with high linear sensitivity. Software is a key part of quantitative image analysis. AutoQuant X3 software provides image deconvolution and interfaces with Imaris (Bitplane) to identify and measure image features, including measuring the distance between features in 3D. CellProfiler software is used to count and measure object intensity in multiple images and interfaces with FCSExpress to perform image cytometry, which allows gating of cell populations like FACS. The latter allows analysis of cell features after gating which is an advantage. ImageJ processing software is also available. BD FACSymphony: Available since September of 2019, the 5-laser BD FACSymphony cytometer is capable of 28 color acquisition, and it is equipped with a High Throughput Sampler. This advanced instrument features an ultra-quiet VPX electronics system that supports up to 50 high-performance photomultiplier tubes (PMTs) and improves detection sensitivity to enable the identification and analysis of rare cell types and events. The version at the Core is equipped with a 355 nm (UV), 405 nm (violet), 488 nm (blue), 561 nm (yellow -green) and 640 nm (red) lasers. When analyzing panels designed with BD Horizon Brilliant dyes, this platform helps to overcome research challenges, such as collecting maximal information from a precious sample, and increases lab throughput with broad phenotyping panels that combine multiple cell -line-specific panels. This highly customizable platform can be configured so that you can select from multiple laser wavelengths and power ratings, and choose the positions of decagon detection arrays to address the requirements of your specific research application. Beckman Coulter Cyan ADP: The CyAn LX Flow Cytometer provides three excitation lines, eleven parameters, complete compensation, and high-speed acquisition, in a bench-top configuration. The 405 nm, 488 nm, and 633 nm lasers are fixed and alignment-free, with spatially separated beams for minimal optical crosstalk. Spatially separated optical apertures, high-performance solid -state detectors, and an obscuration bar minimize unwanted laser radiation. User -interchangeable filters provide experimental flexibility. Eleven acquisition channels (including forward scatter, side scatter, and 9 colors) provide parallel processing of pulse height, pulse length, area, and log for each parameter. BD Accuri C6 Plus: The 2-laser BD Accuri C6 plus cytometer is capable of 4-color acquisition. The BD Accuri is dedicated to BSL2 HIV studies. The BD Accuri C6 Plus is equipped with a blue and red laser, two light scatter detectors, and four fluorescence detectors with optical filters optimized for the detection of many popular fluorochromes. BD FACSAria cell sorter: The 4-laser FACSAria II sorter is capable of 18-color, 4- way sorting and has an automated cell deposition unit for 6, 12, 24, 48 and 96 well sorting. The BD FACSAria cell sorter is a fixed- alignment cuvette flow cell that can detect up to 18 colors, for a total of 20 parameters. Cells passing through the cuvette flow cell are excited by four solid state lasers at 407-nm, 488-nm, 561-nm, and 633-nm. Advanced digital electronics improve sort performance and provide significant advantages in instrument ease-of-use. This cell sorter is contained within a Baxter High Containment Biosafety cabinet within one of the BSL3 laboratories. Facilities  Other Resources Page 16 Contact PDPI: Harty, Ronald N. BDInflux cell sorter: The 5-laser BDInflux cell sorter is capable of 6-way sorting. The BDInflux is primarily dedicated to single cell sorting of malaria parasites and sorting of pluripotent stem cells. Luminex 100 and 200: The Luminex with xMAP (multi -analyte platform) technology systems measure analytes (cytokines, chemokines, etc.) on a multiplex platform. These instruments are operated with dedicated computers running Luminex Xponent software; analysis of reactivity is done with Masterplex QT software. Molecular Services Core instrumentation and services The Molecular Services Core (MSC) is located in the Central Core Facility and is operated by a Director, a full-time manager, and technical support staff. The MSC is dedicated to assist researchers in the development and implementation of state-of-the-art molecular approaches in the support of their research. The MSC has an active Users Group that periodically reviews available services, technology, and operations. The MSC is equipped with instrumentation to conduct most genomic and gene expression applications, and MSC personnel are trained in numerous technical and bioinformatic approaches. Equipment: The MSC provides access to an Illumina iSeq 100 and Illumina MiSeq sequencer for low - to mid- output NGS applications. Fully automated sequencing sample preparations of 8-96 samples are supported with a Sciclone G3 NGSx automated liquid handling system. A 10X Genomics Chromium controller and Fluidigm C1 are available for single cell NGS applications. Additional instrumentation includes a Sage PippinHT for automated size selection and PCR purification, an Agilent 2100 Bioanalyzer and 4200 TapeStation for determining RNA, DNA, and protein integrity, and a Nanodrop spectrophotometer and Invitrogen Qubit Fluorometer for DNA, RNA and protein sample quantitation. Multiple additional thermal cyclers are available to support these instruments. The MSC also provides access to an Illumina iScan with autoloader to allow 24 h service without user intervent ion for high-throughput genotyping and gene expression analyses (allowing the processing of up to 96 multi - sample BeadChips per day). The iScan System supports rapid, sensitive, and accurate imaging of Illuminas array-based genetic analysis products. A Tecan Freedom Evo liquid handling platforms is also available for automation of all Illumina protocols. Multiple PCR machines (996- and 384-well blocks), two AB QuanStudio 5 and one QuantStudio 6 Real Time PCR Systems and an AB QuantStudio 3D Digital PCR System are available to support real -time PCR, TaqMan genotyping, gene expression, absolute quantification, and rare allele detection assays. The Core Facility also has a TECAN GENios multimode plate reader with fluorescence, absorbance and glo w luminescence modes and a high-performance Tecan plate washer for both 96- and 384-well plates. Additionally, a BioRad Chemidoc gel documentation system with UV and chemiluminescence capabilities is available in an electrophoresis clean room. Two Beckman ultracentrifuges are available as well. A dedicated computer server for sequence analysis is available and consists of a Dell PowerEdge R830 server with two Xenon E5-4640 2.1GHz processors (48 cores), 32TB of useable space, and 256 GB RAM. Partek Flow Genomic Analysis Software is supports start-to-finish analysis for next generation sequencing data applications. Services: Services provided by the MSC facility include RNA and DNA isolation, quality determination, and quantitation from a variety of sample sources including blood, tissues, cells, and body fluids, sequencing library preparations supporting whole genome, transcriptome, exome, smRNA, ChIP, and targeted sequencing applications, onsite low and mid-output next generation sequencing services, and standard sequencing data analysis and bioinformatics for DNA, RNA, and smRNA sequencing. Pathology The pathology facility has the following rooms in Building 10: necropsy (288 sq. ft.), hematology (162 sq. ft.), chemistry (160 sq. ft.), tissue processing (162 sq. ft.), slide processing and administration (224 sq. ft.), refrigeratorfreezer room (148 sq. ft.), storage room (132 sq. ft.), multi -use area (bacterial plating, automated immunohistochemistry, and additional clinical pathology microscopy station - 288 sq. ft.), four offices for the pathologists and technicians, and a room that houses the stereology microscope and visiting foreign and Facilities  Other Resources Page 17 Contact PDPI: Harty, Ronald N. American veterinarians and students. It also has a 332 sq. ft. climate controlled slide, block and wet tissue storage room in the quarantine building and a 216 sq. ft. walk in refrigerator located outside and adjacent to the necropsy room. The necropsy facility is organized to safely handle any necropsy (BSL2) or tissue share project. It has a downdraft stainless steel table, a mobile MOPEC HEPA filtered bone saw, and protective clothing for handling hazardous necropsies. Tissue samples can be collected in formalin, paraformaldehyde, or other fixatives as required by a given research project. Tissues can also be collected in various media, OCT embedded for frozen sectioning, snap frozen, or collected fresh. Perfusion fixation of tissues is also available. A Nikon D200 digital camera is used in the necropsy suite to record lesions. Digital gross and microscopic images are stored on the pathology server and are searchable by either pathology accession number or animal ID. The pathologists also provide on-site necropsy support for nonhuman primate studies in the BSL3 and BSL4 containment areas. The histology laboratory provides tissue processing, embedding, paraffin and frozen tissue sectioning, routine and special staining, and immunohistochemistry procedures. The laboratory is equipped with a Tissue-Tek processor and embedding center as well as automatic stainers for routine HE and immunohistochemistry staining. The lab is also equipped with an automated coverslipper, centrifuge, cytospin, cryostat, oven, microscope, dissecting microscope, two microtomes with automatic features, refrigerators, scales, a safe for GLP and animal rule specimens, and other anci llary equipment. A Nikon DXM1200C camera on a Nikon 801 microscope and an Olympus SC100 camera on an Olympus BX50 microscope are used to obtain images from stained slides. The histology laboratory has developed immunohistochemistry procedures for several antigenspathogens, including filovirus, herpesvirus papio, baboon reovirus, alphavirus, flavivirus, amyloid beta- 42, herpes simplex virus, as well as antibody purification. Available antibodies include cluster of differentiation markers (CD) CD68 (KP1), CD68 (3F103), CD20, CD3, S-100, Ki -67, NeuN, Nurr1 (2 antibodies), HSV1, progesterone receptor, estrogen receptor, Von Willebrand Factor, and TUNEL assay for apoptosis. Other IHC procedures are developed as needed. Facilities  Other Resources Page 18 Contact PDPI: Harty, Ronald N. EQUIPMENT (all onsite)  Fox Chase Chemical Diversity Center Inc NMR: Two Varian Mercury Plus 300-MHz Inova NMRs with multinuclear capability (1H, 13C, 31P, 18F). HPLCMS:  Four Micromass ZQ Mass Specs. with Waters 2695 HPLC with 996 diode array detector. Chromatography:  Three Gilson 215 semi -prep HPLC systems, multi -wavelength, automated fraction collection.  Teledyne Isco CombiFlashRf automated chromatography system.  Three Isco Combiflash Sg 100c personal chromatography systems. Evaporation:  Genevac EZ2 Ev aporation System.  FTS Systems Flexi -dry Lyophilyzer.  18 Rotavapors with vacuum pumps (Buchi, Heidolph).  2 VWR Sheldon 1400E Vacuum Ovens with Edwards oil pumps. Hydrogenation: Two Parr 3911 Shaker hydrogenators. Reaction:  Biotage Initiator microwave synthesizer with 60 position autosampler.  Innova platform shaker, 2000.  4 J-Kem Gemini -2 dual temperature controllers with teflon-coated thermocouples.  24Ika Magnetic StirrerHotplates. BL2 Biological and Tissue Culture Labs:  Baker SG400 and Labconco A2 Class II biological safety cabinets.  ABI Prism 7000 Sequence Detection System.  New Brunswick Environmental incubator shaker and incubator.  Biotek Synergy 2 Multi -Detection Microplate Reader.  Molecular Devices SpectraMax 190 microplate reader.  TitertekMultidrop.  LabomediVU 1500 Microscope.  Nikon Eclipse (TE 2000) microscope interfaced with image analysis software  Fisher and VWR Carbon dioxide incubators. Computational Chemistry:  The PA Biotechnology Center has 600Mbs Internet access, more than sufficient.  Two AMD Threadripper 2950X 16 Core Workstations: Both workstations have 64GB of DDR4 RAM, 1x Nvidia GeForce RTX 2070 GPU, and 1TB Samsung 970 Evo NVMe SSD.  One GPU server with 2x Intel Xeon Gold 5218 16 core32 thread 2.3GHz, 96GB of DDR4 RAM, with 5 GPUs (2x Nvidia Titan RTX, 2x Nvidia Titan V, 1x Nvidia 2080Ti), 1x 512GB Samsung 970 PRO NVMe SSD boot drive, and 2x 2TB Samsung 860 EVO SSDsTwo Dell Precision dual-processor Xeon 5680 workstations with Nvidia 470GTX graphics cards;  Schrodingers Small Molecule Drug Discovery Suite (all modules);  GROMACS (molecular dynamics with customized scripts for automation); Storage, Miscellaneous:  Two (2) Justrite 45 Gallon Flammable Safety Cabinets.  Four (4) RefrigeratorFreezers, Frigidaire, GE, Revco.  SciFinder-n, latest, most powerful search engine for chemistry Reagents and starting materials collection available onsite: 30,000 members including 1,000 boronic acids, bar -coded, searchable electronic database. Equipment Page 19 Contact PDPI: Harty, Ronald N. Major Equipment Harty, Ronald N. Major equipment in my laboratory and the adjacent equipment suite includes: 2 Beckman L90 ultracentrifuges, 2 Beckman J21 centrifuges with appropriate rotors, DNA and protein electrophoresis systems, 4 heated water baths, 3 CO 2 cell culture incubators, 2 cell culture bio-cabinets, 2 bacterial shakerincubators, a fume hood, 3 -85C ultralow and 3 -20C freezers, 2 refrigerators, a Savant speedvac concentrator, 2 PCR units, 2 rotator platforms, 3 microfuges, a gel dryer, 3 liquid nitrogen tanks, an inverted Olympus microscope, phosphoimager and related software, STORM scanner, Biotek Synergy microplate reader, X -ray film developer and darkroom. Equipment Page 20 Contact PDPI: Harty, Ronald N. MAJOR EQUIPMENT Texas Biomedical Research Institute Apart from the equipment located in Texas Biomed Core Facilities, Dr. Shtanko also has access to the following equipment: Shtanko Lab Class IIA 4 feet Biosafety cabinets (1) Tissue culture CO2 incubators (1) 370 Incubators for bacterial work (1) Inverted microscopes (1) Refrigerated microcentrifuge (1) Microcentrifuge (1) Water baths (2) Heat blocks (2) Rockers (2) Neon transfection system (1) Geni syngene gel imaging system (1) Viaflo Assist Integra system (1) Impulse Sealer (1) Dryfast Welch 2014B-01 (1) Isotemp FisherScientific 205 (1) ABSL4 Facility Class IIB 6 feet Biosafety cabinets (3) Tissue culture CO2 incubators (4) Hydro Peroxide vapor decontamination systems (2) Beckman ultracentrifuge Optima (1) Refrigerated Benchtop fuge Eppendorf 5810R (1) Microcentrifuge (1) Multidrop Combi SMART dispenser (1) Milli-Q water purification system (1) Select agent tracking phase I and II (2) Select agent geneport system (1) Water baths (2) Heat blocks (7) Rockers (2) Optical ELISA plate reader (1) Luminometer microplate (1) Innova 4000 environmental shaker (1) Piccolo Evos fluorescent microscope (1) BioFlo III fermenters with chillers (2) Multidrop Combi SMART dispenser (1) Milli-Q water purification system (1) AgarosePAGE gel apparatus and power packs (2) Countess Invitrogen C10281 (1) Vortexes (2) Thermoscientific legend XTR centrifuge (1) -20C freezers (1) -80C freezers (1) 4C refrigerators (1) Liquid nitrogen storage system (1) Biophotometer Eppendorf 6132 (1) Uninterrupted power supply (UPS) unit (2) iBlot 2 gel transfer device (1) Balances (2) Xpress Analyzer (2) Abaxis VetScan VS2 (3) Ventilated rabbit cage unit (2) One Cage 2100 ventilated racks and cages (2) bioBUBBLE containment enclosure (4) TissueLyser II and accessories Qiagen (2) Procyte hematology (1) DSI telemetry system large animal (2) NanoDrop One Microvolume UV -Vis Spectrophotometer (1) -80C freezers (3) CytoSMART LUX system (2) 4C refrigerators (2) Liquid nitrogen storage system (2) MAGPIX 96 -well unit (1) Moorman Research Building (where Dr. Shtanko lab is located) Laminar flow hoods (17) Tissue culture CO2 incubators (18) 370 Incubators for bacterial work (3) Nikon Ti Eclipse confocal microscope system (1) Virus Counter 2100 (1) Cherry Biotech microscope system (1) NanoDrop One spectrophotometer (2) Inverted microscopes (7) Refrigerated microcentrifuges (5) Microcentrifuges (25) Benchtop medium speed centrifuges (14) Beckman ultracentrifuge Optima L100K (2) FACSAria flow cytometer (1) Milli-Q water purification systems (2) Countess Invitrogen C10281 (4) Touch thermal cycler C1000 (1) Equipment Page 21 Contact PDPI: Harty, Ronald N. Cold room (1) Dark room (1) BSL2 labs (12) BSL3 labs (3) ABSL4 labs (1) 4C refrigerators (14) Liquid nitrogen storage units (7) -20C freezers (12) -80C freezers (10) pH meters (4) QStudio 6 Fix-Realtime PCR system (1) KrosFlo Research II TFF system (1) Image processing station for Nikon C2 confocal module microscope (1) Odyssey SA IR imaging system (1) Illumina MiSeq System (1) Eppendorf epMotion M5073 automated DNARNA purification system (1) BenchPro plasmid processing station (1) HM Hematology analyzer (1) MagNa Lyser with rotors (1) Abaxis VetScan VS2 (5) Aero3G Biaera aerosol control platform (1) Aerodynamic particle sizer spectrometer 3321TSI (1) Aerosol Diluter 3302A TSI (1) Primate telemetry repeater (1) VS Blood Chemistry analyzer (1) iMark Absorbance microplate reader (2) Luminometer microplate reader s (4) TECAN Genios multimode plate reader (1) Equipment Page 22 Contact PDPI: Harty, Ronald N. OMB Number: 4040- 0010 Expiration Date: 12312022 RESEARCH  RELATED SeniorKey Person Profile (Expanded) PROFILE -Project DirectorPrincipal Investigator Prefix: First Name: Ronald Middle Name N. Last Name: Harty Suffix: Ph.D PositionTitle: Professor Organization Name: University of Pennsylvania Department: Pathobiology and Microbiology Division: School of Veterinary Medicine Street1: Street2: City: Philadelphia County: State: PA: Pennsylvania Province: Country: USA: UNITED STATES Zip  Postal Code: 19104- 0000 Phone Number: Fax Number: E-Mail: Credential, e.g., agency login: Project Role: PDPI Other Project Role Category: Degree Type: PhD Degree Year: 1991 Attach Biographical Sketch: File Name: Harty_Bio.pdf Attach Current  Pending Support: File Name: Page 23 Funding Opportunity Number: PA -20-265 . Received Date: Tracking Number: GRANT13199760 2020 -09-08T12:43:45.000 -04:00 Contact PDPI: Harty, Ronald N. PROFILE -SeniorKey Person Prefix: First Name: Jay Middle Name Edward Last Name: Wrobel Suffix: Ph.D PositionTitle: Vice President, Academic Relations Organization Name: Fox Chase Chemical Diversity Center, Inc. Department: Division: Street1: Pennsylvania Biotechnology Center Street2: City: Doylestown County: State: PA: Pennsylvania Province: Country: USA: UNITED STATES Zip  Postal Code: 18902- 8400 Phone Number: Fax Number: E-Mail: Credential, e.g., agency login: Project Role: Co-Investigator Other Project Role Category: Degree Type: PhD Degree Year: 1983 Attach Biographical Sketch: File Name: Wrobel_biosketch_for_Harty.4.pdf Attach Current  Pending Support: File Name: PROFILE - SeniorKey Person Prefix: First Name: Olena Middle Name Last Name: Shtanko Suffix: PositionTitle: Staff Scientist I Organization Name: Texas Biomedical Research Institute Department: Virology and Immunology Division: Street1: Street2: City: San Antonio County: State: TX: Texas Province: Country: USA: UNITED STATES Zip  Postal Code: 782270000 Fax Number: Phone Number: E-Mail: Credential, e.g., agency login: Project Role: Co-Investigator Other Project Role Category: Degree Type: PHD Degree Year: 2010 Attach Biographical Sketch: File Name: 2._Bio_Shtanko_20 -240_ -_final.pdf Attach Current  Pending Support: File Name: Page 24 Tracking Number: GRANT13199760 Funding Opportunity Number: PA-20-265 . Received Date: 2020-09-08T12:43:45.000- 04:00 Contact PDPI: Harty, Ronald N. PROFILE -SeniorKey Person Prefix: First Name: John Middle Name Last Name: Kulp PositionTitle: Director of Computational Chemistry Organization Name: Fox Chase Chemical Diversity Center, Inc. Department: Divisi Department: Division: Street1: Pennsylvania Biotechnology Center Street2: City: Doylestown County: State: PA: Pennsylvania Province: Country: USA: UNITED STATES Zip  Postal Code: 18901 -8400 Suffix: Ph.D Phone Number: Fax Number: E-Mail: Credential, e.g., agency login: Project Role: Co-Investigator Other Project Role Category: Degree Type: PhD Degree Year: 2006 Attach Biographical Sketch: File Name: Kulp_biosketch_for_Harty.pdf Attach Current  Pending Support: File Name: PROFILE - SeniorKey Person Prefix: First Name: Allen Middle Name Bernard Last Name: Reitz PositionTitle: CEO Organization Name: Fox Chase Chemical Diversity Center Department: Division: Street1: Street2: City: Doylestown County: State: PA: Pennsylvania Province: Country: USA: UNITED STATES Zip  Postal Code: 189020000 Suffix: Ph.D Phone Number: Fax Number: E-Mail: Credential, e.g., agency login: Project Role: Co-Investigator Other Project Role Category: Degree Type: PHD Degree Year: 1982 Attach Biographical Sketch: File Name: Reitz_Biosketch_for_Harty.2.pdf Attach Current  Pending Support: File Name: Page 25 Funding Opportunity Number: PA -20-265 . Received Date: Tracking Number: GRANT13199760 2020 -09-08T12:43:45.000 -04:00 Contact PDPI: Harty, Ronald N. OMB No. 0925-0001 and 0925-0002 (Rev. 032020 Approved Through 02282023) BIOGRAPHICAL SKETCH Provide the following information for the Seniorkey personnel and other significant contributors. Follow this format for each person. DO NOT EXCEED FIVE PAGES. NAME: Harty, Ronald N. eRA COMMONS USER NAME (credential, e.g., agency login): POSITION TITLE: Professor of Microbiology EDUCATIONTRAINING (Begin with baccalaureate or other initial professional education, such as nursing, include postdoctoral training and residency training if applicable. Adddelete rows as necessary.) INSTITUTION AND LOCATION DEGREE (if applicable) Completion Date MMYYYY FIELD OF STUDY University of Lowell, Lowell, MA Louisiana State University Med. Ctr., Shreveport Louisiana State University Med. Ctr., Shreveport Mount Sinai School of Medicine, New York, NY B.S. Ph.D. Postdoctoral Postdoctoral 061985 081991 011993 081998 Biology Virology Virology Virology A. Personal Statement The overall goal of my research on negative-sense RNA viruses is to understand how these viruses assemble and bud from cells, and how host protein recruitment and virus -host interactions contribute to these late stages of virus replication. My current research program is focused on understanding the molecular mechanisms of filovirus (Ebola and Marburg viruses) and arenavirus (Lassa fever) assemblybudding, so that we can identify broad-spectrum inhibitors of virus-host interactions and validate their efficacy in blocking virus budding. Indeed, through a highly collaborative and productive effort in partnership with Dr. Jay Wrobels team at Fox Chase Chemical Diversity Center, and Dr. Olena Shtanko at the BSL-4 laboratory at Texas Biomedical Research Institute, we have identified lead candidate budding inhibitors with potential broad-spectrum activity. In Phase 2, we will rapidly advance and develop these lead candidate therapeutics for NIAID Category A pathogens toward clinical trials. Overall, my expertise, experience, and demonstrated record of successful and productive research in the areas of RNA virus-host interactions and virus egresstransmission has prepared me to lead the proposed collaborative experiments described in this proposal. 1. Harty, R. N. , Brown, M. E., Wang, G., Huibregtse, J., and Hayes, F. P. 2000. A PPxY- motif within the VP40 protein of Ebola virus interacts physically and functionally with a ubiquitin ligase: Implications for filovirus budding. Proc. Natl. Acad. Sci. U.S.A . 97(25):13871-13876. PMCID: PMC17668. 2. Han, Z., Madara, J.J., Herbert, A., Prugar, L.I., Ruthel, G., Lu, J., Liu, Y., Liu, W., Liu, X., Wrobel, J.E., Reitz, A.B., Dye, J.M., Harty, R.N., and Freedman, B.D. (co-corresponding authors). 2015. Calcium regulation of hemorrhagic fever virus budding: Mechanistic implications for host-oriented therapeutic intervention. PLoS Pathog, 11(10):e1005220. PMCID: PMC4634230. 3. Liang, J., Sagum, C.A., Bedford, M.T., Sidhu, S.S., Sudol, M., Han, Z., and Harty, R.N. 2017. Chaperone- mediated autophagy protein BAG3 negatively regulates Ebola and Marburg VP40- mediated egress. PLoS Pathog, Jan. 11;13(1):e1006132. PMCID: PMC5226679. 4. Han, Z., Dash, S., Sagum, C.A., Ruthel, G., Jaladanki, C.K., Berry, C.T., Schwoerer, M.P., Harty, N.M., Freedman, B.D., Bedford, M.T., Fan, H., Sidhu, S.S., Sudol, M., Shtanko, O., and Harty, R.N. 2020. Modular mimicry and engagement of the Hippo pathway by Marburg virus VP40: Implication for filovirus biology and budding. PLoS Pathog., Jan.6;16(1):e1008231. PMCID: PMC6977764. Biosketches Page 26 Conta ct PDPI: Harty, Ronald N. B. Positions and Honors 1998 -2004 Assistant Professor, Dept. of Pathobiology, School of Veterinary Medicine, University of Pennsylvania 2004 -2015 Associate Professor, Dept. of Pathobiology, School of Veterinary Medicine, University of Pennsylvania 2014 -present Head, Laboratory of Infectious Diseases  Immunology, Dept. of Pathobiology, School of Veterinary Medicine, University of Pennsylvania 2015 -present Professor, Dept. of Pathobiology, School of Veterinary Medicine, University of Pennsylvania Other Experience and Professional Memberships 1987 present Member, American Society for Microbiology and American Society for Virology 2001 Pfizer Award for Excellence in Research, University of Pennsylvania 2002 Deans Award for Leadership in Basic Science Education, University of Pennsy lvania 2003 -2007 Member, Special Emphasis Panel, Biodefense and Emerging Pathogens, NIAID, NIH. 2004 -2005 Chair of the Filovirus Workshop Sessions at the American Society for Virology Meetings at McGill University, Montreal, Canada and at Penn State Univer sity, University Park, PA 2004 -present Member  Editorial Board, Journal of Virology; Ad Hoc Reviewer for Virology, Traffic, Molecular Cell, Vaccine, PNAS, PloS Pathogens, Cell. Micro., and J. of Gen. Virol. 2005 Member - Drug Discovery and Mechanisms of Antimicrobial Resistance Special Emphasis Panel Study Section ZRG1 DDR (01) Q, Infectious Diseases and Microbiology IRG, NIH. 2005 -2010 Member -Special Emphasis Panel, ZRG1 F13 NRSA Fellowships, NIH. 2008 -2009 Chair -Special Emphasis Panel, ZRG1 F13 NRSA Fellowships, NIH. 2014 Ad Hoc Reviewer  NIHNIAID Topics in Virology Study Section, ZRG1 IDM-W (applications having conflicts with standing VIR-A and VIR -B study sections) 2014 Ad Hoc Reviewer  NIHNIAID, Pathogenic Eukaryotes (PTHE) Study Section 2015 Ad Hoc Reviewer  NIHNIAID, Drug Discovery and Mechanisms of Antimicrobial Resistance (DDR) Stud y Section ZRG1 IDM -T (82) 2015 Ad Hoc Reviewer  NIH, Special Emphasis Panel ZRG1 AARR-D (02) M 2016 Chair  NIAID, Special Emphasis Panel to review an Ebola P01 application. 2017 Ad Hoc Reviewer  NIHNIAID VIR -B study section 2018 Ad Hoc Reviewer  NIH R13 (Support for Scientific Conferences) Review Panel 2019 Reviewer  FY19 Congressionally Directed Medical Research Programs (CDMRP), Peer Reviewed Medical Research Program (PRMRP) C. Contributions to Science 1. Viral L-domainhost interactions regulate virus egress and spread . Viruses have developed unique and complex molecular mechanisms to ensure efficient egress of mature virions from infected cells. I focused my early studies on unraveling the intricate roles of both viral and host proteins in this process, and particularly the specific recruitment of host factors to promote efficient budding of infectious virus. A better understanding of these virus- host interactions and the mechanisms of virus budding not only will provide fundamental insights into the functions of both viral and host proteins, but also will lead to the emergence of novel strategies to inhibit virion egress and spread. The Late (L) budding domain motifs (PTAP andor PPxY) are highly conserved in the matrix proteins of a wide array of RNA viruses (e.g. filoviruses, arenaviruses, rhabdoviruses, paramyxoviruses, henipaviruses, and retroviruses) and represent attractive and novel targets for the development of therapeutics having broad -spectrum antiviral activity. My lab was first to demonstrate that the Ebola virus VP40 protein plays a central and sufficient role in virion assembly and egress, due in part to the presence of overlapping PTAP and PPEY L-domains. We went on to show that independent expression of VP40 led to the production and egress of virus -like partic les (VLPs) that accurately mimic budding of live infectious virus. We used VP40 VLPs and genetically engineered VSV recombinants that expressed WT and mutant L-domain motifs from Ebola virus VP40, to show that budding is dependent on viral L-domain mediated recruitment of host proteins associated with the ESCRT pathway (Tsg101 and Nedd4) for complete virus -cell fission or separation. Most recently, we have identified for the first time a third functional L-domain motif (YPx (n)L) within Ebola virus VP40 that serves as an alternative L-domain motif to enhance VP40 VLP release by recruiting and interacting with host ESCRT - associated protein Alix. a. Harty, R. N. , Brown, M. E., Wang, G., Huibregtse, J., and Hayes, F. P. 2000. A PPxY- motif within the Biosketches Page 27 Contact PDPI: Harty, Ronald N. VP40 protein of Ebola virus interacts physically and functionally with a ubiquitin ligase: Implications for filovirus budding. Proc. Natl. Acad. Sci. U.S.A . 97(25):13871-13876. PMCID: PMC17668. b. Licata, J. M., Han, Z., Simpson-Holley, M., Wright, N. T., Paragas, J., and Harty, R. N . 2003. Overlapping motifs (PTAP and PPEY) within the Ebola virus VP40 protein function independently as late budding domains: Involvement of host proteins tsg101 and vps4. J. Virol . 77(3), 1812-1819. PMCID: PMC140960 c. Han, Z., Schwoerer, M.P., Hicks, P., Liang, J., Ruthel, G., Berry, C.T., Freedman, B.D., Sagum, C.A., Bedford, M.T., Sidhu, S.S., Sudol, M., and Harty, R.N. 2018. Host protein BAG3 is a negative regulator of Lassa VLP egress. Diseases , Jul 13;6(3). pii: E64. d. Han, Z., Dash, S., Sagum, C.A., Ruthel, G., Jaladanki, C.K., Berry, C.T., Schwoerer, M.P., Harty, N.M., Freedman, B.D., Bedford, M.T., Fan, H., Sidhu, S.S., Sudol, M., Shtanko, O., and Harty, R.N. 2020. Modular mimicry and engagement of the Hippo pathway by Marburg virus VP40: Implication for filovirus biology and budding. PLoS Pathog., Jan.6;16(1):e1008231. PMCID: PMC6977764. 2. Identification and development of host-oriented L-domain inhibitors of virus budding. Based in part on our steady progress in elucidating the molecular aspects and host involvement in virus budding from studies described above, and our long-term goal of developing antivirals, a current major effort in my lab is to identify, develop, and optimize small molecule compounds targeting viral L-domainhost interactions to inhibit virus egress and spread. We postulate that for viruses such as Ebolavirus, administration of such an antiviral therapeutic during an outbreak would inhibit virus dissemination and sprea d in infected individuals, thus slowing disease progression and allowing the individuals immune system time to mount a robust response to effectively combat and clear the infection. Indeed, there is a vital need for the advancement and development of effective and safe therapeutics against emerging, high priority pathogens such as Ebola, Marburg, Lassa fever, and Junín viruses, and since these virus -host interactions represent a common mechanism in a range of emerging RNA viruses, we predict that they represent an Achilles heel in the life cycle of these RNA virus pathogens. Toward this end, we have ongoing, fruitful collaborations with pharmacologists and virologists at the BSL-4 labs at USAMRIID and with medicinal chemists at Fox Chase Chemical Diversity Center, which has led to the identification of two successful lead series of PTAP and PPxY budding inhibitors that exhibit on-target, broad - spectrum antiviral activity against a wide array of RNA viruses. Specifically, we demonstrated that our current lead PTAP inhibitor, compound 0013, blocked egress of both Ebola and Junín VLPs at nanomolar concentrations, and our current lead PPxY inhibitors, compounds 4 and 5, blocked egress of Ebola, Marburg, and Lassa fever VLPs. Most exciting were our findings that these compounds could also block budding of live infectious viruses, including VSV and Junín (Harty lab), rabies (in collaboration with Dr. M. Schnell, TJU), Ebola (Kikwit) and Marburg (Ci67) (in collaboration with Drs. Andrew Herbert and J. Dye, USAMRIID, unpublished data). Currently, we are using Structure Activity Relationship (SAR) in collaboration with medicinal chemists Drs. J. Wrobel and A. Reitz (FCCDC) to identify analogs with enhanced potency and low cytotoxicity, as we seek to further transition one or more full-qualified L-domain inhibitors into more detailed IND-directed pharmacokinetic, pharmacodynamic and toxicity studies. a. Liu, Y., Lee, M. S., Olson, M. A., and Harty, R. N . 2011. Bimolecular complementation to visualize filovirus VP40-host complexes in live mammalian cells: toward the identification of budding inhibitors. Adv. Virol ., pii: 341816. PMCID: PMC3217271. b. Lu, J., Han, Z., Liu, Y., Liu, W., Lee, M.S., Olson, M.A., Ruthel, G., Freedman, B.D., and Harty, R.N. 2014. A host-oriented inhibitor of Junin Argentine hemorrhagic fever virus egress. J. Virol. 88(9):4736- 4743. PMCID: PMC3993831. c. Han, Z., Lu, J., Liu, Y., Davis, B., Lee, M.S., Olson, M.A., Ruthel, G., Freedman, B.D., Schnell, M.J., Wrobel, J., Reitz, A., and Harty, R.N. 2014. Small molecule probes targeting the viral PPxY-host Nedd4 interface block egress of a broad range of RNA viruses. J. Virol . 88(13):7294-7306. PMCID: PMC4054416. d. Loughran, H.M., Han, Z., Wrobel, J.E., Decker, S.E., Ruthel, G., Freedman, B.D., Harty, R.N., and Reitz, A.B. 2016. Quinoxaline-based inhibitors of Ebola and Marburg VP40 Egress. Bioorg. Med Chem Lett., 26:3429-3435. PMCID: PMC4955528. 3. Calcium regulation of hemorrhagic fever virus budding. In non-excitable cells, Ca 2 signals are generated and maintained through a tightly regulated process termed store-operated Ca2 signaling. As ER Ca2 levels decrease, Ca2 dissociates from C -terminal EF hands of the ER membrane -resident protein Stromal Interaction Molecule 1 (STIM1). Ca2 dissociation from STIM1 initiates its oligomerization and localization to ER regions Biosketches Page 28 Conta ct PDPI: Harty, Ronald N. adjacent to the plasma membrane where it interacts with and activates Orai1 Ca2 channels that control extracellular Ca2 influx. While previous studies in my lab implicated Ca2 broadly in control of Ebola VP40 VLP formation, we have now demonstrated for the first time that filovirus VP40 and arenavirus Z matrix proteins trigger host cell Ca 2 signals by activating the ER Ca2 sensing protein STIM1, which then activates plasma membrane Orai1 (Ca2) channels. Furthermore, we demonstrated that VP40 control of VLP production critically depends upon Orai1 -mediated Ca2 entry, as suppressing the expression or inhibiting the function of STIM1 and Orai1, or pharmacologically inhibiting Ca2 permeation of Orai with selective channel blockers profoundly inhibits production of VLPs. Importantly, we also established that Orai1 mediated Ca2 signals regulate transmission and spread of infectious Ebola, Marburg, Lassa fever, and Junín viruses in collaboration with John Dyes group at USAMRIID. Together these results have broad and profound significance because of 1) the insight they provide into mechanisms that control virus egress and transmission, 2 ) their establishment of STIM1 and Orai1 as conserved, essential, and immutable host proteins that underlie Ca2 signals critical for enveloped RNA virus egress, and 3) their validation of Orai blockers as the first of a family of broad-spectrum, host-oriented anti -viral therapeutics. Our ongoing research is focused on 1) defining the mechanism of virus induced calcium signaling, 2) defining the role of calcium in both host and virus dependent steps of budding, and 3 ) pursuing the development of small molecule inhibitors of the Orai1 channel as a host- oriented therapeutic approach to block egress of a wide-array of RNA viruses. a. Han, Z and Harty, R. N . 2007. Influence of calciumcalmodulin on budding of Ebola VLPs: Implications for the involvement of the RasRafMEKERK pathway. Virus Genes , 35(3):511-520. PMID:17570046. b. Madara, J.J., Han, Z., Ruthel, G., Freedman, B.D., and Harty, R.N . 2015. The multifunctional Ebola virus VP40 matrix protein is a promising therapeutic target. Future Virol ., 10(5):537-546. PMCID: PMC4480923. c. Han, Z., Madara, J.J., Herbert, A., Prugar, L.I., Ruthel, G., Lu, J., Liu, Y., Liu, W., Liu, X., Wrobel, J.E., Reitz, A.B., Dye, J.M., Harty, R.N., and Freedman, B.D. (co-corresponding authors). 2015. Calcium regulation of hemorrhagic fever virus budding: Mechanistic implications for host-oriented therapeutic intervention. PLoS Pathogens , 11(10):e1005220. PMCID: PMC4634230. d. Freedman, B.D. and Harty, R.N. 2016. Calcium and filoviruses: A budding relationship. Future Microbiol., 11:713-715. 4. Innate immune defenses of hemorrhagic fever virus infection. Innate immune responses to virus infection provide a critical first line of defense for the host against the invading pathogen. Understanding the complex interplay between the host innate immune defense mechanisms and counteraction by the filoviruses is crucial for developing novel antiviral strategies, vaccines, and therapeutics. We have been interested in the host innate immune response to filoviruses and have investigated several mechanisms of host innate immune mediated defenses involving host proteins TLR4, SOCS1, SOCS3, and ISG15. ISG15 is an interferon stimulated gene that has garnered much attention recently due to its broad-range of antiviral activity against a plethora of pathogens including DNA and RNA viruses. We demonstrated for the first time that ISG15 inhibited budding of EBOV VP40 VLPs in a PPxY L-domain dependent manner, and that such inhibition involved impairment of host Nedd4 ligase activity. We continue to investigate whether ISGylation of additional VP40 host interactors ( e.g. IQGAP1) can adversely affect VP40 function in budding. In addition, we are interrogating the effect of ISG15 expression on inhibition of Junín virus assembly and budding in vitro and in vivo using ISG15 -- mice (in collaboration with Dr. A. Garcia -Sastre, MSSM). These studies will reveal new host innate immune defense mechanisms that may regulate the budding processes of several high priority NIAID Category A pathogens. Moreover, this information will be critical to identify strategies (therapies andor vaccines) designed to tip the scale in favor of the host in the battle between viral pathogens and host innate immune defenses. a. Okumura, A., Pitha, P. M., and Harty, R. N. 2008. ISG15 Inhibits Ebola VP40 VLP Budding in an L- Domain Dependent Manner by Blocking Nedd4 Ligase Activity. Proc.Natl. Acad. Sci. USA , 105(10):3974-9. PMCID: PMC2268823. b. Okumura, A., Pitha, P. M., Yoshimura, A., and Harty, R. N . 2010. Interaction between Ebola virus glycoprotein and host TLR-4 leads to induction of pro-inflammatory cytokines and SOCS1. J. Virol., 84(1):27-33. PMCID: PMC2798428. c. Lu, J., Qu, Y., Liu, Y., Jambusaria, R., Han, Z., Ruthel, G., Freedman, B., and Harty, R. N . 2013. Host IQGAP1 and Ebola virus VP40 interactions facilitate VLP egress. J. Virol ., 87(13): 7777-80. PMCID: PMC3700276. d. Okumura, A., Rasmussen, A.L., Halfmann, P., Feldmann, F., Yoshimura, A., Feldmann, H., Kawaoka, Y., Harty, R.N ., and Katze, M.G. 2015. Suppressor of cytokine signaling 3 is an inducible host factor Biosketches Page 29 Contact PDPI: Harty, Ronald N. during Ebola virus infection and 1 regulates virus egress. J. Virol., 89(20):10399-10406. PMCID: PMC4580175. Complete List of Published Work in MyBibliography: http:www.ncbi.nlm.nih.govsitesmyncbironald.harty.1bibliography44131509publicsortdatedire ctionascending. D. Additional Information: Research Support andor Scholastic Performance Ongoing: R21 AI138052 Harty 011518  123120 Modular Domains of Host Proteins Regulate Filovirus Maturation Major Goal  To elucidate the role of host protein BAG3 in filovirus egress. Role: PI R21 AI129890 Freedman 120117  113120 The Membrane Repair Channel TRPML1 Regulates Ebola Virus Budding Major Goal  To elucidate the role of host channel TRPML1 in regulating Ebola virus egress Role: co-I R41 AI138630 Harty 060118  053121 Small Molecule Therapeutics Targeting Hemorrhagic Fever Viruses Major Goal  To develop PPxY inhibitors of virus budding. Role: PI R21 AI139392 Harty 060818  053121 Dueling PPxY Motifs of Filovirus VP40 and Host Angiomotin: Effects on Innate Immune Defenses and Tight Junction Integrity at Immune Privileged Sites. Major Goal  To examine the effect of VP40 on intestinal epithelium using a CACO2 model. Role: PI Major Goal  To test a unique series of budding inhibitors against FilovirusArenavirus infections. Role: PI R21 EY031465 Harty 040120  033122 Predicted Role of Ebola VP40-Host Interactions in Ocular Pathology and Persistence Major Goal  To determine whether filovirus VP40 contributes to ocular pathology using an eye-on-a-chip model. Role: PI Completed: R33 AI102104 Harty 090112  063019 Host-Oriented Therapeutics Targeting Filovirus Budding. Major Goal  To elucidate the mechanisms and potency of PTAP L-domain inhibitors of RNA virus budding. Role: PI, No overlap with current proposal Biosketches Page 30 Conta ct PDPI: Harty, Ronald N. NAME: Jay Wrobel BIOGRAPHICAL SKETCH eRA COMMONS USER NAME (credential, e.g., agency login): POSITION TITLE : Vice President, Academic Relations, Fox Chase Chemical Diversity Center, Inc. EDUCATIONTRAINING INSTITUTION AND LOCATION DEGREE Completion Date FIELD OF STUDY University of Wisconsin- Milwaukee Cornell University B.S. Ph.D. 1978 1983 Chemistry Organic Chemistry A. Personal Statement I am extremely well suited to for the medicinal chemistry leadership of this proposal. I have 34 years of experience in pharmaceutical industry focused on medicinal chemistry and drug design on a variety of drug discovery programs. In my current position at Fox Chase Chemical Diversity Center (FCCDC), I am involved in creation, leadership and management of medicinal chemistry and drug discovery partnerships with academic and biotech researchers to advance chemical hits to viable preclinical drug candidates up to and including the IND stage. I was formerly Senior Director of Medicinal Chemistry at WyethPfizer in Collegeville, PA where I mentored and guided the efforts of up to 31 medicinal chemists, and worked successfully with outside alliance partners in the design and synthesis of novel agents for multiple drug discovery programs (hit to Preclinical Development). I was directly involved in bringing forward nine development track candidates (Phase 0 and beyond). My hands -on expertise in research includes project and portfolio management, hit triage, hit-to-lead and lead optimization from hits found using enabling technologies such as high-throughput screening, fragment based drug discovery, and virtual screening. In these efforts I have employed ligand and structure based SAR methods, analytical chemistry, multi -step synthesis, parallel synthesis, data management. I obtain and use ADME and PK data in the design of our analogs. I have become involved with Prof. Harty on evaluation of egress inhibitors (namely mammalian Nedd4  viral L domain inhibitors) of RNA viruses as an approach toward new antiviral drugs to combat Ebola, Marburg, Lassa and AIDS. In this capacity I have provided Prof. Harty with medicinal chemistry leadership on a Phase I STTR grant application and along with my colleagues at FCCDC have discovered new egress inhibitor leads that are more potent, stable and suitable for composition of matter patent status than what we (FCCDC and Harty lab) discovered prior to the STTR -funded work (see Journal of Virology , 2014, 88(13), 7294-7306). We have since published an SAR account of our further work (Bioorg. Med. Chem. Lett. 2016 26(15), 3429-35). We have obtained an issued U.S. patent on the initial series and have make great advances during the Phase I STTR period of study resulting in in vivo proof of concept efficacy in a mouse model of Marburg infection. I also have additional experience with infectious diseases including antiviral small molecules therapeutics (Nef inhibitors -HIV) and antibacterial agents against drug resistant pathogens. My role in this project would be to implement and oversee all aspects of the medicinal chemistry program including design of analogs (with participation of the bench chemists), day to day and long term management of medicinal chemists, directing all ADME, PK and drug safety studies with advice from our CRO partners. We will use a project flow scheme I devised to find the best compounds and I will also be involved in any major project decisions with PI Ron Harty. Loughran HM, Han Z, Wrobel JE , Decker SE, Ruthel G, Freedman BD, Harty RN, Reitz AB. Quinoxaline-based inhibitors of Ebola and Marburg VP40 egress. Bioorg Med Chem Lett. 2016 Aug 1;26(15):3429-35. doi: 10.1016j.bmcl.2016.06.053. Epub 2016 Jun 23. PubMed PMID: 27377328; PubMed Central PMCID: PMC4955528 Han Z, Lu J, Liu Y, Davis B, Lee MS, Olson MA, Ruthel G, Freedman BD, Schnell MJ, Wrobel JE, Reitz AB, Harty RN. Small-molecule probes targeting the viral PPxY-host Nedd4 interface block egress of a broad range Biosketches Page 31 Contact PDPI: Harty, Ronald N. of RNA viruses. J Virol. 2014 Jul; 88(13): 7294-306. doi: 10.1128JVI.00591-14. Epub 2014 Apr 16. PubMed PMID: 24741084; PubMed Central PMCID: PMC4054416. Emert-Sedlak LA, Loughran HM, Shi H, Kulp JL 3rd, Shu ST, Zhao J, Day BW, Wrobel JE , Reitz AB, Smithgall TE. Synthesis and evaluation of orally active small molecule HIV-1 Nef antagonists. Bioorg Med Chem Lett. 2016 Mar 1;26(5):1480-4. doi: 10.1016j.bmcl.2016.01.043. Epub 2016 Jan 21. PubMed PMID: 26852364; PubMed Central PMCID: PMC4756635. Rubin H, Selwood T, Yano T, Weaver DG, Loughran HM, Costanzo MJ, Scott RW, Wrobel JE , Freeman KB, Reitz AB. Acinetobacter baumannii OxPhos inhibitors as selective anti-infective agents. Bioorg Med Chem Lett. 2015 Jan 15; 25(2): 378-83. doi: 10.1016j.bmcl.2014.11.020. Epub 2014 Nov 22. PubMed PMID: 25496770. B. Positions and Honors 1983 -1987 Senior Scientist, Medicinal Chemistry, Ayerst, Inc., Princeton, NJ 1987 -1988 Research Associate, Medicinal Chemistry, Wyeth -Ayerst Research, Inc., Princeton, NJ 1988 -1996 Principal Scientist, Chemical Sciences, Wyeth- Ayerst Research, Inc., Princeton, NJ 1996 -2001 Associate Director, Chemical Sciences, Wyeth- Ayerst Research, Inc., Radnor, Pa 2001 -2007 Director, Chemical Sciences, Wyeth Research, Inc., Collegeville, Pa 2007 -2010 Senior Director, Chemical Sciences, WyethPfizer, Inc., Collegeville, Pa 2011 -2012- Senior Research Fellow, Fox Chase Chemical Diversity Center, Inc., Doylestown, Pa 2012-present Vice President, Academic Relations, Fox Chase Chemical Diversity Center, Inc., Doylestown, Pa Member of the American Chemical Society 1978- present Alternate Councilor, American Chemical Society, Princeton, NJ Local Section, 2012-2014 Member, DOD Study Section for review of applications, 2012, 2013. C. Contribution to Science In terms of discovering other novel drug candidates, I have worked extensively in cardiovascular areas on LXR modulators for atherosclerosis . As a LXR Modulator co-team leader and Medicinal Chemistry coordinator for the WyethKaro-Bio LXR alliance, I delivered the first in-human LXR modulator (LXR-623) to phase I clinical studies for atherosclerosis indication in 2006. I also brought forth second and third generation LXR modulators devoid of mechanism and physiochemical -based side effects. Wrobel J, Steffan R, Bowen SM, Magolda R, Matelan E, Unwalla R, Basso M, Clerin V, Gardell SJ, Nambi P, Quinet E, Reminick JI, Vlasuk GP, Wang S, Feingold I, Huselton C, Bonn T, Farnegardh M, Hansson T, Nilsson AG, Wilhelmsson A, Zamaratski E, Evans MJ. Indazole-based liver X receptor (LXR) modulators with maintained atherosclerotic lesion reduction activity but diminished stimulation of hepatic triglyceride synthesis. J Med Chem. 2008 Nov 27; 51(22): 7161-8. doi: 10.1021jm800799q. PubMed PMID: 18973288. Quinet EM, Basso MD, Halpern AR, Yates DW, Steffan RJ, Clerin V, Resmini C, Keith JC, Berrodin TJ, Feingold I, Zhong W, Hartman HB, Evans MJ, Gardell SJ, DiBlasio-Smith E, Mounts WM, LaVallie ER, Wrobel J, Nambi P, Vlasuk GP. LXR ligand lowers LDL cholesterol in primates, is lipid neutral in hamster, and reduces atherosclerosis in mouse. J Lipid Res. 2009 Dec; 50(12): 2358-70. doi: 10.1194jlr.M900037-JLR200. Epub 2009 Mar 24. PubMed PMID: 19318684; PubMed Central PMCID: PMC2781308. Hu B, Unwalla RJ, Goljer I, Jetter JW, Quinet EM, Berrodin TJ, Basso MD, Feingold IB, Nilsson AG, Wilhelmsson A, Evans MJ, Wrobel JE. Identification of phenylsulfone-substituted quinoxaline (WYE-672) as a tissue selective liver X -receptor (LXR) agonist. J Med Chem. 2010 Apr 22; 53(8): 3296-304. doi: 10.1021jm100034x. PubMed PMID: 20350005 I was on a team that evaluated and brought in a FXR candidate from external alliance partner Exelixis. This candidate later became the first non-steroid based FXR agonist (FXR -450) taken to Phase I clinical studies for dyslipidemia in 2006. I later helped discover a FXR follow -on candidate with improved physiochemical properties (WYE-108635) that was taken to advanced preclinical animal toxicity evaluation in 2007. Biosketches Page 32 Conta ct PDPI: Harty, Ronald N. Flatt B, Martin R, Wang TL, Mahaney P, Murphy B, Gu XH, Foster P, Li J, Pircher P, Petrowski M, Schulman I, Westin S, Wrobel J, Yan G, Bischoff E, Daige C, Mohan R. Discovery of XL335 (WAY-362450), a highly potent, selective, and orally active agonist of the farnesoid X receptor (FXR). J Med Chem. 2009 Feb 26; 52(4): 904-7. doi: 10.1021jm8014124. PubMed PMID: 19159286. Lundquist JT, Harnish DC, Kim CY, Mehlmann JF, Unwalla RJ, Phipps KM, Crawley ML, Commons T, Green DM, Xu W, Hum WT, Eta JE, Feingold I, Patel V, Evans MJ, Lai K, Borges -Marcucci L, Mahaney PE, Wrobel JE. Improvement of physiochemical properties of the tetrahydroazepinoindole series of farnesoid X receptor (FXR) agonists: beneficial modulation of lipids in primates. J Med Chem. 2010 Feb 25; 53(4): 1774-87. doi: 10.1021jm901650u. PubMed PMID: 20095622. I co-invented the first non-steroidal PR antagonists that were delivered to development track (Phase 0 and beyond) for contraception or fibroid indications in 2003-2007. And I brought forth the first non-steroid based PR agonist (tanaproget) to human clinical trials for contraception indication in 2001. This compound successfully completed Phase II clinical trials and was licensed to Teva Pharmaceuticals. Fensome A, Adams WR, Adams AL, Berrodin TJ, Cohen J, Huselton C, Illenberger A, Kern JC, Hudak VA, Marella MA, Melenski EG, McComas CC, Mugford CA, Slayden OD, Yudt M, Zhang Z, Zhang P, Zhu Y, Winneker RC, Wrobel JE. Design, synthesis, and SAR of new pyrrole-oxindole progesterone receptor modulators leading to 5-(7-fluoro-3,3-dimethyl -2-oxo-2,3 -dihydro-1H -indol -5-yl) -1-methyl -1H-pyrrole-2-carbonitrile (WAY-255348). J Med Chem. 2008 Mar 27;51(6): 1861-73. doi: 10.1021jm701080t. Epub 2008 Mar 5. PubMed PMID: 18318463. Zhang Z, Lundeen SG, Slayden O, Zhu Y, Cohen J, Berrodin TJ, Bretz J, Chippari S, Wrobel J, Zhang P, Fensome A, Winneker RC, Yudt MR. In vitro and in vivo characterization of a novel nonsteroidal, species -specific progesterone receptor modulator, PRA-910. Ernst Schering Found Symp Proc. 2007; (1):171-97. PubMed PMID: 18540573. Fensome A, Bender R, Chopra R, Cohen J, Collins MA, Hudak V, Malakian K, Lockhead S, Olland A, Svenson K, Terefenko EA, Unwalla RJ, Wilhelm JM, Wolfrom S, Zhu Y, Zhang Z, Zhang P, Winneker RC, Wrobel J. Synthesis and structure-activity relationship of novel 6-aryl-1,4-dihydrobenzod1,3oxazine-2-thiones as progesterone receptor modulators leading to the potent and selective nonsteroidal progesterone receptor agonist tanaproget. J Med Chem. 2005 Aug 11;48(16): 5092-5. PubMed PMID: 16078826. Zhang Z, Olland AM, Zhu Y, Cohen J, Berrodin T, Chippari S, Appavu C, Li S, Wilhem J, Chopra R, Fensome A, Zhang P, Wrobel J, Unwalla RJ, Lyttle CR, Winneker RC. Molecular and pharmacological properties of a potent and selective novel nonsteroidal progesterone receptor agonist tanaproget. J Biol Chem. 2005 Aug 5; 280(31): 28468-75. Epub 2005 Jun 3. PubMed PMID: 15937332. I invented a mixed PPAR α,γ,δ agonist that completed Phase II clinical trials for type II diabetes but was not selected to go further for business reasons. I have an excellent external scientific and technical record based on 75 papers and 81 presentations. I am an inventor on 79 issued U.S. Patents. My publications can be found online at http:www.ncbi.nlm.nih.govmyncbicollectionsbibliography47703212reloaddeleteSuccessdnum1 D. Research Support Current Development of Small Molecule Therapeutics Targeting Hemorrhagic Fever Viruses R41AI138630-01A1 (Role: Key Personnel) 060118  053121 (NCE) This is the Phase I STTR that provided most of the preliminary data that forms the basis of this Phase II STTR application. It was a two year Phase I which is currently in a third year no cost extension. Soft LXR Agonists for Idiopathic Pulmonary Fibrosis R43 HL154852-01 (Role: PI) 072020  063021 Biosketches Page 33 Contact PDPI: Harty, Ronald N. This Phase I SBIR involves the discovery of soft drug LXR agonists for the treatment of Idiopathic Pulmonary Fibrosis (PDF). Overlap  none. Molecule Antagonists of PF4 for the Treatment and Prevention of HIT R42 HL12312602 (Role: PI) 09152019  06302022 This Phase IIB SBIR is directed toward the optimization of novel small molecule PF4 tetramer antagonists for the treatment of Heparin-Induced Thrombocytopenia (HIT). Overlap  none. Completed Inhibitors of the PHD2 Zinc Finger to Treat Anemia R32HL137458 (Role: PI) 04012017  3312018 This Phase I SBIR involves identification of SAR with intent to go to full -fledge drug discovery program in the Phase II portion for novel small molecule inhibitors the PHD2 Zinc Finger as a treatment for anemia Molecule Antagonists of PF4 for the Treatment and Prevention of HIT R42 HL12312602 (Role: Key Personnel) 10012016  10312018 This Phase II STTR is a full-fledge drug discovery program toward the optimization of novel small molecule PF4 tetramer antagonists for the treatment of heparin induced thrombocytopenia. Small Molecule Inhibitors of HIV1 Nef Virulence Factor for Treatment of HIV-Aids 2 R42 GM112516-02A1 (Role: Key Personnel) 10012016  10312018 This Phase II STTR is a full-fledge drug discovery program toward the optimization for novel small molecule inhibitors of the HIV -1 virulence factor, Nef. This viral protein is critical to HIV -1 replication in vivo, immune escape of HIV -infected cells, and AIDS progression Biosketches Page 34 Conta ct PDPI: Harty, Ronald N. OMB No. 0925-0001 and 0925-0002 (Rev. 032020 Approved Through 02282023) BIOGRAPHICAL SKETCH Provide the following information for the Seniorkey personnel and other significant contributors. Follow this format for each person. DO NOT EXCEED FIVE PAGES. NAME: Shtanko, Olena eRA COMMONS USER NAME (credential, e.g., agency login): POSITION TITLE: Staff Scientist I EDUCATIONTRAINING (Begin with baccalaureate or other initial professional education, such as nursing, include postdoctoral training and residency training if applicable. Adddelete rows as necessary.) INSTITUTION AND LOCATION DEGREE (if applicable) Completion Date MMYYYY FIELD OF STUDY Harvard University, Cambridge, MA University of Wisconsin, Madison, WI Texas Biomedical Research Institute, San Antonio, TX A.L.B. Ph.D. Post-Doc 062002 092010 122016 Natural Sciences Microbiology Virology  Antivirals A. Personal Statement I am interested in understanding how filoviruses and other emerging viruses establish productive infection and trigger disease in affected individuals. My thesis research focused on the mechanisms of budding and incorporation of virus nucleocapsids into nascent virions of Mopeia virus, a close relative of Lassa Fever virus. As a Postdoctoral Scientist, I studied mechanisms of entry and replication of highly pathogenic viruses as well as developed and tested novel therapeutics to combat these pathogens. My work has identified cell factors critical for Ebola virus and Crimean-Congo hemorrhagic fever virus entry into cells. Since advancing to the Staff Scientist position, I have established my own laboratory to study mechanisms of pathogenesis and dissemination of filoviruses and other high-containment pathogens. I have also extensively collaborated with a number of investigators, including Dr. Harty, to coordinate and perform virus studies to identify host factors critical for replication and spread of filoviruses as well as test novel antiviral therapeutics using a mouse model of Ebola and Marburg virus disease. I provide sophisticated molecular biology and cell biology techniques for studies of highly pathogenic viruses. I am well -trained in BSL4 procedures and have been working in Texas Biomeds ABSL4 laboratory since 2011. I will be responsible for all BSL4 studies as well as virus data analysis. B. Positions and Honors Positions and Employment 2004 -2010 Graduate Research Assistant, Department of Pathobiological Sciences, Advisor: Yoshihiro Kawaoka, University of Wisconsin, Madison, WI 2011 -2017 Postdoctoral Scientist, Department of Virology and Immunology, Advisor: Robert A. Davey, Texas Biomedical Research Institute, San Antonio, TX 2017 - Staff Scientist I, Department of Virology and Immunology, Texas Biomedical Research Institute, San Antonio, TX Professional Memberships 2008 - Member, American Society for Microbiology 2009 - Member, American Society for Virology 2016 - Member, American Society for Cell Biology Biosketches Page 35 Contact PDPI: Harty, Ronald N. Honors 2012 Douglass Foundation Post-doctoral Award 2014 Travel Award recipient, American Society for Virology Annual Meeting 2017 Texas Biomed Forum Research Support Grant 2017 Texas Biomed Founders Council Grant C. Contributions to Science Characterization of mechanisms of assembly, egress, and spread of emerging virusesUnderstanding how viruses invade and establish productive infection in host cells is essential to identify novel therapeutics and to reveal how the cell itself functions. My work on assembly of Mopeia virus, a close relative of Lassa Fever virus, has revealed that ALIXAIP1, which is required for the biogenesis of multivesicular bodies (MVBs) of the cell, binds to and drives incorporation of virus nucleocapsids into nascent virions. Recently, in collaboration with Dr. Ronald Harty, I have helped to identify angiomotin (Amot), a key regulator of YAP nuclearcytoplasmic localization and function, as essential for egress and spread of authentic Marburg and Ebola viruses. 1. Shtanko O , Imai M, Goto H, Lukashevich IS, Neumann, G, Watanabe T and Kawaoka Y. A role for the C terminus of Mopeia virus nucleoprotein in its incorporation into Z protein-induced virus -like particles. J Virol. 2010; 84(10):5415-5422. PMID: 20200234. PMCID: PMC2863806. 2. Shtanko O, Watanabe S, Jasenosky LD, Watanabe T and Kawaoka Y. ALIXAIP1 is required for NP incorporation into Mopeia Z-induced virus -like particles. J Virol. 2011; 85(7):3631-3641. PMID: 21248028. PMCID: PMC3067881. 3. Han Z, Dash S, Sagum CA, Ruthel G, Berry CT, Freedman BD, Bedford M, Fan H, Sidhu S, Sudol M, Shtanko O and Harty RN. Modular mimi cry and engagement of the Hippo pathway by Marburg virus VP40: implications for filovirus biology and budding. PLoS Path. 2020; 16(1):e1008231. PMID: 31905227. PMCID: PMC6977764. 4. Han Z, Ruthel G, Dash S, Berry CT, Freedman BD, Harty R and Shtanko O . Angiom otin regulates budding and spread of Ebola virus . J Biol Chem . 2020; 295(25):8596-8601. PMID: 32381509. PMCID: PMC7307192. Identification of host factors critical for entry and replication of high-containment viruses My work on Crimean-Congo Hemorrhagic Fever virus spearheaded the identification of the MVB as the site of release of viral genome into the cell cytoplasm. Since disruption of this compartment completely abolishes infection, I have been working on identifying compounds that interfere with MVB function. My research on EBOV has led to the finding that proteins that are known to initiate and coordinate autophagy, a catabolic process where cellular membranes serve as a source of vesicles whose role is to engulf cell contents for degradation, were also essential for macropinocytosis, a primary route of virus uptake. Interestingly, these cellular proteins appear to coordinate macropinosome formation close to the cell surface, an unexpected finding since these proteins were thought to function exclusively internally. I have been involved in several collaborative projects dedicated to mapping cellular networks critical for virus replication. For one such study, in collaboration with Dr. Schlesinger, I have developed assays to manipulate clinically relevant primary human cells to demonstrate the novel finding that E3 ubiquitin-protein ligase RBBP6 binds components of EBOV polymerase complex to modulate virus transcription and replication. 1. Shtanko O , Nikitina RA, Altuntas CZ, Chepurnov AA and Davey RA. Crimean-Congo hemorrhagic fever virus entry into host cells occurs through the multivesicular body and requires ESCRT regulators. PLoS Pathog. 2014; 10(9):e1004390. PMID: 25233119. PMCID: PMC4169490. 2. Shtanko O, Reyes AN, Jackson, WT and Davey RA. Autophagy -associated proteins control Ebola virus internalization into host cells. J Infect Dis . 2018; Jun 27. doi: 10.1093infdisjiy294. PMID: 29947774. PMCID: PMC6249560. 3. Batra J, Hultquist JF, Liu D, Shtanko O, Von Dollen J, Satkamp L, Jang GM, Luthra P, Schwarz TM, Small GI, Arnett E, Anantpadma M, Reyes AN, Leung DW, Kaake R, Haas P, Schmidt CB, Schlesinger LS, LaCount DJ, Davey RA, Amarasinghe GK, Basler CF and Krogan NJ. An Ebola virus -human protein-protein interaction map identifies the host ubiquitin ligase RBBP6 as a negative regulator of infection. Cell . 2018; 175(7):1917-1930. PMID: 30550789. PMCID: PMC6366944. Biosketches Page 36 Conta ct PDPI: Harty, Ronald N. Antivirals discovery to combat filoviruses As part of a multi -collaborative effort, I was involved in establishing methods to perform high-throughput screens to identify novel compounds with antiviral activity. I also initiated development of multiple assays to study the mechanism of virus entry into cells. These efforts have successfully identified novel potent inhibitors of macropinocytosis and consequently virus infection. 1. Anantpadma M, Kouznetsova J, Wang H, Huang R, Kolokoltsov A, Guha R, Lindstrom AR, Shtanko O, Simeonov A, Maloney DJ, Maury W, LaCount DJ, Jadhav A and Davey RA. Large-scale screening and identification of novel Ebola virus and Marburg virus entry inhibitors. Antimicrob Agents Chemother . 2016; 60(8):4471-4481. PMID: 27161622. PMCID: PMC4958205. 2. Shtanko O , Sakurai Y, Noël R, Cintrat JC, Gillet D, Barbier J and Davey RA. Retro-2 and its dihydroquinazolinone derivatives inhibit filovirus infection. Antiviral Res. 2018; 149:154-163. PMID: 29175127. 3. Wu Y, Pons V, Noël R, Kali S, Shtanko O , Davey RA, Popoff MR, Tordo N, Gillet D, Cintrat JC and Barbier J. DABMA: A derivative of ABMA with improved broad-spectrum inhibitory activity of toxins and viruses. ACS Med Chem Lett. 2019; 10(8):1140-1147. PMID: 31413797. PMCID: PMC6691562. Macrophage signaling pathways to control acute infection In collaboration with Dr. Wendy Maury, I investigated how acute infection with Plasmodium parasite, a causative agent of malaria, affected EBOV challenge in a BALBc mouse model of virus disease. The work demonstrated that acute Plasmodium infection protected the animals from lethal viral challenge and that such protection was dependent on parasite-elicited IFN -γ and associated macrophage signaling events. In addition to elucidating a factor critical for the outcome of EBOV disease in malaria-virus coinfected populations, the study carefully analyzed what virus challenge dose was appropriate for research on EBOV pathogenesis. Inoculation of mice with either 1, 10, or 100 PFU resulted in uniform lethality and similar virus loads in blood and major tissues. Interestingly, the protective effect of acute parasitemia was apparent only in 1 PFU -inoculated animals, demonstrating the critical need for appropriate experimental set up. In a separate study, I co-investigated the mechanisms of tissue macrophage-mediated control of acute virus infections. The study identified a previously unappreciated role for macrophage-intrinsic CD40 signaling in controlling EBOV infection. 1. Rogers K, Shtanko O , Vijay R, Mallinger L, Joyner C, Galinski M, Butler N and Maury W. Acute Plasmodium infection promotes interferon-gamma dependent resistance to Ebola virus infection. Cell Reports. 2020; 30(12):4041-4051. PMID: 32209467. PMCID: PMC7172281. 2. Rogers K, Shtanko O, Stunz L, Mallinger L, White J, Schmidt M, Varga S, Butler N, Bishop G and Maury W. Frontline Science: CD40 signaling restricts RNA virus replication in MΦs, leading to rapid innate immune control of acute virus infection. J Leukoc Biol. 2020; May 22. PMID: 32441445. For full publication record, see: https:www.ncbi.nlm.nih.govmyncbi1jeRfa5Kb3tkobibliographypublic D. Additional Information: Research Support andor Scholastic Perfo rmance Ongoing Research Support R41 AI138630 Harty (PI) 06012018-05312021 (2 nd NCE) Development of small molecule therapeutics targeting hemorrhagic fever viruses NIHNIAID The purpose of this study is to develop novel small molecule, broad-spectrum therapeutics against viral infections caused by filoviruses that depend on the PPxY L-domain motif for virus egress and spread of infection. Role: Subaward PI R21 AI144215 Maury (PI) 03112020- 03102022 CD40 regulation of acute virus infection NIHNIAID The purpose of this study is to clarify the role of host macrophages and the CD40 signaling pathway in regulating early immune responses to filovirus infection. Role: Subaward PI Biosketches Page 37 Contact PDPI: Harty, Ronald N. Completed Research Support R21 AI139902 Maury (PI) 06012018- 05312020 Mechanisms and consequences of plasmodiumEbola virus co-infections NIHNIAID This project determined the impact of malaria on EBOV infection and defined the roles of specific pro- and anti - inflammatory cytokines in regulating EBOV disease severity in a mouse co-infection model. Role: Subaward PI HDTRA118C0031 Westfall (PI) 02012019- 12312019 Grow, Expand, and Extract Total RNA Defense Threat Reduction Agency The purpose of this study was to develop a method to effectively extract and inactivate nucleic acids from select agents. Role: Subaward co-PI HHSN272201500015C McDonough (PI) 06232015- 5312019 Targeting therapeutics development to relieve bottlenecks: optimizing lead therapeutic compounds against infectious pathogens NIHNIAID The purpose of this study was to characterize the efficacy of tetrandrine derivatives as inhibitors of filovirus disease using a mouse intraperitoneal Ebola virus exposure model. Role: Subaward PI Shtanko (PI) 12012017- 03312019 This grant explored how VPS34 complex regulates macropinocytosis and EBOV infection in primary human macrophages and iPSC -derived hepatocytes. Role: PI Biosketches Page 38 Conta ct PDPI: Harty, Ronald N. NAME: John L. Kulp III BIOGRAPHICAL SKETCH POSITION TITLE: Director of Computational Chemistry, FCCDC eRA COMMONS USER NAME (credential, e.g., agency login): EDUCATIONTRAINING IN STITUTION AND LOCATION DEGREE (if applicable) Completion Date MMYYYY FIELD OF STUDY Drew University, Madison, NJ New York University, NY, NY Naval Research Laboratory, Washington, DC BA MS, PhD Postdoctoral 052001 052004, 062006 122009 Chemistry, Mathematics Chemistry Chemistry A. PERSONAL STATEMENT I have 11 years of computational chemistry experience with 17 peer reviewed publications. My background includes extensive experience in computer -aided drug design with a specialty focus on computational fragment-based methods. In 2010, I joined BioLeap, a small business that specializes in protein-protein interactions and that has a proprietary fragment-based computational chemistry software platform. I participated in the development of novel inhibitors for DHFR, PCSK9, 11β -HSD, and recA. During that time, I integrated molecular dynamics and docking software into BioLeaps proprietary fragment-based discovery platform. From 2013 to 2014, I was the project leader for BioLeaps PCSK9 program, which consisted of modeling (BioLeap), assays (Broad Institute of MIT and Harvard), and chemical synthesis (WuXi and BioDuro). As Director of Computational Chemistry at FCCDC, I apply structural biology to SAR diversification efforts in areas of innovative pharmacology. For example, I work closely with Dr. Wrobel and his team on multiple projects currently. I will apply my knowledge and expertise in the area of pharmacophore development to achieves the potency, efficacy and ADME gains of this grant application targeting the host Nedd4:virus PPxY interaction for the treatment of Marburg, Ebola and related diseases. B. POSITIONS AND HONORS Positions and Employment 2009 -11 Federal Staff Scientist, Naval Research Laboratory, Washington, DC 2011 -14 Distinguished Senior Scientist, BioLeap, Inc., Pennington, NJ 2013 - Assistant Professor, Baruch S. Blumberg Institute, Doylestown, PA 2014 - CSO, Small Molecule PPI Mimics LLC, College Station, TX 2014 - CEO, Conifer Point Pharmaceuticals LLC, Doylestown, PA 2017 - Director of Computational Chemistry, FCCDC Other Experience and Professional Memberships 2014 Consultant on NIH Phase I STTR grant entitled Small Molecule Inhibitors of HIV1 Nef Virulence Factor for Treatment of HIVAIDS Fox Chase Chemical Diversity Center, R41 GM112516- 01. 2015 Consultant on NIH Direct to Phase II SBIR entitled Development of Small Molecule Therapeutics Against Smallpox and Other Poxviruses Fox Chase Chemical Diversity Center, R44AI115759-01. Honors 2002 -6 MacCracken Fellowship, New York University 2004 Kramer Fellowship, New York University 2006 -9 American Society for Engineering Educations Postdoctoral Fellowship 2009 Young Investigator Award, American Peptide Society Biosketches Page 39 Contact PDPI: Harty, Ronald N. C. CONTRIBUTIONS TO SCIENCE 1. My vision is to create better drugs by rigorous molecular design. Fragment-based lead discovery (FBLD) involves constructing drugs from small chemical building blocks (fragments) that bind to a protein target. Numerous companies, such as Abbott, Astex, and others, are pursuing this strategy by screening fragment libraries, followed by X-ray crystallography or NMR. In contrast, my research starts with a detailed hypothesis of what protein binding interactions are required of a drug for inhibition or activation. Commercial and proprietary design tools, some of which I created, are then used to search ligand- protein or fragment-protein simulation data for poses, ranked by predicted binding affinity, which can be assembled or optimized into custom compounds that satisfy the requirements. The designed compounds are synthesized and tested in assays. With success, this establishes a predictive model to guide subsequent lead optimization. This is a protein-centric strategy, complementary to ligand -centric SAR approaches. This methodology provides access to broad chemical diversity, which is crucial for solving difficult problems in lead identification and optimization.  A Fragment-based Approach to the SAMPL3 Challenge. J.L. Kulp III, S.N. Blumenthal, Q. Wang, R. Bryan, F. Guarnieri, Journal of Computer -Aided Molecular Design 2012, 26(5) , 583-594.  Diverse Fragment Clustering and Water Exclusion Identify Protein Hot Spots, J.L. Kulp III, J.L. Kulp Jr., D.L. Pompliano, F. Guarnieri, Journal of the American Chemical Society 2011, 133(28) , 10740-1074.  Simulations of Nanocylinders Self-Assembled from Cyclic β-Tripeptides, N. Bernstein, J.L. Kulp III, M.A. Cato Jr., T.D. Clark, Journal of Physical Chemistry A , 2010 , 114(44) , 11948-11952. 2. The field of foldamer design promises new routes to important compounds for use in sensors, smart materials, therapeutics and catalysts. The term foldamer refers to a molecule that folds into a structurally stable state in solution. Proteins and peptides are an important class of natural foldamers that carry out a host of essential functions in biology, including molecular recognition, information storage, catalysis, and controlled crystallization of inorganic materials. The desire to mimic such functions with synthetic molecules inspires the field of foldamer design. I developed cyclic β-peptides as stochastic sensing elements and as a model chiral system for studying surface interactions. Other work included a universal method for restraining short peptides in helical conformations by a main-chain hydrogen bond surrogate (HBS) approach. These HBS peptides inhibited a variety of pharmaceutically relevant protein-protein interactions and the resulting technology was licensed by Aileron Therapeutics for clinical development.  Synthesis and characterization of cyclic peptides that are β -helical in trifluoroethanol. K.P. Fears, S.J. Photiadis, J.L. Kulp III and T.D. Clark, Journal of Peptide Science 2014, 20(5) , 366-374.  Engineering a β helical D,L-peptide for folding in polar media. J.L. Kulp III, T.D. Clark. Chemistry: A European Journal 2009, 15(44) , 11867-11877.  Trapping a Folding Intermediate of the α-Helix: Stabilization of the π-Helix. R.N. Chapman, J.L. Kulp III, A. Patgiri, N.R. Kallenbach, C. Bracken, P.S. Arora, Biochemistry 2008, 47(14) , 4189-4195.  Evaluation of Biologically Relevant Short α-Helices Stabilized by a Main -Chain Hydrogen-Bond Surrogate. D. Wang, K. Chen, J.L. Kulp III, P.S. Arora, Journal of the American Chemical Society 2006, 128(28) , 9248-9256. 3. My postdoctoral fellowship and subsequent two year federal employment was in The Molecular Interfaces and Tribology Section of the Department of Chemistry, Naval Research Laboratory. This section studies interdisciplinary questions in basic and applied interface and surface science. Our groups research stretched from advancing broad analytical methods, such as vibrational circular dichroism, to evaluating both prototypical and realistic interfaces to crafting and improving molecular surfaces and peptide architectures for applications such as controlling biological activity, anti -fouling, or lowering contact friction. One focus area was in the design and employment of bioinspired materials that conserve and utilize the exceptional characteristics of biomaterials while combining added functionality, such as fluorescence or conductivity, not originating from natural materials.  Vibrational Circula r-Dichroism Spectroscopy of Homologous Cyclic Peptides Designed to Fold into β Helices of Opposite Chirality, J.L. Kulp III, K.P. Fears, R. Lombardi, D.Y. Petrovykh, J.C. Owrutsky, L.A. Biosketches Page 40 Conta ct PDPI: Harty, Ronald N. Nafie, T.D. Clark, Biointerphases , 2011, 6(1), 1-7.  Characterization of the adhesive plaque of the barnacle Balanus Amphitrite: Anyloid -like nanofibrils are a major component, D.E. Barlow, G.H. Dickinson, B. Orihuela, J.L. Kulp III, D. Rittschof, K.J. Wahl, Langmuir 2010, 26(9) , 6549-6556. Highlighted in Chemical and Engineering News 2010, 88(9) , 34.  Molecular Recognition and Supramolecular Self-Assembly of a Genetically Engineered Gold Binding Peptide on Au 111. C. So, J.L. Kulp III, E.E. Oren, H. Zareie, H. Zareie, C. Tamerler, J.S. Evans, M. Sarikay a, ACS Nano 2009, 3(6), 1525-1531.  Laser Ablation of Micropores for Formation of Artificial Planar Lipid Bilayers. T.J. OShaughnessy, J.E. Hu, J.L. Kulp III, S.M. Daly, F.S. Ligler, Biomedical Microdevices 2007, 9(6), 863-868. 4. The beginning of my computational chemistry career began when I was using nuclear magnetic resonance (NMR) spectroscopy to determine the structures of biological macromolecules like peptides, polypeptides, and proteins. To do this, I would rely on 1D, homo- and hetero-nuclear 2D, and triple resonance, 3D, spectra. Some other examples of this work are cited in scientific contribution 3, above. After extracting all available information about the atomic distances and torsion angles from the NMR spectra, computer -aided structure calculations would convert distance and torsion angle data (constraints) into the 3D atomic model. For these calculations, a molecular dynamics simulation was used with simulated annealing; increasing temperature to generate an ensemble of conformations then decreasing temperature with increasing constraints based on the NMR data. The result is a family of structures that span a relatively narrow conformational space (low RMSD), unless intrinsically flexible, and obeys the constraint cri teria.  Structural Properties of an Artificial Protein that Regulates the Nucleation of Inorganic and Organic Crystals. J.L. Kulp III, T. Minamisawa, K. Shiba, M. Tejani, J.S. Evans, Langmuir 2007, 23(7) , 3857-3863.  Probing the Conformational Features of a Phage Display Polypeptide Sequence Directed Against Single Walled Carbon Nanohorn Surfaces. J.L. Kulp III, K. Shiba, J.S. Evans. Langmuir 2005, 21(25) , 11907- 11914.  Affinity Selection of Peptide Phage Libraries Against Single-Wall Carbon Nanohorns Identifies a Peptide Aptamer with Conformational Variability D. Kase, J.L. Kulp III, M. Yudasaka, J.S. Evans, S. Iijima, K. Shiba, Langmuir 2004, 20(20) , 8939-8941.  Molecular Characterization of a Prokaryotic Polypeptide Sequence that Catalyzes Au Crystal Formation. J.L. Kulp III, M. Sarikaya, J.S. Evans, J. Mat. Chem.2004, 14(14) , 2325-2332. Complete List of Published Work in MyBibliography: http:www.ncbi.nlm.nih.govsitesmyncbijohn.kulp.2bibliography48391280publicsortdatedirectionascen ding D. RESEARCH SUPPORT Cu rrent: Molecular Mechanism of the Dual Effects of Capsid Assembly Effectors on HBV Capsid Assembly and Disassembly (DoDPRMRP). Role: Investigator 912017  8312020 The goal of this project is to obtain molecular insights on the mechanism of two distinct chemotypes of hepatitis B virus (HBV) core protein allosteric modulators (CpAMs) on the assembly and disassembly (or uncoating) of HBV nucleocapsids. Previous: Role: Investigator 712018  6302019 The goal of this project is to progress disubstituted aminothiazoles that demonstrate remarkable selective toxicity for HCC -derived cell lines versus non-HCC liver lines and most other cancer lines. Small Molecule Inhibitors of HIV1 Nef Virulenc e Factor for Treatment of HIV-Aids R42GM112516-01 (PI: Smithgall; Role: Key Personnel) 9012016 8312018 This Phase II STTR involves the discovery of novel small molecule inhibitors of the HIV -1 virulence factor Nef, and has a substantial structural biology and computational chemistry component. Biosketches Page 41 Contact PDPI: Harty, Ronald N. BIOGRAPHICAL SKETCH Provide the following information for the Seniorkey personnel and other significant contributors. Follow this format for each person. DO NOT EXCEED FIVE PAGES . NAME: Reitz, Allen Bernard eRA COMMONS USER NAME (credential, e.g., agency login): POSITION TITLE : Chief Executive Officer EDUCATIONTRAINING (Begin with baccalaureate or other initial professional education, such as nursing, include postdoctoral training and residency training if applicable. Adddelete rows as necessary.) INSTITUTION AND LOCATION DEGREE (if applicable) Completion Date MMYYYY FIELD OF STUDY University of California, Santa Barbara University of California, San Diego University of Pennsylvania (Wharton) B.A. M.S. Ph.D. M.S. 081977 061979 011982 062003 Biochemistry and Molecular Biology Chemistry Chemistry Management of Technology A. Personal Statement Dr. Reitz has had 38 years of experience in the pharmaceutical industry including 26 years with Johnson  Johnson in various drug discovery research and management roles. He is an inventor of eight compounds that have entered clinical trials, including two from FCCDC which he founded in 2008 ( www.fc -cdci.com). For example, he is a coinventor of troriluzole which emerged from an Phase III SBIR -funded project at FCCDC on which he was the Principal Investigator, and which is currently in Phase IIIII clinical trial having recently passed an interim futility analysis for the treatment of Alzheimers disease. He is co-author of 160 scientific publications, co-inventor on 71 issued U.S. patents, Editor -in-Chief of the journal Current Topics in Medicinal Chemistry , on the Editorial Advisory Board of ACS Med. Chem. Lett. Dr. Reitz has extensive experience in project and portfolio management, target validation, hit triage, eADME profiling, and preclinical candidate selection. He is currently serving as an ad hoc reviewer on two of the rapid review COVID -19 study sections in NIAID. Dr. Reitz has worked with Drs. Wrobel and Prof. Harty since the inception of the FCCDC  U. Penn. collaboration of this novel host directed approach to treat Ebola and Marburg infection. He will continue to provide overall guidance as much as possible, working with Dr. Wrobel, and will seek to ensure a successful transition at the end of Phase II to further development and eventual commercialization including through taking advantage of various initiatives of the FDA such as the Neglected Disease Priority Review Voucher program. Loughran HM, Han Z, Wrobel JE, Decker SE, Ruthel G, Freedman BD, Harty RN, Reitz AB. Quinoxaline-based inhibitors of Ebola and Marburg VP40 egress. Bioorg Med Chem Lett. 2016 Aug 1;26(15):3429-35. doi: 10.1016j.bmcl.2016.06.053. PubMed PMID: 27377328; PubMed Central PMCID: PMC4955528 Han Z, Lu J, Liu Y, Davis B, Lee MS, Olson MA, Ruthel G, Freedman BD, Schnell MJ, Wrobel JE, Reitz AB , Harty RN. Smal l-molecule probes targeting the viral PPxY -host Nedd4 interface block egress of a broad range of RNA viruses. J Virol. 2014 Jul; 88(13): 7294-306. doi: 10.1128JVI.00591-14. PubMed PMID: 24741084; PubMed Central PMCID: PMC4054416. B. Positions and Honors Positions and Employment 1982 -1983 Postdoctoral Research Fellow, McNeil Pharmaceutical 1983 -1984 Research Scientist, McNeil Pharmaceutical 1984 -1987 Senior Scientist, Janssen Research Foundation Biosketches Page 42 Conta ct PDPI: Harty, Ronald N. 1987 -1992 Principal Scientist, R W Johnson Pharm. Res. Inst. 1992 -1998 Research Fellow, R W Johnson Pharm. Res. Inst. 1999 -2007 Senior Research Fellow, JNJPRD 2008 Vice President, Research and Development, AlsGen, Inc. 2008-present Founder and CEO, Fox Chase Chemical Diversity Center, Inc. 2009-present Co-Founder and CEO, ALS Biopharma, LLC 2010-present Founder and President, Pennsylvania Drug Discovery Institute (non-profit) Other Experience and Professional Memberships 2015 National Co-Chair of the BIO Boot Camp, associated with the national BIO meeting 2014-present Editorial Advisory Board member, ACS Med. Chem. Lett. 2014 -2016 Co-Editor, Technology Transfer and Entrepreneurship 2008-present Adjunct Professor, Drexel University College of Medicine 2008 -2009 Consultant to the World Health Organization for the Tropical Diseases (TDR) 2005-present Have served on 42 NIH Study Sections including Drug Discovery for the Nervous System (2012- 2014), and am currently an ad hoc member of COVID -19 rapid review study section study sections ZAI1 RG -W (S2) and ZAI1 RG -W (J2), July 14 and Oct. 15 respectively. 2000-present Editor -in-Chief, Current Topics in Medicinal Chemistry 1995-1996 Philadelphia Organic Chemists Club, Assistant Chairman, and then Chairman 1985-1986 Philadelphia Organic Chemists Club, Assistant Secretary, and then Secretary Honors 2016 Cornerstone Award CEO Honoree, Winner for Bucks County, PA 2011 Gold Award, Best Early- Stage Company, Philadelphia Business Journal 2006 Ronald J. Borne Distinguished Lectureship Award, in Recognition for Outstanding Contributions to Medicinal Chemistry, Department of Medicinal Chemistry, School of Pharmacy, University of Mississippi (April 18, 2006). Highlighted on pg. 6 of AACP News 2006, 37(9) . 2003 Industrial Chemistry Award of the Philadelphia Organic Chemists Club.Poster Session, Reception, and Lecture: May 29, 2003 1999 Johnson  Johnson Corporate Office of Science and Technology Excellence in Science Award 1997 Johnson  Johnson Significant Achievement Award for Creating the POP Process to Rapidly Move New Compounds from the Laboratory into Clinical Trials 1994 Philip B. Hofmann Research Scientist Award for Outstanding Achievement 1990 Johnson  Johnson Achievement Award for Outstanding Contributions C. Contribution to Science 1. Target Validation. Research in this area has been to understand the relation of structure to function among related compound libraries, using modern methods of synthetic chemistry and SAR development. Along those lines, FCCDC has submitted 12,000 new chemical entities in the context of individual collaborations with academic and biotech research partners. a. Punchi Hewage, A. N. D.; Yao, H.; Nammalwar, B. Gnanasekaran, K. K.; Lovell, S.; Bunce, R. A.; Eshelman, K.; Phaniraj, S. M.; Lee, M. M.; Peterson, B. R.; Battaile, K. P.; Reitz, A. B.; Rivera, M. Small Molecule Inhibitors of the BfrB-Bfd Interaction Decrease Pseudomonas aeruginona Fitness and Potentiate Fluoroquinoline Activity. J. Am. Chem. Soc. 2019, 141, 8171 -8184. b. Karpova, Y.; Wu, C.; Divan, A.; McDonnell, M. E.; Hewlett, E.; Makhov, P.; Gordon, J.; Ye, M.; Reitz, A. B.; Childers, W. E.; Skorski, T.; Kolenko, V.; Tulin, A. V. Non-NAD-like PARP -1 inhibitors in prostate cancer treatment. Biochem. Pharmacol. 2019, 167, 149-162. c. Robinson, R. M.; Reyes, L.; Duncan, R. M.; Bian, H.; Reitz, A. B .; Manevich, Y.; McClure, J. J.; Champion, M. M.; Chou, C. J.; Sharik, M. E.; Chesi, M.; Bergsagel, P. L.; Dolloff, N. G. Inhibitors of the protein disulfide isomerase family for the treatment of multiple myeloma. Leukemia 2019, 33, 1011- 1022. d. Martin, M. D.; Calcul, L.; Smith, C.; Jinwal, U. K.; Fontaine, S. N.; Darling, A.; Seeley, K.; Woitas, L.; Narayan, M.; Gestwicki, J. E.; Smith, G. R.; Reitz, A. B .; Baker, B. J.; Dickey, C. A. Synthesis, Stereochemical Analysis, and Derivatization of Myricanol Provide New Probes That Promote Autophagic Tau Clearance. ACS Chem. Biol. 2015, 10, 1099 -1109. Biosketches Page 43 Contact PDPI: Harty, Ronald N. 2. Hit to Lead Medicinal Chemistry. Much effort has focused on this area, in which libraries of related compounds are prepared in order to obtain maximum diversity based upon common themes dictated by the structure of the target, pharmacophore analysis, and the creation of new intellectual property. Multiple projects (e.g. BACE, EBNA1) have involved real -time X -ray co -crystallography collaboration and analysis. a. Baxter, E. W.; Conway, K. A.; Kennis, L.; Bischoff, F.; Mercken, M. H.; De Winter, H. L.; Reynolds, C. H.; Tounge, B. A.; Luo, C.; Scott, M. K.; Huang, Y.; Braeken, M.; Pieters, S. M. A.; Berthelot, D. J. C.; Masure, S.; Bruinzeel, W. D.; Jordan, A. D.; Parker, M. H.; Boyd, R. E.; Qu, J.; Alexander, R. S.; Brenneman, D. E.; Reitz, A. B. 2-Amino -3,4-dihydroquinazolines as Inhibitors of BACE-1 ( β-Site APP Cleaving Enzyme): Use of Structure Based Design to Convert a Micromolar Hit into a Nanomolar Lead. J. Med. Chem. 2007, 50, 4261-4264. b. Huang, Y.; Strobel, E. D.; Ho, C. Y.; Reynolds, C. H.; Conway, K. A.; Piesvaux, J. A.; Brenneman, D. E.; Yohrling, G. J.; Arnold, H. M.; Rosenthal, D.; Alexander, R. S.; Tounge, B. A.; Mercken, M.; Vandermeeren, M.; Parker, M. H.; Reitz, A. B.; Baxter, E. W. Macrocyclic BACE Inhibitors: Optimization of a Micromolar Hit to Nanomolar Leads. Bioorg. Med. Chem. Lett. 2010, 203158-3160. c. McDonnell, M. E.; Vera, M. D.; Blass, B. E.; Pelletier, J. C.; King, R. C.; Fernandez -Metzler, C.; Smith, G. R.; Wrobel, J.; Chen, S.; Reitz, A. B. Riluzole prodrugs for melanoma and ALS: design, synthesis, and in vitro metabolic profiling. 2012 , Bioorg. Med. Chem., 20, 5642-5648. d. Rubin, H.; Selwood, T.; Yano, T.; Weaver, D. G.; Loughran, H. M.; Costanzo, M. J.; Scott, R. W.; Wrobel, J. E.; Freeman, K. B.; Reitz, A. B . Acinetobacter baumannii OxPhos inhibitors as selective anti-infective agents. Bioorg. Med. Chem. Lett. 2015, 25, 378-383. 3. Lead Optimization (LO) Medicinal Chemistry Advancing to Preclinical and Clinical Development. Considerable emphasis has been placed on this phase of the drug discovery process, for which eight NMEs have entered human clinical trials. The 71 issued U.S. patents are largely devoted to LO SAR programs. There are three basic considerations for a medicinal chemist which apply to all indications: (1) route of administration (e.g. p.o, i.v., etc.), and (2) dosing regimen (acute, subchronic, chronic), and (3) standard of care (crowded marketplace or keen unmet need). a. Reitz, A. B.; Baxter, E. W.; Bennett, D. J.; Codd, E. E.; Jordan, A. D.; Maryanoff, B. E.; McDonnell, M. E. ; Ortegon, M. E.; Renzi, M. J.; Scott, M. K.; Shank, R. P.; Sherrill, R. G.; Vaught, J. L.; Wustrow, D. J. N-Aryl-N-Benzylpiperazines as Potential Antipsychotic Agents. J. Med. Chem. 1995 , 38, 4211- 4222. Description of mazapertine, which advanced to Phase II clinical trials. b. Parker, M. H.; Smith-Swintosky, V. L.; McComsey, D. F.; Huang, Y.; Brenneman, D.; Klein, B.; Malatynska, E.; White, H. S.; Milewski, M. E.; Herb, M.; Finley, M. F. A.; Liu, Y.; Lubin, M. L.; Qin, N.; Iannucci, R.; Leclercq, L.; Cuyckens, F.; Reitz, A. B .; Maryanoff, B. E. Novel, Broad-Spectrum Anticonvulsants Containing a Sulfamide Group: Advancement of N -(Benzob thien-3 - yl)methylsulfamide (JNJ-26990990) into Human Clinical Studies. J. Med. Chem ., 2009, 52, 7528- 7536. Anticonvulsant JNJ-26990990 which advanced into Phase I clinical trials. c. Ross, T. M.; Battista, K.; Bignan, G. C.; Brenneman, D. E.; Connolly, P.. J.; Liu, J.; Middleton, S. A.; Orsini, M.; Reitz, A. B .; Rosenthal, D. I.; Scott, M. K.; Vaidya, A. H. A selective small molecule NOP (ORL -1 receptor) partial agonist for the treatment of anxiety. Bioorg. Med. Chem. Lett. 2015, 25, 602- 606. Anxiolytic ORL-1 receptor agonist that advanced into Phase I clinical trials. d. Messick, T. E.; Smith, G. R.; Soldan, S. S.; McDonnell, M. E.; Deakyne, J. S.; Malecka, K. A.; Tolvinski, L.; van den Heuvel, A. P. J.; Gu, B. W.; Cassel, J. A.; Tran, D. H.; Wasserman, B. R.; Zhang, Y.; Velvadapu, V.; Zartler, E. R.; Busson, P.; Reitz, A. B .; Lieberman, P. M. Structure-based design of small -molecule inhibitors of EBNA1 DNA binding blocks Epstein-Barr virus latent infection and tumor growth. Sci. Tranl. Med. 2019, 11, pii: eaau5612. doi: 10.1126scitranslmed.aau5612. EBNA1 inhibitor currently in Phase II clinical trials, discovered at FCCDC 4. Synthetic Organic Che mistry. Dr. Reitz was inspired by organic faculty as an undergraduate, which was reinforced by a positive experience as a graduate student with Murray Goodman at UCSD. Highly cited publications include a Chem. Rev. article on the Wittig reaction and Vol. 59 of Organic Reactions on reductive aminations using boron -containing reagents consisting of 700 pages and an entire volume. Research has been conducted in the areas of conformational analysis and new synthetic methods. Biosketches Page 44 Conta ct PDPI: Harty, Ronald N. a. Larsen, B. J.; Rosano, R. J.; Ford -Hutchinson, T. A.; Reitz, A. B .; Wrobel, J. A. A method for C2 arylation of 1,3-indandiones. Tetrahedron 2018, 74, 2762-2768. b. Pelletier, J. C.; Velvadapu, V.; McDonnell, M. E.; Wrobel, J. E.; Reitz, A. B. Intramolecular rearrangement of α -amino acid amide derivatives of 2-aminobenzothiazoles.Tetrahedron Lett. 2014, 55, 4193-4195. c. Maryanoff, B. E.; Reitz, A. B. The Wittig Olefination Reaction and Modifications Involving Phosphoryl - Stabilized Carbanions. Stereochemistry, Mechanism, and Selected Synthetic Aspects. Chem. Rev.1989, 89, 863-927. d. Baxter, E. W; Reitz, A. B. Reductive Aminations of Carbonyl Compounds with Borohydride and Borane Reducing Agents In Organic Reactions : Wiley, New York, p. 1-714, Vol. 59, 2002. Complete List of Published Work in MyBibliography: http:www.ncbi.nlm.nih.govsitesmyncbi1nyMq8FpqE0A5bibliography47313087publicsortdatedirectionascending D. Research Support Ongoing Research Support Riluzole Prodrugs for Melanoma and ALS R44 CA156781-04A1 (PI: Reitz) 9118- 83121 This Phase II SBIR Bridge application involves the further pre-clinical development of troriluzole, an actively - transported (PepT1) prodrug of riluzole, . A Phase II clinical trial for the treatment of solid tumors is included in the grant, to be conducted at the Dana Farber Cancer Institute. Overlap  none. Troriluzole for the Treatment of Cocaine Addiction R41 DA047169-01 (PI: Rawls, Temple University; Role, Key Personnel) 070118-73021 (NCE) In this project, Prof. Rawls is testing whether troriluzole can counter the effects of cocaine in rats. Currently in the no cost extension phase. Overlap  none. Psychoactive bath salts and the glutamate system R01 DA039139-02 (PI: Rawls, Temple University; Role, Key Personnel) 8116- 73121 This R01 from Prof. Rawls at Temple University involves understanding the pharmacology of bath salts in CNS receptor and behavioral assays. Overlap  none. Targeting Telomerase for Pancreatic Cancer Therapeutics R01 CA208439-01 (PI: Skordalakes; Role, Key Personnel) 8118- 73121 This work is to discover new telomerase inhibitors for the treatment of pancreatic cancers. Overlap  none. DART2.0: comprehensive cell type-specific behavioral neuropharmacology RF1 MH117055-01 (PI: Tadross, Duke University; Role, Key Personnel) 91418-9 1321 Prof. Tadross is the PI of a Brain R01 looking to identify new DART bifunctional probes to understand the localization and function of important receptor systems in the CNS. Our role is to perform synthetic chemistry to obtain suitable probes which are then studied in his laboratory. Overlap  none Biosketches Page 45 Contact PDPI: Harty, Ronald N. FCCDC Budget Justification Personnel Jay E. Wrobel, Ph.D. Vice President, Academic Affairs and Senior Research Fellow, Key Personnel, FCCDC Project Leader (Yrs 1-3: 3 cal. months). Dr. Wrobel has 35 years of experience in the pharmaceutical industry focused on drug design and discovery on a variety of drug discovery programs. He was formerly Senior Director of Medicinal Chemistry at WyethPfizer in Collegeville, PA where he mentored and guided the efforts of up to 31 medicinal chemists and worked successfully with outside alliance partners in the design and synthesis of novel agents for multiple drug discovery programs (hit to Preclinical Development). Dr. Wrobel was directly involved in bringing forward nine development track candidates (phase 0 and beyond). He has 78 peer reviewed articles and is an inventor on 83 U.S. Patents. Dr. Wrobel has expertise in project and portfolio management, hit triage, hit -to-lead and lead optimization from hits found using enabling technologies such as high-throughput screening, fragment based drug discovery, and virtual screening. In these efforts he has employed ligand and structure based SAR methods, analytical chemistry, multi -step synthesis, parallel synthesis, data management and ADME analysis. In his current position as VP Academic Relations at Fox Chase Chemical Diversity Center, Dr. Wrobel has collaborated and co-published with Professor Harty on in vitro and in vivo characterization of first-in -class novel inhibitors of filovirus PPxY interaction with host protein Nedd4 as an approach toward broad-spectrum antiviral therapy . Dr. Wrobel will be responsible for FCCDC project design, direction of the project and data interpretation. He will also organize regular project meetings among team members and consultants to maximize efficiency. Allen B. Reitz, Ph.D., CEO, Key Personnel (Yrs 1-3: 0.6 cal. months). Dr. Reitz has had 38 years of demonstrated accomplishment as a medicinal chemist in the pharmaceutical industry, including 26 years with Johnson  Johnson. He is a co-inventor as well as the project leader in most cases of nine compounds that have entered human clinical trials, two of which are currently in the clinic. He has 150 scientific publications and 71 issued U.S. patents, and is the Editor of the journal Current Topics in Medicinal Chemistry . He has extensive experience in project and portfolio management, target validation, hit triage, hit to lead and lead optimization medicinal chemistry, eADME profiling, and preclinical candidate selection. He is also Adj. Prof. at Drexel University, College of Medicine. Dr. Reitz founded FCCDC in 2008 to provide translational biomedical research in the area of early drug discovery research including medicinal chemistry hit to lead and lead optimization research. Dr. Reitz will assist Dr. Wrobel on project design and data interpretation. He will also pay particular attention to developing the required transition beyond Phase II, including a suitable project strategy, implementation of the Commercialization Plan, and interface with potential development partners and other interested parties. John Kulp III, Ph.D., Director of Computational Chemistry, Key Personnel (Yrs 1-3: 0.6 cal. months). Dr. Kulp has 9 years of computational chemistry experience with 17 peer reviewed publications. After his PhD and postdoctoral training he spent 4 years at BioLeap, a small business that specializes in protein-protein interactions and that has a proprietary fragment-based computational chemistry software platform. There he was the project leader for BioLeaps PCSK9 program. He will perform computer -assisted drug design studies, and help to provide insight by the creation of computational pharmacophores during the course of the program. Katie B. Freeman, Ph.D., Director of Biology, FCCDC (Yrs 1 -3: 0.6 cal. monthsyr). Dr. Freeman has extensive experience with numerous in vitro assay formats, in microbiological methodologies, mammalian cell culture, bioanalytical methodologies. She will conduct cytotoxicity assays at FCCDC in liver HepG2 and endothelial HUVEC to compare the relative in vitro therapeutic index between series and individual compounds. In addition, she will perform formulations development as required to support the in vivo evaluation at TBRI. Hong Ye, Scientist (Yrs 1-3 : 12 cal. months), Ms. Ye is an exceptionally talented bench medicinal chemist, who consistently ranks among the top staff at FCCDC in terms of productivity and creativity. She previously had 20 years of experience at Johnson  Johnson in Spring House PA and other companies before joining FCCDC in June 2015. She has been the primary laboratory medicinal chemistry on the Harty MarburgEbola project and will continue to do so working under the direction of Dr. Wrobel with proper attention being paid to labor atory safety, confidentiality, productivity, notebook keeping and spectral data records retention. Budget Justification Attachment Page 55 Contact PDPI: Harty, Ronald N. Leo J. Adalbert, M.B.A. Technical and Business Assistance (TABA). ($50,000 over three years requested) Mr. Adalbert is a Harvard MBA who will be our business development consultant for this project. He has more than 25 years of deep cross -functional commercial and clinical development experience across both small molecules and biologics. Prior to founding StraNexa LLC, Leo held various leadership roles across marketing, business development, and alliance leadership at leading firms such as AstraZeneca, Merck, Astra Merck, and Regeneron. Mr. Adalbert earned his MBA from Harvard Business School while on a fellowship, and holds executive leadership certifications from Northwestern (Kellogg) and the Univers ity of Pennsylvania (Wharton). He will interface with government agencies (FDA, BARDA) and investors as appropriate. Equipment No new equipment is requested. SuppliesMaterials will be purchased as needed for routine medicinal chemistry, including the following: Year 1 Year 2 Year 3 Total Solvent and reagents: Consumables lab supplies: Glassware: Total per year: Other Expenses Contracted Research Organization Expenses: over three years Contract Research Organization (CRO) Expenses  ADMET and PK testing will be performed as in the following schedule. Contractor names are in parentheses, and have all worked effectively with FCCDC in the past. Budget Justification Attachment Page 56 Contact PDPI: Harty, Ronald N. Indirect Cost and Fee Indirect costs of 40 are requested. A fee of 7 is requested. Justification for budget above statutory budget limitations According to National Institutes of Health SBA -Approved SBIRSTTR Topics for Awards over Statutory Budget Limitations as listed in Appendix A of the omnibus 2020 solicitation (https:sbir.nih.govsitesdefaultfilesNIH_Topics_for_Budget_Waivers.pdf), the topic discovery and development of therapeutics for infectious diseases (the prime concern of this proposal) is listed in NIAID, Division of Microbiology and Infectious Disease (DMD) as eligible for budgets over the statutory maximum amounts. Small molecule drug discovery is very expensive and labor intensive, involving the synthesis and evaluation of a large number of test compounds in an iterative fashion. In this STTR Phase II grant application, we embark on a full lead optimization program with the goal of obtaining preclinical drug candidates for the treatment of Marburg, Ebola and related viral infections. Our current lead series are novel and suitable for composition of matter patent protection, and a US patent has issued on the original series which will be supplemented by additional provisional patent applications over time. We expect the optimized compounds at the end of Phase II to have increased potency, and be safe and suitable for oral administration. Budget Justification Attachment Page 57 Contact PDPI: Harty, Ronald N. LE O J.AD ALBE RT GRE A T E R P HI L A D E L P HI A ARE A , PA 1890 1 www.stranexa.com  BIO-PHARMA EXECUTIVE  MARKETING  STRATEGY  GLOBAL COMMERCIALIZATION  BD  Dynamic, creative, and strategic business leader with proven expertise in strategic commercialization, go- to- market strategy, product launches, business development, and consulting across all phases of development, inclusive of biologics and small molecules.  Strong neuroscience, immunology, oncology, and rare diseaseOrphan Drug commercialization and scientific expertise across multiple disease areas.  Outstanding track record of success in identifying and evaluating opportunities, developing a vision for growth, and building differentiated franchises.  Lead teams to exceed expectations utilizing high initiative, strong team -building skills, creative problem -solving, cross -functional leadership, and collaboration. Harvard MBA ( Fellowship ). CORE COMPETENCIES  Partnering  Collaboration  LeadershipVision  New Product Launches  Promotion  Opportunity Identification  Customer, Competitor  Market Analysis  Product Development  Negotiations  Transactions  Capability Creation  InnovationMedical Science  Strategic  Tactical Agility  Critical ThinkingProblem Solving  Talent Development  KOL and Advocacy Group Partnering  Metrics  Governance  Life-Cycle Management PROFESSIONAL EXPERIENCE AND KEY ACCOMPLISHMENTS STRANEXA, LLC ( www.stranexa.com) (2014 -Present) DOYLESTOWN, PA Founder and Principal Lead bio- pharmaceutical strategic commercialization consulting practice with focus on specialty and rare diseaseOrphan Drug markets. Passion for building differentiated brands and capabilities transformative for patients and communities across for-profit and non-profit enterprises through a science -driven, market -led, patient - focused approach.  Led significant projects across multiple domains that consistently exceeded client expectations (portfolio and commercialization strategy, go-to-market strategy, product launch planning, BDalliance leadership, competitor and market analyses, communications strategy and publications, non- profit market development).  Strong scientific foundation across diverse diseases and conditions.  Expert instructor with Biotech Pri mer www.biotechprimer.com (Strategic Commercialization Within BioPharma and Drug Development Immersion ). CSL BEHRING, INC. (2012 -2013) KING OF PRUSSIA, PA Global Marketing Director, Immunoglobulin (IgG) Marketing  Strategic Commercialization (12-13) Consultant (12) Led global marketing strategy, commercialization, launch planning and core tactics for global IgG portfolio with focus on HIZENTRA and PRIVIGEN. Led global commercial strategy for CSL362 (hematological tumors).  Chaired global business team and led major cross- functional initiatives guiding market leadership for key brands (globally PRIVIGEN revenues up 17.9 to $1B; HIZENTRA revenues up 45.2 to $402MM).  Developed innovative strategies and major initiatives to build market leadership and differentiation. AXCAN PHARMA, INC. (2008- 2011) BRIDGEWATER, NJ Director Business Development and Alliance Management (company restructured in 2011)  Successfully divested global PHOTOFRINPHOTOBARR oncology drugdevice business, exceeding internal objectives (strong cash return, elimination of significant ongoing costs, and placement of commercial and RD employees in acquiring firm). Led entire transaction process, including prospectus development, buyer identification, business case preparation, due diligence, negotiations and transition planning.  Created new Alliance Management capability (vision, strategies, metrics, tools and roll-out plan). Effectively managed ongoing and new alliance relationships.  Selected to lead LACTEOL Global Product Team, and helped advance critical lifecycle initiatives and global presence in new markets. Page 1 of 4 Budget Justification Attachment Page 58 Contact PDPI: Harty, Ronald N. LE O J.AD ALBE RT PROFESSIONAL EXPERIENCE AND KEY ACCOMPLISHMENTS (CONTINUED )  ASTRAZENECA PHARMACEUTICALS (1999 2008) WILMINGTON, DE Advanced rapidly through a series of increasingly responsible roles, providing effective leadership in driving major commercialization and business development initiatives (multiple recipient of AstraZeneca Excellence Award). Alliance and Program Leader, Strategic Planning  Business Development  Spearheaded and managed collaboration to commercialize CRESTOR fenofibrate fixed -dose combination. Collaboration progressed six months ahead of schedule and $3.5M under budget.  Instrumental in design and implementation of new Alliance Management capability within the firm. Senior Director, Licensing Commercial Evaluations Managed commercialization strategy in collaboration with Global Product Teams. Led evaluations of Phase IIIII licensingacquisition opportunities across multiple therapeutic areas.  Led product and company alliance and acquisition evaluations covering oncology and neuroscience, driving successful progression of target opportunities through formal diligence to deal completion.  Defined and implemented novel commercialization approaches for various CRESTOR fixed dose combination opportunities. Senior Director, US Marketing  Commercial Operations  Oncology  Infection Directed commercialization strategy, market development, and licensing support for oncologyinfection emerging brands portfolio (seven drug candidates from Discovery through Phase III clinical development covering lung, prostate, breast, and colorectal cancers).  Directed overall strategic direction, including creation of integrated target product profiles and patient - centered claims across entire brand portfolio encompassing multiple pathways (SRC kinase, VEGFEGF, endothelin, vascular targeting agents, and prenylation inhibitors.)  Proactively identified new business opportunities and resolved strategic gaps with global product teams to consistently maintain momentum through key decision-points. Transaction Director, Licensing  North America Led transaction process through all stages involving analyzing, negotiating, closing, and managing transitions of select partnership opportunities (product divestments, co -promotions, and in -licenses.)  Directed all facets of buyer identification, business case prep, due diligence, negotiations, and transition planning. Successfully closed multiple divestment transactions under tight timelines, delivering significant operating income, tax benefits, and business efficiencies based on significant premiums to retention values.  Consistently earned DistinguishedExcellent performance ratings (top 2) and achieved Deal of the Year  recognition. EARLIER RELEVANT EXPERIENCE AND KEY ACCOMPLISHMENTS ELAN PHARMACEUTICALS, INC. SAN FRANCISCO, CA Director, Neuromuscular Marketing (business restructured via SEC obligations) Developed and led execution of strategic and tactical marketing plans to effectively support all stages of lifecyclemarket entry. Developed comprehensive product pre-launch and launch deployment plans that expanded Élans neuromuscular franchise into attractive new market segments. ASTRA -MERCK PHARMACEUTICALS, INC. (JV dissolved) WAYNE, PA Director, Neuroscience Marketing  Business Strategy Marketing Manager, Neuroscience (96-97) Directed franchise development of Astra -Mercks neuroscience business as commercial lead in formulating strategic direction and integration activities with Astra Sweden corporate portfolio.  Established strategic alliances and launched market development activities with patient advocacy groups, medical associations, and payers. Recognized with Leadership Award by Huntingtons Disease Society.  Recipient of Astra -Merck Leadership Award for Excellence for development and implementation of an innovative, comprehensive business and licensing strategy encompassing multiple neurological and psychiatric diseases, indications and product opportunities globally. Page 2 of 4 Budget Justification Attachment Page 59 Contact PDPI: Harty, Ronald N. LE O J.AD ALBE RT EARLIER RELEVANT EXPERIENCE AND KEY ACCOMPLISHMENTS (CONTINUED )  RHONE -POULENC RORER PHARMACEUTICALS, INC. COLLEGEVILLE, PA Senior Product Manager  U.S. Led market development and successful introduction of RILUTEK, the first and only medical therapy approved for the treatment of amyotrophic lateral sclerosis (Lou Gehrigs Disease) prolonging survival. Achieved 95 formulary acceptance within six months and sales 121 to plan. REGENERON PHARMACEUTICALS, INC. TARRYTOWN, NY Manager, Market Development Established commercial direction for the companys first biologic product CNTF (ciliary neurotrophic factor) for the treatment of multiple neurodegenerative diseases. (Product failed in late- Phase III clinical trials.) MERCK HUMAN HEALTH U.S.  a Division of Merck  Company WEST POINT, PA Senior Promotion Manager  Promoted from Promotions Manager Directed $20M product promotion and market development pre-launch and launch program. Drove successful introduction of PROSCAR in the U.S., the first medical therapy treatment of benign prostatic enlargement.  Physician and DTC Program acknowledged with several industry awards; achieved 119 to plan. Marketing Analyst  Promoted from Associate Marketing Analyst Designed, implemented, and analyzed strategic and product -specific marketing research.  Developed and presented recommendations to senior management based on research findings, resulting in $20M in media efficiencies.  Recognized with Merck Leadership Award of Excellence for developing methodology to analyze the impact of changes in promotion mixes and budgets that decreased planning time and increased efficiencies by 33. Professional Sales Representative, Western Pennsylvania Territory Managed $4 million office and hospital sales territory. Conducted educational forums, and presented product and clinical information to nurses, pharmacists and physicians in primary care, cardiology, neurology , orthopedics and ophthalmology. (Achieved 124 of goal) EDUCATION  PROFESSIONAL DEVELOPMENT HARVARD UNIVERSITY GRADUATE SCHOOL OF BUSINESS ADMINISTRATION BOSTON, MA Master of Business Administration (MBA) - Concentration: Marketing  Finance Awarded Harvard Business School Fellowship WEST VIRGINIA UNIVERSITY MORGANTOWN, WV Bachelor of Science in Business Administration  Highest Honors (Summa Cum Laude)  Rhodes Scholarship Finalist Beta Gamma Sigma National Business Honor Society; Phi Kappa Phi  Mortar Board National Honor Societies; Phi Sigma Kappa Fraternity (Repeat Merit Scholar ); Varsity Track Team (400 metersrelay ). CONTINUING EDUCATION:  Harvard Medical School HMX Program (completed with Achievement Honors Pharmacology and Immunology; Genetics and Physiology in progress to complete 2019)  Villanova UniversityBucks County Community College  Completion of full pre-medhealth sciences program with labs (anatomychemistrybiology) -recognized with highest honors and selected for membership in Phi Theta Kappa  Harvard School of Public Health (TH Chan School) o Measurement, Design, and Analysis Methods for Health Economics Research o Navigating the US Bio-Pharmaceutical Sector  Policy, Politics, Pricing, and Payment o Leading in Health Systems: Integrating Effort, Improving Outcomes Page 3 of 4 Budget Justification Attachment Page 60 Contact PDPI: Harty, Ronald N. LE O J.AD ALBE RT  EDUCATION  PROFESSIONAL DEVELOPMENT (CONTINUED )  Northwestern University Graduate School of Management: Graduate of Kellogg Executive Scholar Program  Wharton School (University of Pennsylvania): Graduate of Professional Development Program  Center for Creative Leadership: Leadership Development Program  Dartmouth: Global Leaders Program  GAP International: Leadership for Breakthrough Outcomes  Harvard Law School: Negotiations Strategy Program  Merck  Company: Strategic Marketing Plan Development  MIT: Systems Thinking Program CERTIFICATIONS: Certified Licensing Professional  Certified Strategic Alliance Professional COMMUNITY  PERSONAL  Special Projects Leader  Castleman Disease Collaborative Network (CDCN) www.cdcn.org  Co-Creator  Executive Board member of Camden, NJ Dream Center  (youth community outreach center) www.camdendreamcenter.org  Graduate of Chester County non- profit Leadership Development Program (partnership with United Way)  Competitive masters track and field athlete (400 and 800 meters) and road cyclist Page 4 of 4 Budget Justification Attachment Page 61 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 1 of 20 Quotation Date: 10 August 2020 Quotation Number: APQ -19-0548 Sponsor Jay Wrobel, PhD Vice President, Academic Relations Fox Chase Chemical Diversity Center Doylestown, PA 18902 Phone: Mobile: E-mail: Alliance Pharma, Inc. Ryan Klein, PhD Director, Business Development Malvern, PA 19355 Mobile: Phone: Fax: Email: Web: Non-GLP ADME Studies of Compounds for Fox Chase Chemical Diversity Center (Three Year Plan) 1 Plasma Protein Binding Study By Rapid Equilibrium Dialysis 1.1 Scope of Service To assess the protein binding of compounds in mouse and human plasma using a rapid equilibrium dialysis (RED) device (Thermo Fisher Scientific). 1.2 General Procedures The test compounds will be spiked into plasma samples to achieve a final concentration of 5 µM. Triplicate aliquots of the spiked plasma will be added to the red chambers of the RED device inserts, and phosphate buffered saline will be added into the white chambers of the inserts. The RED device plate will be sealed and incubated for 4 hours at 37C with shaking. Warfarin, a known highly bound drug in plasma, will be tested as the positive control for this assay. After incubation, samples from both the red and white chambers will be analyzed by LC-MSMS. The percentages of free and bound fractions will be determined using the peak area ratios of each analyte to internal standard. 1.3 General Information and Sample Requirement  Molecular weights, including formula weights and exact mass units, of the compounds.  Sufficient amount of the compounds.  Stability information for the compounds such as light, temperature, and solvents, if available.  Safety information for handling the compounds.  Solubility of the compounds, if available. APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 65 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 2 of 20 1.4 Deliverable A Microsoft Excel summary report including raw data, methodology, calculated percent bound, and percent recovery values will be provided. 1.5 Turnaround Time Report will be delivered within 7 business days from receipt of the compounds and any additional items specific to the assay that need to be sourced externally. 1.6 DocumentationRecords Data from this non -GLP study will be preserved for 1 year after submission of the results to the Sponsor. 1.7 Fee Schedule The cost of plasma protein binding analysis is per compound. Item Year 1 Year 2 Year 3 Cost Human Plasma Binding Number of compounds 2 2 2 Cost Mouse Plasma Binding Number of compounds 1 1 1 Cost Total: APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 66 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 3 of 20 2 Metabolic Stability Studies (Intrinsic Clearance) in Liver Microsomes 2.1 Scope of Service To assess the relative metabolic stability (intrinsic clearance) of compounds using mouse, rat, canine and human liver microsomes. 2.2 General Procedures The test compounds at a concentration of 0.5 µM will be incubated with 0.5 mgmL of liver microsomes and a n NADPH- regenerating system (cofactor solution) in potassium phosphate buffer (pH 7.4). At 0, 5, 15, 30, and 45 minutes, an aliquot will be taken, and reactions will be quenched with an acetonitrile solution containing an internal standard. Additionally, controls will be measured that do not contain the cofactor solution. Following completion of the experiment, samples will be analyzed by LC -MSMS. Results will be reported as peak area ratios of each analyte to internal standard. The intrinsi c clearance (CL int) will be determined from the first- order elimination constant by nonlinear regression. 2.3 General Information and Sample Requirement  Molecular weights, including formula weights and exact mass units, of the compounds.  Sufficient amounts of the compounds.  Stability information for the compounds such as light, temperature, and solvents, if available.  Safety information for handling the compounds.  Solubility of the compounds, if available. 2.4 Deliverables A Microsoft Excel summary report of intrinsic clearance (CL int), half -life (t12), rate constant, peak area ratios of each analyte to internal standard, and graphic profiles will be provided. 2.5 Turnaround Time Report will be delivered within 7 business days from Sponsors approval of the study and receipt of all necessary information and assay -specific materialsreagents by Alliance Pharma. 2.6 DocumentationRecords Data from this non-GLP study will be preserved for 1 year after submission of the results to the Sponsor. APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 67 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 4 of 20 2.7 Fee Schedule The cost of metabolic stability analysis using liver microsomes is per compound. Item Year 1 Year 2 Year 3 Cost Microsomal Clearance (Mouse) Number of compounds 15 15 6 Cost Microsomal Clearance (Rat) Number of compounds 2 2 2 Cost Microsomal Clearance (Dog) Number of compounds 2 2 2 Cost Microsomal Clearance (Human) Number of compounds 6 6 10 Cost Total: APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 68 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 5 of 20 3 Direct Cytochrome P450 (CYP 3A4) Inhibition Study 3.1 Scope of Service To evaluate the direct inhibition potential of compounds on the major human hepatic CYP3A4 isoform using pooled human liver microsomes and the isoform -specific probe substrate testosterone. 3.2 General Procedures Pooled human liver microsomes will be used for this study. Incubation mixtures will contain a) potassium phosphate buffer, b) 0.1 mgmL microsomal protein; c) CYP3A4 probe substrate testosterone (12.5 μM); d) specific concentrations of test compounds, blank solvent or positive controls; and (e ) an NADPH- regenerating cofactor solution. The mixtures will be incubated at 37C and 5 CO 2 with orbital shaking at 200 rpm for 5 minutes. Upon completion of the incubation, the reactions will be quenched with an acetonitrile solution containing an internal standard. The samples will be prepared and analyzed by LC -MSMS to monitor for substrate metabolite formation. The enzyme activity in the presence of the compounds will be normalized with the enzyme activity in the absence of the compounds (controls) and expressed as a percentage of activity. The inhibitory potential (IC50) of the compounds will be determined using nonlinear regression. 3.3 General Information and Sample Requirement  Molecular weights, including formula weights and exact mass uni ts, of the compounds.  Sufficient amounts of the compounds.  Stability information for the compounds such as light, temperature, and solvents, if available.  Safety information for handling the compounds.  Solubility of the compounds, if available. 3.4 Deliverables A Microsoft Excel summary report including IC 50 summary tables and plots of test compound concentration versus peak area ratios will be provided. 3.5 Turnaround Time Report will be delivered within 8 business days from Sponsors approval of the study and receipt of all necessary information and assay -specific materialsreagents by Alliance Pharma. 3.6 DocumentationRecords Data from this non-GLP study will be preserved for 1 year after submission of the results to the Sponsor. APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 69 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 6 of 20 3.7 Fee Schedule The cost of CYP inhibition analysis is per compound, per probe substrate (one isoform - specific probe substrate used). Item Year 1 Year 2 Year 3 Cost Direct CYP IC50 (1 substrate) Number of compounds 6 6 4 Cost Total: APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 70 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 7 of 20 4 Direct Cytochrome P450 (CYP) Inhibition Study  Full Panel 4.1 Scope of Service To evaluate the direct inhibition potential of compounds on 7 major human hepatic cytochrome P450 (CYP) isoforms (1A2, 2B6, 2C8, 2C9, 2C19, 2D6 and 3A4) using pooled human liver microsomes and the isoform- specific probe substrates. 4.2 General Procedures Pooled human liver microsomes will be used for this study. The incubation mixtures will contain the following: ( a) potassium phosphate buffer; (b) 0.1 mgmL microsomal protein; (c) isoform -specific probe substrates; (d) specific concentrations of the test compounds, blank solvent, or positive controls (isoform -specific inhibitors); and (e ) an NADPH- regenerating cofactor solution. Table 1 below lists the inhibitors, substrates, and associated concentrations that will be used for each incubation. Table 1: Isoform -Specific Probe Substrates and Inhibitors CYP Isoform Inhibitor Final Concentrations (μM) Substrate Final Concentration (μM) 1A2 Fluvoxamine 0.0033, 0.001, 0.033, 0.1, 0.33, 1, 3.3 Phenacetin 35 2B6 Clopidogrel 0.033, 0.1, 0.33, 1, 3.3, 10, 33 Bupropion 50 2C8 Quercetin 0.033, 0.1, 0.33, 1, 3.3, 10, 33 Amodiaquine 1.5 2C9 Sulfaphenazole 0.0033, 0.01, 0.033, 0.1, 0.33, 1, 3.3 Diclofenac 5 2C19 Ticlopidine 0.033, 0.1, 0.33, 1, 3.3, 10, 33 S-mephenytoin 25 2D6 Quinidine 0.0033, 0.01, 0.033, 0.1, 0.33, 1, 3.3 Dextromethorphan 5 3A4 Ketoconazole 0.00033, 0.001, 0.0033, 0.01, 0.033, 0.1, 0.33 Midazolam 1.25 The mixtures will be incubated at 37C and 5 CO 2 with orbital shaking at 200 rpm for 5 minutes with reactions containing midazolam, for 10 minutes with reactions containing phenacetin, bupropion, diclofenac, or dextromethorphan, and for 20 minutes with reactions containing amodiaquine or S-mephenytoin. Upon completion of the incubation, the reaction s will be quenched with an acetonitrile solution containing an internal standard. The samples will be prepared and analyzed by LC-MSMS to monitor for substrate metabolite formation. The enzyme activity in the presence of the compounds will be normalized with the enzyme activity in the absence of the compounds (controls) and expressed as a percentage of activity. The inhibitory potential (IC 50) of the compounds will be determined using nonlinear regression. APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 71 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 8 of 20 4.3 General Information and Sample Requirement  Molecular weights, including formula weights and exact mass units, of the compounds.  Sufficient amounts of the compounds.  Stability information for the compounds such as light, temperature, and solvents, if available.  Safety information for handling the compounds.  Solubility of the compounds, if available. 4.4 Deliverables A Microsoft Excel summary report including IC50 summary tables and plots of test compound concentration versus peak area ratios will be provided. 4.5 Turnaround Time Report will be delivered within 8 business days from Sponsors approval of the study and receipt of all necessary information and assay -specific materialsreagents by Alliance Pharma. 4.6 DocumentationRecords Data from this non-GLP study will be preserved for 1 year after submission of the results to the Sponsor. 4.7 Fee Schedule The cost of the full-panel CYP inhibition assay is per compound ( per compound, per probe substrate). Item Year 1 Year 2 Year 3 Cost Direct CYP IC 50 (7 substrates) Number of compounds 2 2 2 Cost Total: APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 72 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 9 of 20 5 Plasma Stability Studies 5.1 Scope of Service To assess the relative stability of compounds in mouse and human plasma. 5.2 General Procedures Test compounds will be incubated at a concentration of 1 µM in plasma samples. At 0, 15, 30, 60 and 120 minutes, an aliquot will be removed and reactions will be quenched with an acetonitrile solution containing an internal standard. Following completion of the experiment, samples will be analyzed by LC-MSMS. Results will be reported as peak area ratios of analyte to internal standard. 5.3 General Information and Sample Requirement  Molecular weights, including formula weights and exact mass units, of the compounds.  Sufficient amounts of the compounds.  Stability information for the compounds such as light, temperature, and solvents, if available.  Safety information for handling the compounds.  Solubility of the compounds, if available. 5.4 Deliverables A Microsoft Excel summary report including half- life (t 12), elimination rate constant, peak area ratios of each analyte to internal standard, and graphic profiles will be provided. 5.5 Turnaround Time Report will be delivered within 6 business days from Sponsors approval of the study and receipt of all necessary information and assay -specific materialsreagents by Alliance Pharma. 5.6 DocumentationRecords Data from this non-GLP study will be preserved for 1 year after submission of the results to the Sponsor. APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 73 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 10 of 20 5.7 Fee Schedule The cost of plasma stability analysis is per compound. Item Year 1 Year 2 Year 3 Cost Mouse Plasma Stability Number of compounds 2 2 2 Cost Human Plasma Stability Number of compounds 2 2 1 Cost Total: APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 74 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 11 of 20 6 Aqueous Solubility Study in Phosphate -Buffered Saline 6.1 Scope of Service To assess the solubility of compounds in phosphate- buffered saline (PBS). 6.2 General Procedures Approximately 1 mL of PBS buffer (pH 7.4) will be added to approximately 10 mg of each test compound. The solutions will be sonicated for 30 minutes and vortexed at a low speed for at least 30 minutes. The solutions will then sit at room temperature for 16 -28 hours. After such time, the solutions will be filtered through a 0.22- micron filter followed by diluting 1:10, 1:100, and 1:1000, and 1:10,000 in triplicate in 50 acetonitrile in water (1:1, vv). The final sample solutions will be mixed with an internal standard solution (1:1) followed by LC MSMS analysis. Solubility will be determined according to the calibration curve generated from 5 concentration standards. 6.3 General Information and Sample Requirement  Molecular weights, including formula weights and exact mass units, of the compounds.  Approximately 10 mg of each compound (minimum requirement).  Stability information for the compounds such as light, temperature, and solvents, if available.  Safety information for handling the compounds.  Solubility of the compounds, if available. 6.4 Deliverables A Microsoft Excel summary report including standard curve information and sample solubility data will be provided. 6.5 Turnaround Time Report will be delivered within 7 business days from receipt of compounds by Alliance Pharma. 6.6 DocumentationRecords Data from this non-GLP study will be preserved for 1 year after submission of the report to the Sponsor. APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 75 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 12 of 20 6.7 Fee Schedule The cost of aqueous solubility analysis is per compound. Item Year 1 Year 2 Year 3 Cost Aqueous Solubility Number of compounds 6 6 4 Cost Total: APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 76 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 13 of 20 7 Permeability in MDCKII -MDR1 cells 7.1 Scope of Service To assess the permeability of compounds in MDCKII -MDR1 cells. 7.2 General Procedures The permeability of the test compound in the MDCKII -MDR1 cell monolayer system will be assessed in duplicate in both apical to basolateral (AB) and basolateral to apical (BA) directions. MDCK cells will be seeded onto multiwell permeable supports in culture medium. The permeability study will be conducted approximately 4 days post -seeding. The Transepithelial Electrical Resistance (TEER) of the cell monolayer will serve as a quality contro l for monolayer integrity. On the day of the transport assay, TEER will be measured in all wells. Transport medium will be added to the basolateral and apical sides of the system for a 30-minute incubation to wash the monolayer. After removing transport medium, receiving solution and compound- specific dosing solution (containing Lucifer yellow) will be added to the appropriate wells of the system for AB and BA analysis. The multiwell system will be incubated at 37C with 5 CO 2. After incubation, samples will be collected from both the apical and basolateral sides of the system for LC -MSMS analysis. Lucifer yellow will be fluorometrically determined in all receiving samples and will serve as an in -well quality control for monolayer integrity. 7.3 General Information and Sample Requirements Sponsor shall provide the following:  Molecular weights, including formula weights and exact mass units, of the compounds.  Sufficient amount of each compound.  Stability information (e.g., light, temperature, and solvents) for the compounds, if available.  Safety information for handling the compounds.  Solubility of the compounds, if available. 7.4 Deliverables A spreadsheet summary report (Microsoft Excel file) will be provided that includes the permeability coefficient of the compound from basolateral side to apical side ( Papp, B to A) and from apical side to basolateral side ( Papp, A to B ). 7.5 Turnaround Time The spreadsheet summary report will be delivered to the Sponsor within approximately 11 business days from Sponsors approval of the study and receipt of all necessary information and assay - specific materialsreagents by Alliance Pharma. APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 77 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 14 of 20 7.6 DocumentationRecords Data from this non-GLP study will be preserved for 1 year after submission of the results to the Sponsor. 7.7 Fee Schedule The cost of MDCKII -MDR1 permeability analysis is per compound. Item Year 1 Year 2 Year 3 Cost Permeability Number of compounds 2 2 1 Cost Total: APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 78 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 15 of 20 8 Non-GLP in vivo Pharmacokinetic Studies in BalbC Mice by PO Administration 8.1 Scope of Service To provide the services of in- life animal dosing and sample bioanalysis for pharmacokinetic studies conducted in mice. 8.2 General Procedures One group of 21 mice (n3 mice per time point) will be administered an oral (PO) dose of the compound of interest. Dos ing formulation will be prepared according to Sponsor -provided protocol. Terminal blood plasma samples will be collected from each animal at 7 established time points within a 24 hour period. After all study samples are collected, the samples will be analyzed by LC-MSMS. Calculated concentrations will be reported to the Sponsor. 8.3 General Information and Sample Requirement  Predicted concentration range andor dosing, C max information, if available  Compound molecular weight and its purity information  Sufficient amount of standard compound  Sample stability information, such as light sensitivity, temperature and solvent, if available  Safety information for handling compoundssamples  Solubility of the compound, if available 8.4 Deliverables A report of abnormal in-life observations will be provided to the Sponsor, if applicable. A Microsoft Excel spreadsheet summary of sample analysis results will be delivered to the Sponsor. 8.5 Turnaround Time The pharmacokinetic study will be completed and results will be delivered within approximately 10 business days from receipt of the test compound and all other necessary study materials. 8.6 Documentation and Records Data from this non-GLP study will be preser ved for 1 year after submission of the report to the Sponsor. 8.7 Fee Schedule The cost of the in vivo pharmacokinetic study (dosing and bioanalysis) is per compound. Item Year 1 Year 2 Year 3 Cost In Vivo PO PK in Mice Number of compounds 1 1 1 Cost Total: APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 79 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 16 of 20 9 Non-GLP in vivo Pharmacokinetic Studies in BalbC Mice by IVPO Administration 9.1 Scope of Service To provide the services of in- life animal dosing and sample bioanalysis for pharmacokinetic studies conducted in mice. 9.2 General Procedures Two groups of 21 mice (n3 mice per time point) will be administered an intravenous (IV) or oral (PO) dose of the compound of interest. Dosing formulation will be prepared according to Sponsor- provided protocol. Terminal blood plasma samples will be collected from each animal at 7 established time points within a 24 hour period. After all study samples are collected, the samples will be analyzed by LC- MSMS. Calculated concentrations will be reported to the Sponsor. 9.3 General Information and Sample Requirement  Predicted concentration range andor dosing, C max information, if available  Compound molecular weight and its purity information  Sufficient amount of standard compound  Sample stability information, such as light sensitivity, temperature and solvent, if available  Safety information for handling compoundssamples  Solubility of the compound, if available 9.4 Deliverables A report of abnormal in-life observations will be provided to the Sponsor, if applicable. A Microsoft Excel spreadsheet summary of sample analysis results will be delivered to the Sponsor. 9.5 Turnaround Time The pharmacokinetic study will be completed and results will be delivered within approximately 10 business days from receipt of the test compound and all other necessary study materials. 9.6 Documentation and Records Data from this non-GLP study will be preser ved for 1 year after submission of the report to the Sponsor. APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 80 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 17 of 20 9.7 Fee Schedule The cost of the in vivo pharmacokinetic study (dosing and bioanalysis) is per compound. Item Year 1 Year 2 Year 3 Cost In Vivo IVPO PK in Mice Number of compounds 6 6 6 Cost Total: Total: APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 81 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 18 of 20 10 Total Fee Schedule Assay Year 1 Year 2 Year 3 1. Plasma Protein Binding 2. Microsomal Clearance 3. Direct CYP IC50 (3A4 - Testosterone) 4. Direct CYP IC50 (1A2, 2B6, 2C8, 2C9, 2C19, 2D6 and 3A4-Midazolam) 5. Plasma Stability 6. Aqueous Solubility 7. Permeability 8. In Vivo PK in Mice (PO) 9. In Vivo PK in Mice (IVPO) Annual Total Grand Total APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 82 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 19 of 20 TERMS AND CONDITIONS 1. SERVICES 1.1 Compliance  Alliance Pharma shall perform itself or, as the case may be, through its affiliate(s) for the benefit of Sponsor, the Services described herein in accordance with, as applicable: (i) this Proposal; (ii) applicable standard operating procedures; (iii) curre nt standards of Good Laboratory PracticeGood Clinical Practice (GLPGCP) as applicable in the country where the Services are performed; and (iv) any other applicable professional standards, codes, guidelines, laws, rules, and regulations, including, without limitation, those of any regulatory authority or agency having jurisdiction over the Services (collectively, the  Applicable Laws ). Performance of such Services shall only be initiated after the full execution of this Proposal . 1.2 Mutual non-solicitation  During the term of this Proposal and for a two (2) year period thereafter, neither party nor its respective affiliates shall, directly or indirectly: (i) recruit, solicit or induce, or attempt to recruit, solicit or induce, any employee of the other party to terminate his or her employment or engagement with the other party, or (ii) hire any employee of the other party (including, withou t limitation, employees of Alliance Pharma that are providing services to Sponsor on behalf of Alliance Pharma at the time of hiring). N OTE.This Proposal is valid for sixty (60) calendar days from quotation date. 2. CHEMICAL AND BIOLOGICAL SAMPLES 2.1 Delivery  Sponsor shall deliver to Alliance Pharma, at Sponsors risk and expense and at the location of Alliance Pharma identified in this Proposal or to such other location of Alliance Pharma as identified to Sponsor in writing, the Chemical Samples  and Biological Samples  in sufficient and necessary quantities for Alliance Pharma to perform the Services in compliance with the provisions of Section 1.1 above and as required under the Applicable Laws, along with instructions for the handling, storage, and safety requirements and conditions of such Chemical or Biological Samples. Alliance Pharma will receive shipment(s) of Chemical or Biological Samples only if the equipment required to safely process the samples exists at Alliance Pharma. 2.2 Restricted Use  Alliance Pharma shall neither use nor analyze any Chemical or Biological Samples for any purposes other than to perform the Services without Sponsors prior written consent. 2.3 Storage and Disposal of Chemical Samples  Alliance Pharma will store the Chemical Samples at the appropriate storage conditions as directed by the Sponsor for 3 months after submission of the final deliverable hereunder. Alliance Pharma will dispose of the Chemical Samples following the 3- month expiration date, unless requested otherwise by the Sponsor in writing prior to said date. 2.4 Storage, Return or Destruction of Biological Samples  For regulatory (e.g., GLPGCP) studie s, Alliance Pharma will store the Biological Samples at the appropriate storage conditions as directed by the Sponsor for 6 months after submission of the final service deliverable herein. Alliance Pharma will continue to store the Biological Samples following the 6- month expiration date at the rate of US $0.50 per Biological Sample per month, unless otherwise directed by the Sponsor in writing prior to said date. Sponsor will retain the option of having the Biological Samples destroyed, store d under Alliance Pharmas responsibility, or returned to Sponsor at Sponsors expense. An authorization request form will be sent to Sponsor at sponsors request. For non- regulated studies, Alliance Pharma will destroy Biological Samples two (2) weeks after the Sponsor acknowledges receipt of final Service deliverables unless otherwise requested by the Sponsor in writing prior to such date. If Biological Samples are received by Alliance Pharma that are not intended for bioanalysis under the scope of this Proposal, such as back up samples, the Sponsor will be contacted and offered the option of having the Biological Samples destroyed, stored under Alliance Pharmas responsibility, or returned to the Sponsor at Sponsors expense. If stored under Alliance Pharmas responsibility, the samples, either regulatory or non -regulatory, will be stored and invoiced at the rate of US $0.50 per Biological Sample per month. All sample storage fees are based on standard 1.2  2 mL cryogenic storage vials with a maximum height of 49 mm and diameter of 13.5 mm. Larger sample containers may incur additional fees to be agreed upon between the parties to this Proposal. APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 83 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 20 of 20 3. BUDGET AND PAYMENT TERMS 3.1 Service Fees  In accordance with the following, Sponsor shall provide Alliance Pharma with a payment amount corresponding to the Service fees identified herein. The per -sample pricing provided in this Proposal is based on the number of samples requested by the Sponsor. Pricing may be renegotiated if the sample count decreases from the quoted count by more than 20. Sample re -assaysrepeats requested by Sponsor, dilutional rea -assays, and incurred sample reanalysis will be invoiced at the sample unit rate. Re-assaysrepeats due to failure to meet the pre-defined criteria will be at no charge to Sponsor. However, any special requirements from the Sponsor not included in such budget will be invoiced to the Sponsor at additional cost. 3.2 Pass- Through Costs  Pricing includes all standard laboratory consumables; however, the cost of non- standard items (e.g. chromatographic columns, reference standards, internal standards, SPE plates, commercial ELISA kits, enzymes, specific reagents etc.) purchased by Alliance Pharma will be passed -through to the study sponsor. The fees identified herein may include estimated costs that are expected at the start of the study; however, additional materials may be required. All pass -through costs are subject to a mark -up fee for purchasing, handling and accounting services. 3.3 Installments -Alliance Pharma will invoice upon delivery of the first draft report for each study . (Any additional authorized dollars from contra ct modifications, if any, will be added to the next billing cycle.) 3.4 Payment Terms  Sponsor shall pay each invoice within thirty (30) days of receipt thereof. Any amount owed by Sponsor under this Proposal that is not received by Alliance Pharma on or befo re the date due shall bear interest at a per annum rate of two percent (2) above the prime rate in effect at Bank of America on the date due. Sponsor shall also pay all reasonable collection costs at any time incurred by Alliance Pharma in obtaining payme nt of amounts past due, including reasonable attorneys fees. If any portion of an invoice is disputed, the Sponsor shall pay the undisputed amounts as set forth therein, and the parties shall use good faith efforts to reconcile the disputed portion of the invoice within thirty (30) days of receipt by Sponsor of the applicable invoice. IN WITNESS WHEREOF, this Agreement has been accepted and executed by duly authorized representatives of both parties as follows: Alliance Pharma Inc. SPONSOR: Fox Chase Chemical Diversity Center By: By: Print Name: Ryan Klein, PhD Print Name: Title Director, Business Development Title: Date: Date: APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 84 Turnaround Time. link) for more details. Contact PDPI: Harty, Ronald N. Quotation Customer Delivery Information Customer Invoice Information Fox Chase Chemical Diversity Center Inc Fox Chase Chemical Diversity Center Inc Doylestown, PA 18902, United States 2, Doylestown, PA 18902, United States Name: Accounts Payable Name: Jay Wrobel Phone: Phone: Email: Email: Quotation Number Study Code Customer Account No Eurofins Contact Andrew Lord Date Issued Customer reference Payment Terms Eurofins Contact Phone 04-Aug- 2020 30 Days Quote Valid Until Promo Code Loyalty Category Eurofins Contact Email 03-Sep- 2020 Standard Summary Price Discount Charges Net Price Sales tax Grand Total Price Currency 0.00 0.00 0.00 USD Details Item  Description Assay Cmpd Conc Rep Qty Option TAT Unit price Disc unit Net Price Mode price PP241 SafetyScreen44 Panel, Panlabs Flexible 1 1 2 1.00 Standard 10 Special Requirement: Default test concentrations 10 uM Cmpd  compounds; Conc  Concentrations; Rep  Replicates; Qty  Quantity; TAT  please see Delivery Dates, Turnaround Time section of Terms and Conditions (web 0.00 Additional Charges Special Instructions : NA Orders can be placed by purchase order or credit card. Purchase orders should detail the following company address: Eurofins Panlabs Discovery Services Taiwan Ltd New Taipei City, Taiwan 24891 Please note that the address for sending check payment is as following: Birmingham, AL 35246- 5588 Please note that if arranging payment from a bank outside of Taiwan, any intermediary or senders charges should be borne by the sender. AUTHORIZATION AND ACCEPTANCE Submitting a purchase order with reference to this quotation number or sending a scanned copy of the signed quotation to the address stated below, is considered an acceptance of prices and all other special conditions listed on the quotation. Client Services Email: Fax: By authorization and acceptance of this quotation as indicated above, authorizes Eurofins to perform the work as described in this quotation, subject to the terms and conditions of the MSA by and between the Client and Eurofins or its affiliate, if any, or if none, or in the event that the MSA is not active or found to be invalid as of the date hereof, then the work described in this quotation shall be subject to the then current version of the Eurofins General Terms  Conditions of Sale of Products and Services, as amended from time to time, which can be viewed at the following link: https:www.eurofinsdiscoveryservices.comcmscms-contentmisctcorder-terms-condition-products-and- services-dec19. This quotation constitutes a separate and distinct contract between Client and Eurofins. In no event shall an affiliate of Eurofins be liable for its obligations under this quotation, and Client shall look exclusively to Eurofins in relation to any rights or remedies it may have with respect to this quotation. Eurofins Panlabs Discovery Services Taiwan, Ltd. Wugu District, New Taipei City, 24891, Taiwan Phone Fax Page 1 ofBudget Justification Attachment 2 Contact PDPI: Harty, Ronald N. Quotation AGREED TO AND ACCCEPTED For: Fox Chase Chemical Diversity Center Inc By: Name: Title: Date: Eurofins Panlabs Discovery Services Taiwan, Ltd. Wugu District, New Taipei City, 24891, Taiwan Phone  Fax  Budget Justification Attachment www.eurofins Pdiascgoeve9ry3services.com Page 2 of 2 Contact PDPI: Harty, Ronald N. Quotation Customer Delivery Information Custo mer Invoice Information Fox Chase Chemical Diversity Center Inc Fox Chase Chemical Diversity Center Inc 3805 Old Easton Road Doylestown, PA 18902, United States PA 18902, United States Name: Accounts Payable Name: Jay Wrobel Phone: Phone: Email: Email: Quotation Number Study Code Customer Account No Eurofins Contact Andrew Lord Date Issued Customer reference Payment Terms Eurofins Contact Phone 04-Aug- 2020 30 Days Quote Valid Until Promo Code Loyalty Category Eurofins Contact Email 03-Sep- 2020 Standard Summary Price Discount 0.00 Charges 0.00 Net Price Details Sales tax 0.00 Grand Total Price Item  Description Assay Mode Cmpd Conc Rep Qty Option TAT Unit price Disc unit price Currency USD Net Price PP223 SafetyScreen87 Panel, Panlabs Flexible 1 1 2 1.00 Standard 10 Special Requirement: Default test concentrations 10 uM Cmpd  compounds; Conc  Concentrations; Rep  Replicates; Qty  Quantity; TAT  Turnaround Time. please see Delivery Dates, Turnaround Time section of Terms and Conditions (web link) for more details. 0.00 Additional Charges Special Instructions : NA Orders can be placed by purchase order or credit card. Purchase orders should detail the following company address: Eurofins Panlabs Discovery Services Taiwan Ltd New Taipei City, Taiwan 24891 Please note that the address for sending check payment is as following: Birmingham, AL 35246- 5588 Please note that if arranging payment from a bank outside of Taiwan, any intermediary or senders charges should be borne by the sender. AUTHORIZATION AND ACCEPTANCE Submitting a purchase order with reference to this quotation number or sending a scanned copy of the signed quotation to the address stated below, is considered an acceptance of prices and all other special conditions listed on the quotation. Client Services Email: Fax: By authorization and acceptance of this quotation as indicated above, authorizes Eurofins to perform the work as described in this quotation, subject to the terms and conditions of the MSA by and between the Client and Eurofins or its affiliate, if any, or if none, or in the event that the MSA is not active or found to be invalid as of the date hereof, then the work described in this quotation shall be subject to the then current version of the Eurofins General Terms  Conditions of Sale of Products and Services, as amended from time to time, which can be viewed at the following link: https:www.eurofinsdiscoveryservices.comcmscms-contentmisctcorder-terms-condition-products-and- services-dec19. This quotation constitutes a separate and distinct contract between Client and Eurofins. In no event shall an affiliate of Eurofins be liable for its obligations under this quotation, and Client shall look exclusively to Eurofins in relation to any rights or remedies it may have with respect to this quotation. Eurofins Panlabs Discovery Services Taiwan, Ltd. New Taipei City, 24891, Taiwan Phone Fax www.eurofins Pdiascgoeve9ry4services.com Page 1 of 2Budget Justification Attachment Contact PDPI: Harty, Ronald N. Quotation AGREED TO AND ACCCEPTED For: Fox Chase Chemical Diversity Center Inc By: Name: Title: Date: Eurofins Panlabs Discovery Services Taiwan, Ltd. New Taipei City, 24891, Taiwan Phone Fax Budget Justification Attachment www.eurofins Pdiascgoeve9ry5services.com Page 2 of 2 Contact PDPI: Harty, Ronald N. SUBAWARD BUDGET JUSTIFICATION SeniorKey Personnel Ronald Harty, PhD , PI (25 effort  3.0 ca mo). Dr. Harty is a Professor in the Department of Pathobiology at the University of Pennsylvania School of Veterinary Medicine and a member of the Cell and Molecular Biology Graduate Group. His laboratory is focused on the molecular events that lead to virus assembly and budding and has developed models of virus budding that recapitulate biological events occurring during infection with a number of Category A pathogens. Dr. Hartys laboratory utilizes a wide-array of techniques for these studies such as, virus -like particle (VLP) budding assays, reverse-genetics to generate VSV recombinants, bimolecular complementation, ubiquitination and ISGylation assays, and fluorescence lifetime imaging microscopy (FLIM). Dr. Harty will direct studies to test lead compounds for their ability to block VLP egress from mammalian cells, block the specific virus -host interaction targeted by the lead compound using BiMC, and block budding of live virus using a surrogate model of filovirus and arenavirus infection. Dr. Harty will be responsible for the overall conduct of this study in the role of PI, dedicating 25 effort to the project and being responsible for communicationjoint lab meetings with the Fox Chase Chemical Diversity Center and Texas Biomedical Research Institute to track the progress of the project. Other Personnel Ziying Han, MD, Ph.D . (100 effort  12.0 ca mo). Budget is requested to support Research Specialist Dr. Han, who has provided all of the BSL2 preliminary virolog y and VLP budding data. Dr. Han will be responsible for maintaining the tissue culture models needed for the experiments and for all molecular biological techniques necessary for testing lead compounds for their ability to block VLP and live virus egress and specific virus-host interactions in live mammalian cells using the BiMC assay. Other direct costs: (Year 1), year (Year 2-3) Materials and supplies: is requested in Year 1 to cover the cost of consumables and reagents necessary for the tissue culture and molecular virology studies associated with this project. These funds will also cover the costs of equipment or facility rentaluser fees associated with using the University of Pennsylvania Microscopy Core facility. The imaging core operates on a fee for use cost recovery system that is heavily subsidized by departments and centers in the School of Veterinary Medicine. All users, regardless of their affiliation are charged the same fee for using the Leica SP5 confocal systems, which is hr. This fee helps to defray maintenance and operation costs of the core. Fringe Benefits: Fringe benefits have been requested for the salaries in this budget at the DHHS negotiated rate, which is currently 29.5 Indirect Costs: (Year 1), year (Year 2-3) A total modified direct cost rate has been requested in this budget and is the DHHS negotiated rate. It is currently 62 through June 30, 2021 and then it changes to 62.5 starting July 1, 2020. Budget Justification Attachment Page 106 Contact PDPI: Harty, Ronald N. BUDGET JUSTIFICATION Texas Biomedical Research Institute PERSONNEL Olena Shtanko, Ph.D., Subaward Principal Investigator (2 calendar monthsyear): Dr. Olena Shtanko has extensive experience in researching and handling BSL4 agents, including Ebola, Marburg, and Lassa viruses, both in vitro and a mouse model of filovirus disease, as evidenced by her publications in the field. Dr. Shtanko is fully authorized to work with select agents and has the appropriate Homeland Security and CDC approvals for this work. Dr. Shtanko will be responsible for the overall experimental design and execution of the project at Texas Biomed. She will supervise the conduct of all studies at Texas Biomed, participate in review and interpretation of the data, and prepare all reports and communications with Dr. Harty. In addition, she will perform certain infection and all imaging assays. Gloria Rodriguez, B.S., Research Assistant (4 calendar monthsyear): Gloria Rodriguez, B.S., has training and experience in the cell culture and molecular biology experiments to be executed in the BSL2 setting. She is also fully authorized and experienced to work with replication -competent wild-type viruses in the Texas Biomed BSL4 facility and has been doing so for 5 years. Gloria Rodriguez will help with the extensive cell culture, qPCR, and compound treatments in the BSL2 facility, and will maintain supplies and reagents. She will also perform virus expansion, plaque assays, infection assays, virus inactivation and any other virus -associated work. SUPPLIES A total amount of is budgeted to cover costs of cell culture media, media supplements, culture vessels, and reagents for cell maintenance, donor recruitment to obtain peripheral blood cells, generation and characterization of virus stocks, testing compounds for antiviral activity and cytotoxicity, and plaque assays; molecular biology reagents; antibodies and materials associated with immunofluorescence assays. OTHER DIRECT COSTS In vivo toxicity studies We request a total of for studies aimed to access toxicity of compound treatments in a mouse model. These costs include purchase and shipment of 540 4-5-week old BALBc mice of either sex from the Jackson Laboratories , housing fees at Texas Biomed ABSL2 Vivarium, therapeutic dosing, necropsy, gross pathology, histopathology, required supplies and reagents, and veterinary personnel effort. Budget Justification Attachment Page 117 Contact PDPI: Harty, Ronald N. In vivo compound accumulation studies We request a total of for studies to determine the concentration and accumulation of compounds over time in blood and organs of treated mice. These costs include purchase and shipment of 360 BALBc mice from the Jackson Laboratories , housing fees at Texas Biomed Vivarium, therapeutic dosing, blood and organ harvesting, isolation of plasma from blood, required supplies and reagents, and veterinary personnel effort. A total amount of is requested to ship animal samples to an external source for analysis of compound concentration. ABSL4 efficacy studies using a mouse model of Marburg virus disease Animal procurement (a total of ) is budgeted to purchase and ship 180 4-5-week old BALBc mice of either sex from the Jackson Laboratories. Efficacy studies (a total of ) cover animal housing fees, virus challenge, therapeutic dosing, necropsy, required supplies and reagents, and veterinary personnel effort. The indirect cost rate is the Texas Biomed Federal MTDC Rate of 98. Fringe is calculated at 29.51 according to our federally negotiated rate. Budget Justification Attachment Page 118 Contact PDPI: Harty, Ronald N. Total Direct Costs less Consortium FA NIH policy (NOT -OD-05-004) allows applicants to exclude consortiumcontractual FA costs when determining if an application falls at or beneath any applicable direct cost limit. When a direct cost limit is specified in an FOA, the following table can be used to determine if your application falls within that limit. Categories Budget Period 1 Budget Period 2 Budget Period 3 Budget Period 4 Budget Period 5 TOTALS Total Direct Costs less Consortium FA 0 0 Page 120 Tracking Number: GRANT13199760 Funding Opportunity Number: PA -20-265. Received Date: 2020 -09-08T12:43:45.000 -04:00 Contact PDPI: Harty, Ronald N. OMB Number: 4040- 0010 Expiration date: 12312022 SBIRSTTR Information Agency to which you are applying (select only one)  DOE  HHS  USDA  Other: SBC Control ID: Program Type (select only one)  SBIR  STTR  Both (See agency-specific instructions to determine whether a particular agency allows a single submission for both SBIR and STTR) Application Type (select only one)  Phase I  Phase II  Fast-Track  Direct Phase II  Phase IIA  Phase IIB  Phase IIC  Commercialization Readiness Program (See agency-specific instructions to determine application type participation.) Phase I Letter of Intent Number:  Agency TopicSubtopic: Questions 1-7 must be completed by all SBIR and STTR Applicants: 1a. Do you certify that at the time of award your organization will meet the eligibility criteria for a small business as  Yes  No defined in the funding opportunity announcement 1b. Anticipated Number of personnel to be employed at your organization at the time of award. 28 1c. Is your small business majority owned by venture capital operating companies, hedge funds, or private equity  Yes  No firms 1d. Is your small business a Faculty or Student -Owned entity  Yes  No 2. Does this application include subcontracts with Federal laboratories or any other Federal Government agencies  Yes  No If yes, insert the names of the Federal laboratoriesagencies: 3. Are you located in a HUBZone To find out if your business is in a HUBZone, use the mapping utility provided by  Yes  No the Small Business Administration at its web site: http:www.sba.gov  4. Will all research and development on the project be performed in its entirety in the United States  Yes  No If no, provide an explanation in an attached file. Explanation: 5. Has the applicant andor Program DirectorPrincipal Investigator submitted proposals for essentially equivalent  Yes  No work under other Federal program solicitations or received other Federal awards for essentially equivalent work If yes, insert the names of the other Federal agencies: 6. Disclosure Permission Statement: If this application does not result in an award, is the Government permitted to  Yes  No disclose the title of your proposed project, and the name, address, telephone number and email address of the official signing for the applicant organization to state-level economic development organizations that may be interested in contacting you for further information (e.g., possible collaborations, investment) 7.Commercialization Plan: The following applications require a Commercialization Plan: Phase I (DOE only), Phase II (all agencies), Phase III Fast-Track (all agencies). Include a Commercialization Plan in accordance with the agency announcement andor agency - specific instructions. Attach File: Commercialization_Plan_Harty_Final.pdf Page 121 Tracking Number: GRANT13199760 Funding Opportunity Number: PA-20-265. Received Date: 2020-09-08T12:43:45.000- 04:00 Contact PDPI: Harty, Ronald N. OMB Number: 4040- 0001 Expiration date: 10312019 SBIRSTTR Information SBIR-Specific Questions: Questions 8 and 9 apply only to SBIR applications. If you are submitting ONLY an STTR application, leave questions 8 and 9 blank and proceed to question 10. 8. Have you received SBIR Phase II awards from the Federal Government If yes, provide a company  Yes  No commercialization history in accordance with agency -spec ific instructions using this attachment. Attach File: 9. Will the Project DirectorPrincipal Investigator have hisher primary employment with the small business at the time  Yes  No of award STTR- Specific Questions: Questions 10 -12 apply only to STTR applications. If you are submitting ONLY an SBIR application, leave questions 10 -12 blank. 10. Please indicate whether the answer to BOTH of the following questions is TRUE:  Yes  No (1) Does the Project DirectorPrincipal Investigator have a formal appointment or commitment either with the small business directly (as an employee or a contractor) OR as an employee of the Research Institution, which in turn has made a commitment to the small business through the STTR application process; AND (2) Will the Project DirectorPrincipal Investigator devote at least 10 effort to the proposed project 11. In the joint research and development proposed in this project, does the small business perform at least 40 of  Yes  No the work and the research institution named in the application perform at least 30 of the work 12. Provide DUNS Number of non- profit research partner for STTR. Page 122 Tracking Number: GRANT13199760 Funding Opportunity Number: PA-20-265. Received Date: 2020-09-08T12:43:45.000- 04:00 Contact PDPI: Harty, Ronald N. Development of Small Molecule Therapeutics against RNA Viruses Fox Chase Chemical Diversity Center, Inc. Commercialization Plan We have developed small molecule leads that impede the progression and transmission of RNA viruses including Marburg and Ebola viruses, both potential bioterrorism agents and an area of focus for National Institute of Allergy and Infectious Diseases (NIAID) small business funding. In addition, other viruses that may be therapeutic targets of our agents include Lassa and Human T Cell Leukemia virus. A therapeutic agent arising from our efforts will be taken orally alone or in combination with other agents by persons infected with these RNA viruses, or taken as prophylactic agents for individuals deemed to be in high risk situations such as military or healthcare workers. Our current lead candidate FC-10696 was shown to be a potent inhibitor of virus budding (egress) and transmission in vitro and is active in vivo (mouse). Its mechanism is novel (inhibition of the viral matrix protein interaction with mammalian target cell proteins). In this Phase II STTR application we plan to fully characterize preclinical candidates suitable for IND enabling toxicology studies. We present here our Commercialization Plan from our joint program of study with researchers at University of Pennsylvania and the Texas Biomedical Research Institute, covering the topics listed in the guidelines for STTR submissions. A significant portion of this report was taken from a proprietary Technology Niche Analysis supported by the NIH and prepared for FCCDC by Bruce Inwood of Foresight Science and Technology. 1. Value of the STTR Project, Expected Outcomes and Impact. We review here our key technology objectives for this STTR, commercial need for our technology, our competitive advantage relative to standard of care, and the impact of this technology upon drug discovery and society as a whole. 1.1 Key technology objectives, therapeutic benefit, commercial need and competitive advantage relative to current standard of care of the STTR project. Marburg and Ebola viruses cause hemorrhagic diseases that afflict humans and nonhuman primates. Our proposed drug candidate will be a novel small molecule oral agent used alone or in conjunction with other antiviral agents for individuals infected with filoviruses Marburg and Ebola or other RNA viruses, including arenaviruses (Lassa), which can cause severe hemorrhagic fever syndromes in humans with high mortality rates, or rhabdoviruses (Rabies). Viral hemorrhagic fevers (Ebola, Lassa fever, etc.) are a focus for NIAID SBIRSTTR research and funding (Omnibus Solicitation for SBIRs and STTRs, PHS 2020-, p. 34). These viruses depend on an interaction between their matrix protein PPxY motif and select host proteins for efficient virus egress and spread of infection. Thus, drugs that block or disrupt viral -host interactions required for virus budding are expected to effectively block virus transmission and disease progression. The 2014-2015 outbreak of Ebola in western Africa resulted in over 28,000 infected individuals and over 11,000 deaths (WHO: Ebola Situation Report 2015). More recently the Democratic Republic of the Congo has second largest Ebola epidemic on record, which is ongoing, and as of June, 2020, with more than 2,200 lives lost, 3,400 confirmed infections and 1,160 survivors since the outbreak was declared on 1 August 2018. These unprecedented epidemics have spurred a call to action to discover new, cost effect therapies that combat this deadly pathogen. Among these efforts are several vaccines and antiviral drug candidates. However, the current vaccines are not a complete defense since they must be given pre-exposure. Oral antiviral agents used alone or in combination may be of value for individuals who respond adversely to the vaccine, and could be of value as prophylactic agents for individuals deemed to be in high risk situations such as military or healthcare workers. Therefore, effective therapeutics are needed to safeguard the largely immunologically naive human population by providing immediate protection. We have discovered two novel series of small molecule early leads that have nanomolar potency at inhibiting RNA virus budding and are Commercialization Plan Page 123 Contact PDPI: Harty, Ronald N. effective at reducing spread of live viruses including Marburg and Ebola virus. We are currently pursuing both series in order to increase our chance of success. Our most advanced lead is compound, FC -10696 and close structural analogs have the following attributes, tested to the extent afforded by the Phase I level of funding:  Potent activity (IC 50  100 nM) in the Marburg and Ebola VLP cellular budding (egress) assays,  Evidence that analogs of FC -10696 directly target the host cell and virus molecular machinery that effect viral cellular egress,  Live virus activity against RNA viruses (Marburg, Ebola),  Metabolic stability in mouse and human liver microsomes,  No cytochrome P450 3A4 activity which translates to a lower risk of drug- drug interactions,  Good drug levels upon IP administration in mice, and  Proof of concept in vivo reductions of Marburg virus (IP administration) in mouse models. However, it is important to note that improved analogs and additional novel composition of matter are expected to be discovered during the course of the Phase II STTR period of study, with fully characterized ADME properties suitable as antiviral oral drug candidates for preclinical development. 1.2 Expected outcomes: path to commercialization including timelines. The goal of this STTR Phase II application is to produce one or more preclinical drug candidates by medicinal chemistry synthesis optimization strategies that would be ready for GMP scale up and IND enabling studies. At the end of this award period we would have one or more drug candidates that are:  Potent, orally available inhibitors of RNA virus egress, due to inhibition of the viral matrix protein PPxY interaction with cellular Nedd4,  Broad spectrum activity agains t RNA viruses (Ebola, Marburg, Ebola, Lassa, RSV, etc).  Are not cytotoxic at concentrations 20-fold or higher than the live virus assay IC50,  Have suitable ADME and PK properties in mice for once a day oral administration,  Have oral activity in a mouse antiviral assay,  Have little or no cytochrome P450 inhibitory (Cyp) activity against the seven Cyp isoforms most likely to cause exposure variability and drug- drug interactions (DDIs),  Have little or no activity in a manual patch clamp assay to assess hERG (human Ether-à-go-go-Related Gene product), suggesting a greatly lower risk of the compound causing cardiac arrhythmias,  Have little or no activity in an Ames testing with and without metabolic activation, suggesting a greatly lower risk of the compound being mutagenic,  Have chemical syntheses that are efficient and scalable, and  Provide the basis for one or more composition of matter U.S. patent applications. We anticipate being in an ideal position to pursue IND enabling studies with our candidate compound and will require additional funding to support IND filing and early clinical development. Many of the future IND -enabling activities would be funded from sources listed later in this application. Such post -Phase II SBIR activities include:  Preparation of 300 g of non-GMP active pharmaceutical agent,  Selection of a suitable salt form (if necessary), vehicle and formulation, and evaluation of crystallinity and the propensity for polymorph formation,  Analytical methods development,  Drug substance stability testing, including standard pharmaceutics development work -up involving shelf- life and stress testing,  File additional provisional and PCT patent applications as necessary, and  Prepare 5 kg of GMP quality active pharmaceutical ingredient. Commercialization Plan Page 124 Contact PDPI: Harty, Ronald N. With the completion of GMP and non-GMP batches we plan to initiate the necessary studies to support an IND filing, specifically investigation of metabolites, in vitro toxicity and acute in vivo safety pharmacology and toxicity assessment. Activities include:  Metabolite ID (human, rat and monkey), characterization for toxicity and biological testing,  Full PK parameters in three relevant safety pharmacology species, mice, rabbit, and non-human primate (NHP), chosen since they are used in relevant in vivo pharmacology studies,  Investigation of Cyp450 inhibition and induction and transporter modulation in detail, for the potential for unexpected drug-drug interactions,  Enhanced survival in two or more GLP double blind placebo -controlled animal models of Ebola according to the Animal Rule of Efficacy (some of these studies might be done during the IND phase). Viral load and survival will also be assessed,  Completion of in vitro GLP toxicology (hERG, Ames, genotoxicity),  Conducting dose ranging and 28-day safety profiles in mice and NHPs,  Safety Pharmacology: Cardiovascular safety study in dogs, CNS safety in rats, respiratory safety in rats (if needed),  Reproductive toxicology in mice and rabbits (possibly do afterduring Phase I studies).  Conducting a pre-IND meeting with the FDA to establish and understand expectations, and  Initiating preparation of an IND to facilitate the transition into clinical evaluation. We have established a world-class PPxY Project Leadership Team (PPLT) to interpret data and provide strategic direction on a real time basis. The PPLT will regularly provide advice and guidance (at least monthly) in person, by Zoom, or by email. The PPLT will interpret data most appropriately and coordinate experimental design in close coordination with the Harty group and FCCDC management. The PPLT will conduct more formal quarterly meetings that Prof. Harty will chair, and any member of the PPLT may add items to the agenda prepared in adv ance of the meeting. The PPLT shall consist of the following members: Ronald N. Harty, Ph.D., Professor, University of Pennsylvania. Dr. Harty is Professor of Pathobiology and Microbiology, at the School of Veterinary Medicine. His research interests are focused on understanding the molecular mechanisms of filovirus (Ebola and Marburg viruses) and arenavirus (Lassa fever and Junin) assemblybudding, in order to identify broad- spectrum inhibitors of virus -host interactions. His research has directly led to the discovery of the antiviral agents described in this application. He is also interested in understanding the innate immune responses to virus infection that provide a critical first line of defense for the host against the invading pathogen. Prof. Harty has co-authored 80 scientific publications. Jay E. Wrobel, Ph.D., Vice President, Business Development, FCCDC. Dr. Wrobel has spent 37 years as a medicinal chemist in the pharmaceutical industry, 26 years at Wyeth (later Pfizer), with his last position as Senior Director of Medicinal Chemistry at the Collegeville, PA facility. At Wyeth, he mentored and guided the efforts of 31 Ph.D.MS medicinal chemists and worked successfully with outside alliance partners. He was directly involved in bringing forward nine development track candidates (Phase 0 and beyond) in a variety of therapeutic areas . He has co-authored 78 scientific publications and is an inventor on 83 issued US patents. Bruce Freedman, VMD, Ph.D., Associate Professor, University of Pennsylvania. Dr. Freedman is an Associate Professor of Pathobiology and Microbiology, at the School of Veterinary Medicine. Dr. Freedman is Director of the Microscopy Core facility at PennVet which houses state- of-the-art confocal and live cell imaging instruments that will be critical for assessing the efficacy of our budding inhibitors. Dr. Freedman is a long-standing collaborator with Dr. Ronald Harty, the PI on this proposal. Olena Shtanko, Ph.D., Staff Scientist, Texas Biomedical Research Institute. Dr. Shtanko researches fundamental molecular processes used by Ebola and Marburg viruses to disseminate within the host to understand the extraordinary capacity of these emerging viruses to trigger disease in affected individuals, with the aim to design and test antiviral therapies. Dr. Shtanko has over 16 years of conducting research on viruses, Commercialization Plan Page 125 Contact PDPI: Harty, Ronald N. with 9 years spent at the maximum containment laboratory (BSL-4) at Texas Biomedical Research Institute. Dr. Shtankos work has identified factors critical for Ebola virus and Crimean- Congo Hemorrhagic Fever virus entry into cells. Dr. Shtanko also extensively collaborates with a number of investigators to coordinate and perform virus studies to identify host factors critical for replication and spread of high-containment viruses as well as test novel antiviral therapeutics using a small animal model of Ebola and Marburg virus disease. Allen B. Reitz, Ph.D. CEO, FCCDC. Dr. Reitz has spent 38 years as a medicinal chemist in the pharmaceutical industry, 26 years with Johnson and Johnson (Spring House, PA). At JJ he mentored and guided a staff of 16 Ph.D.MS medicinal chemists and was co-inventor on 7 compounds that have entered human clinical trials. He has co-authored 150 scientific publications and is an inventor on 71 issued US patents. He is Editor -in-Chief and founding editor of Current Topics in Medicinal Chemistry . He has established two biotechnology companies: FCCDC and ALS Biopharma. FCCDC has already successfully partnered a separate technology platform for the treatment of Alzheimers disease, which has passed an interim futility analysis in Phase IIIII. Leo J. Adalbert, M.B.A. Mr. Adalbert will be a business development consultant for this project. An insightful, collaborative, and creative leader, Leo carries more than 25 years of deep cross -functional commercial and clinical development experience across both small molecules and biologics. Core therapeutic areas of expertise include cardiovascular, neuroscience, oncology, immunology, and GI, as well as biomarkers. Prior to founding StraNexa LLC, Leo held various leadership roles across marketing, business development, and alliance leadership at leading firms such as AstraZeneca, Merck, Astra Merck, and Regeneron. Leo earned an MBA from Harvard Business School (fellowship), and executive leadership certifications from Northwestern University (Kellogg) and University of Pennsylvania (Wharton). He also completed extensive coursework in biological sciences (Villanova UniversityBucks CC), earning high honors (Phi Theta Kappa distinction), and is a graduate of the HMX Program at Harvard Medical School (pharmacology and immunology). FCCDC has established a world-class Scientific Advisory Board, as posted on the company website, who are on call for questions regarding drug development strategy, flowchart design, chemistry and pharmacology. Nicholas A. Meanwell, Ph.D., Bristol-Myers Sq uibb Research  Development, Princeton, NJ. Nicholas A. Meanwell is currently Vice President, Discovery Chemistry Platforms, Small Molecule Drug Discovery where he has led drug discovery programs in the cardiovascular, neurosciences and virology therapeuti c areas, work that has resulted in the advancement of over 30 clinical candidates for the prevention of thrombosis, the treatment of stroke and therapy for viral infections, including human immunodeficiency virus-1 (HIV-1), hepatitis C virus (HCV) and respiratory syncytial virus (RSV). Significant compounds in the HCV arena include daclatasvir (Daklinza ), asunaprevir (Sunvepra), Daklinza and Sunvepra. Dr. Meanwell has authoredco-authored more than 250 publications, review articles, book chapters and editorials and is named as an inventorco- inventor on 138 issued U.S. Patents. Dr. Meanwell is the co-recipient of a PhRMA Research and Hope Award for Biopharmaceutical Industry Research, 2014 and the recipient of the 2015 Philip S. Portoghese Medicinal Chemistry Lectureship Award. He was inducted into the ACS Division of Medicinal Chemistry Hall of Fame on August 18th, 2015 and he is a member of the team that was recognized by a 2017 Heroes of Chemistry Award sponsored by the American Chemical Society. Eileen Jaffe, Ph.D., Fox Chase Cancer Center, Temple University School of Medicine  Drexel University School of Medicine. Eileen Jaffe received her PhD in Biochemistry from the University of Pennsylvania, working with Mildred Cohn. She was then a Postdoctoral Fellow at Harvard University with Jeremy Knowles. Since 2003 she has been at the Fox Chase Cancer Center in Philadelphia, PA, and is currently also an Adjunct Professor of Biochemistry at Temple University School of Medicine and Drexel University College of Medicine. Her research interests lie in understanding protein structure-function relationships using both biochemical and biophysical approaches. Prof. Jaffe is currently focused on the roles of protein quaternary structure and quaternary structure rearrangements in the control of protein function as this relates to phenylalanine hydroxylase and phenylketonuria. She has been associated with FCCDC since 2008. Patrick Lam, Ph.D., Baruch S. Blumberg Institute, Drexel University College of Medicine  Lam Drug Discovery Consulting. Patrick Lam is currently a Distinguished Professor of Chemistry at Baruch S. Blumberg Institute and Adjunct Professor at Drexel University College of Medicine. He is responsible for the discovery of a total of eight Commercialization Plan Page 126 Contact PDPI: Harty, Ronald N. clinical candidates. At Bristol Myers Squibb, Dr. Lam was the group leaderco-inventor responsible for the discovery of EliquisApixaban, a novel Factor Xa anticoagulant with annual sales of $2 B. Eliquis was chosen as the Best New Medicine of 2012 by Med Ad News. He and his team were awarded 2015 American Chemical Society Heroes of Chemistry Award for the discovery of Eliquis. Patrick is also internationally known as the co- discoverer of the powerful Chan-Lam Coupling Reaction, which is complementary to the Nobel -prize winning Suzuki -Miyaura Coupling Reaction. Patrick has authored 98 papersreviewsbook chapters and is an inventor on 36 patentspatent applications. He has presented 110 invited seminars worl dwide. Assuming that our candidate has progressed to IND status, our next objective will be clinical evaluation. Collectively, the management of FCCDC has been responsible for more than a dozen compounds that have entered human clinical trials, with one currently in Phase III and two in Phase II. Based on this experience, we propose the following path to commercialization with the associated timelines.  A pre-IND meeting will be held with the FDA prior to initiation of IND enabling studies, to obtain guidance as to proper preclinical and clinical study design and requirements. We expect to file an Investigational New Drug (IND) in 2Q2024, with First in Human (FIH) dosing to begin 1Q2025. If we have partnered with an appropriate mid-level or major pharmaceutical company, then our development partner will monitor and lead the drug development process.  Since our primary indication is as a Marburg antiviral medication, effectiveness of our candidate may not easily be determined in human clinical trials for ethical reasons, particularly if there is not an outbreak of the virus. Thus, we will resort to the Animal Rule (21CFR Part 314 Subpart 1) as the regulatory pathway for approval of this drug.  Initial Phase I clinical evaluation will take place in normal individuals looking for safety and tolerability in a single ascending and multiple ascending dose format and drug-drug interactions. Clinical read-outs will include standard hemodynamic parameters involved in cardiovascular safety testing. Adverse events would be monitored. The study would most likely take 12 months, enroll 40 participants and cost $3 million.  Phase II trial beginning 1Q2027 will be an intervention trial for healthy patients with human T cell leukemia virus, (a virus with a similar mode of egress to Marburg). A dose of candidate compound would be administered orally with a placebo. The endpoints would be change in viral load. The study would most likely take 10 months, enroll 140 participants and cost $10 million.  A New Drug Application (NDA) will be filed with the U.S. FDA containing all of the CMC and other ICH regulatory and safety pharmacology documentation required for commercialization. This would be schedule to occur at 2Q2029.  Initial FDA approval would be for use as an antiviral therapeutic agent for Marburg infection. This would occur on or about 1Q2030. The Target Product Profile for our candidate is shown in Table 1. Table 1. Desired Product Profile Preferred Minimally acceptable Efficacy ( survive) 95 85 Efficacy (virus) Undetectable levels of virus Undetectable levels of virus Adverse events Comparable to placebo Minor Administration Oral Oral Dosing schedule Once daily Twice a day Drug resistance None None Stability No special packaging required Modest precautions acceptable  An IV formulation may be preferable for the extremely ill, and a liquid formulation useful for children Commercialization Plan Page 127 Contact PDPI: Harty, Ronald N. 1.3. Impact on drug discovery and society . In the outbreak of a terrorist exposure to Marburg, Ebola or related virus, thousands if not greater numbers of people could die before vaccines become available to treat the affected population. Our technology, an oral antiviral medicine for Ebola, Marburg, Lassa and other deadly viruses, could be used alone or with other antiviral medications to protect unvaccinated individuals for the time required to mount an immune response to the vaccine. These agents may also be of value for individuals who respond adversely to the vaccine, and could be of value as prophylactic agents for individuals deemed to be in high risk situations such as military or healthcare workers. In addition, our agent may be effective against the related human T cell leukemia virus that can lead to a rare but highly aggressive cancer. This agent also has the potential to be used as an HIV medication used in conjunction with other HIV antiviral agents. Since current HIV medications can have reduced efficacy over time due to resistance mechanisms, other agents used in combination can reduce resistance and side effects, new drugs are of great need. Finally, our agents that block Nedd4 activity, such as FC -10696, have the potential to show clinical effectiveness against other areas of medical need including various cancers, and obesity (Huang, 2019, Li, 2015; Wan, 2019, Ye, 2014; Zou, 2015). Continued successful prosecution of this project will allow Fox Chase Chemical Diversity Center to benefit financially and be able to expand and hire additional staff, thereby creating jobs. In addition, the development, clinical validation and commercial exploitation of this platform technology will serve as a great testament to the value of the SBIRSTTR program. In summary, patients will be treated with an effective and safe therapy that had not been available before, Fox Chase Chemical Diversity Center and partners will realize potentially substantial financial return, and the STTR program will have served its mission to society. 2. Company Fox Chase Chemical Diversity Center, Inc., (FCCDC, www.fc -cdci.com) founded in 2008, is an emerging biotechnology company located in Doylestown, PA whose mission is to advance basic scientific discoveries by providing value-added early drug discovery and medicinal chemistry research support for the translation of viable preclinical drug and diagnostic candidates prior to eventual entry into human clinical trials. Our goal is to partner with suitable biomedical research collaborators on selected projects or act as their companys proprietary medicinal chemistry research service. We bring a unique and highly experienced team of internal scientists and collaborative external partners in order to leverage cutting edge scientific advances. The staff at FCCDC has vast drug discovery experience, drawing from a diverse background of pharmaceutical and biotech companies including Johnson  Johnson, Wyeth and Merck. FCCDC has two fully functional, multisite medicinal chemistry laboratories with two 300-MHz NMRs, multiple LCMS machines, microwave reactor, and multiple Gilson semi - prep HPLCs for compound and library purification. We have prepared 12,000 new chemical entities (NCEs) so far, with two compounds advancing into human clinical trials, both funded initially by SBIR grant support. FCCDC has a 30,000 member onsite chemical inventory of reagents and starting materials for ready use, which are bar-coded, computer searchable, and segregated according to chemical class. As the basis for our internal technology platform, we have the ability to empower research programs at multiple stages of their development: provide insight into probe molecule identification and synthesis, in silico identification of custom and commercial high throughput screening libraries, perform hit triage, hit to lead medicinal chemistry, and lead optimization. Importantly, we also prepare or assist in preparation of intellectual property documents. We seek to improve efficiencies in early drug discovery by quickly derisking the overall process using modern methods and wisdom regarding drug suitability and compound library evaluation. 3.Market, Customers and Competition 3.1 Market. In response to credible scenarios of bioterrorist attacks using a filovirus or other infectious agents, the US Congress enacted Project BioShield legislation that was signed in 2004 which provided $5.6 billion over 10 years to encourage the private sector to develop medical countermeasures against chemical, biological and other terrorism agents (Grotton, 2011). Following this legislation, the Biomedical Advanced Research and Development Authority (BARDA) was created two years later to oversee BioShields advanced development and procurement efforts. In 2013 Congress renewed laws to extend funding by passing the Pandemic All Hazards Commercialization Plan Page 128 Contact PDPI: Harty, Ronald N. Preparedness Reauthorization Act that provided an additional $2.8 billion and extended BARDA until 2018 (Genome Web Daily News, 2013). As an example, the US has granted Fast Track designation to the Arbutus Ebola drug and has funded $140 million for RD on EVD (PR Newswire 2014). BioCryst was awarded $12 million for development of BCX4430 (Barda 2015). In addition to the Barda program, the commercial market for future natural outbreaks is growing at a CAGR of 95.2 and the market is expected to reach $56 million by 2020 (Allied Market Research 2015). Lassa fever is another viral disease treatable with our egress inhibitors . This malady is caused by the Lassa virus and results in 300,000 to 500,000 cases annually, mainly in western Africa and causes about 5,000 deaths each year ( Ogbua 2007) . Transmission of Lassa virus to humans occurs most commonly through ingestion or inhalation due to rodent infestation. A drug in this category could also be effective against human T cell leukemia virus (HTLV), which can lead to adult T-cell leukemialymphoma (ATLL). ATLL is a rare and fast-growing T-cell lymphoma that can be found in the blood (leukemia), lymph nodes (lymphoma), skin, or multiple areas of the body (Lymphoma Research Foundation, 2015). Around 20 million people worldwide are infected with HTLV. It is common in Japan, South and Central America, the Caribbean and Western Africa, and a large percentage (30) of African Americans are infected in certain areas of the US (e.g. New Jersey and New Orleans). It is transmitted through sexual contact or exposure to infected blood and blood products. Only a small percentage of carriers (5) actually come down with ATLL however, but there is no way to predict those that do. Survival depends on the ATLL subtype and var ies from 4-6 months to more than five years (Gonçalves, 2010). Treatment options include standard anti -cancer protocols but at present no therapy targets the virus. Other diseases and disorders from HTLV include the incapacitating neurological disease (HTL V-associated myelopathytropical spastic paraparesis HAMTSP), uveitis, rheumatic syndromes, and predisposition to helminth and bacterial infections (Gonçalves, 2010). As mentioned earlier, the compounds in this application may block other biological functions of Nedd4. For example, Nedd4 is overexpressed in a variety of cancers including bladder, breast, prostate, gastric carcinomas, colorectal, breast and non-small cell lung carcinomas and inhibits several tumor suppressor systems (e.g. PTEN, Beclin). Suppression of Nedd4 has been shown to reduce tumor cell growth (Boase, 2015; Wan, 2019, Ye, 2014; Zou 2015). Thus, these compounds could find applications in the treatment of such cancers. Nedd4 also negatively regulates interferon (IFN) -induced transmembrane protein 3 (IFITM3) that restricts cellular infection by influenza virus. Therefore, blocking Nedd4 could lead to therapeutic treatment of influenza (Chesarino, 2015) 3.2 Customers. As mentioned previously, BARDA is the most obvious customer for our RNA virus antiviral agent; however several potential governmental partners could provide development funds andor resources toward development including the National Institute of Allergy and Infectious Diseases (NIAID), Office of Biodefens e Research (OBRA), Department of Defense (DOD), The Defense Threat Reduction Agency (DTRA), The United Sates Army Medical Research Institute for Infectious Diseases (USAMRIID), Defense Science and Technology Laboratories (DSTL), and the Centers for Disease Control and Prevention (CDC). In addition to pursuing the government funding options outlined above, we may seek to partner with small biotechnology companies such as Siga, Tekmira, Chimerix, MappBio Pharm, Athene, Biocryst and Tetraphase, and large companies such as Merck, Johnson  Johnson, and GSK who currently are developing one or more therapeutics targeting a bioterrorism threat as most of these companies have a stated company goal of identifying and developing additional therapeutics in this area. 3.3 Competition. Since there are currently no other agents that inhibit virus egress that are being developed as broad spectrum RNA antiviral agents, an oral drug product based on FC -10696 is potentially a first-in -class treatment either as a mono-therapeutic prophylactic agent, or in combination with other agents currently in development for treatment of RNA viral infections. With respect to Ebola efforts, presently there are several therapeutics agents being evaluated that are summarized here. Commercialization Plan Page 129 Contact PDPI: Harty, Ronald N.  Recombinant vesicular stomatitis virus Zaire Ebola virus (rVSV-ZEBOV), sold under the brand name Ervebo (Merck) is a vaccine that prevents Zaire-strain Ebola. When used in ring vaccination, Ervebo has shown a high level of protection. Around half the people given the vaccine have mild to moderate adverse effects that include headache, fatigue, and muscle pain. Ervebo does not protect against other species of Ebolavirus or Marburg virus. Also, rural distribution has been a challenge and there is no data on longevity of the vaccines protection and how often an additional dose boost is required to extend immunity.  Johnson and Johnson is developing a vaccine regiment that consists of two doses leveraging different vaccines. One is the Ad26.ZEBOV monovalent vaccine based on adenovirus type 26 (Ad26) vector expressing the glycoprotein (GP) of the Ebola virus Mayinga variant (Ad26.ZEBOV). The other is multivalent Modified Vaccinia Virus Ankara (MVA) strain containing ZEBOV, SEBOV, Marburg Virus GP and Tai Forest nucleoprotein inserts (MVA-BN -Filo). More than 6,500 individuals have now participated in clinical studies for the Ebola vaccine across the U.S., Europe, and Africa. So far, the data from these studies suggest that the vaccine stimulates a robust immune response and has a favorable safety profile. Janssen has also assembled a stockpile of 1.5 million vaccine regimens for potential use in public health emergencies.  ZMapp (MappBio USA) is a cocktail of three monoclonal antibodies with neutralizing activity against Ebola virus in animal models. Phase II trial is underway. A similar produce MIL-77 (MabWorks, China) is undergoing Phase I study. A trial conducted in 71 patients, 36 of whom received ZMapp in addition to the standard of care, did not show a significant effect on mortality. At 28 days, death had occurred in 84 of 169 (49.7) of patients in the ZMapp group.  REGN-EB3 (Regeneron) is an experimental biopharmaceutical antiviral drug that is a cocktail of three monoclonal antibodies (REGN3470, 3471 and 3479) for treatment of Ebola virus disease. It is being developed, tested and manufactured as part of an agreement established in 2015 with BARDA, part of the Office of the Assistant Secretary for Preparedness and Response at the U.S. Department of Health and Human Services. REGN -EB3 is currently under clinical development and its safety and efficacy have not been fully evaluated by any regulatory authority. It was developed against the Zaire species of Ebola virus, but the Sudan and Bundibugyo strains have also caused outbreaks and it is unlikely that EB3 would be effective against these strains. It was tested in the PALM trial that had enrolled nearly 700 participants across four Ebola Treatment Centers in the DRC. At 28 days, death occurred in 52 of 155 (33.5) patients in the REGN-EB3 group.  mAb114 is a monoclonal antibody (Ridgeback Biotherapeutics LP) that is being evaluated as a treatment for Ebola virus disease. A license for mAb114 was obtained by in 2018 from the National Institutes of Health - National Institute of Allergy and Infectious Diseases. In the PALM trial that had enrolled nearly 700 participants across four Ebola Treatment Centers in the DRC. At 28 days, death occurred in 61 of 174 (35.1) patients in the MAb114 group.  Galidesivir (BCX4430, BioCryst) is a broad-spectrum small molecule antiviral in advanced development under the Animal Rule for the treatment of viruses including Ebola, Marburg, Yellow Fever and Zika viruses. A Phase 1 clinical safety and pharmacokinetics study in healthy subjects has been completed, and in animal studies, galidesivir has demonstrated survival benefits. Galidesivir is a viral RNA -dependent RNA polymerase inhibitor that has also demonstrated broad-spectrum activity in vitro against more than 20 ribonucleic acid (RNA) viruses in nine different families, including filoviruses, togaviruses, bunyaviruses, arenaviruses, paramyxoviruses, coronaviruses and flaviviruses. Galidesivir may be suitable for administration by intravenous, intramuscular, and oral routes. There remains a critical need for additional agents, particularly with alternative modes of action. First many of the therapeutic agents above, particularly among the vaccine, antibody, siRNA andor antisense oligonucleotides are specific for individual viruses and sometimes only one or more strains within a virus. Biological agents require iv or infusion administration, and also need to be kept in the cold during storage, which are not ideal conditions in many remote areas of virus prevalence. Our product would be broad spectrum for RNA viruse s that have PPxY motifs. Our product differentiates further from vaccines in that it would be an oral agent with a great probability of more economic manufacturing and ease of distribution. Our agent could be Commercialization Plan Page 130 Contact PDPI: Harty, Ronald N. used in conjunction with other oral agents, like the cocktail approach to HIV drug administration, as synergistic therapeutic effects and a decrease in the risk of drug resistance are possible. 4. Intellectual Property Protection FCCDC and the Trustees of the University of Pennsylvania obtained U.S. Patent 10,160,756, December 25, 2018. This application contains composition of matter and use claims specific to earlier analogs of FC -10696. FC-10696 and its present suite of analogs are outside the scope of this patent and will be subject to new composition of matter applications. We will seek to create additional value in this project by the continued prosecution of the other U.S. provisional patent applications and those that would be expected to emerge in the future. We will file PCT applications and pursue patent protection in major foreign markets, including in Europe, Asia and the BRIC countries (Brazil, Russia, India, China). No special restrictions or licenses have been or are imposed on the rights of the key personnel, FCCDC or the University of Pennsylvania that fall outside of the",
    "intellectual_merit": "Harvard Medical School HMX Program (completed with Achievement Honors Pharmacology and Immunology;",
    "broader_impacts": "4.c. If this project has an actual or potential impact on the environment, has an exemption been authorized or an  Yes",
    "keywords": []
  },
  "research_plan": {
    "objectives": [
      "Harty, Ronald N. 1.3. Impact on drug discovery and society . In the outbreak of a terrorist exposure to Marburg, Ebola or related virus, thousands if not greater numbers of people could die before vaccines become available to treat the affected population. Our technology, an oral antiviral medicine for Ebola, Marburg, Lassa and other deadly viruses, could be used alone or with other antiviral medications to protect unvaccinated individuals for the time required to mount an immune response to the vaccine. These agents may also be of value for individuals who respond adversely to the vaccine, and could be of value as prophylactic agents for individuals deemed to be in high risk situations such as military or healthcare workers. In addition, our agent may be effective against the related human T cell leukemia virus that can lead to a rare but highly aggressive cancer. This agent also has the potential to be used as an HIV medication used in conjunction with other HIV antiviral agents. Since current HIV medications can have reduced efficacy over time due to resistance mechanisms, other agents used in combination can reduce resistance and side effects, new drugs are of great need. Finally, our agents that block Nedd4 activity, such as FC -10696, have the potential to show clinical effectiveness against other areas of medical need including various cancers, and obesity (Huang, 2019, Li, 2015; Wan, 2019, Ye, 2014; Zou, 2015). Continued successful prosecution of this project will allow Fox Chase Chemical Diversity Center to benefit financially and be able to expand and hire additional staff, thereby creating jobs. In addition, the development, clinical validation and commercial exploitation of this platform technology will serve as a great testament to the value of the SBIRSTTR program. In summary, patients will be treated with an effective and safe therapy that had not been available before, Fox Chase Chemical Diversity Center and partners will realize potentially substantial financial return, and the STTR program will have served its mission to society. 2. Company Fox Chase Chemical Diversity Center, Inc., (FCCDC, www.fc -cdci.com) founded in 2008, is an emerging biotechnology company located in Doylestown, PA whose mission is to advance basic scientific discoveries by providing value-added early drug discovery and medicinal chemistry research support for the translation of viable preclinical drug and diagnostic candidates prior to eventual entry into human clinical trials. Our goal is to partner with suitable biomedical research collaborators on selected projects or act as their companys proprietary medicinal chemistry research service. We bring a unique and highly experienced team of internal scientists and collaborative external partners in order to leverage cutting edge scientific advances. The staff at FCCDC has vast drug discovery experience, drawing from a diverse background of pharmaceutical and biotech companies including Johnson  Johnson, Wyeth and Merck. FCCDC has two fully functional, multisite medicinal chemistry laboratories with two 300-MHz NMRs, multiple LCMS machines, microwave reactor, and multiple Gilson semi - prep HPLCs for compound and library purification. We have prepared 12,000 new chemical entities (NCEs) so far, with two compounds advancing into human clinical trials, both funded initially by SBIR grant support. FCCDC has a 30,000 member onsite chemical inventory of reagents and starting materials for ready use, which are bar-coded, computer searchable, and segregated according to chemical class. As the basis for our internal technology platform, we have the ability to empower research programs at multiple stages of their development: provide insight into probe molecule identification and synthesis, in silico identification of custom and commercial high throughput screening libraries, perform hit triage, hit to lead medicinal chemistry, and lead optimization. Importantly, we also prepare or assist in preparation of intellectual property documents. We seek to improve efficiencies in early drug discovery by quickly derisking the overall process using modern methods and wisdom regarding drug suitability and compound library evaluation. 3.Market, Customers and Competition 3.1 Market. In response to credible scenarios of bioterrorist attacks using a filovirus or other infectious agents, the US Congress enacted Project BioShield legislation that was signed in 2004 which provided $5.6 billion over 10 years to encourage the private sector to develop medical countermeasures against chemical, biological and other terrorism agents (Grotton, 2011). Following this legislation, the Biomedical Advanced Research and Development Authority (BARDA) was created two years later to oversee BioShields advanced development and procurement efforts. In 2013 Congress renewed laws to extend funding by passing the Pandemic All Hazards Commercialization Plan Page 128 Contact PDPI: Harty, Ronald N. Preparedness Reauthorization Act that provided an additional $2.8 billion and extended BARDA until 2018 (Genome Web Daily News, 2013). As an example, the US has granted Fast Track designation to the Arbutus Ebola drug and has funded $140 million for RD on EVD (PR Newswire 2014). BioCryst was awarded $12 million for development of BCX4430 (Barda 2015). In addition to the Barda program, the commercial market for future natural outbreaks is growing at a CAGR of 95.2 and the market is expected to reach $56 million by 2020 (Allied Market Research 2015). Lassa fever is another viral disease treatable with our egress inhibitors . This malady is caused by the Lassa virus and results in 300,000 to 500,000 cases annually, mainly in western Africa and causes about 5,000 deaths each year ( Ogbua 2007) . Transmission of Lassa virus to humans occurs most commonly through ingestion or inhalation due to rodent infestation. A drug in this category could also be effective against human T cell leukemia virus (HTLV), which can lead to adult T-cell leukemialymphoma (ATLL). ATLL is a rare and fast-growing T-cell lymphoma that can be found in the blood (leukemia), lymph nodes (lymphoma), skin, or multiple areas of the body (Lymphoma Research Foundation, 2015). Around 20 million people worldwide are infected with HTLV. It is common in Japan, South and Central America, the Caribbean and Western Africa, and a large percentage (30) of African Americans are infected in certain areas of the US (e.g. New Jersey and New Orleans). It is transmitted through sexual contact or exposure to infected blood and blood products. Only a small percentage of carriers (5) actually come down with ATLL however, but there is no way to predict those that do. Survival depends on the ATLL subtype and var ies from 4-6 months to more than five years (Gonçalves, 2010). Treatment options include standard anti -cancer protocols but at present no therapy targets the virus. Other diseases and disorders from HTLV include the incapacitating neurological disease (HTL V-associated myelopathytropical spastic paraparesis HAMTSP), uveitis, rheumatic syndromes, and predisposition to helminth and bacterial infections (Gonçalves, 2010). As mentioned earlier, the compounds in this application may block other biological functions of Nedd4. For example, Nedd4 is overexpressed in a variety of cancers including bladder, breast, prostate, gastric carcinomas, colorectal, breast and non-small cell lung carcinomas and inhibits several tumor suppressor systems (e.g. PTEN, Beclin). Suppression of Nedd4 has been shown to reduce tumor cell growth (Boase, 2015; Wan, 2019, Ye, 2014; Zou 2015). Thus, these compounds could find applications in the treatment of such cancers. Nedd4 also negatively regulates interferon (IFN) -induced transmembrane protein 3 (IFITM3) that restricts cellular infection by influenza virus. Therefore, blocking Nedd4 could lead to therapeutic treatment of influenza (Chesarino, 2015) 3.2 Customers. As mentioned previously, BARDA is the most obvious customer for our RNA virus antiviral agent; however several potential governmental partners could provide development funds andor resources toward development including the National Institute of Allergy and Infectious Diseases (NIAID), Office of Biodefens e Research (OBRA), Department of Defense (DOD), The Defense Threat Reduction Agency (DTRA), The United Sates Army Medical Research Institute for Infectious Diseases (USAMRIID), Defense Science and Technology Laboratories (DSTL), and the Centers for Disease Control and Prevention (CDC). In addition to pursuing the government funding options outlined above, we may seek to partner with small biotechnology companies such as Siga, Tekmira, Chimerix, MappBio Pharm, Athene, Biocryst and Tetraphase, and large companies such as Merck, Johnson  Johnson, and GSK who currently are developing one or more therapeutics targeting a bioterrorism threat as most of these companies have a stated company goal of identifying and developing additional therapeutics in this area. 3.3 Competition. Since there are currently no other agents that inhibit virus egress that are being developed as broad spectrum RNA antiviral agents, an oral drug product based on FC -10696 is potentially a first-in -class treatment either as a mono-therapeutic prophylactic agent, or in combination with other agents currently in development for treatment of RNA viral infections. With respect to Ebola efforts, presently there are several therapeutics agents being evaluated that are summarized here. Commercialization Plan Page 129 Contact PDPI: Harty, Ronald N.  Recombinant vesicular stomatitis virus Zaire Ebola virus (rVSV-ZEBOV), sold under the brand name Ervebo (Merck) is a vaccine that prevents Zaire-strain Ebola. When used in ring vaccination, Ervebo has shown a high level of protection. Around half the people given the vaccine have mild to moderate adverse effects that include headache, fatigue, and muscle pain. Ervebo does not protect against other species of Ebolavirus or Marburg virus. Also, rural distribution has been a challenge and there is no data on longevity of the vaccines protection and how often an additional dose boost is required to extend immunity.  Johnson and Johnson is developing a vaccine regiment that consists of two doses leveraging different vaccines. One is the Ad26.ZEBOV monovalent vaccine based on adenovirus type 26 (Ad26) vector expressing the glycoprotein (GP) of the Ebola virus Mayinga variant (Ad26.ZEBOV). The other is multivalent Modified Vaccinia Virus Ankara (MVA) strain containing ZEBOV, SEBOV, Marburg Virus GP and Tai Forest nucleoprotein inserts (MVA-BN -Filo). More than 6,500 individuals have now participated in clinical studies for the Ebola vaccine across the U.S., Europe, and Africa. So far, the data from these studies suggest that the vaccine stimulates a robust immune response and has a favorable safety profile. Janssen has also assembled a stockpile of 1.5 million vaccine regimens for potential use in public health emergencies.  ZMapp (MappBio USA) is a cocktail of three monoclonal antibodies with neutralizing activity against Ebola virus in animal models. Phase II trial is underway. A similar produce MIL-77 (MabWorks, China) is undergoing Phase I study. A trial conducted in 71 patients, 36 of whom received ZMapp in addition to the standard of care, did not show a significant effect on mortality. At 28 days, death had occurred in 84 of 169 (49.7) of patients in the ZMapp group.  REGN-EB3 (Regeneron) is an experimental biopharmaceutical antiviral drug that is a cocktail of three monoclonal antibodies (REGN3470, 3471 and 3479) for treatment of Ebola virus disease. It is being developed, tested and manufactured as part of an agreement established in 2015 with BARDA, part of the Office of the Assistant Secretary for Preparedness and Response at the U.S. Department of Health and Human Services. REGN -EB3 is currently under clinical development and its safety and efficacy have not been fully evaluated by any regulatory authority. It was developed against the Zaire species of Ebola virus, but the Sudan and Bundibugyo strains have also caused outbreaks and it is unlikely that EB3 would be effective against these strains. It was tested in the PALM trial that had enrolled nearly 700 participants across four Ebola Treatment Centers in the DRC. At 28 days, death occurred in 52 of 155 (33.5) patients in the REGN-EB3 group.  mAb114 is a monoclonal antibody (Ridgeback Biotherapeutics LP) that is being evaluated as a treatment for Ebola virus disease. A license for mAb114 was obtained by in 2018 from the National Institutes of Health - National Institute of Allergy and Infectious Diseases. In the PALM trial that had enrolled nearly 700 participants across four Ebola Treatment Centers in the DRC. At 28 days, death occurred in 61 of 174 (35.1) patients in the MAb114 group.  Galidesivir (BCX4430, BioCryst) is a broad-spectrum small molecule antiviral in advanced development under the Animal Rule for the treatment of viruses including Ebola, Marburg, Yellow Fever and Zika viruses. A Phase 1 clinical safety and pharmacokinetics study in healthy subjects has been completed, and in animal studies, galidesivir has demonstrated survival benefits. Galidesivir is a viral RNA -dependent RNA polymerase inhibitor that has also demonstrated broad-spectrum activity in vitro against more than 20 ribonucleic acid (RNA) viruses in nine different families, including filoviruses, togaviruses, bunyaviruses, arenaviruses, paramyxoviruses, coronaviruses and flaviviruses. Galidesivir may be suitable for administration by intravenous, intramuscular, and oral routes. There remains a critical need for additional agents, particularly with alternative modes of action. First many of the therapeutic agents above, particularly among the vaccine, antibody, siRNA andor antisense oligonucleotides are specific for individual viruses and sometimes only one or more strains within a virus. Biological agents require iv or infusion administration, and also need to be kept in the cold during storage, which are not ideal conditions in many remote areas of virus prevalence. Our product would be broad spectrum for RNA viruse s that have PPxY motifs. Our product differentiates further from vaccines in that it would be an oral agent with a great probability of more economic manufacturing and ease of distribution. Our agent could be Commercialization Plan Page 130 Contact PDPI: Harty, Ronald N. used in conjunction with other oral agents, like the cocktail",
      "Host-Pathogen Interactions is the study of the basic biology of infection in humans and animals  the immediate i ntersection of host and",
      "PI, No overlap with current proposal",
      "viral PPxY interaction, thereby inhibiting virus budding and spread. Our initial lead",
      "Marketing  Finance",
      "co-I",
      "Harty, Ronald N.  Recombinant vesicular stomatitis virus Zaire Ebola virus (rVSV-ZEBOV), sold under the brand name Ervebo (Merck) is a vaccine that prevents Zaire-strain Ebola. When used in ring vaccination, Ervebo has shown a",
      "Harty, Ronald N. LE O J.AD ALBE RT PROFESSIONAL EXPERIENCE AND KEY ACCOMPLISHMENTS (CONTINUED )  ASTRAZENECA PHARMACEUTICALS (1999 2008) WILMINGTON, DE Advanced rapidly through a series of increasingly responsible roles, providing effective leadership in driving major commercialization and business development initiatives (multiple recipient of AstraZeneca Excellence Award). Alliance and Program Leader, Strategic Planning  Business Development  Spearheaded and managed collaboration to commercialize CRESTOR fenofibrate fixed -dose combination. Collaboration progressed six months ahead of schedule and $3.5M under budget.  Instrumental in design and implementation of new Alliance Management capability within the firm. Senior Director, Licensing Commercial Evaluations Managed commercialization strategy in collaboration with Global Product Teams. Led evaluations of Phase IIIII licensingacquisition opportunities across multiple therapeutic areas.  Led product and company alliance and acquisition evaluations covering oncology and neuroscience, driving successful progression of target opportunities through formal diligence to deal completion.  Defined and implemented novel commercialization",
      "Harty , Ronald N.",
      "PPxY inhibitors that target this virus -host interaction, and Dr. Shtankos team",
      "Implications for",
      "Provisions to minimize discomfort, distress, pain and injury All animals will be observed by professional technicians trained in lab animal care and use. An attending veterinarian is available at all time for emergencies. In the event that any aspect of this study causes more than normally expected brief pain or distress to the animals, the study director shall determine, in consultation with the attending veterinarian, if administration of appropriate sedatives, analgesics, or anesthetics, or other therapies would be contraindicated by the objectives of the study and document the resultant course of action. Animals that experience severe or chronic pain or distress that cannot be relieved will be euthanized. Point 4: Euthanasia Animals will typically be sacrificed after the last clinical o bservation noted or blood sample has been collected. All rats will be euthanized by cervical dislocation under isoflurane anesthesia. Isoflurane ensures that the rats are unconscious, while dislocation ensures quick death. This minimizes animal distress, i s effective, and efficient; it is consistent with the recommendations of the AVMA Guidelines on Euthanasia. Vertebrate Animals Page 171 Contact PDPI: Harty, Ronald N. VERTEBRATE ANIMALS SECTION (VAS) p. 1 of 1 1 2 3 4 Description of procedures In vivo pharmacokinetics (PK) assessment will be performed to determine the dose and dosing schedule for additional in vivo studies. PK work will be performed at Alliance Pharma (Malvern, PA), a CRO specializing in drug metabolism and PK (DMPK). C57Bl6 mice will be used. Justifications The proposed animal experiments are critical to determine PK parameters after intravenous (IV) and intratracheal (IT) drug administration, including half -life, peak plasma and lung tissue concentration (C max), and the integral of t he concentration -time curve in both plasma and lung tissue (AUC, a measure of drug exposure) to determine the dosing schedules to be used in future studies, and to enable an understanding of how drug exposure relates to efficacy in models of lung disease. To assess the PK profile of each compound, mice will be dosed IV andor IT with each compound, and plasma concentrations will be determined at up to eight time -points over 24 hours (n3 mice per time -point) using LC -MSMS. Total number of mice to be used for PK studies is 192 over a two year period. Minimization of Pain and Distress All p rocedures are designed to minimize discomfort and pain. Mice will be fed and watered appropriately throughout the entire protocol. Each cage also contains a nestlet for enrichment purposes. Levels of paindistress and endpoints are discussed by the IACUC as an integral part of the protocol review process. The IACUC ensures that the procedures described in the protocol are categorized appropriately. As an IACUC member, the Attending Veterinarian reviews all animal care and use protocols with an emphasis on those that require any anesthetics or analgesics. The AV and investigators discuss the appropriate selection and regimen for anesthetics and analge sics for the animal procedures in the protocol. The recommendations are then transferred into the protocol for IACUC review and approval. Euthanasia Any mice exhibiting pain or discomfort lasting longer than a few seconds that cannot be instantly relieved will be immediately euthanized. No animals are euthanized in the presence of their conspecifics (same room). Animals are placed in a clear, plexiglass chamber of known size and CO2 is introduced at a flow rate of 10 -30 (based on chamber size) via a calibrated flow meter. When animals are euthanized, death is assured by direct observation to ensure an absence of respiration. In addition, the chest is palpated to ensure an absence of a heartbeat. Methods of euthanasia used at Alliance are in compliance with the AVMA Guidelines for the Euthanasia of Animals: 2013 Edition. Verteb rAatle nimalV Page 172 liAance_ s AS VAS Alliance Contact PDPI: Harty, Ronald N. SELECT AGENTS Texas Biomedical Research Institute (Texas Biomed) BSL4 agents licensed for use in this proposal. Low passage human isolates of Ebola virus variant Mayinga, Marburg virus strain Musoke, Lassa virus strain Josiah as well as mouse-adapted Marburg virus strain Angola will be used in this study and are available in the repository at Texas Biomed. Work with replication competent viruses will be performed in the Texas Biomed ABSL4 facility by Dr. Olena Shtanko, Ms. Gloria Rodriguez (Research Assistant), and ABSL4 veterinary staff. Dr. Shtanko, Ms. Rodriguez, and veterinary staff are authorized for work with these agents and have appropriate Homeland Security and CDC approval for this work. Dr. Shtanko is a CDC -registered PI and has been working with high-containment viruses for 9 years. Ms. Rodriguez has been working with filoviruses for 5 years. Animal work (mouse, guinea pigs and non- human primate) with Ebola, Marburg, and Lassa viruses has been performed at Texas Biomed for 15 years. The Biohazard  Safety Committee, Animal Care  Use Committee, and the Institutional Review Board to protect the rights and welfare of human research subjects have approved the protocols detailing the work described herein. Registration status of all entities where Select Agent(s) will be used. Select agent work will only be conducted at Texas Biomed. Texas Biomed is a select agent registered entity with Health and Human Services (HHS), Centers for Disease Control and Prevention (CDC) and U.S. Department of Agriculture, Animal Plant Health Inspection Service, National Select Agent Program. Texas Biomed has been inspected by the CDC National Select Agent Program for use of HHS Select Agents and Toxins, Overlap Select Agents and Toxins and USDA Select Agents and Toxins. Per the requirements of 42 CFR 73, the most recent certificate of registration was issued on November 20, 2017 for use of select agents at BSL2, 3, and 4 and ABSL3 and 4. Texas Biomed registration number is 20171120- 1976. Texas Biomed has a Responsible Official (RO) and an Alternate Responsible Official. Facilities where the Select Agents will be used. We have one ABSL4 full -suit lab which is registered for CDC and USDA select agent work on human and animal pathogens. Access to the ABSL4 area is secured via three successive magnetic cardkeypad controlled airlock passages: The first is located at the entry foyer to the BSL3 suite, the second at the entry foyer to the outer changer room, and the third at the entry foyer to the ABSL4 area. The ABSL4 area is equipped with an entry and exit decontamination airlock and an emergency exitdecontamination chamber. The latter also serves as an equipment decontamination chamber to allow for the repair, removal andor replacement of faulty equipment without requiring a complete laboratory decontamination process. All supply and exhaust air for the ABSL4 passes through dual (and tandem) HEPA filters. The laboratory contains 3 x class IIB biological safety hoods, 4 x water-jacketed CO2 incubators, low -, high- and ultra-speed centrifuges equipped with both analytical and preparative scale rotors, a microfuge, 2 x 4C refrigerators, 2 x liquid nitrogen storage systems, and 3 x -80 C freezers. The ABSL4 also has 2 CytoSMART Lux systems, an optical ELISA plate reader, MAGPIX multiplexing 96-well format unit, luminometer, Innova 4000 environmental shaker, microplate shaker, BioFlo III fermenters with chillers, Evos fluorescent microscope, pass - through autoclave, dunk tank, facsimile machine and networked computers. Housing for animals in the ABSL4 space includes 2 rodent racks able to accommodate 36 cages for guinea pigs or up to 480 mice. Non-human primate (NHP) housing consists of squeeze back cages to house up to 12 marmosets and 24 macaques which are placed within a microisolated enclosure. Vetscan and Procyte hematology and blood chemistry Abaxis analyzers are available in the space for analysis of blood collected from infected animals. Coagulation parameters can be measured in the lab with Idexx analyzers. Specialized equipment for monitoring biopotentials of NHPs in the ABSL4 includes the DSI (Data Science International) Telemetry System with analysis software. A dedicated procedure area is located within the lab to perform routine animal sampling and full necropsies. Critical parameters (air supply and exhaust flow rates, pressure differentials, normal and emergency breathing air, decontamination systems, etc.) are continuously monitored. Personnel within the lab communicate, and are actively monitored by outside personnel, through a scrambled UHF communication system worn inside the laboratory suit. The 500 sq. ft. ABSL4 laboratory staging area includes an air locked suit change room, two inner change rooms with pass -through showers, and an outer clean change room equipped with sinks, Select Agent Research Page 173 Contact PDPI: Harty, Ronald N. lockers, toilets, etc. Critical mechanical support devices are located in an attached 3,000 sq. ft. mechanical support complex. All critical equipment and devices (valves, etc.) are redundant, including duplicate air-supply and exhaust systems, decontamination shower systems, as well as the compressors, pumps, valves, heater cores, etc. that comprise the liquid waste decontamination system. All critical mechanical devices operate on a system with automatic emergency electricity backup generators. The ABSL4 area (including mechanical support rooms) is equipped with an elaborate intrusion detection system that includes electronic door and roof hatch sensors, motion detectors, etc. Texas Biomed maintains an experienced and trained staff of scientists, veterinarians, research technicians and veterinary technicians available to perform studies at high biocontainment. These individuals have demonstrated proficiency at conducting in vitro and animal studies with the agent identified in the proposal. The ABSL4 Operations and Safety Manuals specify policies, procedures, and standard operating procedures (SOPs) for the safe handling of biological materials in biosafety laboratories. The policies, procedures, and SOPs comply with applicable Federal, State, and municipal regulations and with the guidelines Biosafety in Microbiological and Biomedical Laboratories issued by the CDC and the National Institute of Health (NIH). Employees are trained from these manuals on each facility mechanical systems, biosafety, biocontainment and security. Employees are also trained according to project specific and Departmental SOPs. These procedures apply to all that use, generate, store, or dispose of potentially infectious materials in Texas Biomed biosafety laboratories and to persons who must enter these laboratories to perform services. Select agent use, transfer or possession is forbidden without the permission of the Responsible Official (RO) and until the required forms filed and written approval received from the CDC Select Agent Program. Upon approval, ABSL4 investigators desiring to work on a BSL4 project must also submit an application to the Biohazard and Safety Committee for approval. The Committee is responsible for evaluating the facility, equipment, and staff capabilities to perform work in a safe manner. Infectious cultures and inventory stocks are stored inside the ABSL4 laboratory in locked refrigerators and freezers. An electronic Inventory (eInventory) system is used to document usage of infectious stocks. All infectious material stored in refrigerators or freezers is properly labeled and stored in containers capable of withstanding thermal shock of freezing and thawing. Each container is labeled with the identity of the infectious agent, the date of the preparation, and a barcode that links the material to the more inclusive information contained in the inventory database. When work is completed, all infectious cultures are removed from workbenches and cabinets and stored in a designated refrigerator or freezer. Materials to be discarded are placed in a sealable container filled with a suitable disinfectant. The container is placed in a discard pan containing the disinfectant. Discard pans are placed on a cart and transported to the autoclave. Labware containing infectious liquids are stored and transported in leak -proof containers large enough to contain the fluid in case of leakage. Select Agent Research Page 174 Contact PDPI: Harty, Ronald N. References: 1. Bieniasz, P.D., Late budding domains and host proteins in enveloped virus release. Virology, 2006. 344(1): p. 55-63. 2. Calistri, A., et al., Role of multivesicular bodies and their components in the egress of enveloped RNA viruses. Rev Med Virol, 2009. 19(1): p. 31-45. 3. Chen, B.J. and R.A. Lamb, Mechanisms for enveloped virus budding: can some viruses do without an ESCRT Virology, 2008. 372(2): p. 221-32. 4. Hartlieb, B. and W. Weissenhorn, Filovirus assembly and budding. Virology, 2006. 344(1): p. 64-70. 5. Harty, R.N., No exit: targeting the budding process to inhibit filovirus replication. Antiviral Res, 2009. 81(3): p. 189-97. 6. Jasenosky, L.D. and Y. Kawaoka, Filovirus budding. Virus Res, 2004. 106(2): p. 181-8. 7. Liu, Y. and R.N. Harty, Viral and host proteins that modulate filovirus budding. Future Virol, 2010. 5(4): p. 481-491. 8. Urata, S. and J.C. de la Torre, Arenavirus budding. Adv Virol, 2011. 2011: p. 180326. 9. Chen, H., et al., TSG101: a novel anti -HIV-1 drug target. Curr Med Chem, 2010. 17(8): p. 750-8. 10. Jiang, Y., X. Liu, and E. De Clercq, New therapeutic",
      "Harty, Ronald N.",
      "Potent, orally available inhibitors of RNA virus egress, due to inhibition of the viral matrix protein PPxY",
      "Low nM (100 nM) potency for VLP and BiMC in vitro comparable to our current leads, FC -10696;  Inhibition of live virus proliferation (IC 50 100 nM);  Minimal compound effects on other cellular Nedd4 activities;  Little or no cytotoxicity in mammalian cell lines at 50-fold higher than minimum effective concentrations;  Suitable ADME and PK properties for once",
      "Role: Investigator 712018  6302019",
      "Harty, Ronald N. effects associated both with high molecular weight and lipophilicity. All compounds will be synthesized, purified and characterized at FCCDC labs. FCCDC has the capacity to prepare up to 250 g of any chemical analog. We will scale up, in multigram amounts, analogs that meet the criteria on the flow scheme for in vivo or other non - GLP assays . In addition, we will synthesize possible metabolites found during the metabolite identification processes. Our mandate is to improve aqueous solubility of our analogs while continuing to improve their potency, therapeutic index, ADME, PK and safety properties. We will focus on the following four areas: 1) Understand stereospecificity of FC -10696. Of our three lead compounds, FC -10696 has the best microsomal stability and solubility. Thus, we will synthesize both R and S enantiomers from the readily available starting materials S and R 1-1,1-biphenyl-4-ylethanol, respectively. These will be evaluated for potency, stability and solubility to determine the most promising enantiomer for further studies. We will also determine IV and PO PK in mice of the best enantiomer to determine if it could be an early front runner in our pursuit of a preclinical candidate. 2) Quarternary methylene analogs of FC -10696. Analogs C and or D in Fig. 9 avoid the chiral center of FC - 10696 altogether and we expect good potency in our primary assays based on previous SAR, as well as good stability in mouse microsomes. Our previous method of preparing compounds of this type utilized 4-(1 -bromo-1 - methyl -ethyl)-biphenyl to alkylate a quinoxaline thiol and led to FC -12004 in Table 1 among others, which gave poor yields of desired products ( 5). However, using a different synthetic",
      "Smithgall; Role: Key Personnel) 9012016 8312018",
      "Ebola Situation Report 2015). More recently the Democratic Republic of the Congo has second largest Ebola epidemic on record, which is ongoing, and as of June, 2020, with more than 2,200 lives lost, 3,400 confirmed infections and 1,160 survivors since the outbreak was declared on 1 August 2018. These unprecedented epidemics have spurred a call to action to discover new, cost effect therapies that combat this deadly pathogen. Among these efforts are several vaccines and antiviral drug candidates. However, the current vaccines are not a complete defense since they must be given pre-exposure. Oral antiviral agents used alone or in combination may be of value for individuals who respond adversely to the vaccine, and could be of value as prophylactic agents for individuals deemed to be in high risk situations such as military or healthcare workers. Therefore, effective therapeutics are needed to safeguard the largely immunologically naive human population by providing immediate protection. We have discovered two novel series of small molecule early leads that have nanomolar potency at inhibiting RNA virus budding and are Commercialization Plan Page 123 Contact PDPI: Harty, Ronald N. effective at reducing spread of live viruses including Marburg and Ebola virus. We are currently pursuing both series in order to increase our chance of success. Our most advanced lead is compound, FC -10696 and close structural analogs have the following attributes, tested to the extent afforded by the Phase I level of funding:  Potent activity (IC 50  100 nM) in the Marburg and Ebola VLP cellular budding (egress) assays,  Evidence that analogs of FC -10696 directly target the host cell and virus molecular machinery that effect viral cellular egress,  Live virus activity against RNA viruses (Marburg, Ebola),  Metabolic stability in mouse and human liver microsomes,  No cytochrome P450 3A4 activity which translates to a lower risk of drug- drug interactions,  Good drug levels upon IP administration in mice, and  Proof of concept in vivo reductions of Marburg virus (IP administration) in mouse models. However, it is important to note that improved analogs and additional novel composition of matter are expected to be discovered during the course of the Phase II STTR period of study, with fully characterized ADME properties suitable as antiviral oral drug candidates for preclinical development. 1.2 Expected outcomes: path to commercialization including timelines. The goal of this STTR Phase II application is to produce one or more preclinical drug candidates by medicinal chemistry synthesis optimization strategies that would be ready for GMP scale up and IND enabling studies. At the end of this award period we would have one or more drug candidates that are:  Potent, orally available inhibitors of RNA virus egress, due to inhibition of the viral matrix protein PPxY interaction with cellular Nedd4,  Broad spectrum activity agains t RNA viruses (Ebola, Marburg, Ebola, Lassa, RSV, etc).  Are not cytotoxic at concentrations 20-fold or higher than the live virus assay IC50,  Have suitable ADME and PK properties in mice for once a day oral administration,  Have oral activity in a mouse antiviral assay,  Have little or no cytochrome P450 inhibitory (Cyp) activity against the seven Cyp isoforms most likely to cause exposure variability and drug- drug interactions (DDIs),  Have little or no activity in a manual patch clamp assay to assess hERG (human Ether-à-go-go-Related Gene product), suggesting a greatly lower risk of the compound causing cardiac arrhythmias,  Have little or no activity in an Ames testing with and without metabolic activation, suggesting a greatly lower risk of the compound being mutagenic,  Have chemical syntheses that are efficient and scalable, and  Provide the basis for one or more composition of matter U.S. patent applications. We anticipate being in an ideal position to pursue IND enabling studies with our candidate compound and will require additional funding to support IND filing and early clinical development. Many of the future IND -enabling activities would be funded from sources listed later in this application. Such post -Phase II SBIR activities include:  Preparation of 300 g of non-GMP active pharmaceutical agent,  Selection of a suitable salt form (if necessary), vehicle and formulation, and evaluation of crystallinity and the propensity for polymorph formation,  Analytical methods development,  Drug substance stability testing, including standard pharmaceutics development work -up involving shelf- life and stress testing,  File additional provisional and PCT patent applications as necessary, and  Prepare 5 kg of GMP quality active pharmaceutical ingredient. Commercialization Plan Page 124 Contact PDPI: Harty, Ronald N. With the completion of GMP and non-GMP batches we plan to initiate the necessary studies to support an IND filing, specifically investigation of metabolites, in vitro toxicity and acute in vivo safety pharmacology and toxicity assessment. Activities include:  Metabolite ID (human, rat and monkey), characterization for toxicity and biological testing,  Full PK parameters in three relevant safety pharmacology species, mice, rabbit, and non-human primate (NHP), chosen since they are used in relevant in vivo pharmacology studies,  Investigation of Cyp450 inhibition and induction and transporter modulation in detail, for the potential for unexpected drug-drug interactions,  Enhanced survival in two or more GLP double blind placebo -controlled animal models of Ebola according to the Animal Rule of Efficacy (some of these studies might be done during the IND phase). Viral load and survival will also be assessed,  Completion of in vitro GLP toxicology (hERG, Ames, genotoxicity),  Conducting dose ranging and 28-day safety profiles in mice and NHPs,  Safety Pharmacology: Cardiovascular safety study in dogs, CNS safety in rats, respiratory safety in rats (if needed),  Reproductive toxicology in mice and rabbits (possibly do afterduring Phase I studies).  Conducting a pre-IND meeting with the FDA to establish and understand expectations, and  Initiating preparation of an IND to facilitate the transition into clinical evaluation. We have established a world-class PPxY Project Leadership Team (PPLT) to interpret data and provide strategic direction on a real time basis. The PPLT will regularly provide advice and guidance (at least monthly) in person, by Zoom, or by email. The PPLT will interpret data most appropriately and coordinate experimental design in close coordination with the Harty group and FCCDC management. The PPLT will conduct more formal quarterly meetings that Prof. Harty will chair, and any member of the PPLT may add items to the agenda prepared in adv ance of the meeting. The PPLT shall consist of the following members: Ronald N. Harty, Ph.D., Professor, University of Pennsylvania. Dr. Harty is Professor of Pathobiology and Microbiology, at the School of Veterinary Medicine. His research interests are focused on understanding the molecular mechanisms of filovirus (Ebola and Marburg viruses) and arenavirus (Lassa fever and Junin) assemblybudding, in order to identify broad- spectrum inhibitors of virus -host interactions. His research has directly led to the discovery of the antiviral agents described in this application. He is also interested in understanding the innate immune responses to virus infection that provide a critical first line of defense for the host against the invading pathogen. Prof. Harty has co-authored 80 scientific publications. Jay E. Wrobel, Ph.D., Vice President, Business Development, FCCDC. Dr. Wrobel has spent 37 years as a medicinal chemist in the pharmaceutical industry, 26 years at Wyeth (later Pfizer), with his last position as Senior Director of Medicinal Chemistry at the Collegeville, PA facility. At Wyeth, he mentored and guided the efforts of 31 Ph.D.MS medicinal chemists and worked successfully with outside alliance partners. He was directly involved in bringing forward nine development track candidates (Phase 0 and beyond) in a variety of therapeutic areas . He has co-authored 78 scientific publications and is an inventor on 83 issued US patents. Bruce Freedman, VMD, Ph.D., Associate Professor, University of Pennsylvania. Dr. Freedman is an Associate Professor of Pathobiology and Microbiology, at the School of Veterinary Medicine. Dr. Freedman is Director of the Microscopy Core facility at PennVet which houses state- of-the-art confocal and live cell imaging instruments that will be critical for assessing the efficacy of our budding inhibitors. Dr. Freedman is a long-standing collaborator with Dr. Ronald Harty, the PI on this proposal. Olena Shtanko, Ph.D., Staff Scientist, Texas Biomedical Research Institute. Dr. Shtanko researches fundamental molecular processes used by Ebola and Marburg viruses to disseminate within the host to understand the extraordinary capacity of these emerging viruses to trigger disease in affected individuals, with the aim to design and test antiviral therapies. Dr. Shtanko has over 16 years of conducting research on viruses, Commercialization Plan Page 125 Contact PDPI: Harty, Ronald N. with 9 years spent at the maximum containment laboratory (BSL-4) at Texas Biomedical Research Institute. Dr. Shtankos work has identified factors critical for Ebola virus and Crimean- Congo Hemorrhagic Fever virus entry into cells. Dr. Shtanko also extensively collaborates with a number of investigators to coordinate and perform virus studies to identify host factors critical for replication and spread of high-containment viruses as well as test novel antiviral therapeutics using a small animal model of Ebola and Marburg virus disease. Allen B. Reitz, Ph.D. CEO, FCCDC. Dr. Reitz has spent 38 years as a medicinal chemist in the pharmaceutical industry, 26 years with Johnson and Johnson (Spring House, PA). At JJ he mentored and guided a staff of 16 Ph.D.MS medicinal chemists and was co-inventor on 7 compounds that have entered human clinical trials. He has co-authored 150 scientific publications and is an inventor on 71 issued US patents. He is Editor -in-Chief and founding editor of Current Topics in Medicinal Chemistry . He has established two biotechnology companies: FCCDC and ALS Biopharma. FCCDC has already successfully partnered a separate technology platform for the treatment of Alzheimers disease, which has passed an interim futility analysis in Phase IIIII. Leo J. Adalbert, M.B.A. Mr. Adalbert will be a business development consultant for this project. An insightful, collaborative, and creative leader, Leo carries more than 25 years of deep cross -functional commercial and clinical development experience across both small molecules and biologics. Core therapeutic areas of expertise include cardiovascular, neuroscience, oncology, immunology, and GI, as well as biomarkers. Prior to founding StraNexa LLC, Leo held various leadership roles across marketing, business development, and alliance leadership at leading firms such as AstraZeneca, Merck, Astra Merck, and Regeneron. Leo earned an MBA from Harvard Business School (fellowship), and executive leadership certifications from Northwestern University (Kellogg) and University of Pennsylvania (Wharton). He also completed extensive coursework in biological sciences (Villanova UniversityBucks CC), earning high honors (Phi Theta Kappa distinction), and is a graduate of the HMX Program at Harvard Medical School (pharmacology and immunology). FCCDC has established a world-class Scientific Advisory Board, as posted on the company website, who are on call for questions regarding drug development strategy, flowchart design, chemistry and pharmacology. Nicholas A. Meanwell, Ph.D., Bristol-Myers Sq uibb Research  Development, Princeton, NJ. Nicholas A. Meanwell is currently Vice President, Discovery Chemistry Platforms, Small Molecule Drug Discovery where he has led drug discovery programs in the cardiovascular, neurosciences and virology therapeuti c areas, work that has resulted in the advancement of over 30 clinical candidates for the prevention of thrombosis, the treatment of stroke and therapy for viral infections, including human immunodeficiency virus-1 (HIV-1), hepatitis C virus (HCV) and respiratory syncytial virus (RSV). Significant compounds in the HCV arena include daclatasvir (Daklinza ), asunaprevir (Sunvepra), Daklinza and Sunvepra. Dr. Meanwell has authoredco-authored more than 250 publications, review articles, book chapters and editorials and is named as an inventorco- inventor on 138 issued U.S. Patents. Dr. Meanwell is the co-recipient of a PhRMA Research and Hope Award for Biopharmaceutical Industry Research, 2014 and the recipient of the 2015 Philip S. Portoghese Medicinal Chemistry Lectureship Award. He was inducted into the ACS Division of Medicinal Chemistry Hall of Fame on August 18th, 2015 and he is a member of the team that was recognized by a 2017 Heroes of Chemistry Award sponsored by the American Chemical Society. Eileen Jaffe, Ph.D., Fox Chase Cancer Center, Temple University School of Medicine  Drexel University School of Medicine. Eileen Jaffe received her PhD in Biochemistry from the University of Pennsylvania, working with Mildred Cohn. She was then a Postdoctoral Fellow at Harvard University with Jeremy Knowles. Since 2003 she has been at the Fox Chase Cancer Center in Philadelphia, PA, and is currently also an Adjunct Professor of Biochemistry at Temple University School of Medicine and Drexel University College of Medicine. Her research interests lie in understanding protein structure-function relationships using both biochemical and biophysical",
      "Lead optimization medicinal chemistry . Our current generation of PPxYNedd4 inhibitors",
      "provide insight into probe molecule identification and synthesis, in silico",
      "Available since September of 2019, the 5-laser BD FACSymphony cytometer is capable of 28 color acquisition, and it is equipped with a High Throughput Sampler. This advanced instrument features an ultra-quiet VPX electronics system that supports up to 50 high-performance photomultiplier tubes (PMTs) and improves detection sensitivity to enable the identification and analysis of rare cell types and events. The version at the Core is equipped with a 355 nm (UV), 405 nm (violet), 488 nm (blue), 561 nm (yellow -green) and 640 nm (red) lasers. When analyzing panels designed with BD Horizon Brilliant dyes, this platform helps to overcome research challenges, such as collecting maximal information from a precious sample, and increases lab throughput with broad phenotyping panels that combine multiple cell -line-specific panels. This highly customizable platform can be configured so that you can select from multiple laser wavelengths and power ratings, and choose the positions of decagon detection arrays to address the requirements of your specific research application. Beckman Coulter Cyan ADP: The CyAn LX Flow Cytometer provides three excitation lines, eleven parameters, complete compensation, and high-speed acquisition, in a bench-top configuration. The 405 nm, 488 nm, and 633 nm lasers are fixed and alignment-free, with spatially separated beams for minimal optical crosstalk. Spatially separated optical apertures, high-performance solid -state detectors, and an obscuration bar minimize unwanted laser radiation. User -interchangeable filters provide experimental flexibility. Eleven acquisition channels (including forward scatter, side scatter, and 9 colors) provide parallel processing of pulse height, pulse length, area, and log for each parameter. BD Accuri C6 Plus: The 2-laser BD Accuri C6 plus cytometer is capable of 4-color acquisition. The BD Accuri is dedicated to BSL2 HIV studies. The BD Accuri C6 Plus is equipped with a blue and red laser, two light scatter detectors, and four fluorescence detectors with optical filters optimized for the detection of many popular fluorochromes. BD FACSAria cell sorter: The 4-laser FACSAria II sorter is capable of 18-color, 4- way sorting and has an automated cell deposition unit for 6, 12, 24, 48 and 96 well sorting. The BD FACSAria cell sorter is a fixed- alignment cuvette flow cell that can detect up to 18 colors, for a total of 20 parameters. Cells passing through the cuvette flow cell are excited by four solid state lasers at 407-nm, 488-nm, 561-nm, and 633-nm. Advanced digital electronics improve sort performance and provide significant advantages in instrument ease-of-use. This cell sorter is contained within a Baxter High Containment Biosafety cabinet within one of the BSL3 laboratories. Facilities  Other Resources Page 16 Contact PDPI: Harty, Ronald N. BDInflux cell sorter: The 5-laser BDInflux cell sorter is capable of 6-way sorting. The BDInflux is primarily dedicated to single cell sorting of malaria parasites and sorting of pluripotent stem cells. Luminex 100 and 200: The Luminex with xMAP (multi -analyte platform) technology systems measure analytes (cytokines, chemokines, etc.) on a multiplex platform. These instruments are operated with dedicated computers running Luminex Xponent software; analysis of reactivity is done with Masterplex QT software. Molecular Services Core instrumentation and services The Molecular Services Core (MSC) is located in the Central Core Facility and is operated by a Director, a full-time manager, and technical support staff. The MSC is dedicated to assist researchers in the development and implementation of state-of-the-art molecular",
      "Provisions to minimize discomfort, distress, pain and injury",
      "PPxY complex as potent, broad-spectrum antivirals. For this grant, our team, that",
      "PI",
      "toward the identification of budding inhibitors."
    ],
    "methodology": "A European Journal 2009, 15(44) , 11867-11877.  Trapping a Folding Intermediate of the α-Helix: Stabilization of the π-Helix. R.N. Chapman, J.L. Kulp III, A. Patgiri, N.R. Kallenbach, C. Bracken, P.S. Arora, Biochemistry 2008, 47(14) , 4189-4195.  Evaluation of Biologically Relevant Short α-Helices Stabilized by a Main -Chain Hydrogen-Bond Surrogate. D. Wang, K. Chen, J.L. Kulp III, P.S. Arora, Journal of the American Chemical Society 2006, 128(28) , 9248-9256. 3. My postdoctoral fellowship and subsequent two year federal employment was in The Molecular Interfaces and Tribology Section of the Department of Chemistry, Naval Research Laboratory. This section studies interdisciplinary questions in basic and applied interface and surface science. Our groups research stretched from advancing broad analytical methods, such as vibrational circular dichroism, to evaluating both prototypical and realistic interfaces to crafting and improving molecular surfaces and peptide architectures for applications such as controlling biological activity, anti -fouling, or lowering contact friction. One focus area was in the design and employment of bioinspired materials that conserve and utilize the exceptional characteristics of biomaterials while combining added functionality, such as fluorescence or conductivity, not originating from natural materials.  Vibrational Circula r-Dichroism Spectroscopy of Homologous Cyclic Peptides Designed to Fold into β Helices of Opposite Chirality, J.L. Kulp III, K.P. Fears, R. Lombardi, D.Y. Petrovykh, J.C. Owrutsky, L.A. Biosketches Page 40 Conta ct PDPI: Harty, Ronald N. Nafie, T.D. Clark, Biointerphases , 2011, 6(1), 1-7.  Characterization of the adhesive plaque of the barnacle Balanus Amphitrite: Anyloid -like nanofibrils are a major component, D.E. Barlow, G.H. Dickinson, B. Orihuela, J.L. Kulp III, D. Rittschof, K.J. Wahl, Langmuir 2010, 26(9) , 6549-6556. Highlighted in Chemical and Engineering News 2010, 88(9) , 34.  Molecular Recognition and Supramolecular Self-Assembly of a Genetically Engineered Gold Binding Peptide on Au 111. C. So, J.L. Kulp III, E.E. Oren, H. Zareie, H. Zareie, C. Tamerler, J.S. Evans, M. Sarikay a, ACS Nano 2009, 3(6), 1525-1531.  Laser Ablation of Micropores for Formation of Artificial Planar Lipid Bilayers. T.J. OShaughnessy, J.E. Hu, J.L. Kulp III, S.M. Daly, F.S. Ligler, Biomedical Microdevices 2007, 9(6), 863-868. 4. The beginning of my computational chemistry career began when I was using nuclear magnetic resonance (NMR) spectroscopy to determine the structures of biological macromolecules like peptides, polypeptides, and proteins. To do this, I would rely on 1D, homo- and hetero-nuclear 2D, and triple resonance, 3D, spectra. Some other examples of this work are cited in scientific contribution 3, above. After extracting all available information about the atomic distances and torsion angles from the NMR spectra, computer -aided structure calculations would convert distance and torsion angle data (constraints) into the 3D atomic model. For these calculations, a molecular dynamics simulation was used with simulated annealing; increasing temperature to generate an ensemble of conformations then decreasing temperature with increasing constraints based on the NMR data. The result is a family of structures that span a relatively narrow conformational space (low RMSD), unless intrinsically flexible, and obeys the constraint cri teria.  Structural Properties of an Artificial Protein that Regulates the Nucleation of Inorganic and Organic Crystals. J.L. Kulp III, T. Minamisawa, K. Shiba, M. Tejani, J.S. Evans, Langmuir 2007, 23(7) , 3857-3863.  Probing the Conformational Features of a Phage Display Polypeptide Sequence Directed Against Single Walled Carbon Nanohorn Surfaces. J.L. Kulp III, K. Shiba, J.S. Evans. Langmuir 2005, 21(25) , 11907- 11914.  Affinity Selection of Peptide Phage Libraries Against Single-Wall Carbon Nanohorns Identifies a Peptide Aptamer with Conformational Variability D. Kase, J.L. Kulp III, M. Yudasaka, J.S. Evans, S. Iijima, K. Shiba, Langmuir 2004, 20(20) , 8939-8941.  Molecular Characterization of a Prokaryotic Polypeptide Sequence that Catalyzes Au Crystal Formation. J.L. Kulp III, M. Sarikaya, J.S. Evans, J. Mat. Chem.2004, 14(14) , 2325-2332. Complete List of Published Work in MyBibliography: http:www.ncbi.nlm.nih.govsitesmyncbijohn.kulp.2bibliography48391280publicsortdatedirectionascen ding D. RESEARCH SUPPORT Cu rrent: Molecular Mechanism of the Dual Effects of Capsid Assembly Effectors on HBV Capsid Assembly and Disassembly (DoDPRMRP). Role: Investigator 912017  8312020 The goal of this project is to obtain molecular insights on the mechanism of two distinct chemotypes of hepatitis B virus (HBV) core protein allosteric modulators (CpAMs) on the assembly and disassembly (or uncoating) of HBV nucleocapsids. Previous: Role: Investigator 712018  6302019 The goal of this project is to progress disubstituted aminothiazoles that demonstrate remarkable selective toxicity for HCC -derived cell lines versus non-HCC liver lines and most other cancer lines. Small Molecule Inhibitors of HIV1 Nef Virulenc e Factor for Treatment of HIV-Aids R42GM112516-01 (PI: Smithgall; Role: Key Personnel) 9012016 8312018 This Phase II STTR involves the discovery of novel small molecule inhibitors of the HIV -1 virulence factor Nef, and has a substantial structural biology and computational chemistry component. Biosketches Page 41 Contact PDPI: Harty, Ronald N. BIOGRAPHICAL SKETCH Provide the following information for the Seniorkey personnel and other significant contributors. Follow this format for each person. DO NOT EXCEED FIVE PAGES . NAME: Reitz, Allen Bernard eRA COMMONS USER NAME (credential, e.g., agency login): POSITION TITLE : Chief Executive Officer EDUCATIONTRAINING (Begin with baccalaureate or other initial professional education, such as nursing, include postdoctoral training and residency training if applicable. Adddelete rows as necessary.) INSTITUTION AND LOCATION DEGREE (if applicable) Completion Date MMYYYY FIELD OF STUDY University of California, Santa Barbara University of California, San Diego University of Pennsylvania (Wharton) B.A. M.S. Ph.D. M.S. 081977 061979 011982 062003 Biochemistry and Molecular Biology Chemistry Chemistry Management of Technology A. Personal Statement Dr. Reitz has had 38 years of experience in the pharmaceutical industry including 26 years with Johnson  Johnson in various drug discovery research and management roles. He is an inventor of eight compounds that have entered clinical trials, including two from FCCDC which he founded in 2008 ( www.fc -cdci.com). For example, he is a coinventor of troriluzole which emerged from an Phase III SBIR -funded project at FCCDC on which he was the Principal Investigator, and which is currently in Phase IIIII clinical trial having recently passed an interim futility analysis for the treatment of Alzheimers disease. He is co-author of 160 scientific publications, co-inventor on 71 issued U.S. patents, Editor -in-Chief of the journal Current Topics in Medicinal Chemistry , on the Editorial Advisory Board of ACS Med. Chem. Lett. Dr. Reitz has extensive experience in project and portfolio management, target validation, hit triage, eADME profiling, and preclinical candidate selection. He is currently serving as an ad hoc reviewer on two of the rapid review COVID -19 study sections in NIAID. Dr. Reitz has worked with Drs. Wrobel and Prof. Harty since the inception of the FCCDC  U. Penn. collaboration of this novel host directed approach to treat Ebola and Marburg infection. He will continue to provide overall guidance as much as possible, working with Dr. Wrobel, and will seek to ensure a successful transition at the end of Phase II to further development and eventual commercialization including through taking advantage of various initiatives of the FDA such as the Neglected Disease Priority Review Voucher program. Loughran HM, Han Z, Wrobel JE, Decker SE, Ruthel G, Freedman BD, Harty RN, Reitz AB. Quinoxaline-based inhibitors of Ebola and Marburg VP40 egress. Bioorg Med Chem Lett. 2016 Aug 1;26(15):3429-35. doi: 10.1016j.bmcl.2016.06.053. PubMed PMID: 27377328; PubMed Central PMCID: PMC4955528 Han Z, Lu J, Liu Y, Davis B, Lee MS, Olson MA, Ruthel G, Freedman BD, Schnell MJ, Wrobel JE, Reitz AB , Harty RN. Smal l-molecule probes targeting the viral PPxY -host Nedd4 interface block egress of a broad range of RNA viruses. J Virol. 2014 Jul; 88(13): 7294-306. doi: 10.1128JVI.00591-14. PubMed PMID: 24741084; PubMed Central PMCID: PMC4054416. B. Positions and Honors Positions and Employment 1982 -1983 Postdoctoral Research Fellow, McNeil Pharmaceutical 1983 -1984 Research Scientist, McNeil Pharmaceutical 1984 -1987 Senior Scientist, Janssen Research Foundation Biosketches Page 42 Conta ct PDPI: Harty, Ronald N. 1987 -1992 Principal Scientist, R W Johnson Pharm. Res. Inst. 1992 -1998 Research Fellow, R W Johnson Pharm. Res. Inst. 1999 -2007 Senior Research Fellow, JNJPRD 2008 Vice President, Research and Development, AlsGen, Inc. 2008-present Founder and CEO, Fox Chase Chemical Diversity Center, Inc. 2009-present Co-Founder and CEO, ALS Biopharma, LLC 2010-present Founder and President, Pennsylvania Drug Discovery Institute (non-profit) Other Experience and Professional Memberships 2015 National Co-Chair of the BIO Boot Camp, associated with the national BIO meeting 2014-present Editorial Advisory Board member, ACS Med. Chem. Lett. 2014 -2016 Co-Editor, Technology Transfer and Entrepreneurship 2008-present Adjunct Professor, Drexel University College of Medicine 2008 -2009 Consultant to the World Health Organization for the Tropical Diseases (TDR) 2005-present Have served on 42 NIH Study Sections including Drug Discovery for the Nervous System (2012- 2014), and am currently an ad hoc member of COVID -19 rapid review study section study sections ZAI1 RG -W (S2) and ZAI1 RG -W (J2), July 14 and Oct. 15 respectively. 2000-present Editor -in-Chief, Current Topics in Medicinal Chemistry 1995-1996 Philadelphia Organic Chemists Club, Assistant Chairman, and then Chairman 1985-1986 Philadelphia Organic Chemists Club, Assistant Secretary, and then Secretary Honors 2016 Cornerstone Award CEO Honoree, Winner for Bucks County, PA 2011 Gold Award, Best Early- Stage Company, Philadelphia Business Journal 2006 Ronald J. Borne Distinguished Lectureship Award, in Recognition for Outstanding Contributions to Medicinal Chemistry, Department of Medicinal Chemistry, School of Pharmacy, University of Mississippi (April 18, 2006). Highlighted on pg. 6 of AACP News 2006, 37(9) . 2003 Industrial Chemistry Award of the Philadelphia Organic Chemists Club.Poster Session, Reception, and Lecture: May 29, 2003 1999 Johnson  Johnson Corporate Office of Science and Technology Excellence in Science Award 1997 Johnson  Johnson Significant Achievement Award for Creating the POP Process to Rapidly Move New Compounds from the Laboratory into Clinical Trials 1994 Philip B. Hofmann Research Scientist Award for Outstanding Achievement 1990 Johnson  Johnson Achievement Award for Outstanding Contributions C. Contribution to Science 1. Target Validation. Research in this area has been to understand the relation of structure to function among related compound libraries, using modern methods of synthetic chemistry and SAR development. Along those lines, FCCDC has submitted 12,000 new chemical entities in the context of individual collaborations with academic and biotech research partners. a. Punchi Hewage, A. N. D.; Yao, H.; Nammalwar, B. Gnanasekaran, K. K.; Lovell, S.; Bunce, R. A.; Eshelman, K.; Phaniraj, S. M.; Lee, M. M.; Peterson, B. R.; Battaile, K. P.; Reitz, A. B.; Rivera, M. Small Molecule Inhibitors of the BfrB-Bfd Interaction Decrease Pseudomonas aeruginona Fitness and Potentiate Fluoroquinoline Activity. J. Am. Chem. Soc. 2019, 141, 8171 -8184. b. Karpova, Y.; Wu, C.; Divan, A.; McDonnell, M. E.; Hewlett, E.; Makhov, P.; Gordon, J.; Ye, M.; Reitz, A. B.; Childers, W. E.; Skorski, T.; Kolenko, V.; Tulin, A. V. Non-NAD-like PARP -1 inhibitors in prostate cancer treatment. Biochem. Pharmacol. 2019, 167, 149-162. c. Robinson, R. M.; Reyes, L.; Duncan, R. M.; Bian, H.; Reitz, A. B .; Manevich, Y.; McClure, J. J.; Champion, M. M.; Chou, C. J.; Sharik, M. E.; Chesi, M.; Bergsagel, P. L.; Dolloff, N. G. Inhibitors of the protein disulfide isomerase family for the treatment of multiple myeloma. Leukemia 2019, 33, 1011- 1022. d. Martin, M. D.; Calcul, L.; Smith, C.; Jinwal, U. K.; Fontaine, S. N.; Darling, A.; Seeley, K.; Woitas, L.; Narayan, M.; Gestwicki, J. E.; Smith, G. R.; Reitz, A. B .; Baker, B. J.; Dickey, C. A. Synthesis, Stereochemical Analysis, and Derivatization of Myricanol Provide New Probes That Promote Autophagic Tau Clearance. ACS Chem. Biol. 2015, 10, 1099 -1109. Biosketches Page 43 Contact PDPI: Harty, Ronald N. 2. Hit to Lead Medicinal Chemistry. Much effort has focused on this area, in which libraries of related compounds are prepared in order to obtain maximum diversity based upon common themes dictated by the structure of the target, pharmacophore analysis, and the creation of new intellectual property. Multiple projects (e.g. BACE, EBNA1) have involved real -time X -ray co -crystallography collaboration and analysis. a. Baxter, E. W.; Conway, K. A.; Kennis, L.; Bischoff, F.; Mercken, M. H.; De Winter, H. L.; Reynolds, C. H.; Tounge, B. A.; Luo, C.; Scott, M. K.; Huang, Y.; Braeken, M.; Pieters, S. M. A.; Berthelot, D. J. C.; Masure, S.; Bruinzeel, W. D.; Jordan, A. D.; Parker, M. H.; Boyd, R. E.; Qu, J.; Alexander, R. S.; Brenneman, D. E.; Reitz, A. B. 2-Amino -3,4-dihydroquinazolines as Inhibitors of BACE-1 ( β-Site APP Cleaving Enzyme): Use of Structure Based Design to Convert a Micromolar Hit into a Nanomolar Lead. J. Med. Chem. 2007, 50, 4261-4264. b. Huang, Y.; Strobel, E. D.; Ho, C. Y.; Reynolds, C. H.; Conway, K. A.; Piesvaux, J. A.; Brenneman, D. E.; Yohrling, G. J.; Arnold, H. M.; Rosenthal, D.; Alexander, R. S.; Tounge, B. A.; Mercken, M.; Vandermeeren, M.; Parker, M. H.; Reitz, A. B.; Baxter, E. W. Macrocyclic BACE Inhibitors: Optimization of a Micromolar Hit to Nanomolar Leads. Bioorg. Med. Chem. Lett. 2010, 203158-3160. c. McDonnell, M. E.; Vera, M. D.; Blass, B. E.; Pelletier, J. C.; King, R. C.; Fernandez -Metzler, C.; Smith, G. R.; Wrobel, J.; Chen, S.; Reitz, A. B. Riluzole prodrugs for melanoma and ALS: design, synthesis, and in vitro metabolic profiling. 2012 , Bioorg. Med. Chem., 20, 5642-5648. d. Rubin, H.; Selwood, T.; Yano, T.; Weaver, D. G.; Loughran, H. M.; Costanzo, M. J.; Scott, R. W.; Wrobel, J. E.; Freeman, K. B.; Reitz, A. B . Acinetobacter baumannii OxPhos inhibitors as selective anti-infective agents. Bioorg. Med. Chem. Lett. 2015, 25, 378-383. 3. Lead Optimization (LO) Medicinal Chemistry Advancing to Preclinical and Clinical Development. Considerable emphasis has been placed on this phase of the drug discovery process, for which eight NMEs have entered human clinical trials. The 71 issued U.S. patents are largely devoted to LO SAR programs. There are three basic considerations for a medicinal chemist which apply to all indications: (1) route of administration (e.g. p.o, i.v., etc.), and (2) dosing regimen (acute, subchronic, chronic), and (3) standard of care (crowded marketplace or keen unmet need). a. Reitz, A. B.; Baxter, E. W.; Bennett, D. J.; Codd, E. E.; Jordan, A. D.; Maryanoff, B. E.; McDonnell, M. E. ; Ortegon, M. E.; Renzi, M. J.; Scott, M. K.; Shank, R. P.; Sherrill, R. G.; Vaught, J. L.; Wustrow, D. J. N-Aryl-N-Benzylpiperazines as Potential Antipsychotic Agents. J. Med. Chem. 1995 , 38, 4211- 4222. Description of mazapertine, which advanced to Phase II clinical trials. b. Parker, M. H.; Smith-Swintosky, V. L.; McComsey, D. F.; Huang, Y.; Brenneman, D.; Klein, B.; Malatynska, E.; White, H. S.; Milewski, M. E.; Herb, M.; Finley, M. F. A.; Liu, Y.; Lubin, M. L.; Qin, N.; Iannucci, R.; Leclercq, L.; Cuyckens, F.; Reitz, A. B .; Maryanoff, B. E. Novel, Broad-Spectrum Anticonvulsants Containing a Sulfamide Group: Advancement of N -(Benzob thien-3 - yl)methylsulfamide (JNJ-26990990) into Human Clinical Studies. J. Med. Chem ., 2009, 52, 7528- 7536. Anticonvulsant JNJ-26990990 which advanced into Phase I clinical trials. c. Ross, T. M.; Battista, K.; Bignan, G. C.; Brenneman, D. E.; Connolly, P.. J.; Liu, J.; Middleton, S. A.; Orsini, M.; Reitz, A. B .; Rosenthal, D. I.; Scott, M. K.; Vaidya, A. H. A selective small molecule NOP (ORL -1 receptor) partial agonist for the treatment of anxiety. Bioorg. Med. Chem. Lett. 2015, 25, 602- 606. Anxiolytic ORL-1 receptor agonist that advanced into Phase I clinical trials. d. Messick, T. E.; Smith, G. R.; Soldan, S. S.; McDonnell, M. E.; Deakyne, J. S.; Malecka, K. A.; Tolvinski, L.; van den Heuvel, A. P. J.; Gu, B. W.; Cassel, J. A.; Tran, D. H.; Wasserman, B. R.; Zhang, Y.; Velvadapu, V.; Zartler, E. R.; Busson, P.; Reitz, A. B .; Lieberman, P. M. Structure-based design of small -molecule inhibitors of EBNA1 DNA binding blocks Epstein-Barr virus latent infection and tumor growth. Sci. Tranl. Med. 2019, 11, pii: eaau5612. doi: 10.1126scitranslmed.aau5612. EBNA1 inhibitor currently in Phase II clinical trials, discovered at FCCDC 4. Synthetic Organic Che mistry. Dr. Reitz was inspired by organic faculty as an undergraduate, which was reinforced by a positive experience as a graduate student with Murray Goodman at UCSD. Highly cited publications include a Chem. Rev. article on the Wittig reaction and Vol. 59 of Organic Reactions on reductive aminations using boron -containing reagents consisting of 700 pages and an entire volume. Research has been conducted in the areas of conformational analysis and new synthetic methods. Biosketches Page 44 Conta ct PDPI: Harty, Ronald N. a. Larsen, B. J.; Rosano, R. J.; Ford -Hutchinson, T. A.; Reitz, A. B .; Wrobel, J. A. A method for C2 arylation of 1,3-indandiones. Tetrahedron 2018, 74, 2762-2768. b. Pelletier, J. C.; Velvadapu, V.; McDonnell, M. E.; Wrobel, J. E.; Reitz, A. B. Intramolecular rearrangement of α -amino acid amide derivatives of 2-aminobenzothiazoles.Tetrahedron Lett. 2014, 55, 4193-4195. c. Maryanoff, B. E.; Reitz, A. B. The Wittig Olefination Reaction and Modifications Involving Phosphoryl - Stabilized Carbanions. Stereochemistry, Mechanism, and Selected Synthetic Aspects. Chem. Rev.1989, 89, 863-927. d. Baxter, E. W; Reitz, A. B. Reductive Aminations of Carbonyl Compounds with Borohydride and Borane Reducing Agents In Organic Reactions : Wiley, New York, p. 1-714, Vol. 59, 2002. Complete List of Published Work in MyBibliography: http:www.ncbi.nlm.nih.govsitesmyncbi1nyMq8FpqE0A5bibliography47313087publicsortdatedirectionascending D. Research Support Ongoing Research Support Riluzole Prodrugs for Melanoma and ALS R44 CA156781-04A1 (PI: Reitz) 9118- 83121 This Phase II SBIR Bridge application involves the further pre-clinical development of troriluzole, an actively - transported (PepT1) prodrug of riluzole, . A Phase II clinical trial for the treatment of solid tumors is included in the grant, to be conducted at the Dana Farber Cancer Institute. Overlap  none. Troriluzole for the Treatment of Cocaine Addiction R41 DA047169-01 (PI: Rawls, Temple University; Role, Key Personnel) 070118-73021 (NCE) In this project, Prof. Rawls is testing whether troriluzole can counter the effects of cocaine in rats. Currently in the no cost extension phase. Overlap  none. Psychoactive bath salts and the glutamate system R01 DA039139-02 (PI: Rawls, Temple University; Role, Key Personnel) 8116- 73121 This R01 from Prof. Rawls at Temple University involves understanding the pharmacology of bath salts in CNS receptor and behavioral assays. Overlap  none. Targeting Telomerase for Pancreatic Cancer Therapeutics R01 CA208439-01 (PI: Skordalakes; Role, Key Personnel) 8118- 73121 This work is to discover new telomerase inhibitors for the treatment of pancreatic cancers. Overlap  none. DART2.0: comprehensive cell type-specific behavioral neuropharmacology RF1 MH117055-01 (PI: Tadross, Duke University; Role, Key Personnel) 91418-9 1321 Prof. Tadross is the PI of a Brain R01 looking to identify new DART bifunctional probes to understand the localization and function of important receptor systems in the CNS. Our role is to perform synthetic chemistry to obtain suitable probes which are then studied in his laboratory. Overlap  none Biosketches Page 45 Contact PDPI: Harty, Ronald N. FCCDC Budget Justification Personnel Jay E. Wrobel, Ph.D. Vice President, Academic Affairs and Senior Research Fellow, Key Personnel, FCCDC Project Leader (Yrs 1-3: 3 cal. months). Dr. Wrobel has 35 years of experience in the pharmaceutical industry focused on drug design and discovery on a variety of drug discovery programs. He was formerly Senior Director of Medicinal Chemistry at WyethPfizer in Collegeville, PA where he mentored and guided the efforts of up to 31 medicinal chemists and worked successfully with outside alliance partners in the design and synthesis of novel agents for multiple drug discovery programs (hit to Preclinical Development). Dr. Wrobel was directly involved in bringing forward nine development track candidates (phase 0 and beyond). He has 78 peer reviewed articles and is an inventor on 83 U.S. Patents. Dr. Wrobel has expertise in project and portfolio management, hit triage, hit -to-lead and lead optimization from hits found using enabling technologies such as high-throughput screening, fragment based drug discovery, and virtual screening. In these efforts he has employed ligand and structure based SAR methods, analytical chemistry, multi -step synthesis, parallel synthesis, data management and ADME analysis. In his current position as VP Academic Relations at Fox Chase Chemical Diversity Center, Dr. Wrobel has collaborated and co-published with Professor Harty on in vitro and in vivo characterization of first-in -class novel inhibitors of filovirus PPxY interaction with host protein Nedd4 as an approach toward broad-spectrum antiviral therapy . Dr. Wrobel will be responsible for FCCDC project design, direction of the project and data interpretation. He will also organize regular project meetings among team members and consultants to maximize efficiency. Allen B. Reitz, Ph.D., CEO, Key Personnel (Yrs 1-3: 0.6 cal. months). Dr. Reitz has had 38 years of demonstrated accomplishment as a medicinal chemist in the pharmaceutical industry, including 26 years with Johnson  Johnson. He is a co-inventor as well as the project leader in most cases of nine compounds that have entered human clinical trials, two of which are currently in the clinic. He has 150 scientific publications and 71 issued U.S. patents, and is the Editor of the journal Current Topics in Medicinal Chemistry . He has extensive experience in project and portfolio management, target validation, hit triage, hit to lead and lead optimization medicinal chemistry, eADME profiling, and preclinical candidate selection. He is also Adj. Prof. at Drexel University, College of Medicine. Dr. Reitz founded FCCDC in 2008 to provide translational biomedical research in the area of early drug discovery research including medicinal chemistry hit to lead and lead optimization research. Dr. Reitz will assist Dr. Wrobel on project design and data interpretation. He will also pay particular attention to developing the required transition beyond Phase II, including a suitable project strategy, implementation of the Commercialization Plan, and interface with potential development partners and other interested parties. John Kulp III, Ph.D., Director of Computational Chemistry, Key Personnel (Yrs 1-3: 0.6 cal. months). Dr. Kulp has 9 years of computational chemistry experience with 17 peer reviewed publications. After his PhD and postdoctoral training he spent 4 years at BioLeap, a small business that specializes in protein-protein interactions and that has a proprietary fragment-based computational chemistry software platform. There he was the project leader for BioLeaps PCSK9 program. He will perform computer -assisted drug design studies, and help to provide insight by the creation of computational pharmacophores during the course of the program. Katie B. Freeman, Ph.D., Director of Biology, FCCDC (Yrs 1 -3: 0.6 cal. monthsyr). Dr. Freeman has extensive experience with numerous in vitro assay formats, in microbiological methodologies, mammalian cell culture, bioanalytical methodologies. She will conduct cytotoxicity assays at FCCDC in liver HepG2 and endothelial HUVEC to compare the relative in vitro therapeutic index between series and individual compounds. In addition, she will perform formulations development as required to support the in vivo evaluation at TBRI. Hong Ye, Scientist (Yrs 1-3 : 12 cal. months), Ms. Ye is an exceptionally talented bench medicinal chemist, who consistently ranks among the top staff at FCCDC in terms of productivity and creativity. She previously had 20 years of experience at Johnson  Johnson in Spring House PA and other companies before joining FCCDC in June 2015. She has been the primary laboratory medicinal chemistry on the Harty MarburgEbola project and will continue to do so working under the direction of Dr. Wrobel with proper attention being paid to labor atory safety, confidentiality, productivity, notebook keeping and spectral data records retention. Budget Justification Attachment Page 55 Contact PDPI: Harty, Ronald N. Leo J. Adalbert, M.B.A. Technical and Business Assistance (TABA). ($50,000 over three years requested) Mr. Adalbert is a Harvard MBA who will be our business development consultant for this project. He has more than 25 years of deep cross -functional commercial and clinical development experience across both small molecules and biologics. Prior to founding StraNexa LLC, Leo held various leadership roles across marketing, business development, and alliance leadership at leading firms such as AstraZeneca, Merck, Astra Merck, and Regeneron. Mr. Adalbert earned his MBA from Harvard Business School while on a fellowship, and holds executive leadership certifications from Northwestern (Kellogg) and the Univers ity of Pennsylvania (Wharton). He will interface with government agencies (FDA, BARDA) and investors as appropriate. Equipment No new equipment is requested. SuppliesMaterials will be purchased as needed for routine medicinal chemistry, including the following: Year 1 Year 2 Year 3 Total Solvent and reagents: Consumables lab supplies: Glassware: Total per year: Other Expenses Contracted Research Organization Expenses: over three years Contract Research Organization (CRO) Expenses  ADMET and PK testing will be performed as in the following schedule. Contractor names are in parentheses, and have all worked effectively with FCCDC in the past. Budget Justification Attachment Page 56 Contact PDPI: Harty, Ronald N. Indirect Cost and Fee Indirect costs of 40 are requested. A fee of 7 is requested. Justification for budget above statutory budget limitations According to National Institutes of Health SBA -Approved SBIRSTTR Topics for Awards over Statutory Budget Limitations as listed in Appendix A of the omnibus 2020 solicitation (https:sbir.nih.govsitesdefaultfilesNIH_Topics_for_Budget_Waivers.pdf), the topic discovery and development of therapeutics for infectious diseases (the prime concern of this proposal) is listed in NIAID, Division of Microbiology and Infectious Disease (DMD) as eligible for budgets over the statutory maximum amounts. Small molecule drug discovery is very expensive and labor intensive, involving the synthesis and evaluation of a large number of test compounds in an iterative fashion. In this STTR Phase II grant application, we embark on a full lead optimization program with the goal of obtaining preclinical drug candidates for the treatment of Marburg, Ebola and related viral infections. Our current lead series are novel and suitable for composition of matter patent protection, and a US patent has issued on the original series which will be supplemented by additional provisional patent applications over time. We expect the optimized compounds at the end of Phase II to have increased potency, and be safe and suitable for oral administration. Budget Justification Attachment Page 57 Contact PDPI: Harty, Ronald N. LE O J.AD ALBE RT GRE A T E R P HI L A D E L P HI A ARE A , PA 1890 1 www.stranexa.com  BIO-PHARMA EXECUTIVE  MARKETING  STRATEGY  GLOBAL COMMERCIALIZATION  BD  Dynamic, creative, and strategic business leader with proven expertise in strategic commercialization, go- to- market strategy, product launches, business development, and consulting across all phases of development, inclusive of biologics and small molecules.  Strong neuroscience, immunology, oncology, and rare diseaseOrphan Drug commercialization and scientific expertise across multiple disease areas.  Outstanding track record of success in identifying and evaluating opportunities, developing a vision for growth, and building differentiated franchises.  Lead teams to exceed expectations utilizing high initiative, strong team -building skills, creative problem -solving, cross -functional leadership, and collaboration. Harvard MBA ( Fellowship ). CORE COMPETENCIES  Partnering  Collaboration  LeadershipVision  New Product Launches  Promotion  Opportunity Identification  Customer, Competitor  Market Analysis  Product Development  Negotiations  Transactions  Capability Creation  InnovationMedical Science  Strategic  Tactical Agility  Critical ThinkingProblem Solving  Talent Development  KOL and Advocacy Group Partnering  Metrics  Governance  Life-Cycle Management PROFESSIONAL EXPERIENCE AND KEY ACCOMPLISHMENTS STRANEXA, LLC ( www.stranexa.com) (2014 -Present) DOYLESTOWN, PA Founder and Principal Lead bio- pharmaceutical strategic commercialization consulting practice with focus on specialty and rare diseaseOrphan Drug markets. Passion for building differentiated brands and capabilities transformative for patients and communities across for-profit and non-profit enterprises through a science -driven, market -led, patient - focused approach.  Led significant projects across multiple domains that consistently exceeded client expectations (portfolio and commercialization strategy, go-to-market strategy, product launch planning, BDalliance leadership, competitor and market analyses, communications strategy and publications, non- profit market development).  Strong scientific foundation across diverse diseases and conditions.  Expert instructor with Biotech Pri mer www.biotechprimer.com (Strategic Commercialization Within BioPharma and Drug Development Immersion ). CSL BEHRING, INC. (2012 -2013) KING OF PRUSSIA, PA Global Marketing Director, Immunoglobulin (IgG) Marketing  Strategic Commercialization (12-13) Consultant (12) Led global marketing strategy, commercialization, launch planning and core tactics for global IgG portfolio with focus on HIZENTRA and PRIVIGEN. Led global commercial strategy for CSL362 (hematological tumors).  Chaired global business team and led major cross- functional initiatives guiding market leadership for key brands (globally PRIVIGEN revenues up 17.9 to $1B; HIZENTRA revenues up 45.2 to $402MM).  Developed innovative strategies and major initiatives to build market leadership and differentiation. AXCAN PHARMA, INC. (2008- 2011) BRIDGEWATER, NJ Director Business Development and Alliance Management (company restructured in 2011)  Successfully divested global PHOTOFRINPHOTOBARR oncology drugdevice business, exceeding internal objectives (strong cash return, elimination of significant ongoing costs, and placement of commercial and RD employees in acquiring firm). Led entire transaction process, including prospectus development, buyer identification, business case preparation, due diligence, negotiations and transition planning.  Created new Alliance Management capability (vision, strategies, metrics, tools and roll-out plan). Effectively managed ongoing and new alliance relationships.  Selected to lead LACTEOL Global Product Team, and helped advance critical lifecycle initiatives and global presence in new markets. Page 1 of 4 Budget Justification Attachment Page 58 Contact PDPI: Harty, Ronald N. LE O J.AD ALBE RT PROFESSIONAL EXPERIENCE AND KEY ACCOMPLISHMENTS (CONTINUED )  ASTRAZENECA PHARMACEUTICALS (1999 2008) WILMINGTON, DE Advanced rapidly through a series of increasingly responsible roles, providing effective leadership in driving major commercialization and business development initiatives (multiple recipient of AstraZeneca Excellence Award). Alliance and Program Leader, Strategic Planning  Business Development  Spearheaded and managed collaboration to commercialize CRESTOR fenofibrate fixed -dose combination. Collaboration progressed six months ahead of schedule and $3.5M under budget.  Instrumental in design and implementation of new Alliance Management capability within the firm. Senior Director, Licensing Commercial Evaluations Managed commercialization strategy in collaboration with Global Product Teams. Led evaluations of Phase IIIII licensingacquisition opportunities across multiple therapeutic areas.  Led product and company alliance and acquisition evaluations covering oncology and neuroscience, driving successful progression of target opportunities through formal diligence to deal completion.  Defined and implemented novel commercialization approaches for various CRESTOR fixed dose combination opportunities. Senior Director, US Marketing  Commercial Operations  Oncology  Infection Directed commercialization strategy, market development, and licensing support for oncologyinfection emerging brands portfolio (seven drug candidates from Discovery through Phase III clinical development covering lung, prostate, breast, and colorectal cancers).  Directed overall strategic direction, including creation of integrated target product profiles and patient - centered claims across entire brand portfolio encompassing multiple pathways (SRC kinase, VEGFEGF, endothelin, vascular targeting agents, and prenylation inhibitors.)  Proactively identified new business opportunities and resolved strategic gaps with global product teams to consistently maintain momentum through key decision-points. Transaction Director, Licensing  North America Led transaction process through all stages involving analyzing, negotiating, closing, and managing transitions of select partnership opportunities (product divestments, co -promotions, and in -licenses.)  Directed all facets of buyer identification, business case prep, due diligence, negotiations, and transition planning. Successfully closed multiple divestment transactions under tight",
    "timeline": "Harty, Ronald N. LE O J.AD ALBE RT EARLIER RELEVANT EXPERIENCE AND KEY ACCOMPLISHMENTS (CONTINUED )  RHONE -POULENC RORER PHARMACEUTICALS, INC. COLLEGEVILLE, PA Senior Product Manager  U.S. Led market development and successful introduction of RILUTEK, the first and only medical therapy approved for the treatment of amyotrophic lateral sclerosis (Lou Gehrigs Disease) prolonging survival. Achieved 95 formulary acceptance within six months and sales 121 to plan. REGENERON PHARMACEUTICALS, INC. TARRYTOWN, NY Manager, Market Development Established commercial direction for the companys first biologic product CNTF (ciliary neurotrophic factor) for the treatment of multiple neurodegenerative diseases. (Product failed in late- Phase III clinical trials.) MERCK HUMAN HEALTH U.S.  a Division of Merck  Company WEST POINT, PA Senior Promotion Manager  Promoted from Promotions Manager Directed $20M product promotion and market development pre-launch and launch program. Drove successful introduction of PROSCAR in the U.S., the first medical therapy treatment of benign prostatic enlargement.  Physician and DTC Program acknowledged with several industry awards; achieved 119 to plan. Marketing Analyst  Promoted from Associate Marketing Analyst Designed, implemented, and analyzed strategic and product -specific marketing research.  Developed and presented recommendations to senior management based on research findings, resulting in $20M in media efficiencies.  Recognized with Merck Leadership Award of Excellence for developing methodology to analyze the impact of changes in promotion mixes and budgets that decreased planning time and increased efficiencies by 33. Professional Sales Representative, Western Pennsylvania Territory Managed $4 million office and hospital sales territory. Conducted educational forums, and presented product and clinical information to nurses, pharmacists and physicians in primary care, cardiology, neurology , orthopedics and ophthalmology. (Achieved 124 of goal) EDUCATION  PROFESSIONAL DEVELOPMENT HARVARD UNIVERSITY GRADUATE SCHOOL OF BUSINESS ADMINISTRATION BOSTON, MA Master of Business Administration (MBA) - Concentration: Marketing  Finance Awarded Harvard Business School Fellowship WEST VIRGINIA UNIVERSITY MORGANTOWN, WV Bachelor of Science in Business Administration  Highest Honors (Summa Cum Laude)  Rhodes Scholarship Finalist Beta Gamma Sigma National Business Honor Society; Phi Kappa Phi  Mortar Board National Honor Societies; Phi Sigma Kappa Fraternity (Repeat Merit Scholar ); Varsity Track Team (400 metersrelay ). CONTINUING EDUCATION:  Harvard Medical School HMX Program (completed with Achievement Honors Pharmacology and Immunology; Genetics and Physiology in progress to complete 2019)  Villanova UniversityBucks County Community College  Completion of full pre-medhealth sciences program with labs (anatomychemistrybiology) -recognized with highest honors and selected for membership in Phi Theta Kappa  Harvard School of Public Health (TH Chan School) o Measurement, Design, and Analysis Methods for Health Economics Research o Navigating the US Bio-Pharmaceutical Sector  Policy, Politics, Pricing, and Payment o Leading in Health Systems: Integrating Effort, Improving Outcomes Page 3 of 4 Budget Justification Attachment Page 60 Contact PDPI: Harty, Ronald N. LE O J.AD ALBE RT  EDUCATION  PROFESSIONAL DEVELOPMENT (CONTINUED )  Northwestern University Graduate School of Management: Graduate of Kellogg Executive Scholar Program  Wharton School (University of Pennsylvania): Graduate of Professional Development Program  Center for Creative Leadership: Leadership Development Program  Dartmouth: Global Leaders Program  GAP International: Leadership for Breakthrough Outcomes  Harvard Law School: Negotiations Strategy Program  Merck  Company: Strategic Marketing Plan Development  MIT: Systems Thinking Program CERTIFICATIONS: Certified Licensing Professional  Certified Strategic Alliance Professional COMMUNITY  PERSONAL  Special Projects Leader  Castleman Disease Collaborative Network (CDCN) www.cdcn.org  Co-Creator  Executive Board member of Camden, NJ Dream Center  (youth community outreach center) www.camdendreamcenter.org  Graduate of Chester County non- profit Leadership Development Program (partnership with United Way)  Competitive masters track and field athlete (400 and 800 meters) and road cyclist Page 4 of 4 Budget Justification Attachment Page 61 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 1 of 20 Quotation Date: 10 August 2020 Quotation Number: APQ -19-0548 Sponsor Jay Wrobel, PhD Vice President, Academic Relations Fox Chase Chemical Diversity Center Doylestown, PA 18902 Phone: Mobile: E-mail: Alliance Pharma, Inc. Ryan Klein, PhD Director, Business Development Malvern, PA 19355 Mobile: Phone: Fax: Email: Web: Non-GLP ADME Studies of Compounds for Fox Chase Chemical Diversity Center (Three Year Plan) 1 Plasma Protein Binding Study By Rapid Equilibrium Dialysis 1.1 Scope of Service To assess the protein binding of compounds in mouse and human plasma using a rapid equilibrium dialysis (RED) device (Thermo Fisher Scientific). 1.2 General Procedures The test compounds will be spiked into plasma samples to achieve a final concentration of 5 µM. Triplicate aliquots of the spiked plasma will be added to the red chambers of the RED device inserts, and phosphate buffered saline will be added into the white chambers of the inserts. The RED device plate will be sealed and incubated for 4 hours at 37C with shaking. Warfarin, a known highly bound drug in plasma, will be tested as the positive control for this assay. After incubation, samples from both the red and white chambers will be analyzed by LC-MSMS. The percentages of free and bound fractions will be determined using the peak area ratios of each analyte to internal standard. 1.3 General Information and Sample Requirement  Molecular weights, including formula weights and exact mass units, of the compounds.  Sufficient amount of the compounds.  Stability information for the compounds such as light, temperature, and solvents, if available.  Safety information for handling the compounds.  Solubility of the compounds, if available. APQ -20-0548_DM_FoxChase_vFinal Budget Justification Attachment Page 65 Contact PDPI: Harty, Ronald N. STUDY PROPOSAL p. 2 of 20 1.4 Deliverable A Microsoft Excel summary report including raw data, methodology, calculated percent bound, and percent recovery values will be provided. 1.5 Turnaround Time Report will be delivered within 7 business days from receipt of the compounds and any additional items specific to the assay that need to be sourced externally. 1.6 DocumentationRecords Data from this non -GLP study will be preserved for 1 year after submission of the",
    "innovation": "Harty, Ronald N. LE O J.AD ALBE RT PROFESSIONAL EXPERIENCE AND KEY ACCOMPLISHMENTS (CONTINUED )  ASTRAZENECA PHARMACEUTICALS (1999 2008) WILMINGTON, DE Advanced rapidly through a series of increasingly responsible roles, providing effective leadership in driving major commercialization and business development initiatives (multiple recipient of AstraZeneca Excellence Award). Alliance and Program Leader, Strategic Planning  Business Development  Spearheaded and managed collaboration to commercialize CRESTOR fenofibrate fixed -dose combination. Collaboration progressed six months ahead of schedule and $3.5M under budget.  Instrumental in design and implementation of new Alliance Management capability within the firm. Senior Director, Licensing Commercial Evaluations Managed commercialization strategy in collaboration with Global Product Teams. Led evaluations of Phase IIIII licensingacquisition opportunities across multiple therapeutic areas.  Led product and company alliance and acquisition evaluations covering oncology and neuroscience, driving successful progression of target opportunities through formal diligence to deal completion.  Defined and implemented novel commercialization approaches for various CRESTOR fixed dose combination opportunities. Senior Director, US Marketing  Commercial Operations  Oncology  Infection Directed commercialization strategy, market development, and licensing support for oncologyinfection emerging brands portfolio (seven drug candidates from Discovery through Phase III clinical development covering lung, prostate, breast, and colorectal cancers).  Directed overall strategic direction, including creation of integrated target product profiles and patient - centered claims across entire brand portfolio encompassing multiple pathways (SRC kinase, VEGFEGF, endothelin, vascular targeting agents, and prenylation inhibitors.)  Proactively identified new business opportunities and resolved strategic gaps with global product teams to consistently maintain momentum through key decision-points. Transaction Director, Licensing  North America Led transaction process through all stages involving analyzing, negotiating, closing, and managing transitions of select partnership opportunities (product divestments, co -promotions, and in -licenses.)  Directed all facets of buyer identification, business case prep, due diligence, negotiations, and transition planning. Successfully closed multiple divestment transactions under tight timelines, delivering significant operating income, tax benefits, and business efficiencies based on significant premiums to retention values.  Consistently earned DistinguishedExcellent performance ratings (top 2) and achieved Deal of the Year  recognition. EARLIER RELEVANT EXPERIENCE AND KEY ACCOMPLISHMENTS ELAN PHARMACEUTICALS, INC. SAN FRANCISCO, CA Director, Neuromuscular Marketing (business restructured via SEC obligations) Developed and led execution of strategic and tactical marketing plans to effectively support all stages of lifecyclemarket entry. Developed comprehensive product pre-launch and launch deployment plans that expanded Élans neuromuscular franchise into attractive new market segments. ASTRA -MERCK PHARMACEUTICALS, INC. (JV dissolved) WAYNE, PA Director, Neuroscience Marketing  Business Strategy Marketing Manager, Neuroscience (96-97) Directed franchise development of Astra -Mercks neuroscience business as commercial lead in formulating strategic direction and integration activities with Astra Sweden corporate portfolio.  Established strategic alliances and launched market development activities with patient advocacy groups, medical associations, and payers. Recognized with Leadership Award by Huntingtons Disease Society.  Recipient of Astra -Merck Leadership Award for Excellence for development and implementation of an innovative, comprehensive business and licensing strategy encompassing multiple neurological and psychiatric diseases, indications and product opportunities globally. Page 2 of 4 Budget Justification Attachment Page 59 Contact PDPI: Harty, Ronald N. LE O J.AD ALBE RT EARLIER RELEVANT EXPERIENCE AND KEY ACCOMPLISHMENTS (CONTINUED )  RHONE -POULENC RORER PHARMACEUTICALS, INC. COLLEGEVILLE, PA Senior Product Manager  U.S. Led market development and successful introduction of RILUTEK, the first and only medical therapy approved for the treatment of amyotrophic lateral sclerosis (Lou Gehrigs Disease) prolonging survival. Achieved 95 formulary acceptance within six months and sales 121 to plan. REGENERON PHARMACEUTICALS, INC. TARRYTOWN, NY Manager, Market Development Established commercial direction for the companys first biologic product CNTF (ciliary neurotrophic factor) for the treatment of multiple neurodegenerative diseases. (Product failed in late- Phase III clinical trials.) MERCK HUMAN HEALTH U.S.  a Division of Merck  Company WEST POINT, PA Senior Promotion Manager  Promoted from Promotions Manager Directed $20M product promotion and market development pre-launch and launch program. Drove successful introduction of PROSCAR in the U.S., the first medical therapy treatment of benign prostatic enlargement.  Physician and DTC Program acknowledged with several industry awards; achieved 119 to plan. Marketing Analyst  Promoted from Associate Marketing Analyst Designed, implemented, and analyzed strategic and product -specific marketing research.  Developed and presented recommendations to senior management based on research findings, resulting in $20M in media efficiencies.  Recognized with Merck Leadership Award of Excellence for developing methodology to analyze the"
  },
  "compliance": {
    "human_subjects": "No",
    "vertebrate_animals": "No",
    "biohazards": "No"
  },
  "personnel": {
    "co_investigators": [
      "Degree Type: PhD Degree Year: 1983",
      "Harty, Ronald N.",
      "Degree Type: PHD Degree Year: 2010",
      "https:www.niaid.nih.govgrants -contractssample- applications",
      "optimizing lead therapeutic compounds against",
      "Degree Type: PhD Degree Year: 2006",
      "Degree Type: PHD Degree Year: 1982",
      "Divisi Co-Investigator"
    ],
    "senior_personnel": [],
    "collaborators": [
      "0.6 cal. months). Dr. Reitz has had 38 years of",
      "6.b. Optional Explanation:  No",
      "Host-Pathogen Interactions is the study of the basic biology of infection in humans and animals  the immediate i ntersection of host and",
      "ABSL4 laboratory",
      "p. 2251-64.",
      "p. e32030.",
      "Implications for",
      "0.6 cal. months). Dr.",
      "No model organisms will be generated.",
      "Harty, Ronald N.",
      "provide insight into probe molecule identification and synthesis, in silico",
      "GRANT13199760 Funding Opportunity Number: PA-20-265. Received Date:",
      "Most important: Ebola and Marburg are disease targets that are eligible for the U.S. Neglected Disease",
      "3429-35. doi:",
      "In the scenario where we license rights to our product to a development partner, we expect that additional trained",
      "Adult T Cell Leukemia  Lymphoma,"
    ]
  },
  "references": [
    {

    {
      "id": 345,
      "citation": "Attach Biographical Sketch: File Name: Wrobel_biosketch_for_Harty.4.pdf",
      "type": "unknown"
    },


    },
    }
  ],
  "validation": {
    "completeness_score": 0.8148148148148148,
    "missing_required_fields": [
      "administrative_info.proposal_id",
      "administrative_info.nsf_program",
      "administrative_info.requested_amount",
      "project_description.keywords",
      "personnel.senior_personnel"
    ],
    "data_quality_issues": []
  }
}"""

    return example_proposal

# Instructions for Google Colab users
print("🚀 Enhanced Claude API Grant Reviewer for Google Colab")
print("=" * 60)
print()
print("📋 SETUP INSTRUCTIONS:")
print("1. Get your Anthropic API key from: https://console.anthropic.com/")
print("2. Run: reviewer = setup_claude_reviewer('your-api-key-here')")
print("3. Use: review_grant_proposal(reviewer, your_proposal_json)")
print()
print("💡 QUICK START:")
print("""
# Step 1: Initialize the reviewer
reviewer = setup_claude_reviewer('your-anthropic-api-key')

# Step 2: Prepare your proposal JSON
proposal_json = '''
{
  "document_info": {
    "file_path": "/content/R43-Brooks-Application.pdf",
    "document_type": "nsf_grant",
    "extraction_timestamp": "2025-07-10T23:35:30.142617"
  },
  "administrative_info": {
    "proposal_id": "1224457",
    "nsf_program": "",
    "submission_date": "6",
    "proposal_title": "High -throughput, multiplexed characterization and modeling of",
    "pi_name": "4040-0001",
    "pi_email": "deaweb@niaid.nih.gov",
    "institution": "Brooks, Benjamin Delbert",
    "requested_amount": ""
  },
  "budget": {
    "total_budget": "1",
    "personnel_costs": ",",
    "equipment_costs": "13",
    "travel_costs": "1",
    "indirect_costs": "1"
  },
  "project_description": {
    "overview": "",
    "intellectual_merit": "6825 -6836.",
    "broader_impacts": "4.c. If this project has an actual or potential impact on the environment, has an exemption been authorized or an",
    "keywords": []
  },
  "research_plan": {
    "objectives": [
      "1) to use biochemical and immunological approaches to explore how",
      "1) Develop the user interface and HT data analysissoftware. 2)",
      "antigen recognition at atomic resolution, these",
      "To evaluate recombinant proteins of vaccine and smallpox developed by C -PERL using baculoviruses",
      "antigen binding more broadly and more deeply than previously possible at this throughput. . In Phase",
      "12111 -63012",
      "co -inves tigator Contact PDPI: Brooks, Benjamin Delbert",
      "PI 1R21AI119741 (Griswold) 070116  06301 8",
      "gD2306t is consi dered a Gold S tandard to compare efficacy of vaccine candidates.",
      "Brooks, Benjamin Delbert",
      "antigen binding promises to fill pipelines with epitopically diverse",
      "PI",
      "7197 -73114",
      "PI Contact PDPI: Brooks, Benjamin Delbert",
      "Experimental . A preliminary epitope mapping study was perfor med by Wasatch and the",
      "co -investigator 1P01AI120756 (Tomaras, Duke) 040116  033121"
    ],
    "methodology": "gD, gB, a heterodimer of gHgL, and either of two cell receptors  herpes virus entry mediator (HVEM) or nectin- 1 14. Absence of any single protein abolishes entry and cell -cell Figure A.1. (A) Previously established relationships of the gD mAbs used in these studies. ( )  regions of gD involved in binding. Magenta  neutralizing mAbs. (B) Representation of extracellular domain of gD used in these studies. Nectin -1 disrupting mutations are in red. Contact PDPI: Brooks, Benjamin Delbert Research Strategy Page 66 fusion. Each step in this cascade of events leading to fusion is subject to blocking by epitope- specific virus - neutralizing mAbs. The Cohen- Eisen berg labs have crystallized and solved the structures of each of the entry proteins and the cell receptors , so that epitopes may be mapped onto the 3 -D structures of the proteins. In this phase I project, we will focus on the gD glycoprotein, for which the Cohen- Eisenberg labs have developed a large library of antibodies against both type I and type II HSV (Figure A.1). In addition to generally gaining a broader and deeper understanding of the binding patterns in these antibodies and other variants as much still remains uncharacterized in the library (especially specific epitopes), we will study an intriguing question about antigen diversity raised by a recent vaccine trial. GlaxoSmithKline developed a vaccine using gD from type II HSV; unfortunately the vaccine only provided protection against Type II HSV for 35 of the study participants , but surprisingly it provided o ver 80 protection against Type I HSV15. Studying the large -scale impacts of antigenic variability on antibody recognition will be made possible by the proposed approaches. Moreover, this work stands uniquely at the intersection between basic studies of recognition and immune responses to glycoproteins responsible for virus entry. Strength of the premise. The importance of characterizing antibody:antigen binding, particularly localizing epitopes, has been well established 5,7,11. Due to the time and expense of structure determination discussed above6,12, a variety of other methods have been developed, but they do not provide both the throughput and resolution of the proposed approach. For example H -D exchange mass spectrometry is still relatively low throughput6,12, while peptide arrays do not use native antigenic structures6,12. In addition, a lanine scanning is laborious and of a low hit rate16, while at the other extreme shotgun mutagenesis provides a lot of data but not of the same resolution and quality as SPR and due to random variation so literally hit -or-miss while not capturing impacts due to natural variation17. Purely computational approaches for antibody epitope prediction are not yet of sufficient accuracy to be generally useful18,19, though the steady improvement in antibody modeling and docking2027 provides some hope particularly for antibody -specific approaches, especially when integrated with experimental data 28 as we propose to do here. In particular, the proposed approach builds upon the well -established Wasatch SPR platform, for multiplexed evaluation of a matrix of either antibody:antigen binding or antibody:antibody competition for a single antigen. The latter approach is commonly termed  binning, as antibodies possessing similar blocking profiles are placed in the same bin 7. It is now commonly being conducted at the earliest possible stages in drug discovery 7,1012; e.g., Yasmina Abidche recently detailed how epitope binning using instrument packages from Wasatch and ForteBio is improving Pfizer -Rinats drug discovery processes29. Thus our proposed method builds upon this strong technological platform in order to address a significant gap in large- scale, rapid, detailed characterization of antibody:antigen binding. B. Innovation Researchers from both the vaccine community and the antibody therapeutic community need better, faster, and more scalable techniques to characterize antibody epitopes, across entire arrays of antibodies and antigenic variants. Since neither purely computational nor purely experimental techniques meet these goals, our innovative integrated methodology promises to combine the best characteristics of each side. The high - throughput, multiplexed analysis of antibody:antigen binding promises to fill pipelines with epitopically diverse candidates, inherently leading to more functional diversity and increased odds of success at the clinical stage.7 Specific innovations of our proposed approach include the following:  Integration of computational docking with large- scale antibody:antigen binding studies. Computational methods alone are not yet sufficient to localize epitopes and binding mechanisms, and neither are binding studies alone, but the integration of these techniques promises to provide complementary insights enabling modeling of the binding.  Computational design of sets of mutations to test specific binding hypotheses. In contrast to alanine scanning, single point mutation, and site saturation, our methods will design mutations based on prior knowledge and modeling, so as to best (most efficiently and confidently) evaluate putative binding modes.  Exploitation of natural and artificial antigenic variability . In order to localize epitopes, our methods will analyze the relationship between sequence differences and binding differences, inferring which sets of residues are likely to play key roles. Contact PDPI: Brooks, Benjamin Delbert Research Strategy Page 67  Systematic analysis of antibody competition assays across multiple antigens . While antibody competition with respect to a single antigen has become a standard approach, our approach will combine competition results across multiple variants to better characterize communities.  Development of reference panels . Our computational methods will select among natural variants and design new ones, so as to best support both coarse and fine grained epitope localization.  Computational analysis methods for large- scale antibody competition and binding studies . Our platform breaks new ground in characterizing antibody:antig en binding by integrated analysis of the large - scale data provided by the Wasatch SPR. C. Approach We will pursue two complementary approaches for the systematic characterization of antibody:antigen binding, requiring different experimental effort and yielding representations of different generality and specificity (Fig. C. 1). An antibody vs. antibody binning experiment (Fig. C.1, top) evaluates competition between antibodies in binding an antigen, and can thereby reveal which antibodies are likely to be recognizing overlapping regions on the antigen. Thus competition results from a large binning experiment, as Wasatch SPR can produce, are naturally represented in terms of communities of antibodies that tend to compete with each other. In our preliminary results below, we showed that binning identified communities consistent with previous low -throughput experiments conducted over many years, and that binning across antigenic variants resulted provided important information about the epitopic landscape of the protein, has been traditional ly been much more difficult to obtain. In our proposed work, we will extend from using binning only to identify antibody communities to also simultaneously character ize their rough footprints on the antigen. Consider a hypothetical scenario where two antigenic variants differ by only one amino acid, and the antibodies in a community bind one variant but not the other. Intuitively, we can infer that the variable amino acid is likely to be important for the binding of those antibodies. This intui tion also drives the popular but limited alanine scanning approach to epitope16, more focused site-directed mutagenesis methods17,30 as well literally hit -or- miss approaches like shotgun mutagenesis16. Our proposed approach will exploit this insight in the more natural scenario, which is also generally more helpful, where there are multiple antigen variants and each pair of antigens differs at multiple positions (Fig. C.2) . An antibody vs. antigen binding experiment (Fig. C.1, bottom) Figure C.1 . Cartoon intuition for the approaches. (top, aim 1) Binning- based inference of antibody communities and general epitope localization. Each matrix captures binning results between each pair of antibodies (rows and columns) competing to bind a single antigen; red indicates high competition, yellow medium, green low, and gray indeterminate. The data enable the inference of antibody communities from mutual competition, as well as general regions on the antigen reco gnized by each community from competition differences with respect to antigenic sequence variation. (bottom, aim 2 ) Binding -based inference of fine epitope localization. The single matrix represents binding between each antibody:antigen pair (antibodies as rows, antigenic variants as columns), with a quantitative binding measurement indicated in grayscale. Once again, the impact of antigenic sequence variation on the experimental results enables localization of antibody binding, but here at a finer resolution due to the larger extent of antigenic variability and the incorporation of additional antibody and antigen structural modeling. Figure C.2 . Superposition of gD1 structure and gD2 homology model, with backbones in ribbon mode and side chains shown o nly for positions with different amino acids . Contact PDPI: Brooks, Benjamin Delbert Research Strategy Page 68 evaluates the strength of a particular interaction; Wasatch SPR can do this in parallel for a whole matrix of pairs of antibodies and antigens. In our preliminary results below, we show that the SPR instrument readily enables identification of linear epitopes via antibody:peptide binding. In our proposed work, we will more precisely localize the hotspots mediating antibody:antigen recognition, with respect to native antigen conformations rather than individual peptides , and we will scale up to whole panels of antibodies . As de scribed intuitively above, impacts of antigenic variation on antibody binding enables inference of residues important for recognition. Here, in characterizing the epitope of a particular antibody, we will have a richer set of variants enabling finer grained localization. Our proposed approach will leverage this data with and without reference to predictions of putative binding modes by computational docking methods. Furthermore, we will assess the augmentation of preexisting natural antigenic variants with new ones that have been computationally designed to be informative in revealing epitopes. Overall, the contrast in the two approaches is that our binning method will generalize across antibodies, providing coarse localization information such that, assuming we have a sufficiently comprehensive panel, new antibodies are likely to fall into the same communities with the same interaction patterns. On the other hand, our mapping method will finely localize a specific interaction in a way that captures details of particular antibody:antigen pairs. In both cases, we leverage and assess the impact of antigenic variability, though using more extensive variability (at consequently more expense) for binding than for binning. In the binding case, we also consider use of antibody sequence (again, at more expense) and computationally hypothesized binding modes, and target the design of artificial antigenic variants to provide further information. Aim 1. Define communities of antibodies with similar antigen binding patterns. Preliminary Data. A preliminary epitope binning study was perfor med by Wasatch and the Cohe nEisenberg teams to evaluate the effectiveness of the high-throughput system for characterizing epitopes of a library of mAbs with multiple variants of an antigen. The results set the stage for this Phase I SBIR project. The study assessed four different gD constructs (truncations of length 285 and 306, for both type I and type 2 Figure C.3. Antibody community maps from four gD variants. A community plot is a network plot (e.g. social networks) generated from the epitope binning heat maps. The different colors represent 4 Wasatch derived unique communities formed by the binning characteristic s of 40 different gD monoclonal antibodies obtained from 6 different laboratories. Circles represent binding in both directions (surface attached and in solution) whereas squares represent binding only in one direct. Chord length indicates degree of relate dness s defined by relative completion in the binning assay. Dashed chord indicate that there is asymmetry in the relationship (i.e. blocking does not reciprocate). Figure C.4. Structural localization of antibody epitopes for antibodies of different specificities. The colors on the 3 -D structures represent the presumed location of two major epitopes IA and IB (pink and red) derived by genetic and mutagenic analysis. The C -termin us shown in black, lies along the intersection of IA and IB and also intersects the Nectin -1 receptor binding site. The N -terminus (shown in yellow) is presented to show the complexity of the Nectin -1 binding face. Contact PDPI: Brooks, Benjamin Delbert Research Strategy Page 69 HSV). The classical sandwich binning assay format was used, with antibody competition used to identify pairs of antibodies with overlapping HSV epitopes via Wasatchs CFM and Ibis MX96 surface plasmon resonance imager (SPRi) biosensor. Communities of co -competing antibodies were then identified, separately for each antigen (Figure C.3) which shows how the four general binding communities are related for each mutant . While the communities are generally the same between type 1 and 2 for gD 285, important structural differences in the proteins are exposed in gD 306 between type 1 and 2 two as shown in the differences between the community maps. Importantly, these differ ences in communities relate to important structural and epitope details that had been identified by previous low -throughput methods (Figure C.4). Proposed Work: Experimental. The CohenEisenberg Labs at the University of Pennsylvania has available multiple gD variants and multiple antibodies (Fig. A,1). These constructs as well as constructs obtained from Integral Molecular (see letters of support) will be used to conduct a set of epitope binning experiment s for up to 60 selected antibodies across at least 10 gD variants , to feed into the computational analysis . Proposed Work: Computational . Our goal is to find communities of related antibodies and corresponding regions on the antigen that mediate their binding. We will focus on the case where antibody communities are already defined by binning as in our preliminary results, since we have shown that that method yields communities consistent with external characterizations of related antibodies. But now we will have communities identified for multiple different antigenic variant s, with amino acid differences in the antigen leading to perturbations in binding (see again Fig. C.3) and thereby the competition results and identified antibody clusters. Under the assumption that antibodies in a community recognize nearby residues in the antigens structure (since they compete for binding to the antigen), those residues should be spatially proximal on the antigenic surface. Thus the approach is to cluster, with respect to the antigenic surface, the variable residues that perturb a community. We will implement and assess standard clustering techniques (hierarchical, k-means, etc.), with custom metrics that account for a combination of residue proximity on the surface and",
    "timeline": "Brooks, Benjamin Delbert",
    "innovation": "GM Accession Number: 3968518 IPF: 10005562 Organization: WASATCH MICROFLUIDICS Former Number: Department: IRGSRG: ZRG1 IMST -K (14)B AIDS: N Expedited: N Subtotal Direct Costs (excludes consortium FA) Year 1: Animals: N Humans: N Clinical Trial: N Current HS Code: 10 HESC: N New Investigator: N Early Stage Investigator: N SeniorKey Personnel: Organization: Role Category: Benjamin Brooks Ph.D WASATCH MICROFLUIDICS PDPI Chris Bailey -Kellog Dartmouth College MPI GARY COHEN University of Pennsylvania MPI Roselyn Eisenberg University of Pennsylvania MPI Always follow your funding opportunitys instructions for application format. Although this application demonstrate s good grantsmanship, time has passed since the grantee applied. The samples may not reflect the latest format or rules. NIAID posts new samples periodically : https:www.niaid.nih.govgrants-contractssample-applications The text of the applicatio n is copyrighted. The awarde e provided express permission for NIAID to post this grant application and summary statement for educational purposes. The awardee allow s you to use the material (e.g., data, writing, graphics) they shared in the applications for nonprofit educational purposes only, provided the material remains unchanged and the principal investigators , awardee organizations, and NIH NIAID are credited. Freedom of Information Act (FOIA). NIAID is strongly committed to protecting the integrity and confidentiality of the peer review process. When NIH responds to FOIA requests for grant applications and summary statements, the material will be subject to FOIA exemptions and include substantial redactions. NIH must protect all confidential commercial or financial information, reviewer comments and deliberations, and personal privacy information. Note on Section 508 Conformance and Accessibility. We have reformatted this sample to improve accessibility for people with disabilities and users of assistive technology. If you have trouble accessing the content, contact the NIAID Office of Knowledge and Educational Resources at deaweb@niaid.nih.gov . OMB Number: 4040-0001 Expiration Date: 06302016 Tracking Number: GRANT12244575Funding Opportunity Number: PA-16-302 . Received Date: 2016-09-06T17:27:06.000-04:00APPLICATION FOR FEDERAL ASSISTANCE SF 424 (RR)3. DATE RECEIVED BY STATE State Application Identifier 1. TYPE OF SUBMISSION 4.a. Federal Identifier  Pre-application  Application  ChangedCorrected Applicationb. Agency Routing Number 2. DATE SUBMITTED 2016-09-01Application Identifier c. Previous Grants.gov Tracking Number 5. APPLICANT INFORMATION Organizational DUNS: Legal Name: WASATCH MICROFLUIDICS Department: Division: Wasatch Microfluidics Street1: Street2: City: County: State: Province: Country: ZIP  Postal Code: Person to be contacted on matters involving this application Prefix: Prof. First Name: Benjamin Middle Name: Last Name: Brooks Suffix: PositionTitle: Street1: Street2: City: County: State: Province: Country: ZIP  Postal Code: Phone Number: Fax Number: Email: 6. EMPLOYER IDENTIFICATION NUMBER (EIN) or (TIN)  7. TYPE OF APPLICANT R: Small Business Other (Specify): Small Business Organization Type  Women Owned  Socially and Economically Disadvantaged 8. TYPE OF APPLICATION If Revision, mark appropriate box(es).  New  Resubmission  A. Increase Award  B. Decrease Award  C. Increase Duration  Renewal  Continuation  Revision  D. Decrease Duration  E. Other (specify) : Is this application being submitted to other agencies YesNoWhat other Agencies 9. NAME OF FEDERAL AGENCY National Institutes of Health10. CATALOG OF FEDERAL DOMESTIC ASSISTANCE NUMBER TITLE: 11. DESCRIPTIVE TITLE OF APPLICANTS PROJECT High-throughput, multiplexed characterization and modeling of antibody:antigen binding, with application to HSV 12. PROPOSED PROJECT Start Date Ending Date 07012017 0630201813. CONGRESSIONAL DISTRICTS OF APPLICANT UT-002 Contact PDPI: Brooks, Benjamin Delbert Tracking Number: GRANT12244575Funding Opportunity Number: PA-16-302 . Received Date: 2016-09-06T17:27:06.000-04:00SF 424 (RR) APPLICATION FOR FEDERAL ASSISTANCE Page 2 14. PROJECT DIRECTORPRINCIPAL INVESTIGATOR CONTACT INFORMATION Prefix: First Name: Benjamin Middle Name: Delbert Last Name: Brooks Suffix: Ph.D PositionTitle: Vice President of Research Organization Name: WASATCH MICROFLUIDICS Department: Division: Street1: Street2: City: County: State: Province: Country: ZIP  Postal Code: Phone Number: Fax Number: Email: 15. ESTIMATED PROJECT FUNDING a. Total Federal Funds Requested $ b. Total Non-Federal Funds $0.00 c. Total Federal  Non-Federal Funds $ d. Estimated Program Income $0.0016. IS APPLICATION SUBJECT TO REVIEW BY STATE EXECUTIVE ORDER 12372 PROCESS a.YESTHIS PREAPPLICATIONAPPLICATION WAS MADE AVAILABLE TO THE STATE EXECUTIVE ORDER 12372 PROCESS FOR REVIEW ON: DATE: b.NOPROGRAM IS NOT COVERED BY E.O. 12372; OR PROGRAM HAS NOT BEEN SELECTED BY STATE FOR REVIEW 17. By signing this application, I certify (1) to the statements contained in the list of certifications and (2) that the statements herein are true, complete and accurate to the best of my knowledge. I also provide the required assurances  and agree to comply with any resulting terms if I accept an award. I am aware that any false, fictitious, or fraudulent statements or claims may subject me to criminal, civil, or administrative penalties. (U.S. Code, Title 18, Section 1001)  I agree  The list of certifications and assurances, or an Internet site where you may obtain this list, is contained in the announcement or agency specific instructions. 18. SFLLL or OTHER EXPLANATORY DOCUMENTATION File Name: 19. AUTHORIZED REPRESENTATIVE Prefix: First Name: Josh Middle Name: Last Name: Eckman Suffix: PositionTitle: President Organization Name: Wasatch Microfluidics Department: Division: Street1: Street2: City: County: State: Province: Country: ZIP  Postal Code: Phone Number: Fax Number: Email: Signature of Authorized Representative Josh EckmanDate Signed 09062016 20. PRE-APPLICATION File Name: Mime Type: 21. COVER LETTER ATTACHMENT File Name:Cover_Letter_PA-16-302.pdf Mime Type: applicationpdf 424 RR and PHS-398 Specific Table Of ContentsPage Numbers SF 424 RR Cover Page----------------------------------------------------------------------------------------- 1 Table of Contents------------------------------------------------------------------------- 3 Performance Sites--------------------------------------------------------------------------------------------- 4 Research  Related Other Project Information------------------------------------------------------------------ 6 Project SummaryAbstract(Description)----------------------------------------------------- 7 Project Narrative------------------------------------------------------------------------- 8 Facilities  Other Resources-------------------------------------------------------------- 9 Equipment--------------------------------------------------------------------------------- 13 Other Attachments------------------------------------------------------------------------- 14 CBK_-_Personnel_Justification_PA-16-302------------------------------- 14 SBC_000348985--------------------------------------------------------- 15 Research  Related SeniorKey Person-------------------------------------------------------------------------- 16 Research  Related Budget Year - 1---------------------------------------------------------------------------- 39 Budget Justification------------------------------------------------------------------------------------------ 42 Research  Related Cumulative Budget-------------------------------------------------------------------------- 45 Research  Related Budget - Consortium Budget (Subaward 1)---------------------------------------------------- 46 Research  Related Budget - Consortium Budget (Subaward 2)---------------------------------------------------- 51 Total Direct Costs Less Consortium FA------------------------------------------------------------------------ 56 SBIR STTR Information----------------------------------------------------------------------------------------- 57 Research  Development Outside the U.S.--------------------------------------------------- 59 Prior SBIR Phase II Awards---------------------------------------------------------------- 61 PHS398 Cover Page Supplement---------------------------------------------------------------------------------- 62 PHS 398 Research Plan----------------------------------------------------------------------------------------- 64 Specific Aims----------------------------------------------------------------------------- 65 Research Strategy------------------------------------------------------------------------- 66 Multiple PDPI Leadership Plan------------------------------------------------------------ 72 Bibliography  References Cited----------------------------------------------------------- 73 Letters of Support------------------------------------------------------------------------ 75Contact PDPI: Brooks, Benjamin Delbert Table of Contents Page 3 Contact PDPI: Brooks, Benjamin Delbert OMB Number: 4040-0010 Expiration Date: 06302016 Tracking Number: GRANT12244575Funding Opportunity Number: PA-16-302 . Received Date: 2016-09-06T17:27:06.000-04:00ProjectPerformance Site Location(s) ProjectPerformance Site Primary Location  I am submitting an application as an individual, and not on behalf of a company, state, local or tribal government, academia, or other type of organization. Organization Name: Wasatch Microfluidics Duns Number: Street1: Street2: City: County: State: Province: Country: Zip  Postal Code: ProjectPerformance Site Congressional District: UT-002 ProjectPerformance Site Location 1  I am submitting an application as an individual, and not on behalf of a company, state, local or tribal government, academia, or other type of organization. Organization Name: Dartmouth College DUNS Number: Street1: Street2: City: County: State: Province: Country: Zip  Postal Code: ProjectPerformance Site Congressional District: ProjectPerformance Site Location 2  I am submitting an application as an individual, and not on behalf of a company, state, local or tribal government, academia, or other type of organization. Organization Name: Trustees of the University of Pennsylvania DUNS Number: Street1: Street2: City: County: State: Province: Country: Zip  Postal Code: ProjectPerformance Site Congressional District: PA-002 Page 4 Tracking Number: GRANT12244575Funding Opportunity Number: PA-16-302 . Received Date: 2016-09-06T17:27:06.000-04:00File Name Mime Type Additional Location(s)Contact PDPI: Brooks, Benjamin Delbert Page 5 Tracking Number: GRANT12244575Funding Opportunity Number: PA-16-302. Received Date: 2016-09-06T17:27:06.000-04:00OMB Number: 4040-0001 Expiration Date: 06302016 RESEARCH  RELATED Other Project Information 1. Are Human Subjects Involved YesNo 1.a. If YES to Human Subjects Is the Project Exempt from Federal regulations YesNo If YES, check appropriate exemption number: 123456 If NO, is the IRB review Pending YesNo IRB Approval Date: Human Subject Assurance Number 2. Are Vertebrate Animals Used YesNo 2.a. If YES to Vertebrate Animals Is the IACUC review Pending YesNo IACUC Approval Date: Animal Welfare Assurance Number 3. Is proprietaryprivileged information included in the application YesNo 4.a. Does this project have an actual or potential"
  },
  "compliance": {
    "human_subjects": "No",
    "vertebrate_animals": "No",
    "biohazards": "No"
  },
  "personnel": {
    "co_investigators": [],
    "senior_personnel": [
      "Cohen and Eisenberg Lab"
    ],
    "collaborators": [
      "6.b. Optional Explanation:",
      "215.573.8432",
      "antigen pair charact erize how well they bind, not",
      "Brooks, Benjamin Delbert"
    ]
  },
  "references": [
    {

'''

# Step 3: Get the review
review = review_grant_proposal(reviewer, proposal_json)

# Step 4: (Optional) Compare with summary statement
summary_statement_json = """ {
  "document_info": {
    "file_path": "/content/harty-summary-statement.pdf",
    "document_type": "nih_summary_statement",
    "extraction_timestamp": "2025-07-10T23:47:20.924548"
  },
  "administrative_info": {
    "application_number": "",
    "formerly": "3R41AI138630-",
    "principal_investigator": "HARTY, RONALD N  \nA",
    "applicant_organization": "",
    "review_group": "ZRG1  BCMB -G (10)",
    "meeting_date": "11/16/2020",
    "council": "JAN 2021 PCC",
    "rfa_pa": "PA20",
    "requested_start": "04/01/2021",
    "project_title": "Development of Small  Molecule  Therapeutics Targeting Hemorrhagic Fever  Viruses  \n \nSRG Action:",
    "impact_score": "",
    "human_subjects": "30-Human  subjects involved  - Certified,  no SRG  concerns",
    "animal_subjects": "30-Vertebrate animals involved - no SRG concerns noted"
  },
  "budget_info": {},
  "project_description": {
    "abstract": "",
    "public_health_relevance": ""
  },
  "resume_and_summary": {
    "summary_of_discussion": "A very strong and experienced in drug discovery group from the Fox Chase Chemical Diversity Center proposes to continue the development of small molecules for the treatment of hemorrhagic fever viruses. The significance of the proposed work is exceptionally high as effective therapies for emerging human RNA viruses are urgently needed. The Phase I portion of the project was very successful and yielded a lead series with promising properties, including in vivo proof-of-concept. There is a solid foundation for further optimization of drug-like properties and demonstration of in vivo efficacy. The overall approach is well-conceived, logically presented and sufficiently detailed; supplemental information shows that the lead molecule also blocks release of live infectious SARS-CoV-2 from infected human lung cells, which is consistent with the mechanism of action. Few minor weaknesses associated with the unclear ‘new’ scaffolds do not detract from the very high enthusiasm the panel expressed for this comprehensive and likely impactful proposal."
  },
  "critiques": [
    {
      "critique_number": "1",
      "overall_impact": "The overall impact of this Phase II STTR proposal to develop effective therapies for RNA viruses caused by Marburg and Ebola, is very high. These are also rare diseases, and the investigators can benefit from that designation along with being eligible for the US neglected disease voucher program. The scientific rigor from the data obtained from the Phase I study is strong with the identification of two lead series of compounds and an advanced lead compound with very good potency. The strategy to block the viral-host interactions between the matrix protein PPxY and select host proteins for efficient virus egress (cellular budding) and spreading infection appears innovative. The strong premise of the project plan with the potential to identify a preclinical candidate that could lead to acute toxicity study phase of the pre-IND. The details of a screening cascade, target product profile and go no-go decisions, and a robust commercialization plan with options, provide confidence that this well rounded team can execute the current objectives and has the high potential to nominate a clinical candidate from this project. The resources lined up and the facilities that will be used to conduct the proposed studies and the strength of a world class team assembled around this project has made this a highly impactful proposal.",
      "candidate_strengths": [
        "There is an unmet need to develop effective therapeutics to protect immunologically naïve individuals form the RNA virus infections as the Marburg and Ebola viruses cause hemorrhagic diseases, which is a rare disease.",
        "Currently available vaccines are not a complete defense since they must be given pre- exposure.",
        "The therapeutic development program for these diseases is eligible for the US neglected disease voucher program."
      ],
      "candidate_weaknesses": [
        "None noted."
      ],
      "career_plan_strengths": [],
      "career_plan_weaknesses": [],
      "research_plan_strengths": [],
      "research_plan_weaknesses": [],
      "mentor_strengths": [],
      "mentor_weaknesses": [],
      "environment_strengths": [
        "There is an unmet need to develop effective therapeutics to protect immunologically naïve individuals form the RNA virus infections as the Marburg and Ebola viruses cause hemorrhagic diseases, which is a rare disease.",
        "Currently available vaccines are not a complete defense since they must be given pre- exposure.",
        "The therapeutic development program for these diseases is eligible for the US neglected disease voucher program."
      ],
      "environment_weaknesses": [
        "None noted."
      ]
    },
    {
      "critique_number": "2",
      "overall_impact": "This Phase II STTR application aims to develop novel small molecule, broad-spectrum antivirals against infections caused by filoviruses, arenaviruses, and other viruses that depend on the PPxY L-domain motif for virus egress and spread of infection [Marburg (MARV), Ebola (EBOV), and Lassa fever (LAFV)]. There are no commercially available small molecule therapeutics for these viral infections. The proposed anti-virals will fill a significant unmet need. The team has discovered several chemical series that block the host Nedd4/virus PPxY complex (critical regulator of viral transmission) and have identified an analog capable of blocking in vivo activity in a Marburg virus challenged mouse model as proof of concept. In the current proposal they will use their current lead inhibitor series, (e.g., FC-10696) to generate novel candidates for IND-enablement studies. To accomplish this goal, 2 R42 AI138630-03 6 ZRG1 BCMB-G (10) HARTY, R scientists at the Fox Chase Chemical Diversity Center, Inc. (FCCDC; pharmaceutical and medicinal chemistry), the Harty Lab at the University of Pennsylvania (experimental aspects of antiviral therapy), and the lab of Olena Shtanko at Texas Biomedical Research Institute (evaluating compounds against live viruses in vitro and in vivo under BSL-4 conditions) have combined their considerable experience and expertise. The team of researchers has extensive expertise in relevant areas and are a major strength. Preliminary data provided in the proposal indicate that the approach is promising and newly designed leads exhibit antiviral activity. The overall approach is well conceived, is based on solid literature precedent, and addresses the major aspects required of a drug discovery program. In addition, supplements (Additional Materials) recently submitted by the PI suggest the compounds under development may have the potential to serve as broad-spectrum antivirals against emerging pathogens (e.g., SARS-Co-V-2). There is considerable enthusiasm for the project with only minor concerns.",
      "candidate_strengths": [
        "The project addresses an important, current need for effective anti-viral therapy.",
        "The novel agents presented potently interrupt the host Nedd4: viral PPxY interaction to inhibit virus budding and spread.",
        "If the project is successful it could lead to a novel, much needed antiviral treatment.",
        "The scientific premise is well supported by literature reports.",
        "The team has experience with antivirals and a history in the drug discovery area."
      ],
      "candidate_weaknesses": [
        "None noted."
      ],
      "career_plan_strengths": [],
      "career_plan_weaknesses": [],
      "research_plan_strengths": [],
      "research_plan_weaknesses": [],
      "mentor_strengths": [],
      "mentor_weaknesses": [],
      "environment_strengths": [
        "The project addresses an important, current need for effective anti-viral therapy.",
        "The novel agents presented potently interrupt the host Nedd4: viral PPxY interaction to inhibit virus budding and spread.",
        "If the project is successful it could lead to a novel, much needed antiviral treatment.",
        "The scientific premise is well supported by literature reports.",
        "The team has experience with antivirals and a history in the drug discovery area."
      ],
      "environment_weaknesses": [
        "None noted."
      ]
    },
    {
      "critique_number": "3",
      "overall_impact": "This is a strong proposal for the development of small molecules for treatment of hemorrhagic fever viruses. Their Phase I work yielded a lead series with promising properties, including in vivo proof-of-concept. Phase II is focused on further optimization of drug-like properties and demonstration of in vivo efficacy. The supplemental information shows that the lead molecule also blocks release of live infectious SARS-CoV-2 from infected human lung cells, which is consistent with the MOA. Overall, a strong proposal with clear goals for optimization and an effective screening tier that will give confidence in the develop-ability of an optimized lead.",
      "candidate_strengths": [],
      "candidate_weaknesses": [],
      "career_plan_strengths": [],
      "career_plan_weaknesses": [],
      "research_plan_strengths": [],
      "research_plan_weaknesses": [],
      "mentor_strengths": [],
      "mentor_weaknesses": [],
      "environment_strengths": [
        "High unmet medical need for treatment of highly pathogenic viruses such as Ebola, Marburg, and Lassa viruses; potential for application to SARS-CoV-2 (supplementary material).",
        "Application to viral treatment in both community settings and in response to bioterrorist use of the hemorrhagic viruses."
      ],
      "environment_weaknesses": [
        "None noted."
      ]
    }
  ],
  "compliance_info": {
    "human_subjects_protection": "ACCEPTABLE INCLUSION OF WOMEN PLAN: ACCEPTABLE INCLUSION OF MINORITIES PLAN: ACCEPTABLE",
    "vertebrate_animal": "",
    "biohazard": "",
    "responsible_conduct_training": "",
    "data_sharing_plan": "",
    "resource_authentication": ""
  },
  "budget_recommendations": {
    "committee_recommendations": "The budget was recommended as requested."
  }
}"""

print()
print("📝 DEMO:")
print("Run demo_grant_reviewer() to get example proposal JSON")
print()
print("🔧 Available models:")
print("- claude-3-5-sonnet-20241022 (default, recommended)")
print("- claude-3-haiku-20240307 (faster, less detailed)")
print("- claude-3-opus-20240229 (most thorough)")
print()
print("🆕 NEW FEATURES:")
print("- Enhanced critique structure similar to NIH summary statements")
print("- Comparative analysis with existing summary statements")
print("- More detailed assessment categories")
print("- Actionable recommendations for improvement")

In [ ]:
import os
import anthropic
from google.colab import userdata

class ClaudeGrantReviewer:
    """
    Enhanced AI Grant Reviewer that uses Claude API to analyze grant proposals
    and provide detailed critiques similar to NIH summary statements.
    """

    def __init__(self, api_key: str = None):
        """
        Initialize the Claude Grant Reviewer.

        Args:
            api_key: Anthropic API key. If None, will try to get from Colab secrets.
        """
        if api_key is None:
            api_key = userdata.get('ANTHROPIC_API_KEY')
            if api_key is None:
                raise ValueError("API key must be provided either as parameter or stored as 'ANTHROPIC_API_KEY' in Colab secrets.")

        self.client = anthropic.Anthropic(api_key=api_key)
        self.system_prompt = self._get_system_prompt()

    def _get_system_prompt(self) -> str:
        """
        Placeholder for system prompt logic.
        Replace with actual prompt generation based on document type or config.
        """
        # Example placeholder prompt
        return """You are an AI assistant specialized in reviewing grant proposals.
Provide a detailed critique of the grant based on the provided content,
following the structure of an NIH summary statement critique."""